In [1]:
import os
# Configure the environment
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = '/opt/spark'
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NASA_Transportation").getOrCreate() 
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
#from __future__ import print_function 
import json
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
import json
import pyspark.sql.functions as psf
from pyspark.sql.functions import lit, col, upper, udf
import hashlib
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from decimal import Decimal

In [2]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=NASA_Transportation>

In [3]:


# Create Spark session
spark = SparkSession.builder \
    .getOrCreate()

In [176]:
#read the maritime lookup
maritime_transport_coverage_location = "D://notebooks/NASA/Transport_coverage.csv"


df_tra_cov_lkp = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep",",").option("quote ",'"')\
            .option("esc","\\")\
            .load(maritime_transport_coverage_location)

df_tra_cov_lkp.show(10, False)

+----------------------------------------------------------------+-------------------------------------+----------+---------+------+------------+
|URI                                                             |Label                                |Definition|Notation |Status|AcceptedDate|
+----------------------------------------------------------------+-------------------------------------+----------+---------+------+------------+
|http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/TOTAL    |Total transport                      |null      |TOTAL    |valid |2013-08-09  |
|http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/NAT      |National transport                   |null      |NAT      |valid |2013-08-09  |
|http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL     |International transport              |null      |INTL     |valid |2013-08-09  |
|http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL_LD  |International transport - loaded     |null      |INTL_LD  

In [5]:
# +----------------------------------------------------------------+-------------------------------------+----------+---------+------+------------+
# |URI                                                             |Label                                |Definition|Notation |Status|AcceptedDate|
# +----------------------------------------------------------------+-------------------------------------+----------+---------+------+------------+
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/TOTAL    |Total transport                      |null      |TOTAL    |valid |2013-08-09  |
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/NAT      |National transport                   |null      |NAT      |valid |2013-08-09  |
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL     |International transport              |null      |INTL     |valid |2013-08-09  |
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL_LD  |International transport - loaded     |null      |INTL_LD  |valid |2013-08-09  |
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL_NLD |International transport - unloaded   |null      |INTL_NLD |valid |2013-08-09  |
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL_CTRD|International transport - cross-trade|null      |INTL_CTRD|valid |2013-08-09  |
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL_CAB |International transport - cabotage   |null      |INTL_CAB |valid |2013-08-09  |
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL_IN  |International transport - incoming   |null      |INTL_IN  |valid |2013-08-09  |
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL_OUT |International transport - outgoing   |null      |INTL_OUT |valid |2013-08-09  |
# |http://dd.eionet.europa.eu/vocabulary/eurostat/tra_cov/INTL_IEU |International transport Intra-EU     |null      |INTL_IEU |valid |2013-08-09  |
# +----------------------------------------------------------------+-------------------------------------+----------+---------+------+------------+

In [6]:
#read the maritime lookup
maritime_entity_location = "D://notebooks/NASA/Maritime_entity.csv"


df_mar_lkp = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep",",").option("quote ",'"')\
            .option("esc","\\")\
            .load(maritime_entity_location)

df_mar_lkp.show(10, False)

+--------+----------------------------------------------------------------------+--------+
|notation|prefLabel                                                             |country |
+--------+----------------------------------------------------------------------+--------+
|EU28    |European Union - 28 countries (2013-2020)                             |Europe  |
|EU27    |European Union - 27 countries (2007-2013)                             |Europe  |
|EU25    |European Union - 25 countries (2004-2006)                             |Europe  |
|EU15    |European Union - 15 countries (1995-2004)                             |Europe  |
|EA15    |Euro area (BE, DE, IE, EL, ES, FR, IT, CY, LU, MT, NL, AT, PT, SI, FI)|Europe  |
|EA13    |Euro area (BE, DE, IE, EL, ES, FR, IT, LU, NL, AT, PT, SI, FI)        |Europe  |
|EA12    |Euro area (BE, DE, IE, EL, ES, FR, IT, LU, NL, AT, PT, FI)            |Europe  |
|BE      |Belgium                                                               |Belgium |

In [7]:
# +--------+----------------------------------------------------------------------+
# |notation|prefLabel                                                             |
# +--------+----------------------------------------------------------------------+
# |EU28    |European Union - 28 countries (2013-2020)                             |
# |EU27    |European Union - 27 countries (2007-2013)                             |
# |EU25    |European Union - 25 countries (2004-2006)                             |
# |EU15    |European Union - 15 countries (1995-2004)                             |
# |EA15    |Euro area (BE, DE, IE, EL, ES, FR, IT, CY, LU, MT, NL, AT, PT, SI, FI)|
# |EA13    |Euro area (BE, DE, IE, EL, ES, FR, IT, LU, NL, AT, PT, SI, FI)        |
# |EA12    |Euro area (BE, DE, IE, EL, ES, FR, IT, LU, NL, AT, PT, FI)            |
# |BE      |Belgium                                                               |
# |BG      |Bulgaria                                                              |
# |DK      |Denmark                                                               |
# +--------+----------------------------------------------------------------------+

In [296]:
#read the maritime lookup
Traffic_and_transport_measurement_location = "D://notebooks/NASA/Traffic_and_transport_measurement.csv"


Traffic_and_transport_lkp = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep",",").option("quote ",'"')\
            .option("esc","\\")\
            .load(Traffic_and_transport_measurement_location)

for col in Traffic_and_transport_lkp.columns:
    Traffic_and_transport_lkp = Traffic_and_transport_lkp.withColumnRenamed(col,col.replace(" ", ""))

Traffic_and_transport_lkp.show(10, False)

+--------------------------------------------------------------------+-----------------------------------+----------+------------+-------+------------+
|URI                                                                 |Label                              |Definition|Notation    |Status |AcceptedDate|
+--------------------------------------------------------------------+-----------------------------------+----------+------------+-------+------------+
|http://dd.eionet.europa.eu/vocabulary/eurostat/tra_meas/TOTAL       |Total                              |null      |TOTAL       |valid  |2010-04-20  |
|http://dd.eionet.europa.eu/vocabulary/eurostat/tra_meas/PASS        |Passengers embarked and disembarked|null      |PASS        |invalid|2010-04-20  |
|http://dd.eionet.europa.eu/vocabulary/eurostat/tra_meas/PASS_ONLY   |Passenger only                     |null      |PASS_ONLY   |invalid|2010-04-20  |
|http://dd.eionet.europa.eu/vocabulary/eurostat/tra_meas/PASS_EMB    |Passengers embarke

In [127]:
#read the maritime lookup
ICAO_airport_codes_location = "D://notebooks/NASA/UE_Airports.csv"


df_ICAO_airport_codes_lkp = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep",",").option("quote ",'"')\
            .option("esc","\\")\
            .load(ICAO_airport_codes_location)

df_ICAO_airport_codes_lkp.show(10, False)

+----+----+------------------------------------+------------+---------------+
|ICAO|IATA|AirportName                         |Municipality|Country        |
+----+----+------------------------------------+------------+---------------+
|EKYT|AAL |Aalborg Airport                     |Aalborg     |Denmark        |
|EGPD|ABZ |Aberdeen Dyce Airport               |Aberdeen    |United Kingdom |
|LEMD|MAD |Adolfo Suárez Madrid–Barajas Airport|Madrid      |Spain          |
|LEAL|ALC |Alicante International Airport      |Alicante    |Spain          |
|EHAM|AMS |Amsterdam Airport Schiphol          |Amsterdam   |The Netherlands|
|LFOA|null|Avord (BA 702) Air Base             |Avord       |France         |
|LEBL|BCN |Barcelona–El Prat Airport           |Barcelona   |Spain          |
|LIBD|BRI |Bari Karol Wojtyła Airport          |Bari        |Italy          |
|EGAA|BFS |Belfast International Airport       |Belfast     |United Kingdom |
|LYBE|BEG |Belgrade Nikola Tesla Airport       |Belgrade    |Ser

In [8]:
# read the airport codes lookup
import urllib.request

url = "https://pkgstore.datahub.io/core/airport-codes/airport-codes/archive/edda13b18a6832d040c1ff19fbd4a8fd/airport-codes.csv"
response = urllib.request.urlopen(url)
data = response.read()      
text = data.decode('utf-8')  

air_code_lkp = spark.read.csv(sc.parallelize(text.splitlines()), header = True)


In [9]:
air_code_lkp.filter("continent='EU'").show(10 , False)

+-------+--------+------------------------------+------------+---------+-----------+----------+------------------+--------+---------+----------+----------------------------------+
|ident  |type    |name                          |elevation_ft|continent|iso_country|iso_region|municipality      |gps_code|iata_code|local_code|coordinates                       |
+-------+--------+------------------------------+------------+---------+-----------+----------+------------------+--------+---------+----------+----------------------------------+
|ABE    |closed  |RAF Calveley                  |null        |EU       |GB         |GB-ENG    |Cheshire          |null    |null     |null      |-2.603889, 53.113333              |
|ABL    |closed  |RNAS/RAF Calshot              |null        |EU       |GB         |GB-ENG    |Hampshire         |null    |null     |null      |-1.30677223206, 50.8199131549     |
|AD-0001|heliport|Camí Heliport                 |null        |EU       |AD         |AD-04     |La Ma

In [146]:
# df_air_codes_merged = .join(air_code_lkp, ((psf.lower(air_code_lkp.name)== psf.lower(df_ICAO_airport_codes_lkp.Municipality)) & (psf.lower(psf.trim(air_code_lkp.municipality))== psf.lower(psf.trim(df_ICAO_airport_codes_lkp.Municipality)))), how = 'inner')

# # df_air_codes_merged = df_air_codes_merged.('code_airport')

In [148]:
df_ICAO_airport_codes_lkp.show(10, False)

+----+----+------------------------------------+------------+---------------+
|ICAO|IATA|AirportName                         |Municipality|Country        |
+----+----+------------------------------------+------------+---------------+
|EKYT|AAL |Aalborg Airport                     |Aalborg     |Denmark        |
|EGPD|ABZ |Aberdeen Dyce Airport               |Aberdeen    |United Kingdom |
|LEMD|MAD |Adolfo Suárez Madrid–Barajas Airport|Madrid      |Spain          |
|LEAL|ALC |Alicante International Airport      |Alicante    |Spain          |
|EHAM|AMS |Amsterdam Airport Schiphol          |Amsterdam   |The Netherlands|
|LFOA|null|Avord (BA 702) Air Base             |Avord       |France         |
|LEBL|BCN |Barcelona–El Prat Airport           |Barcelona   |Spain          |
|LIBD|BRI |Bari Karol Wojtyła Airport          |Bari        |Italy          |
|EGAA|BFS |Belfast International Airport       |Belfast     |United Kingdom |
|LYBE|BEG |Belgrade Nikola Tesla Airport       |Belgrade    |Ser

In [48]:
# read country codes lookup
import urllib.request

country_codes_location ='https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'
#= "https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv"
response_code = urllib.request.urlopen(country_codes_location)
data_country_codes = response_code.read()      
country_codes = data_country_codes.decode('utf-8')  


df_country_codes = spark.read.csv(sc.parallelize(country_codes.splitlines()), header = True)
df_country_codes = df_country_codes.withColumnRenamed("name", "Country_name")
df_country_codes = df_country_codes.withColumnRenamed("alpha-3", "Country_code")
df_country_codes = df_country_codes.withColumnRenamed("alpha-2", "Country_iso_code")

In [49]:
df_country_codes.show(1, False)

+------------+----------------+------------+------------+-------------+------+-------------+-------------------+-----------+---------------+------------------------+
|Country_name|Country_iso_code|Country_code|country-code|iso_3166-2   |region|sub-region   |intermediate-region|region-code|sub-region-code|intermediate-region-code|
+------------+----------------+------------+------------+-------------+------+-------------+-------------------+-----------+---------------+------------------------+
|Afghanistan |AF              |AFG         |004         |ISO 3166-2:AF|Asia  |Southern Asia|null               |142        |034            |null                    |
+------------+----------------+------------+------------+-------------+------+-------------+-------------------+-----------+---------------+------------------------+
only showing top 1 row



In [ ]:
# +------------+----------------+------------+------------+-------------+------+-------------+-------------------+-----------+---------------+------------------------+
# |Country_name|Country_iso_code|Country_code|country-code|iso_3166-2   |region|sub-region   |intermediate-region|region-code|sub-region-code|intermediate-region-code|
# +------------+----------------+------------+------------+-------------+------+-------------+-------------------+-----------+---------------+------------------------+
# |Afghanistan |AF              |AFG         |004         |ISO 3166-2:AF|Asia  |Southern Asia|null               |142        |034            |null                    |
# +------------+----------------+------------+------------+-------------+------+-------------+-------------------+-----------+---------------+------------------------+

In [155]:
df_maritime_codes = df_mar_lkp.join(df_country_codes, on=(psf.lower(df_mar_lkp.country)==psf.lower(df_country_codes.Country_name)), how ='inner' ).select(df_country_codes.Country_code, *[c for c in df_mar_lkp.columns])

In [156]:
df_maritime_codes.show(10, False)

+------------+---------+-------------------------------------------------------------------------+-------+
|Country_code|notation |prefLabel                                                                |country|
+------------+---------+-------------------------------------------------------------------------+-------+
|BEL         |BE_0BE888|Belgium - other ports                                                    |Belgium|
|BEL         |BE_0BE002|Belgium - other inland ports (other than Zeekanaal Brussel-Schelde ports)|Belgium|
|BEL         |BE_0BE001|Zeekanaal Brussel-Schelde - ports                                        |Belgium|
|BEL         |BE_0BEZEE|Zeebrugge                                                                |Belgium|
|BEL         |BE_0BEOST|Oostende (Ostend)                                                        |Belgium|
|BEL         |BE_0BENIE|Nieuwpoort                                                               |Belgium|
|BEL         |BE_0BELGG|Li�ge        

In [159]:
df_airport_codes = df_ICAO_airport_codes_lkp.join(df_country_codes, on=(psf.lower(df_ICAO_airport_codes_lkp.Country)==psf.lower(df_country_codes.Country_name)), how ='inner' ).select(df_country_codes.Country_code, psf.concat(df_country_codes.Country_iso_code, lit('_'), df_ICAO_airport_codes_lkp.ICAO).alias('code_ICAO'), *[c for c in df_ICAO_airport_codes_lkp.columns])

In [160]:
df_airport_codes.show(10, False)

+------------+---------+----+----+------------------------------------------+------------+----------------------+
|Country_code|code_ICAO|ICAO|IATA|AirportName                               |Municipality|Country               |
+------------+---------+----+----+------------------------------------------+------------+----------------------+
|ALB         |AL_LATI  |LATI|TIA |Tirana International Airport Mother Teresa|Tirana      |Albania               |
|AUT         |AT_LOWW  |LOWW|VIE |Vienna International Airport              |Vienna      |Austria               |
|BLR         |BY_UMMS  |UMMS|MSQ |Minsk National Airport                    |Minsk       |Belarus               |
|BEL         |BE_EBLG  |EBLG|LGG |Liège Airport                             |Liège       |Belgium               |
|BEL         |BE_EBCI  |EBCI|CRL |Brussels South Charleroi Airport          |Brussels    |Belgium               |
|BEL         |BE_EBBR  |EBBR|BRU |Brussels Airport                          |Brussels   

In [16]:
# +------------+-----+--------------+--------------------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------------------------+
# |Country_code|ident|type          |name                            |elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|coordinates                           |
# +------------+-----+--------------+--------------------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------------------------+
# |YEM         |OY74 |medium_airport|Al Anad Air Base                |900         |AS       |YE         |YE-LA     |null        |null    |null     |null      |44.765098571777344, 13.174699783325195|
# |YEM         |OY75 |small_airport |Canoxy Airport                  |3300        |AS       |YE         |YE-HD     |null        |OY75    |null     |OY75      |49.0806999206543, 15.601799964904785  |
# |YEM         |OYAA |medium_airport|Aden International Airport      |7           |AS       |YE         |YE-AD     |Aden        |OYAA    |ADE      |null      |45.02880096435547, 12.829500198364258 |
# |YEM         |OYAB |small_airport |Abbse Airport                   |651         |AS       |YE         |YE-SN     |Abbse       |OYAB    |EAB      |null      |43.176667, 16.010833                  |
# |YEM         |OYAT |medium_airport|Ataq Airport                    |3735        |AS       |YE         |YE-AB     |null        |OYAT    |AXK      |null      |46.82619857788086, 14.551300048828125 |
# |YEM         |OYBA |small_airport |Al Badie Airport                |908         |AS       |YE         |YE-HD     |null        |OYBA    |null     |null      |50.83689880371094, 18.71929931640625  |
# |YEM         |OYBI |small_airport |Al-Bayda Airport                |6120        |AS       |YE         |YE-BA     |Al-Bayda    |OYBI    |BYD      |null      |45.441079, 14.105974                  |
# |YEM         |OYBN |medium_airport|Beihan Airport                  |3800        |AS       |YE         |YE-AB     |null        |OYBN    |BHN      |null      |45.72010040283203, 14.781999588012695 |
# |YEM         |OYBQ |small_airport |Al-Bough Airport                |3800        |AS       |YE         |YE-SD     |Al-Bough    |OYBQ    |BUK      |null      |44.621700286899994, 17.3467006683     |
# |YEM         |OYGD |medium_airport|Al Ghaidah International Airport|134         |AS       |YE         |YE-MR     |null        |OYGD    |AAY      |null      |52.17499923706055, 16.191699981689453 |
# +------------+-----+--------------+--------------------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------------------------+

In [286]:
#read the gross weight of ports file
Gross_weight_of_ports_location = "D://notebooks/NASA/Datasets/Gross_weight_of_ports_mar_go_qm.tsv//mar_go_qm.tsv"


df_GWOP = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep","\t").option("quote ",'"')\
            .option("esc","\\")\
            .load(Gross_weight_of_ports_location)

# replace ':' with 'N/A'
df_GWOP_1 = df_GWOP.replace(": ", None).replace(" ", "")
# replace blank spaces from column names
for col in df_GWOP.columns:
    df_GWOP_1 = df_GWOP_1.withColumnRenamed(col,col.replace(" ", "")).withColumnRenamed(col,col.replace("\\", "_per_"))
    


In [287]:


split_col = psf.split(df_GWOP_1['direct,tra_cov,unit,rep_mar_per_time'], ',')
df_GWOP_2 = df_GWOP_1.withColumn('direct', split_col.getItem(0))\
                    .withColumn('tra_cov', split_col.getItem(1))\
                    .withColumn('unit', split_col.getItem(2))\
                    .withColumn('rep_mar_per_time', split_col.getItem(3))\
                    .select( "direct", "tra_cov", "unit", "rep_mar_per_time", "2020Q1", "2019Q4", "2019Q3",
                           "2019Q2", "2019Q1")
df_GWOP_2 =  df_GWOP_2.withColumn('unit', psf.when(df_GWOP_2.unit == 'THS_T', 'Thousand tonnes'))

In [288]:
df_GWOP_labeled = df_GWOP_2.join(df_maritime_codes,  df_GWOP_2.rep_mar_per_time == df_maritime_codes.notation, how='inner').select(df_maritime_codes.country, df_maritime_codes.Country_code, *[c for c in df_GWOP_2.columns])

In [289]:
df_GWOP_labeled_tra = df_GWOP_labeled.join(df_tra_cov_lkp, psf.upper(df_GWOP_labeled.tra_cov) == psf.upper(df_tra_cov_lkp.Notation), how = 'inner').select(df_tra_cov_lkp.Label, *[c for c in df_GWOP_labeled.columns])

In [102]:
df_GWOP_labeled_tra.drop('tra_cov', 'rep_mar_per_time', 'Label', 'direct').distinct().show(10, False)

#Croatia <> 'N/A' -> data available 

+--------+------------+---------------+------+------+------+------+------+
|country |Country_code|unit           |2020Q1|2019Q4|2019Q3|2019Q2|2019Q1|
+--------+------------+---------------+------+------+------+------+------+
|Belgium |BEL         |Thousand tonnes|N/A   |N/A   |2697  |2861  |2398  |
|Belgium |BEL         |Thousand tonnes|N/A   |N/A   |1034  |1279  |1028  |
|Belgium |BEL         |Thousand tonnes|N/A   |N/A   |19144 |20286 |18181 |
|Belgium |BEL         |Thousand tonnes|N/A   |N/A   |17955 |20166 |18611 |
|Bulgaria|BGR         |Thousand tonnes|N/A   |1253  |2269  |1061  |1241  |
|Denmark |DNK         |Thousand tonnes|N/A   |9     |5     |81    |122   |
|Denmark |DNK         |Thousand tonnes|N/A   |91    |132   |113   |102   |
|Denmark |DNK         |Thousand tonnes|N/A   |116   |146   |205   |177   |
|Finland |FIN         |Thousand tonnes|N/A   |N/A   |343   |165   |272   |
|Finland |FIN         |Thousand tonnes|N/A   |N/A   |258   |184   |282   |
+--------+------------+--

In [290]:
df_GWOP_labeled_2 = df_GWOP_labeled_tra\
.withColumn('2020M03', psf.when(df_GWOP_labeled_tra['2020Q1'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2020Q1']/3).cast('int'))\
.withColumn('2020M02', psf.when(df_GWOP_labeled_tra['2020Q1'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2020Q1']/3).cast('int'))\
.withColumn('2020M01', psf.when(df_GWOP_labeled_tra['2020Q1'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2020Q1']/3).cast('int'))\
.withColumn('2019M12', psf.when(df_GWOP_labeled_tra['2019Q4'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q4']/3).cast('int'))\
.withColumn('2019M11', psf.when(df_GWOP_labeled_tra['2019Q4'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q4']/3).cast('int'))\
.withColumn('2019M10', psf.when(df_GWOP_labeled_tra['2019Q4'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q4']/3).cast('int'))\
.withColumn('2019M09', psf.when(df_GWOP_labeled_tra['2019Q3'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q3']/3).cast('int'))\
.withColumn('2019M08', psf.when(df_GWOP_labeled_tra['2019Q3'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q3']/3).cast('int'))\
.withColumn('2019M07', psf.when(df_GWOP_labeled_tra['2019Q3'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q3']/3).cast('int'))\
.withColumn('2019M06', psf.when(df_GWOP_labeled_tra['2019Q2'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q2']/3).cast('int'))\
.withColumn('2019M05', psf.when(df_GWOP_labeled_tra['2019Q2'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q2']/3).cast('int'))\
.withColumn('2019M04', psf.when(df_GWOP_labeled_tra['2019Q2'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q2']/3).cast('int'))\
.withColumn('2019M03', psf.when(df_GWOP_labeled_tra['2019Q1'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q1']/3).cast('int'))\
.withColumn('2019M02', psf.when(df_GWOP_labeled_tra['2019Q1'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q1']/3).cast('int'))\
.withColumn('2019M01', psf.when(df_GWOP_labeled_tra['2019Q1'] =='N/A', None).otherwise(df_GWOP_labeled_tra['2019Q1']/3).cast('int'))

In [291]:
df_GWOP_labeled_3 = df_GWOP_labeled_2.drop('tra_cov', 'rep_mar_per_time', 'Label', 'direct')

In [292]:
df_GWOP_labeled_3.show(10,  False)

+-------+------------+---------------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|country|Country_code|unit           |2020Q1|2019Q4|2019Q3|2019Q2|2019Q1|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
+-------+------------+---------------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|Belgium|BEL         |Thousand tonnes|null  |null  |null  |null  |null  |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |
|Belgium|BEL         |Thousand tonnes|null  |null  |7081  |7134  |6508  |null   |null   |null   |null   |null   |null   |2360   |2360   |2360   |2378   |2378   |2378   |2169   |2169   |2169   |
|Belgium|BEL         |Thousand

In [293]:
df_GWOP_labeled_4 = df_GWOP_labeled_3.replace(None, "N/A")
    

df_GWOP_labeled_4.show(10, False)

ValueError: to_replace should be a bool, float, int, long, string, list, tuple, or dict. Got <class 'NoneType'>

In [ ]:
+-------+------------+---------------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|country|Country_code|unit           |2020Q1|2019Q4|2019Q3|2019Q2|2019Q1|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
+-------+------------+---------------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |N/A   |N/A   |N/A   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |null   |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |7081  |7134  |6508  |null   |null   |null   |null   |null   |null   |2360   |2360   |2360   |2378   |2378   |2378   |2169   |2169   |2169   |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |334   |372   |252   |null   |null   |null   |null   |null   |null   |111    |111    |111    |124    |124    |124    |84     |84     |84     |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |2588  |2716  |2127  |null   |null   |null   |null   |null   |null   |862    |862    |862    |905    |905    |905    |709    |709    |709    |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |4890  |4941  |4720  |null   |null   |null   |null   |null   |null   |1630   |1630   |1630   |1647   |1647   |1647   |1573   |1573   |1573   |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |2588  |2716  |2127  |null   |null   |null   |null   |null   |null   |862    |862    |862    |905    |905    |905    |709    |709    |709    |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |2588  |2716  |2128  |null   |null   |null   |null   |null   |null   |862    |862    |862    |905    |905    |905    |709    |709    |709    |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |2668  |2771  |2269  |null   |null   |null   |null   |null   |null   |889    |889    |889    |923    |923    |923    |756    |756    |756    |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |4159  |4047  |4129  |null   |null   |null   |null   |null   |null   |1386   |1386   |1386   |1349   |1349   |1349   |1376   |1376   |1376   |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |1857  |1822  |1536  |null   |null   |null   |null   |null   |null   |619    |619    |619    |607    |607    |607    |512    |512    |512    |
+-------+------------+---------------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+

In [ ]:
+-------+------------+---------------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|country|Country_code|unit           |2020Q1|2019Q4|2019Q3|2019Q2|2019Q1|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09           |2019M08           |2019M07           |2019M06          |2019M05          |2019M04          |2019M03           |2019M02           |2019M01           |
+-------+------------+---------------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |N/A   |N/A   |N/A   |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A               |N/A               |N/A               |N/A              |N/A              |N/A              |N/A               |N/A               |N/A               |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |7081  |7134  |6508  |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |2360.3333333333335|2360.3333333333335|2360.3333333333335|2378.0           |2378.0           |2378.0           |2169.3333333333335|2169.3333333333335|2169.3333333333335|
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |334   |372   |252   |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |111.33333333333333|111.33333333333333|111.33333333333333|124.0            |124.0            |124.0            |84.0              |84.0              |84.0              |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |2588  |2716  |2127  |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |862.6666666666666 |862.6666666666666 |862.6666666666666 |905.3333333333334|905.3333333333334|905.3333333333334|709.0             |709.0             |709.0             |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |4890  |4941  |4720  |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1630.0            |1630.0            |1630.0            |1647.0           |1647.0           |1647.0           |1573.3333333333333|1573.3333333333333|1573.3333333333333|
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |2588  |2716  |2127  |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |862.6666666666666 |862.6666666666666 |862.6666666666666 |905.3333333333334|905.3333333333334|905.3333333333334|709.0             |709.0             |709.0             |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |2588  |2716  |2128  |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |862.6666666666666 |862.6666666666666 |862.6666666666666 |905.3333333333334|905.3333333333334|905.3333333333334|709.3333333333334 |709.3333333333334 |709.3333333333334 |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |2668  |2771  |2269  |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |889.3333333333334 |889.3333333333334 |889.3333333333334 |923.6666666666666|923.6666666666666|923.6666666666666|756.3333333333334 |756.3333333333334 |756.3333333333334 |
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |4159  |4047  |4129  |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1386.3333333333333|1386.3333333333333|1386.3333333333333|1349.0           |1349.0           |1349.0           |1376.3333333333333|1376.3333333333333|1376.3333333333333|
|Belgium|BEL         |Thousand tonnes|N/A   |N/A   |1857  |1822  |1536  |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |619.0             |619.0             |619.0             |607.3333333333334|607.3333333333334|607.3333333333334|512.0             |512.0             |512.0             |
+-------+------------+---------------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+

In [295]:
df_GWOP_labeled_3.repartition(1).write.option("delimiter", ",").mode('overwrite').option("header", "true").csv("D://notebooks/NASA/Output/Gross_weight_of_ports_maritime_data.csv")



In [22]:
# +--------+------------+---------------+------+------+------+------+------+
# |country |Country_code|unit           |2020Q1|2019Q4|2019Q3|2019Q2|2019Q1|
# +--------+------------+---------------+------+------+------+------+------+
# |Belgium |BEL         |Thousand tonnes|N/A   |N/A   |2697  |2861  |2398  |
# |Belgium |BEL         |Thousand tonnes|N/A   |N/A   |1034  |1279  |1028  |
# |Belgium |BEL         |Thousand tonnes|N/A   |N/A   |19144 |20286 |18181 |
# |Belgium |BEL         |Thousand tonnes|N/A   |N/A   |17955 |20166 |18611 |
# |Bulgaria|BGR         |Thousand tonnes|N/A   |1253  |2269  |1061  |1241  |
# |Denmark |DNK         |Thousand tonnes|N/A   |9     |5     |81    |122   |
# |Denmark |DNK         |Thousand tonnes|N/A   |91    |132   |113   |102   |
# |Denmark |DNK         |Thousand tonnes|N/A   |116   |146   |205   |177   |
# |Finland |FIN         |Thousand tonnes|N/A   |N/A   |343   |165   |272   |
# |Finland |FIN         |Thousand tonnes|N/A   |N/A   |258   |184   |282   |
# +--------+------------+---------------+------+------+------+------+------+

In [103]:
#read the airport traffic data - Airport traffic data by reporting airport and airlines
#https://ec.europa.eu/eurostat/web/products-datasets/product?code=avia_tf_apal 

Airport_traffic_data_location = "D://notebooks/NASA/Datasets/Airport_traffic_data_avia_tf_apal.tsv//avia_tf_apal.tsv"


df_AT = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep","\t").option("quote ",'"')\
            .option("esc","\\")\
            .load(Airport_traffic_data_location)

# replace ':' with 'N/A'
df_AT_1 = df_AT.replace(": ", "N/A").replace(" ", "")
# replace blank spaces from column names
for col in df_AT.columns:
    df_AT_1 = df_AT_1.withColumnRenamed(col,col.replace(" ", "")).withColumnRenamed(col,col.replace("\\", "_per_"))

In [104]:


split_col = psf.split(df_AT_1['unit,tra_meas,airline,rep_airp_per_time'], ',')
df_AT_2 = df_AT_1.withColumn('unit', split_col.getItem(0))\
                    .withColumn('tra_meas', split_col.getItem(1))\
                    .withColumn('airline', split_col.getItem(2))\
                    .withColumn('rep_airp_per_time', split_col.getItem(3))\
                    .select("unit", "tra_meas", "airline", "rep_airp_per_time", "2020M04", "2020M03", "2020M02",
                           "2020M01", "2019M12", "2019M11", "2019M10", "2019M09", "2019M08", "2019M07", "2019M06",
                           "2019M05", "2019M04", "2019M03", "2019M02", "2019M01")

In [26]:
# +----+--------+-------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
# |unit|tra_meas|airline|rep_airp_per_time|2020M04|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
# +----+--------+-------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
# |MOVE|ACM     |LIC_EU |IS_BIAR          |N/A    |10     |10     |N/A    |8      |2      |10     |13     |13     |17     |12     |13     |8      |2      |6      |N/A    |
# |MOVE|ACM     |LIC_EU |IS_BIKF          |154    |842    |1061   |1111   |1292   |1255   |1174   |1144   |1399   |1349   |1129   |915    |829    |1915   |1794   |2014   |
# |MOVE|ACM     |LIC_EU |IS_BIRK          |2      |15     |8      |8      |14     |17     |10     |30     |59     |40     |23     |30     |13     |14     |7      |4      |
# |MOVE|ACM     |LIC_EU |MT_LMML          |86     |1623   |3001   |3191   |3518   |3456   |4603   |4766   |4990   |N/A    |N/A    |N/A    |4396   |3497   |2894   |3171   |
# |MOVE|ACM     |LIC_NEU|IS_BIAR          |210    |351    |401    |355    |386    |433    |474    |487    |581    |547    |508    |563    |484    |486    |439    |465    |
# |MOVE|ACM     |LIC_NEU|IS_BIKF          |249    |1288   |1713   |1672   |1924   |2033   |2579   |3351   |4128   |4238   |4104   |3211   |2351   |2089   |1734   |2023   |
# |MOVE|ACM     |LIC_NEU|IS_BIRK          |489    |833    |1025   |914    |1017   |1188   |1372   |1524   |2246   |2192   |1861   |1618   |1239   |1274   |1150   |1187   |
# |MOVE|ACM     |LIC_NEU|MT_LMML          |12     |167    |294    |288    |299    |316    |440    |437    |470    |N/A    |N/A    |N/A    |217    |223    |196    |225    |
# |MOVE|ACM     |TOTAL  |CY_LCLK          |N/A    |2465   |3675   |3981   |3837   |4345   |6642   |6994   |7477   |7299   |6833   |6205   |5222   |3996   |3261   |3523   |
# |MOVE|ACM     |TOTAL  |CY_LCPH          |N/A    |747    |993    |1069   |1038   |1307   |2095   |2185   |2303   |2306   |2291   |2115   |1745   |1127   |858    |858    |
# +----+--------+-------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+

In [162]:
df_AT_labeled = df_AT_2.join(df_airport_codes,  df_AT_2.rep_airp_per_time == df_airport_codes.code_ICAO, how = 'inner').select(df_airport_codes.Country, df_airport_codes.Country_code, *[c for c in df_AT_2.columns])

In [174]:
Traffic_and_transport_lkp.printSchema()

root
 |-- URI	Label	Notation	Status	AcceptedDate: string (nullable = true)



In [178]:
df_AT_labeled_2 = df_AT_labeled.join(Traffic_and_transport_lkp, on= (df_AT_labeled.tra_meas==Traffic_and_transport_lkp.Notation), how= 'inner').select(Traffic_and_transport_lkp.Label, *[c for c in df_AT_labeled.columns])

In [183]:
df_AT_labeled_3 = df_AT_labeled_2.drop('rep_airp_per_time', 'airline', 'tra_meas')

In [294]:
df_AT_labeled_3.repartition(1).write.option("delimiter", ",").mode('overwrite').option("header", "true").csv("D://notebooks/NASA/Output/Airport_traffic_data.csv")



In [180]:
df_AT_labeled_3 = df_AT_labeled_2.drop('rep_airp_per_time', 'airline', 'tra_meas').filter("2020M03<>'N/A'").show(10, False)

+-----------------------------------------------------------------------------------------------------------+-------+------------+----+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|Label                                                                                                      |Country|Country_code|unit|2020M04|2020M03|2020M02 |2020M01 |2019M12 |2019M11 |2019M10 |2019M09 |2019M08 |2019M07 |2019M06 |2019M05 |2019M04 |2019M03 |2019M02|2019M01|
+-----------------------------------------------------------------------------------------------------------+-------+------------+----+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|Freight and mail loaded and unloaded                                                                       |Croatia|HRV         |T   |N/A    |649    |668     |747     |823

In [192]:
from scipy.stats import pearsonr
import pandas as pd 

# create a new DataFrame where the values for the indices and columns
# align on the diagonals
c = pd.DataFrame(columns = df_AT_labeled_3.columns, index = df_AT_labeled_3.columns)

# since we know set(a.columns) == set(b.columns), we can just iterate
# through the columns in a (although a more robust way would be to iterate
# through the intersection of the two sets of columns, in the case your actual dataframes' columns don't match up

correl_signif = pearsonr(df_AT_labeled_3["Country"], df_pollution_SO2["Country"]) # correlation of those two Series
correl = correl_signif[0] # grab the actual Pearson R value from the tuple from above
c.loc["Country", "Country"] = correl   

TypeError: len() of unsized object

In [193]:
df_AT_labeled_3['Country'].corr(df_pollution_SO2['Country'])

TypeError: 'Column' object is not callable

In [194]:
df_AT_labeled_3.corr()

TypeError: corr() missing 2 required positional arguments: 'col1' and 'col2'

In [187]:
df_AT_labeled_corr = df_AT_labeled_3.corrwith(df_pollution_SO2.Country)

AttributeError: 'DataFrame' object has no attribute 'corrwith'

In [186]:
df_pollution_SO2.show(1, False)

+-------+----+--------+-------+--------+-------+-------+--------+-------+--------+--------+--------+-------+---------+--------+--------+-------+-------+--------+--------+-------+---------+--------+-------+--------+-------+-------+--------+-------+--------+--------+--------+--------+--------+-------+---------+--------+--------+-------+-------+--------+--------+-------+---------+--------+--------+-------+-------+--------+---------+---------+---------+------------+-----------+------------+-----------+-----------+------------+-----------+------------+------------+------------+-----------+-------------+------------+------------+-----------+-----------+------------+------------+-----------+-------------+------------+-----------+------------+-----------+-----------+------------+-----------+------------+------------+------------+------------+------------+-----------+-------------+------------+------------+-----------+-----------+------------+------------+-----------+-------------+------------+

In [32]:
+-----------------------------------------------------------------------------------------------------------+-------+------------+----+----------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|Label                                                                                                      |Country|Country_code|unit|tra_meas  |2020M04|2020M03|2020M02 |2020M01 |2019M12 |2019M11 |2019M10 |2019M09 |2019M08 |2019M07 |2019M06 |2019M05 |2019M04 |2019M03 |2019M02|2019M01|
+-----------------------------------------------------------------------------------------------------------+-------+------------+----+----------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|Freight and mail loaded and unloaded                                                                       |Croatia|HRV         |T   |FRM_LD_NLD|N/A    |649    |668     |747     |823     |846     |1073    |923     |883     |1042    |967     |1164    |962     |766     |709    |691    |
|Direct transit passengers                                                                                  |Croatia|HRV         |PAS |PAS_TRS   |N/A    |53     |0       |198     |0       |1113    |638     |796     |728     |317     |456     |389     |39      |1081    |664    |1025   |
|Transfer passengers                                                                                        |Croatia|HRV         |PAS |PAS_TRF   |N/A    |9373   |16583   |16691   |17529   |21846   |23995   |30470   |36116   |35911   |29276   |24669   |21446   |22321   |16987  |17923  |
|Passengers carried                                                                                         |Croatia|HRV         |PAS |PAS_CRD   |N/A    |95716  |183242  |201277  |230125  |244346  |328891  |347811  |374036  |363936  |332955  |309377  |279230  |230671  |179463 |189136 |
|Commercial aircraft movements (passenger, freight and mail, commercial general aviation operations)        |Croatia|HRV         |MOVE|CACM      |N/A    |1919   |2624    |2805    |2849    |2953    |3623    |3739    |4012    |3913    |3656    |3758    |3379    |2833    |2467   |2749   |
|Aircraft movements (passenger, freight and mail, commercial and non-commercial general aviation operations)|Croatia|HRV         |MOVE|ACM       |N/A    |2213   |2935    |3076    |3326    |3302    |4013    |4157    |4390    |4319    |4025    |4231    |3731    |3293    |2801   |3018   |
|Freight and mail loaded and unloaded                                                                       |Czechia|CZE         |T   |FRM_LD_NLD|N/A    |5247   |6056    |6819    |7259    |7169    |7666    |7411    |7578    |7040    |6980    |6569    |5909    |6387    |5588   |6263   |
|Direct transit passengers                                                                                  |Czechia|CZE         |PAS |PAS_TRS   |N/A    |478    |282     |727     |820     |509     |852     |444     |1185    |1234    |1025    |1541    |1203    |991     |948    |840    |
|Transfer passengers                                                                                        |Czechia|CZE         |PAS |PAS_TRF   |N/A    |2034   |5752    |9682    |9787    |12219   |18455   |30528   |41594   |30268   |20354   |11870   |9415    |6660    |5012   |5606   |
|Passengers carried                                                                                         |Czechia|CZE         |PAS |PAS_CRD   |N/A    |432640 |1007093 |1048999 |1312597 |1248385 |1582715 |1846065 |1994114 |1964543 |1773637 |1482696 |1387134 |1229434 |975990 |976146 |
+-----------------------------------------------------------------------------------------------------------+-------+------------+----+----------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+

+-------+----+----+--------+-------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|country|CODE|unit|tra_meas|airline|rep_airp_per_time|2020M04|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
+-------+----+----+--------+-------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
+-------+----+----+--------+-------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+



In [115]:
#read the air pass file -  Air passenger transport by main airports in each reporting country
#https://data.europa.eu/euodp/en/data/dataset/Ez3kc1cjABsZ83uw1uw 

import pyspark.sql.functions as psf

Air_passenger_transport_location = "D://notebooks/NASA/Datasets/Air_passenger_transport_avia_paoa.tsv//avia_paoa.tsv"


df_APT = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep","\t").option("quote ",'"')\
            .option("esc","\\")\
            .load(Air_passenger_transport_location)


# replace ':' with 'N/A'
df_APT_1 = df_APT.replace(": ", "N/A").replace(" ", "")
# replace blank spaces from column names
for col in df_APT.columns:
    df_APT_1 = df_APT_1.withColumnRenamed(col,col.replace(" ", "")).withColumnRenamed(col,col.replace("\\", "_per_"))
    

In [116]:


split_col = psf.split(df_APT_1['unit,tra_meas,rep_airp,schedule,tra_cov_per_time'], ',')
df_APT_2 = df_APT_1.withColumn('unit', split_col.getItem(0))\
                    .withColumn('tra_meas', split_col.getItem(1))\
                    .withColumn('rep_airp', split_col.getItem(2))\
                    .withColumn('schedule', split_col.getItem(3))\
                    .withColumn('tra_cov_per_time', split_col.getItem(4))\
                    .select("unit", "tra_meas", "rep_airp", "schedule", "tra_cov_per_time", "2020M04", "2020M03", "2020M02",
                           "2020M01", "2019M12", "2019M11", "2019M10", "2019M09", "2019M08", "2019M07", "2019M06",
                           "2019M05", "2019M04", "2019M03", "2019M02", "2019M01")


In [196]:
df_APT_2_labeled = df_APT_2.join(df_airport_codes,  df_APT_2.rep_airp == df_airport_codes.code_ICAO, how = 'inner').select(df_airport_codes.Country, df_airport_codes.Country_code, *[c for c in df_APT_2.columns])

In [199]:
df_APT_labeled_2 = df_APT_2_labeled.join(Traffic_and_transport_lkp, on= (df_APT_2_labeled.tra_meas==Traffic_and_transport_lkp.Notation), how= 'inner').select(Traffic_and_transport_lkp.Label, *[c for c in df_APT_2_labeled.columns])

In [253]:
df_APT_labeled_2.drop('rep_airp','tra_meas','tra_cov_per_time').show(10, False)

+-------------------------------+-------+------------+----+--------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|Label                          |Country|Country_code|unit|schedule|2020M04|2020M03|2020M02|2020M01|2019M12 |2019M11 |2019M10 |2019M09 |2019M08 |2019M07 |2019M06 |2019M05 |2019M04 |2019M03 |2019M02|2019M01|
+-------------------------------+-------+------------+----+--------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|Passengers carried (departures)|Austria|AUT         |PAS |TOT     |N/A    |N/A    |N/A    |N/A    |1246932 |1170369 |1440652 |1471001 |1576911 |1612275 |1509613 |1434286 |1370870 |1170534 |926683 |928619 |
|Passengers carried (departures)|Austria|AUT         |PAS |TOT     |N/A    |N/A    |N/A    |N/A    |20617   |22844   |25161   |28395   |21681   |25198   |26224   |21674   |

In [259]:
df_APT_labeled_3.select("unit").distinct().show(10, False)

+----------+
|unit      |
+----------+
|PASSENGERS|
|FLIGHT    |
+----------+



In [255]:
df_APT_labeled_3 = df_APT_labeled_2.withColumn('unit', psf.when(df_APT_labeled_2.unit =='PAS', 'PASSENGERS').otherwise(df_APT_labeled_2.unit))

In [260]:
df_APT_labeled_3.repartition(1).write.option("delimiter", ",").mode('overwrite').option("header", "true").csv("D://notebooks/NASA/Output/Air_passenger_transport.csv")



In [ ]:
+-------------------------------+-------+------------+----+--------+----------------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|Label                          |Country|Country_code|unit|schedule|tra_cov_per_time|2020M04|2020M03|2020M02|2020M01|2019M12 |2019M11 |2019M10 |2019M09 |2019M08 |2019M07 |2019M06 |2019M05 |2019M04 |2019M03 |2019M02|2019M01|
+-------------------------------+-------+------------+----+--------+----------------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|Passengers carried (departures)|Austria|AUT         |PAS |TOT     |TOTAL           |N/A    |N/A    |N/A    |N/A    |1246932 |1170369 |1440652 |1471001 |1576911 |1612275 |1509613 |1434286 |1370870 |1170534 |926683 |928619 |
|Passengers carried (departures)|Austria|AUT         |PAS |TOT     |NAT             |N/A    |N/A    |N/A    |N/A    |20617   |22844   |25161   |28395   |21681   |25198   |26224   |21674   |21496   |22890   |19929  |18792  |
|Passengers carried (departures)|Austria|AUT         |PAS |TOT     |INTL_XEU28      |N/A    |N/A    |N/A    |N/A    |N/A     |N/A     |N/A     |N/A     |N/A     |N/A     |476506  |433490  |435966  |367669  |304437 |307428 |
|Passengers carried (departures)|Austria|AUT         |PAS |TOT     |INTL_XEU27_2020 |N/A    |N/A    |N/A    |N/A    |508957  |444001  |559357  |533595  |613195  |639652  |550995  |506754  |510149  |428305  |357541 |363840 |
|Passengers carried (departures)|Austria|AUT         |PAS |TOT     |INTL_IEU28      |N/A    |N/A    |N/A    |N/A    |N/A     |N/A     |N/A     |N/A     |N/A     |N/A     |1006883 |979122  |913408  |779975  |602317 |602399 |
|Passengers carried (departures)|Austria|AUT         |PAS |TOT     |INTL_IEU27_2020 |N/A    |N/A    |N/A    |N/A    |717358  |703524  |856134  |909011  |942035  |947425  |932394  |905858  |839225  |719339  |549213 |545987 |
|Passengers carried (departures)|Austria|AUT         |PAS |TOT     |INTL            |N/A    |N/A    |N/A    |N/A    |1226315 |1147525 |1415491 |1442606 |1555230 |1587077 |1483389 |1412612 |1349374 |1147644 |906754 |909827 |
|Passengers carried (departures)|Austria|AUT         |PAS |SCHED   |TOTAL           |N/A    |N/A    |N/A    |N/A    |1238026 |1163440 |1425900 |1451584 |1545019 |1582203 |1484557 |1416903 |1359565 |1162337 |919259 |920170 |
|Passengers carried (departures)|Austria|AUT         |PAS |SCHED   |NAT             |N/A    |N/A    |N/A    |N/A    |20177   |22844   |25161   |28386   |21681   |25198   |26137   |21674   |21453   |22890   |19929  |18722  |
|Passengers carried (departures)|Austria|AUT         |PAS |SCHED   |INTL_XEU28      |N/A    |N/A    |N/A    |N/A    |N/A     |N/A     |N/A     |N/A     |N/A     |N/A     |465546  |424754  |430128  |360431  |299617 |301335 |
+-------------------------------+-------+------------+----+--------+----------------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+

In [ ]:
# +------+--------+--------+--------+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
# |unit  |tra_meas|rep_airp|schedule|tra_cov_per_time|2020M04|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
# +------+--------+--------+--------+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
# |FLIGHT|CAF_PAS |CY_LCLK |N_SCHED |INTL            |N/A    |176    |201    |143    |188    |264    |762    |790    |798    |811    |771    |701    |368    |221    |151    |153    |
# |FLIGHT|CAF_PAS |CY_LCLK |N_SCHED |INTL_IEU27_2020 |N/A    |71     |80     |48     |77     |111    |338    |339    |393    |389    |354    |309    |173    |78     |60     |56     |
# |FLIGHT|CAF_PAS |CY_LCLK |N_SCHED |INTL_XEU27_2020 |N/A    |105    |121    |95     |111    |153    |424    |451    |405    |422    |417    |392    |195    |143    |91     |97     |
# |FLIGHT|CAF_PAS |CY_LCLK |N_SCHED |NAT             |N/A    |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |
# |FLIGHT|CAF_PAS |CY_LCLK |N_SCHED |TOTAL           |N/A    |176    |201    |143    |188    |264    |762    |790    |798    |811    |771    |701    |368    |221    |151    |153    |
# |FLIGHT|CAF_PAS |CY_LCLK |SCHED   |INTL            |N/A    |1512   |2601   |2712   |2813   |3110   |4851   |5345   |5780   |5628   |5279   |4607   |3966   |2910   |2339   |2586   |
# |FLIGHT|CAF_PAS |CY_LCLK |SCHED   |INTL_IEU27_2020 |N/A    |761    |1339   |1337   |1419   |1410   |1905   |2063   |2118   |2095   |1946   |1783   |1640   |1444   |1174   |1293   |
# |FLIGHT|CAF_PAS |CY_LCLK |SCHED   |INTL_XEU27_2020 |N/A    |751    |1262   |1375   |1394   |1700   |2946   |3282   |3662   |3533   |3333   |2824   |2326   |1466   |1165   |1293   |
# |FLIGHT|CAF_PAS |CY_LCLK |SCHED   |NAT             |N/A    |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |
# |FLIGHT|CAF_PAS |CY_LCLK |SCHED   |TOTAL           |N/A    |1512   |2601   |2712   |2813   |3110   |4851   |5345   |5780   |5628   |5279   |4607   |3966   |2910   |2339   |2586   |
# +------+--------+--------+--------+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+

In [213]:
#read the file - International intra-EU freight and mail air transport by main airports in each reporting country and EU partner country
# https://data.europa.eu/euodp/en/data/dataset/LzQnng3eGm3hbz8HfjcIQ

International_air_transport_location = "D://notebooks/NASA/Datasets/International_air_transport_avia_goinac.tsv//avia_goinac.tsv"


df_IAT = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep","\t").option("quote ",'"')\
            .option("esc","\\")\
            .load(International_air_transport_location)

# replace ':' with 'N/A'
df_IAT_1 = df_IAT.replace(": ", "N/A").replace(" ", "")
# replace blank spaces from column names
for col in df_IAT.columns:
    df_IAT_1 = df_IAT_1.withColumnRenamed(col,col.replace(" ", "")).withColumnRenamed(col,col.replace("\\", "_per_"))

In [214]:


split_col = psf.split(df_IAT_1['unit,tra_meas,rep_airp,partner_per_time'], ',')
df_IAT_2 = df_IAT_1.withColumn('unit', split_col.getItem(0))\
                    .withColumn('tra_meas', split_col.getItem(1))\
                    .withColumn('rep_airp', split_col.getItem(2))\
                    .withColumn('partner_per_time', split_col.getItem(3))\
                    .select("unit", "tra_meas", "rep_airp", "partner_per_time", "2020M04", "2020M03", "2020M02",
                           "2020M01", "2019M12", "2019M11", "2019M10", "2019M09", "2019M08", "2019M07", "2019M06",
                           "2019M05", "2019M04", "2019M03", "2019M02", "2019M01")

In [215]:
df_IAT_2_labeled = df_IAT_2.join(df_airport_codes,  df_IAT_2.rep_airp == df_airport_codes.code_ICAO, how = 'inner').select(df_airport_codes.Country, df_airport_codes.Country_code, *[c for c in df_IAT_2.columns])

In [216]:
df_IAT_labeled_2 = df_IAT_2_labeled.join(Traffic_and_transport_lkp, on= (df_IAT_2_labeled.tra_meas==Traffic_and_transport_lkp.Notation), how= 'inner').select(Traffic_and_transport_lkp.Label, *[c for c in df_IAT_2_labeled.columns])

In [226]:
df_maritime_codes.show(10, False)

+------------+---------+-------------------------------------------------------------------------+-------+
|Country_code|notation |prefLabel                                                                |country|
+------------+---------+-------------------------------------------------------------------------+-------+
|BEL         |BE_0BE888|Belgium - other ports                                                    |Belgium|
|BEL         |BE_0BE002|Belgium - other inland ports (other than Zeekanaal Brussel-Schelde ports)|Belgium|
|BEL         |BE_0BE001|Zeekanaal Brussel-Schelde - ports                                        |Belgium|
|BEL         |BE_0BEZEE|Zeebrugge                                                                |Belgium|
|BEL         |BE_0BEOST|Oostende (Ostend)                                                        |Belgium|
|BEL         |BE_0BENIE|Nieuwpoort                                                               |Belgium|
|BEL         |BE_0BELGG|Li�ge        

In [233]:
df_maritime_codes_mer = df_maritime_codes.withColumnRenamed('country', 'country_mer').withColumnRenamed('Country_code', 'Country_code_mer')

In [240]:


df_IAT_labeled_3 = df_IAT_labeled_2.join(df_maritime_codes_mer, on= ((df_IAT_2_labeled.Country==df_maritime_codes_mer.country_mer)) , how= 'inner').select(df_maritime_codes_mer.Country_code_mer.alias('partener_country_code'), *[c for c in df_IAT_2_labeled.columns])

In [241]:
df_IAT_labeled_3.show(10, False)

+---------------------+-------+------------+----+--------+--------+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|partener_country_code|Country|Country_code|unit|tra_meas|rep_airp|partner_per_time|2020M04|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
+---------------------+-------+------------+----+--------+--------+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |

In [247]:
df_IAT_labeled_4 = df_IAT_labeled_3.drop('tra_meas', 'rep_airp','partner_per_time')

In [ ]:
+---------------------+-------+------------+----+--------+--------+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|partener_country_code|Country|Country_code|unit|tra_meas|rep_airp|partner_per_time|2020M04|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
+---------------------+-------+------------+----+--------+--------+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |T   |FRM_NLD |SE_ESSA |UK              |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
+---------------------+-------+------------+----+--------+--------+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+

In [248]:
df_IAT_labeled_4 = df_IAT_labeled_4.withColumn('unit', psf.when(df_IAT_labeled_4.unit =='T', 'TONNES').otherwise(df_IAT_labeled_4.unit))

# +------+
# |unit  |
# +------+
# |FLIGHT|
# |TONNES|
# +------+




In [251]:
df_IAT_labeled_4.show(10, False)

+---------------------+-------+------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|partener_country_code|Country|Country_code|unit  |2020M04|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
+---------------------+-------+------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N

In [ ]:
+---------------------+-------+------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|partener_country_code|Country|Country_code|unit  |2020M04|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
+---------------------+-------+------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
|SWE                  |Sweden |SWE         |TONNES|N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |1      |1      |1      |3      |1      |8      |
+---------------------+-------+------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+

In [252]:
df_IAT_labeled_4.repartition(1).write.option("delimiter", ",").mode('overwrite').option("header", "true").csv("D://notebooks/NASA/Output/International_air_transport.csv")



In [210]:
df_IAT_labeled_2.drop('tra_meas', 'rep_airp').filter("2020M03 <> 'N/A'").show(10, False)

+-------------------------+-------+------------+----+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|Label                    |Country|Country_code|unit|partner_per_time|2020M04|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
+-------------------------+-------+------------+----+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|Freight and mail unloaded|Croatia|HRV         |T   |UK              |N/A    |12     |12     |9      |13     |13     |15     |11     |11     |10     |10     |12     |13     |16     |9      |13     |
|Freight and mail unloaded|Croatia|HRV         |T   |SI              |N/A    |0      |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |
|Frei

In [ ]:
# +-------------------------+-------+------------+----+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
# |Label                    |Country|Country_code|unit|partner_per_time|2020M04|2020M03|2020M02|2020M01|2019M12|2019M11|2019M10|2019M09|2019M08|2019M07|2019M06|2019M05|2019M04|2019M03|2019M02|2019M01|
# +-------------------------+-------+------------+----+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
# |Freight and mail unloaded|Croatia|HRV         |T   |UK              |N/A    |12     |12     |9      |13     |13     |15     |11     |11     |10     |10     |12     |13     |16     |9      |13     |
# |Freight and mail unloaded|Croatia|HRV         |T   |SI              |N/A    |0      |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |N/A    |
# |Freight and mail unloaded|Croatia|HRV         |T   |PL              |N/A    |6      |14     |11     |9      |14     |13     |13     |13     |9      |8      |10     |8      |11     |8      |7      |
# |Freight and mail unloaded|Croatia|HRV         |T   |NL              |N/A    |19     |17     |21     |23     |22     |18     |14     |16     |14     |18     |18     |20     |21     |17     |22     |
# |Freight and mail unloaded|Croatia|HRV         |T   |IT              |N/A    |52     |141    |165    |183    |171    |178    |172    |163    |204    |192    |196    |212    |183    |160    |170    |
# |Freight and mail unloaded|Croatia|HRV         |T   |FR              |N/A    |5      |9      |7      |10     |8      |9      |8      |9      |9      |8      |8      |8      |8      |8      |8      |
# |Freight and mail unloaded|Croatia|HRV         |T   |EU27_2020       |N/A    |291    |283    |339    |358    |336    |339    |309    |281    |344    |309    |348    |351    |281    |253    |271    |
# |Freight and mail unloaded|Croatia|HRV         |T   |EU27_2007       |N/A    |303    |295    |348    |371    |349    |354    |320    |292    |354    |319    |360    |364    |297    |262    |284    |
# |Freight and mail unloaded|Croatia|HRV         |T   |ES              |N/A    |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |0      |
# |Freight and mail unloaded|Croatia|HRV         |T   |EL              |N/A    |0      |0      |0      |0      |0      |0      |0      |0      |1      |0      |0      |0      |0      |0      |0      |
# +-------------------------+-------+------------+----+----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+

In [261]:
#read the volume_of_containers_transported_martitime file - Volume of containers transported to/from main ports - quarterly data (2019 - 2020)
#https://ec.europa.eu/eurostat/web/transport/data/database?p_p_id=NavTreeportletprod_WAR_NavTreeportletprod_INSTANCE_yjUOJMEUlFPI&p_p_lifecycle=0&p_p_state=pop_up&p_p_mode=view&p_p_col_id=column-2&p_p_col_count=1&_NavTreeportletprod_WAR_NavTreeportletprod_INSTANCE_yjUOJMEUlFPI_nodeInfoService=true&nodeId=272692
volume_of_containers_transported_martitime_location = "D://notebooks/NASA/Datasets/volume_of_containers_transported_martitime_mar_go_qm_c2020.tsv//mar_go_qm_c2020.tsv"


df_containers_volume = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep","\t").option("quote ",'"')\
            .option("esc","\\")\
            .load(volume_of_containers_transported_martitime_location)

# replace ':' with 'N/A'
df_containers_volume_1 = df_containers_volume.replace(": ", "N/A")
# replace blank spaces from column names
for col in df_containers_volume.columns:
    df_containers_volume_1 = df_containers_volume_1.withColumnRenamed(col,col.replace(" ", "")).withColumnRenamed(col,col.replace("\\", "_per_"))

In [262]:
df_containers_volume_1.show(10, False)

+---------------------------------------------------+------+------+------+------+------+
|par_mar,direct,cargo,loadstat,unit,rep_mar_per_time|2020Q1|2019Q4|2019Q3|2019Q2|2019Q1|
+---------------------------------------------------+------+------+------+------+------+
|AE,IN,LCNT_20,EMPTY,TEU,BE                         |N/A   |N/A   |82    |68    |228   |
|AE,IN,LCNT_20,EMPTY,TEU,BE_0BEANR                  |N/A   |N/A   |82    |68    |228   |
|AE,IN,LCNT_20,EMPTY,TEU,CY_0CYLMS                  |N/A   |0     |0     |0     |0     |
|AE,IN,LCNT_20,EMPTY,TEU,DE                         |N/A   |N/A   |224   |112   |259   |
|AE,IN,LCNT_20,EMPTY,TEU,DE_1                       |N/A   |N/A   |224   |112   |259   |
|AE,IN,LCNT_20,EMPTY,TEU,DE_1DEBRV                  |N/A   |N/A   |172   |66    |215   |
|AE,IN,LCNT_20,EMPTY,TEU,DE_1DEHAM                  |N/A   |N/A   |52    |46    |44    |
|AE,IN,LCNT_20,EMPTY,TEU,EL                         |N/A   |N/A   |10    |12    |1     |
|AE,IN,LCNT_20,EMPTY,

In [ ]:


split_col = psf.split(df_containers_volume_1['par_mar,direct,cargo,loadstat,unit,rep_mar_per_time'], ',')
df_containers_volume_2 = df_containers_volume_1\
                    .withColumn('par_mar', split_col.getItem(0))\
                    .withColumn('direct', split_col.getItem(1))\
                    .withColumn('cargo', split_col.getItem(2))\
                    .withColumn('loadstat', split_col.getItem(3))\
                    .withColumn('unit', split_col.getItem(4))\
                    .withColumn('rep_mar_per_time', split_col.getItem(5))\
                    .drop('par_mar,direct,cargo,loadstat,unit,rep_mar_per_time')

In [ ]:
df_containers_volume_2.filter("2020Q1<> 'N/A'").show(10, False)

In [ ]:
#read the Vessels_in_main_ports_maritime file - Vessels in main ports by type and size of vessels (based on inwards declarations) - quarterly data
#https://ec.europa.eu/eurostat/web/transport/data/database?p_p_id=NavTreeportletprod_WAR_NavTreeportletprod_INSTANCE_yjUOJMEUlFPI&p_p_lifecycle=0&p_p_state=pop_up&p_p_mode=view&p_p_col_id=column-2&p_p_col_count=1&_NavTreeportletprod_WAR_NavTreeportletprod_INSTANCE_yjUOJMEUlFPI_nodeInfoService=true&nodeId=268970

Vessels_in_main_ports_maritime_location = "D://notebooks/NASA/Datasets/Vessels_in_main_ports_maritime_mar_tf_qm.tsv//mar_tf_qm.tsv"


df_vessels_in_ports = spark.read.format("com.databricks.spark.csv")\
            .option("header","true")\
            .option("inferSchema","false")\
            .option("sep","\t").option("quote ",'"')\
            .option("esc","\\")\
            .load(Vessels_in_main_ports_maritime_location)

# replace ':' with 'N/A'
df_vessels_in_ports_1 = df_vessels_in_ports.replace(": ", "N/A")
# replace blank spaces from column names
for col in df_vessels_in_ports.columns:
    df_vessels_in_ports_1 = df_vessels_in_ports_1.withColumnRenamed(col,col.replace(" ", "")).withColumnRenamed(col,col.replace("\\", "_per_"))

In [ ]:
df_vessels_in_ports_1.show(1, False)

In [ ]:


split_col = psf.split(df_vessels_in_ports_1['tonnage,vessel,unit,rep_mar_per_time'], ',')
df_vessels_in_ports_2 = df_vessels_in_ports_1\
                    .withColumn('tonnage', split_col.getItem(0))\
                    .withColumn('vessel', split_col.getItem(1))\
                    .withColumn('unit', split_col.getItem(2))\
                    .withColumn('rep_mar_per_time', split_col.getItem(3))\
                    .select("tonnage", "vessel", "unit", "rep_mar_per_time", "2020Q1", "2019Q4", "2019Q3", "2019Q2", "2019Q1")

In [ ]:
df_vessels_in_ports_2.filter("2020Q1<>'N/A'").show(100, False) 

In [62]:
info=[['Country','Date','COCL','COEM','COLS','COPD','COSC','LWI','TO3','Var_COCL','Var_COEM','Var_COLS','Var_COPD','Var_COSC','Var_LWI','Var_TO3'],
['Austria','201901','0.00052217644','5.2215263e-11','2.130699e-11','5.2717713e-12','107.294075','0.9999939','332.40884','5.4041827e-10','6.9557666e-28','4.0904974e-24','2.1660355e-25','527.91626','-3.722289e-11','1222.4336'],
['Bulgaria','201901','0.0006045318','8.269609e-11','3.923956e-11','1.1001666e-11','113.75886','0.9999939','341.49298','7.5443485e-10','6.65716e-23','7.133116e-24','1.269314e-24','334.50278','-3.722289e-11','978.8043'],
['Cyprus','201901','0.00059760467','1.4871522e-10','5.958477e-11','1.9645681e-11','94.4257','0.9167252','305.65863','4.373992e-10','2.4329632e-26','9.375241e-24','1.207587e-24','47.345406','8.456168e-09','785.3675'],
['Czech Republic','201901','0.0006154364','1.7012135e-10','2.177097e-11','5.48942e-12','131.08618','0.9999939','340.81232','8.0829826e-10','5.0434563e-28','6.953719e-24','2.5713298e-25','596.02234','-3.722289e-11','1151.0533'],
['Estonia','201901','0.0006380527','1.6174933e-11','9.3608645e-12','2.2614347e-12','106.662506','0.9999939','374.64996','1.5200797e-09','2.6961476e-28','3.8502727e-24','1.2840298e-25','271.30743','-3.722289e-11','2523.6577'],
['France','201901','0.00056703395','2.855465e-11','3.511119e-11','1.0216282e-11','95.81483','0.9999939','315.19217','6.465581e-10','9.4196525e-26','5.8265763e-24','3.9676087e-25','180.34047','-3.722289e-11','1611.2158'],
['Germany','201901','0.0006090712','1.00464324e-10','2.220764e-11','5.749967e-12','121.656456','0.9999939','336.80597','8.810675e-10','8.708368e-28','8.24095e-24','3.005337e-25','665.05975','-3.722289e-11','1714.8175'],
['Greece','201901','0.00055132376','4.138155e-11','4.378858e-11','1.28057035e-11','96.02888','0.9999939','334.66504','7.3790435e-10','4.779986e-25','9.594584e-24','1.2982887e-24','148.54712','-3.722289e-11','1234.2467'],
['Hungary','201901','0.0006545789','2.1642674e-10','2.853232e-11','7.301519e-12','141.49945','0.9999939','342.7284','1.1828429e-09','2.6629448e-27','8.114126e-24','3.581297e-25','825.91736','-3.722289e-11','907.59033'],
['Latvia','201901','0.00064038223','1.1901728e-10','1.067056e-11','2.5878201e-12','114.9396','0.81152093','367.70514','1.2600979e-09','5.6486707e-28','3.8589206e-24','1.3833682e-25','338.05054','0.03649116','1983.8116'],
['Lithuania','201901','0.0006396529','1.1830766e-10','1.3692339e-11','3.3349788e-12','120.63592','0.9999939','359.45782','1.1394082e-09','2.1249762e-27','4.5135813e-24','1.6299854e-25','523.3459','-3.722289e-11','1404.956'],
['Luxembourg','201901','0.0006026182','2.3358573e-10','2.6141353e-11','7.0457308e-12','136.39137','0.9999939','328.4721','7.652708e-10','2.1619254e-23','8.233839e-24','2.9506975e-25','1257.4581','-3.722289e-11','1616.4618'],
['Malta','201901','0.00062568794','1.7656987e-11','6.091111e-11','1.999252e-11','88.45673','0.0','325.09708','8.471649e-10','0.0','1.1520946e-23','9.904985e-25','54.01576','0.0','1542.0587'],
['Netherlands','201901','0.00064984546','6.1583794e-10','2.2993706e-11','6.2776117e-12','167.84668','0.9999939','330.26318','1.5160646e-09','2.2917465e-26','9.982196e-24','3.2764512e-25','6147.388','-3.722289e-11','1545.0626'],
['Poland','201901','0.0006379761','1.8155788e-10','1.8556271e-11','4.6525605e-12','137.38911','0.9999939','346.52863','1.1596507e-09','0.0','6.075912e-24','1.738816e-25','1572.8195','-3.722289e-11','1006.4285'],
['Portugal','201901','0.00058970484','3.99797e-11','5.466886e-11','2.0610902e-11','90.01556','0.9999939','290.33868','6.747239e-10','5.6970747e-22','1.5913409e-23','1.593783e-24','100.20128','-3.722289e-11','182.53363'],
['Romania','201901','0.0005947813','5.1722602e-11','2.632596e-11','6.7436547e-12','119.4007','0.9999939','341.00604','7.5537954e-10','2.4190986e-27','3.4791795e-24','5.0988133e-25','242.13509','-3.722289e-11','835.90533'],
['Slovakia','201901','0.0006081663','3.899871e-11','2.3666335e-11','6.01505e-12','122.1179','0.9999939','342.53674','7.0624434e-10','9.666625e-28','4.5809287e-24','2.2948102e-25','744.3249','-3.722289e-11','848.39136'],
['Slovenia','201901','0.00060788536','1.679723e-10','3.1083684e-11','8.005234e-12','129.05624','0.9999939','334.42578','9.0202934e-10','0.0','1.04364404e-23','4.221359e-25','554.7524','-3.722289e-11','1212.7684'],
['Spain','201901','0.0005533143','1.2568336e-09','4.879376e-11','1.6539032e-11','143.47897','0.9999939','292.753','5.5560484e-10','2.1906083e-25','1.3637193e-23','1.13514e-24','2403.967','-3.722289e-11','266.00797'],
['Sweden','201901','0.0006342875','2.0584423e-11','7.955578e-12','1.981192e-12','111.03694','0.78684926','370.3193','1.4768224e-09','0.0','4.8126073e-24','1.3107466e-25','787.93866','0.0005483832','2592.0583'],
['Ireland','201901','0.0006082353','1.6966385e-11','1.8203869e-11','5.306613e-12','84.82069','0.9999939','315.5038','6.232886e-10','7.821728e-26','9.1818416e-24','4.6841427e-25','45.94617','-3.722289e-11','1471.9684'],
['Italy','201901','0.0006212496','5.8481825e-10','5.0410055e-11','1.4901991e-11','125.1788','0.9999939','329.16254','6.8403677e-10','7.1391936e-24','1.961009e-23','1.6605536e-24','363.713','-3.722289e-11','1540.1017'],
['Denmark','201901','0.00062975194','1.0511265e-10','1.38156145e-11','3.669879e-12','105.7877','0.9999939','345.5067','1.2060741e-09','1.7652096e-28','7.74494e-24','2.196666e-25','771.1376','-3.722289e-11','2048.1106'],
['Finland','201901','0.00062612153','4.4745856e-11','5.7766947e-12','1.366847e-12','112.40272','0.9999939','392.01538','1.0259168e-09','1.9689569e-25','2.4882365e-24','7.6350784e-26','667.5431','-3.722289e-11','3160.4734'],
['Belgium','201901','0.000635819','7.168065e-10','2.8695114e-11','7.921012e-12','168.42384','0.9999939','326.73376','1.0660616e-09','3.6151495e-26','9.7577834e-24','3.4364874e-25','2714.0752','-3.722289e-11','1596.8489'],
['Croatia','201901','0.0006042942','3.5723396e-11','3.500352e-11','9.275619e-12','113.55788','0.99441576','333.59778','7.0290473e-10','5.706756e-22','1.0158474e-23','5.982548e-25','290.23398','1.255986e-08','1294.662'],
['Austria','201902','0.00053157704','5.2480686e-11','3.4027225e-11','9.8518e-12','98.546036','0.99999505','322.5209','6.844735e-10','3.8052847e-26','5.5219695e-23','6.966558e-24','346.5214','-2.4396596e-11','1774.4662'],
['Bulgaria','201902','0.0006123481','1.463353e-10','5.555243e-11','1.6488672e-11','114.00639','0.99999505','343.45532','1.6937206e-09','2.5685396e-20','1.1101295e-22','5.3495084e-24','410.74234','-2.4396596e-11','3134.02'],
['Cyprus','201902','0.00060390163','1.4978647e-10','7.042722e-11','2.3977532e-11','96.15004','0.9167249','321.35672','1.1788851e-09','2.9471276e-25','2.66036e-23','2.5799516e-24','55.662647','8.223374e-09','1417.0311'],
['Czech Republic','201902','0.0006285433','1.7547166e-10','3.6514992e-11','1.07846145e-11','144.8766','0.99999505','327.9902','1.0096988e-09','2.595169e-22','6.965942e-23','9.106173e-24','917.1898','-2.4396596e-11','1814.3982'],
['Estonia','201902','0.0006529261','1.6158503e-11','1.875126e-11','5.0530196e-12','114.49732','0.99999505','359.8883','1.2711254e-09','3.0520867e-27','1.5773408e-23','1.9669884e-24','598.4262','-2.4396596e-11','1935.0709'],
['France','201902','0.0005794715','3.269552e-11','4.9009113e-11','1.6271057e-11','94.93699','0.99999505','310.0737','8.7871255e-10','6.819657e-23','6.535247e-23','1.5846654e-23','135.31342','-2.4396596e-11','1520.1089'],
['Germany','201902','0.0006222572','1.010544e-10','3.838487e-11','1.1746731e-11','134.89497','0.99999505','326.11945','1.0618316e-09','3.5009092e-25','6.995102e-23','1.19905674e-23','1048.5068','-2.4396596e-11','2074.0547'],
['Greece','201902','0.00056480506','4.5792977e-11','6.1778305e-11','1.8786361e-11','99.08035','0.99999505','344.7729','1.2487145e-09','5.0510673e-24','9.091241e-23','5.1332635e-24','93.04102','-2.4396596e-11','2829.7107'],
['Hungary','201902','0.00065903645','2.9838998e-10','4.222897e-11','1.2396013e-11','142.5747','0.99999505','334.97394','1.208338e-09','5.6536845e-20','7.7972015e-23','7.746075e-24','1019.16656','-2.4396596e-11','1902.2565'],
['Latvia','201902','0.00065287907','1.1898951e-10','2.0465395e-11','5.5947634e-12','121.285034','0.7527889','355.113','9.99012e-10','4.5829922e-27','2.0174868e-23','2.352707e-24','546.21466','0.0116321435','2059.6208'],
['Lithuania','201902','0.0006516705','1.1883726e-10','2.4342198e-11','6.755393e-12','125.80485','0.99999505','351.59726','8.452619e-10','1.0171815e-23','3.0875748e-23','3.2513518e-24','676.2417','-2.4396596e-11','2181.7532'],
['Luxembourg','201902','0.00061424763','2.3619068e-10','4.2074313e-11','1.3332455e-11','162.12823','0.99999505','323.35043','1.1109904e-09','9.417446e-23','6.4158423e-23','1.3641868e-23','3215.6719','-2.4396596e-11','2380.7493'],
['Malta','201902','0.0006385028','1.7662063e-11','7.488723e-11','2.5040131e-11','95.539825','0.0','330.6687','1.1448095e-09','1.0303489e-29','8.0423965e-23','4.6117763e-24','72.78974','0.0','1752.1223'],
['Netherlands','201902','0.0006648926','6.170434e-10','3.9881515e-11','1.2615024e-11','193.37251','0.99999505','328.5421','1.4113388e-09','3.5648423e-24','7.438957e-23','1.3849595e-23','6835.054','-2.4396596e-11','2478.1633'],
['Poland','201902','0.0006498174','1.8182382e-10','3.0226762e-11','8.7324835e-12','144.85526','0.99999505','341.76712','8.693705e-10','1.5672431e-25','4.7805088e-23','5.3524212e-24','1290.0457','-2.4396596e-11','1867.3275'],
['Portugal','201902','0.00059582095','4.5372574e-11','7.205216e-11','2.7944529e-11','92.68801','0.99999505','288.39655','6.7261874e-10','1.361837e-23','8.087605e-23','1.9198621e-23','47.399536','-2.4396596e-11','1128.7264'],
['Romania','201902','0.00059194665','5.2299224e-11','3.835108e-11','1.097672e-11','112.52769','0.99999505','340.61276','1.1893277e-09','7.5127065e-25','6.705762e-23','4.09903e-24','263.3138','-2.4396596e-11','2788.4248'],
['Slovakia','201902','0.0006162055','4.3711024e-11','3.517456e-11','1.0319711e-11','117.69092','0.99999505','335.46622','8.880737e-10','1.04013046e-22','5.020151e-23','5.8930852e-24','414.66626','-2.4396596e-11','1875.0012'],
['Slovenia','201902','0.0006089347','1.6810536e-10','4.7486036e-11','1.4362266e-11','122.85963','0.99999505','327.59048','1.0814796e-09','3.7233014e-25','1.0940632e-22','1.3655073e-23','581.6753','-2.4396596e-11','1600.5895'],
['Spain','201902','0.0005645347','1.2639635e-09','6.656106e-11','2.3422642e-11','158.34149','0.99999505','296.8319','6.1102695e-10','1.3900076e-22','9.2367484e-23','1.6718864e-23','2003.7358','-2.4396596e-11','1300.2682'],
['Sweden','201902','0.00065885513','2.0596858e-11','1.8057295e-11','4.9313205e-12','121.26602','0.78420997','358.63007','1.3212035e-09','2.6190182e-28','1.7387783e-23','2.4336826e-24','757.1706','0.0026682597','2116.275'],
['Ireland','201902','0.0006256064','1.8135589e-11','3.5207053e-11','1.1022462e-11','92.30462','0.99999505','334.25772','9.201277e-10','1.6420285e-25','4.9454122e-23','1.0466203e-23','97.43933','-2.4396596e-11','2848.9307'],
['Italy','201902','0.00062814174','5.896767e-10','7.125678e-11','2.356361e-11','131.55988','0.99999505','327.06738','9.930975e-10','1.2985277e-22','1.1713086e-22','1.4250084e-23','851.1905','-2.4396596e-11','1121.0286'],
['Denmark','201902','0.0006524434','1.0513089e-10','2.8456432e-11','8.4963946e-12','129.2936','0.99999505','339.39734','5.8953453e-10','1.0087116e-27','5.1286148e-23','8.463884e-24','2099.524','-2.4396596e-11','2723.287'],
['Finland','201902','0.0006418192','4.2704125e-11','1.3295919e-11','3.4247544e-12','110.50793','0.99999505','372.95215','1.2430553e-09','1.7361045e-25','8.589481e-24','1.0242353e-24','394.06863','-2.4396596e-11','1764.6497'],
['Belgium','201902','0.00063752837','7.1796485e-10','4.5653214e-11','1.4789142e-11','150.61545','0.99999505','325.38782','1.3248145e-09','3.5160453e-25','7.4602645e-23','1.632164e-23','2430.8628','-2.4396596e-11','2480.8435'],
['Croatia','201902','0.0006049585','8.091756e-11','5.120805e-11','1.57639e-11','113.50781','0.99441797','328.132','9.335622e-10','2.4067332e-20','1.0680322e-22','1.1805743e-23','380.03506','1.3756789e-08','1517.9233'],
['Austria','201903','0.00054045627','5.4617356e-11','7.338434e-11','2.158666e-11','98.010765','1.000009','346.1357','9.784841e-10','3.1009834e-24','2.8939876e-22','2.714276e-23','258.75626','-7.945089e-11','1341.2334'],
['Bulgaria','201903','0.0006173732','1.0457073e-10','9.6896983e-11','3.147686e-11','111.390465','1.000009','338.47916','9.970241e-10','9.601734e-22','2.9273982e-22','3.059966e-23','244.82016','-7.945089e-11','804.5141'],
['Cyprus','201903','0.0005891028','1.5226702e-10','9.17928e-11','3.39901e-11','91.2594','0.91665065','332.38812','4.1056705e-10','5.6414086e-25','1.0063855e-22','1.3622033e-23','35.6142','3.8425516e-09','968.19403'],
['Czech Republic','201903','0.0006282996','1.7548274e-10','7.97292e-11','2.3886258e-11','115.45246','1.000009','359.2961','1.1640343e-09','2.785588e-23','3.5312148e-22','3.8520504e-23','557.675','-7.945089e-11','2011.4467'],
['Estonia','201903','0.00063931424','1.7629941e-11','4.3298604e-11','1.1854129e-11','96.58494','1.000009','393.93274','6.4986105e-10','2.4806834e-24','1.204613e-22','1.3925588e-23','65.55976','-7.945089e-11','2888.733'],
['France','201903','0.0005856074','6.32346e-11','8.94673e-11','2.95098e-11','92.12492','1.000009','336.05197','7.470765e-10','1.26962805e-20','2.2580703e-22','2.6228357e-23','131.23416','-7.945089e-11','910.375'],
['Germany','201903','0.0006202105','1.07045324e-10','8.0310314e-11','2.4098562e-11','110.27947','1.000009','360.9982','1.1671558e-09','2.1295751e-23','3.0211146e-22','3.4637497e-23','353.3171','-7.945089e-11','2240.6826'],
['Greece','201903','0.0005544642','6.2648706e-11','9.1309516e-11','3.1538862e-11','94.77389','1.000009','332.56186','7.32135e-10','3.2323226e-22','1.9884328e-22','1.4303099e-23','140.89236','-7.945089e-11','1021.17065'],
['Hungary','201903','0.0006586137','2.4103328e-10','8.765887e-11','2.7691845e-11','121.109764','1.000009','351.02884','9.066887e-10','3.9868437e-21','4.0498662e-22','4.7634274e-23','664.6762','-7.945089e-11','1341.4634'],
['Latvia','201903','0.0006420327','1.2053006e-10','4.7747757e-11','1.3326979e-11','101.161934','0.73517245','391.64468','6.7312106e-10','2.580286e-24','1.6288832e-22','1.8391704e-23','99.18997','0.0032007028','2874.532'],
['Lithuania','201903','0.0006431533','1.2189613e-10','5.6741938e-11','1.6210458e-11','103.00014','1.000009','384.62762','7.343777e-10','1.466177e-23','2.3214212e-22','2.7164974e-23','148.48157','-7.945089e-11','2827.3245'],
['Luxembourg','201903','0.0006166536','2.420674e-10','8.295012e-11','2.5229003e-11','119.31753','1.000009','352.44455','1.4638667e-09','1.5233129e-22','3.1564773e-22','3.0961705e-23','1075.5076','-7.945089e-11','1945.6907'],
['Malta','201903','0.0006238533','1.7665468e-11','1.00501725e-10','3.7587287e-11','93.36264','0.0','318.58136','1.3582898e-09','7.8804005e-30','2.1990688e-22','1.3519404e-23','137.5643','0.0','827.0002'],
['Netherlands','201903','0.0006569207','6.1993066e-10','8.0063234e-11','2.4692505e-11','138.51173','1.000009','361.21222','1.9491573e-09','2.3847721e-23','3.1495381e-22','3.4829414e-23','2997.5632','-7.945089e-11','2510.9248'],
['Poland','201903','0.00064115593','1.871432e-10','6.7116916e-11','2.0067052e-11','108.961784','1.000009','369.78812','8.136917e-10','2.4259024e-23','3.0313067e-22','3.5987128e-23','229.3197','-7.945089e-11','2310.5498'],
['Portugal','201903','0.0006086054','7.087859e-11','1.130786e-10','4.401304e-11','94.73086','1.000009','315.61087','1.0411993e-09','1.6126592e-21','2.4017273e-22','2.538049e-23','321.1446','-7.945089e-11','1463.3218'],
['Romania','201903','0.0006030982','1.178394e-10','7.97207e-11','2.4636501e-11','108.51534','1.000009','349.23428','5.914451e-10','1.3133328e-20','3.1936167e-22','3.721376e-23','236.11562','-7.945089e-11','1110.1643'],
['Slovakia','201903','0.00062075304','5.020701e-11','7.580353e-11','2.3267666e-11','104.98873','1.000009','355.3981','8.6016627e-10','1.0688063e-22','3.452217e-22','4.1145997e-23','285.3087','-7.945089e-11','1565.4166'],
['Slovenia','201903','0.0006172243','1.7026437e-10','9.915166e-11','3.1293225e-11','114.90523','1.000009','345.95575','1.303412e-09','4.845319e-23','3.88264e-22','4.0815442e-23','356.3794','-7.945089e-11','1172.1719'],
['Spain','201903','0.00057256635','1.279579e-09','1.0642452e-10','3.768284e-11','140.40327','1.000009','315.71558','6.3232364e-10','4.804396e-22','2.3893468e-22','2.1160194e-23','1943.3346','-7.945089e-11','1229.6837'],
['Sweden','201903','0.00064373383','2.0774894e-11','4.206588e-11','1.1521337e-11','104.77282','0.77519476','390.51965','7.526472e-10','2.1069144e-26','1.1442408e-22','1.3757444e-23','406.8798','8.3934436e-05','2648.9136'],
['Ireland','201903','0.00062680355','2.0004987e-11','6.726372e-11','2.1468107e-11','85.78634','1.000009','360.5127','1.1457185e-09','5.5777115e-25','1.378228e-22','2.157132e-23','46.308697','-7.945089e-11','3139.4812'],
['Italy','201903','0.0006253339','5.990685e-10','1.1418124e-10','4.0259233e-11','126.228004','1.000009','336.91202','1.0179199e-09','3.2312662e-22','3.5109059e-22','3.348793e-23','939.77765','-7.945089e-11','1102.8109'],
['Denmark','201903','0.0006345784','1.0544913e-10','6.3322805e-11','1.9119113e-11','96.20971','1.000009','376.4727','1.1701499e-09','6.8328747e-26','2.4326534e-22','2.9444113e-23','164.54681','-7.945089e-11','2657.3816'],
['Finland','201903','0.00062706677','4.3503093e-11','3.249081e-11','8.434623e-12','99.50253','1.000009','395.01025','7.0472017e-10','1.7777708e-24','9.043864e-23','9.1740745e-24','105.32163','-7.945089e-11','2728.9036'],
['Belgium','201903','0.00064625725','7.2320233e-10','8.8414207e-11','2.7377219e-11','137.3493','1.000009','353.78152','1.9759352e-09','3.3372025e-23','3.6305777e-22','3.4672838e-23','2710.239','-7.945089e-11','2084.9612'],
['Croatia','201903','0.0006114808','7.7308465e-11','9.773065e-11','3.1646016e-11','108.07872','0.99442846','343.89258','1.1066896e-09','1.5656458e-20','2.8167694e-22','3.390314e-23','539.52637','8.941917e-09','1073.8392'],
['Austria','201904','0.00056287705','6.4478436e-11','1.2930597e-10','3.903388e-11','111.775635','1.0000064','354.74158','6.5210093e-10','1.5858575e-23','3.8351585e-22','4.7222448e-23','293.04968','-4.1264103e-11','647.57733'],
['Bulgaria','201904','0.0006300942','1.2004596e-10','1.6483835e-10','5.4210775e-11','106.66225','1.0000064','361.00153','8.738239e-10','8.119146e-22','6.5952583e-22','1.1951897e-22','110.73082','-4.1264103e-11','495.43436'],
['Cyprus','201904','0.000616358','1.5481115e-10','1.3866149e-10','5.2077904e-11','95.03059','0.9166337','347.30768','2.6031594e-10','7.661617e-25','2.3056067e-22','7.957054e-23','87.85239','3.261831e-09','877.2152'],
['Czech Republic','201904','0.00065989624','2.0445773e-10','1.501542e-10','4.697838e-11','122.767975','1.0000064','354.03583','7.440956e-10','1.8194289e-22','6.7282595e-22','8.922255e-23','366.46503','-4.1264103e-11','892.3771'],
['Estonia','201904','0.00071431923','3.1080798e-11','1.0423202e-10','2.9641144e-11','111.84814','1.0000064','357.13705','2.3434308e-09','1.01883917e-22','4.718817e-22','5.871291e-23','657.37085','-4.1264103e-11','1442.3448'],
['France','201904','0.0006112499','6.8077335e-11','1.343224e-10','4.310458e-11','99.2443','1.0000064','367.91766','1.5413987e-09','1.3404482e-22','3.606829e-22','5.0298267e-23','193.93199','-4.1264103e-11','393.52875'],
['Germany','201904','0.00065417204','1.3543702e-10','1.4310585e-10','4.4450978e-11','121.627174','1.0000064','361.86655','7.453145e-10','2.74679e-22','4.3755487e-22','7.9746246e-23','362.80304','-4.1264103e-11','1222.9406'],
['Greece','201904','0.0005871792','9.043867e-11','1.3917154e-10','4.6816218e-11','99.30504','1.0000064','353.167','1.0317261e-09','9.420997e-22','2.719414e-22','6.609692e-23','130.37265','-4.1264103e-11','625.4389'],
['Hungary','201904','0.0006875395','2.477053e-10','1.7061084e-10','5.4762375e-11','123.966545','1.0000064','357.00687','1.1274874e-09','2.1179918e-22','8.3540415e-22','1.0893079e-22','380.01923','-4.1264103e-11','645.4982'],
['Latvia','201904','0.0007174236','1.402365e-10','1.1622539e-10','3.3273016e-11','118.13634','0.7288869','363.53943','2.3806188e-09','7.1641465e-22','5.3415334e-22','7.224748e-23','750.173','-4.3699921e-10','1668.8453'],
['Lithuania','201904','0.00071015913','1.5660172e-10','1.35043e-10','3.9412213e-11','122.75539','1.0000064','355.16614','1.6457764e-09','1.8563985e-21','6.010434e-22','9.2397915e-23','764.87646','-4.1264103e-11','1030.8553'],
['Luxembourg','201904','0.00064924516','2.5344649e-10','1.4039912e-10','4.3166297e-11','135.93527','1.0000064','369.91528','1.5602281e-09','7.0929867e-23','4.523835e-22','7.553221e-23','1321.1218','-4.1264103e-11','884.3384'],
['Malta','201904','0.0006527271','1.7672619e-11','1.4439908e-10','5.4909653e-11','91.687225','0.0','340.50452','1.2746767e-09','8.0779355e-30','2.4923112e-22','7.228696e-23','57.48956','0.0','511.0352'],
['Netherlands','201904','0.0007037851','6.4472566e-10','1.4557716e-10','4.4953846e-11','167.4439','1.0000064','374.2379','2.1292024e-09','1.4425745e-21','5.2149784e-22','9.2444786e-23','4885.1504','-4.1264103e-11','1214.2123'],
['Poland','201904','0.0006839523','2.1843789e-10','1.455496e-10','4.55328e-11','120.71698','1.0000064','349.98682','7.6009743e-10','3.589732e-22','6.376603e-22','1.1806131e-22','468.24966','-4.1264103e-11','866.89185'],
['Portugal','201904','0.0006338998','9.864745e-11','1.5494796e-10','5.5317365e-11','95.281364','1.0000064','352.93506','1.2138851e-09','1.6960444e-22','1.4160853e-22','5.523182e-23','140.68597','-4.1264103e-11','1108.445'],
['Romania','201904','0.0006237935','1.00872005e-10','1.5738567e-10','4.8992293e-11','107.13208','1.0000064','356.82767','9.10656e-10','2.5941646e-21','6.8940617e-22','8.825365e-23','159.64807','-4.1264103e-11','415.17093'],
['Slovakia','201904','0.0006552242','7.1059526e-11','1.564686e-10','4.856362e-11','113.69063','1.0000064','351.11288','9.403349e-10','2.3393252e-22','7.705232e-22','9.5390436e-23','222.38745','-4.1264103e-11','523.9523'],
['Slovenia','201904','0.0006402139','1.6913025e-10','1.6293951e-10','5.0969555e-11','119.34345','1.0000064','359.67728','1.2405473e-09','2.4719376e-25','4.734034e-22','6.1789064e-23','370.40402','-4.1264103e-11','591.2835'],
['Spain','201904','0.0005885566','1.296352e-09','1.5257687e-10','5.1170384e-11','134.98152','1.0000064','359.02924','8.916322e-10','3.2920145e-22','2.3265028e-22','6.510524e-23','1258.7999','-4.1264103e-11','1031.4054'],
['Sweden','201904','0.0007136328','2.1809636e-11','9.7717646e-11','2.8489168e-11','113.534035','0.7735199','355.1302','1.6859606e-09','3.6250303e-25','3.1429057e-22','5.1209663e-23','707.4815','1.0196768e-08','1743.9806'],
['Ireland','201904','0.0006667504','3.174705e-11','1.0423325e-10','3.328922e-11','107.41528','1.0000064','374.7188','3.383796e-09','6.860524e-22','1.8451358e-22','4.1561007e-23','630.2854','-4.1264103e-11','1266.024'],
['Italy','201904','0.0006580943','6.1576044e-10','1.7544759e-10','5.887644e-11','133.33907','1.0000064','361.53745','1.9801307e-09','3.7500723e-22','4.955073e-22','7.380979e-23','1079.0876','-4.1264103e-11','571.1558'],
['Denmark','201904','0.0006837586','1.0848282e-10','1.2615092e-10','3.9821476e-11','112.43694','1.0000064','370.42044','9.0901686e-10','2.396887e-23','3.3934595e-22','7.8173913e-23','517.44434','-4.1264103e-11','1747.5873'],
['Finland','201904','0.00069754745','5.7049e-11','8.660239e-11','2.4984006e-11','103.72149','1.0000064','352.17828','1.7553955e-09','1.0586253e-22','2.746583e-22','4.4949944e-23','329.80817','-4.1264103e-11','1797.0571'],
['Belgium','201904','0.0006904037','7.349778e-10','1.5100365e-10','4.6264735e-11','165.45079','1.0000064','374.264','3.8183257e-09','3.072884e-23','4.9958094e-22','8.832819e-23','4378.715','-4.1264103e-11','994.2813'],
['Croatia','201904','0.0006346078','3.7523082e-11','1.5482282e-10','4.8993892e-11','110.90018','0.9944336','361.0277','1.7190184e-09','8.253581e-22','4.0123016e-22','5.2238394e-23','376.00058','5.9660734e-09','580.9738'],
['Austria','201905','0.00054391834','8.4395976e-11','2.0130683e-10','6.476893e-11','99.97739','1.0000206','359.42493','1.1285446e-09','8.0853036e-23','4.6967723e-22','1.0037342e-22','145.38557','-4.2317816e-10','551.88947'],
['Bulgaria','201905','0.00060673524','1.6578108e-10','2.593441e-10','9.7115455e-11','106.85381','1.0000206','347.99374','1.0867455e-09','1.4364693e-22','7.0850554e-22','2.0430354e-22','247.22838','-4.2317816e-10','423.41693'],
['Cyprus','201905','0.0005815753','1.6076457e-10','1.930219e-10','9.222115e-11','91.00702','0.9166239','323.0823','4.2837497e-10','2.9728555e-22','2.613709e-22','2.5770286e-22','112.99928','2.7093954e-09','253.27904'],
['Czech Republic','201905','0.00062617485','2.6860533e-10','2.3962235e-10','8.252033e-11','118.379684','1.0000206','362.47433','1.0184416e-09','5.827257e-22','6.0179384e-22','2.0179451e-22','652.4005','-4.2317816e-10','778.98535'],
['Estonia','201905','0.000615127','7.7594015e-11','1.5376442e-10','5.4855082e-11','91.56729','1.0000206','371.58655','8.382201e-10','4.633607e-22','2.126703e-22','1.0392999e-22','97.37825','-4.2317816e-10','2124.783'],
['France','201905','0.00057638506','1.183082e-10','1.9965807e-10','7.265628e-11','92.0511','1.0000206','353.676','1.114883e-09','4.1409707e-22','3.8075254e-22','1.0265887e-22','145.18881','-4.2317816e-10','594.0486'],
['Germany','201905','0.00060874125','1.9432932e-10','2.2572215e-10','7.777484e-11','110.55173','1.0000206','366.98053','1.2628835e-09','6.0860677e-22','4.058136e-22','1.8042627e-22','460.2652','-4.2317816e-10','1323.9382'],
['Greece','201905','0.0005544405','1.4168199e-10','1.9577873e-10','7.5048516e-11','94.60663','1.0000206','345.17715','6.252847e-10','2.2312187e-21','2.194579e-22','9.011094e-23','124.226685','-4.2317816e-10','368.38437'],
['Hungary','201905','0.0006584817','2.8370734e-10','2.698803e-10','9.448935e-11','121.24362','1.0000206','363.07596','1.2068517e-09','1.6568451e-22','8.563355e-22','1.9507991e-22','607.65137','-4.2317816e-10','355.40277'],
['Latvia','201905','0.0006262796','1.7920529e-10','1.6844828e-10','6.040337e-11','96.85638','0.7288758','365.5494','6.790788e-10','6.287091e-22','2.2404596e-22','1.2031009e-22','142.01819','2.9773644e-09','1577.9111'],
['Lithuania','201905','0.0006347348','2.0085164e-10','1.9312688e-10','6.932368e-11','100.183556','1.0000206','358.11713','6.669511e-10','3.5729404e-22','2.9987978e-22','1.4542932e-22','173.94925','-4.2317816e-10','880.7585'],
['Luxembourg','201905','0.0006065947','2.8867625e-10','2.2644346e-10','7.841005e-11','122.313515','1.0000206','364.0163','1.6693918e-09','3.4430613e-22','5.3857667e-22','1.8236432e-22','1706.0408','-4.2317816e-10','1121.388'],
['Malta','201905','0.00061883323','1.7680137e-11','1.9495511e-10','8.1998734e-11','86.456764','0.0','348.53348','7.394476e-10','9.6666246e-30','2.737223e-22','8.891775e-23','41.63809','0.0','309.1937'],
['Netherlands','201905','0.00064122403','6.474209e-10','2.2986821e-10','8.140367e-11','133.40556','1.0000206','366.45273','1.7778866e-09','5.636625e-22','5.660455e-22','2.0459958e-22','2721.7683','-4.2317816e-10','1236.3812'],
['Poland','201905','0.0006402936','2.808065e-10','2.3825575e-10','8.5017854e-11','111.24032','1.0000206','361.04922','9.113082e-10','5.853845e-22','8.362174e-22','2.4652524e-22','545.88763','-4.2317816e-10','717.05927'],
['Portugal','201905','0.00059300644','1.4765536e-10','1.830264e-10','8.280825e-11','91.368935','1.0000206','331.25818','9.60234e-10','1.4603902e-22','8.585173e-23','7.521168e-23','209.13454','-4.2317816e-10','380.05026'],
['Romania','201905','0.0005959141','1.2937669e-10','2.4015526e-10','8.435051e-11','106.99274','1.0000206','353.52536','1.0586144e-09','2.7309453e-22','5.8877756e-22','1.4578133e-22','256.466','-4.2317816e-10','423.49423'],
['Slovakia','201905','0.000627784','1.13999477e-10','2.48839e-10','8.5268376e-11','109.288605','1.0000206','363.13107','1.3661914e-09','2.8412773e-22','8.241406e-22','1.6862321e-22','345.87338','-4.2317816e-10','416.2779'],
['Slovenia','201905','0.00061459455','1.7179219e-10','2.3488234e-10','7.9681525e-11','107.71388','1.0000206','361.1844','9.877597e-10','3.2493215e-23','5.2830775e-22','1.0823286e-22','273.3071','-4.2317816e-10','389.59186'],
['Spain','201905','0.0005549761','1.3527334e-09','1.9308577e-10','8.103702e-11','124.109314','1.0000206','338.9159','1.1488398e-09','1.1078307e-21','1.7339997e-22','8.774102e-23','1323.8427','-4.2317816e-10','292.6134'],
['Sweden','201905','0.0006098973','2.591418e-11','1.3986677e-10','4.931984e-11','96.24801','0.77352005','378.84042','7.755573e-10','5.6508394e-24','1.5386143e-22','9.147473e-23','268.49182','1.2775221e-08','2279.9487'],
['Ireland','201905','0.00060621864','2.990262e-11','1.6273284e-10','6.149982e-11','83.18456','1.0000206','353.39838','1.0535726e-09','8.394704e-24','1.9730731e-22','1.1397924e-22','147.91338','-4.2317816e-10','1189.8882'],
['Italy','201905','0.0006274217','6.538185e-10','2.3898653e-10','8.6314136e-11','123.02951','1.0000206','361.89273','1.3689643e-09','3.1263158e-22','4.3730445e-22','7.9963075e-23','883.7321','-4.2317816e-10','409.42477'],
['Denmark','201905','0.0006095386','1.1471827e-10','1.8082226e-10','6.476083e-11','87.58132','1.0000206','378.74606','9.194939e-10','6.673167e-24','2.2651024e-22','1.2123118e-22','131.9064','-4.2317816e-10','2216.1326'],
['Finland','201905','0.00060221506','1.2336031e-10','1.3088253e-10','4.571015e-11','90.405685','1.0000206','376.1887','8.5149066e-10','1.1405484e-21','1.3110661e-22','7.650075e-23','107.34549','-4.2317816e-10','1973.1348'],
['Belgium','201905','0.0006404542','7.623766e-10','2.4148397e-10','8.4253673e-11','148.00352','1.0000206','363.83255','2.0008668e-09','1.1288509e-22','6.4216373e-22','2.1536365e-22','2950.2847','-4.2317816e-10','1112.013'],
['Croatia','201905','0.0006097278','3.2020265e-11','2.2001947e-10','7.5413106e-11','98.190384','0.99444175','359.79758','1.298864e-09','2.4237026e-24','4.874555e-22','9.077192e-23','86.84792','8.099363e-09','377.40707'],
['Austria','201906','0.00050166325','1.2855313e-10','2.3185033e-10','1.00770794e-10','101.01491','1.000018','321.0134','8.86241e-10','1.3785338e-21','3.3822925e-22','6.038412e-23','309.61932','-3.2376546e-10','158.65675'],
['Bulgaria','201906','0.0005757813','1.98845e-10','2.7239666e-10','1.2210045e-10','103.12473','1.000018','331.72433','7.444828e-10','7.9172903e-22','1.987189e-22','3.56224e-23','246.1269','-3.2376546e-10','234.06242'],
['Cyprus','201906','0.00055085577','1.6403819e-10','2.4407618e-10','1.3216692e-10','84.40024','0.9166653','305.39072','3.5870748e-10','2.8306734e-23','4.1234461e-22','1.4248685e-22','52.870087','1.391709e-09','268.21387'],
['Czech Republic','201906','0.00058819144','3.6242306e-10','2.9120814e-10','1.3105105e-10','125.351036','1.000018','323.33734','1.3379214e-09','8.4452717e-22','5.804614e-22','1.0479341e-22','1062.8256','-3.2376546e-10','120.00711'],
['Estonia','201906','0.0005748423','1.6001044e-10','1.7104572e-10','7.571863e-11','89.60371','1.000018','329.31433','1.2583036e-09','3.684006e-22','9.9378337e-23','1.8264301e-23','206.40338','-3.2376546e-10','356.7565'],
['France','201906','0.0005207687','1.974134e-10','1.9963015e-10','9.345441e-11','88.42009','1.000018','334.81372','6.756657e-10','5.1841086e-22','7.857897e-23','1.04119126e-22','250.6198','-3.2376546e-10','837.7382'],
['Germany','201906','0.00057158957','2.6266206e-10','2.589783e-10','1.1650686e-10','112.90123','1.000018','327.12146','1.2718349e-09','4.8881603e-22','4.095974e-22','1.0532567e-22','675.0611','-3.2376546e-10','265.5115'],
['Greece','201906','0.000536015','1.72049e-10','2.0603372e-10','9.2382046e-11','93.99815','1.000018','329.16913','5.2985e-10','6.5037965e-23','1.0898413e-22','2.5094511e-23','145.83543','-3.2376546e-10','228.12544'],
['Hungary','201906','0.00060992414','3.306168e-10','3.0621952e-10','1.4048476e-10','117.25818','1.000018','326.77017','8.3268914e-10','2.0342922e-22','3.9228675e-22','1.0371968e-22','612.4643','-3.2376546e-10','202.25107'],
['Latvia','201906','0.0005807503','2.4412353e-10','1.9183367e-10','8.6363396e-11','97.027145','0.728872','325.7094','1.3322344e-09','3.2771243e-22','1.6101146e-22','3.1414606e-23','303.84628','1.0678675e-08','305.49573'],
['Lithuania','201906','0.00058625446','2.8585417e-10','2.2779006e-10','1.0367e-10','104.65957','1.000018','323.37836','1.5356953e-09','6.0484104e-22','3.0903387e-22','6.0213084e-23','497.85223','-3.2376546e-10','235.41733'],
['Luxembourg','201906','0.0005614258','3.4029676e-10','2.5453156e-10','1.1376425e-10','125.35188','1.000018','332.9362','1.4158105e-09','3.056209e-22','3.836256e-22','1.4547073e-22','2399.1711','-3.2376546e-10','589.8143'],
['Malta','201906','0.0005763856','1.7683277e-11','2.2284895e-10','1.1977454e-10','84.27278','0.0','324.4983','1.0101034e-09','1.0602291e-29','4.2082756e-22','1.1764429e-22','77.61949','0.0','228.47704'],
['Netherlands','201906','0.00060464366','6.6290035e-10','2.6616875e-10','1.1734656e-10','142.82877','1.000018','339.85965','2.8446063e-09','4.0458692e-22','7.085479e-22','1.8146598e-22','4723.5522','-3.2376546e-10','723.42706'],
['Poland','201906','0.00060178776','3.7578804e-10','2.973957e-10','1.3599022e-10','115.6651','1.000018','323.05872','1.3427509e-09','8.72254e-22','5.181047e-22','1.163042e-22','1065.8549','-3.2376546e-10','177.34258'],
['Portugal','201906','0.00054832804','1.814648e-10','1.9545965e-10','9.523582e-11','81.54978','1.000018','338.78308','7.882008e-10','1.2927297e-22','5.957245e-23','4.5855112e-23','155.92183','-3.2376546e-10','509.28894'],
['Romania','201906','0.0005559672','1.8354582e-10','2.5356026e-10','1.1118431e-10','102.15382','1.000018','327.11856','9.2491326e-10','2.1578872e-22','1.9254988e-22','3.923484e-23','324.6692','-3.2376546e-10','217.735'],
['Slovakia','201906','0.0005827686','1.8158583e-10','2.8362773e-10','1.2625283e-10','106.99277','1.000018','324.39496','7.704229e-10','4.7895716e-22','3.1208625e-22','7.6801327e-23','366.94022','-3.2376546e-10','201.16447'],
['Slovenia','201906','0.00056553906','1.7354461e-10','2.5609145e-10','1.1592134e-10','107.3131','1.000018','325.67612','1.0622554e-09','7.311386e-24','3.2501622e-22','5.1181857e-23','362.00867','-3.2376546e-10','178.25659'],
['Spain','201906','0.00052978884','1.5975103e-09','2.2014975e-10','1.0387969e-10','132.44891','1.000018','331.51965','6.57603e-10','7.015783e-19','1.5145496e-22','1.1310512e-22','2257.614','-3.2376546e-10','729.0435'],
['Sweden','201906','0.00058273546','3.554561e-11','1.757619e-10','7.673479e-11','92.71154','0.77351445','333.2818','1.3426527e-09','4.7044304e-24','1.0286799e-22','3.244495e-23','431.6585','3.9710866e-09','389.05884'],
['Ireland','201906','0.0005409432','4.1687542e-11','1.8762852e-10','8.129087e-11','73.07207','1.000018','363.69403','1.2017061e-09','9.565242e-24','1.8493756e-22','7.8997175e-23','106.056305','-3.2376546e-10','776.9978'],
['Italy','201906','0.00058365485','7.105838e-10','2.4502514e-10','1.1989497e-10','131.27165','1.000018','327.07294','9.75909e-10','5.8449745e-21','2.8151947e-22','5.4893896e-23','1434.9995','-3.2376546e-10','133.53049'],
['Denmark','201906','0.0005791766','1.2461404e-10','2.1249089e-10','9.511037e-11','85.90069','1.000018','337.92056','1.7748188e-09','9.915113e-24','2.3361112e-22','7.70611e-23','350.21567','-3.2376546e-10','392.51294'],
['Finland','201906','0.00056649005','2.5247193e-10','1.6157102e-10','6.8114146e-11','92.10785','1.000018','337.72498','1.7762765e-09','7.246121e-22','9.0030486e-23','2.3956786e-23','271.52347','-3.2376546e-10','396.43765'],
['Belgium','201906','0.0005853382','8.050468e-10','2.7148472e-10','1.2119768e-10','137.02571','1.000018','338.33646','1.8682853e-09','2.1366587e-22','6.1294543e-22','1.8736196e-22','2713.2764','-3.2376546e-10','835.32495'],
['Croatia','201906','0.00055609335','3.8240244e-11','2.3735372e-10','1.1046137e-10','97.77098','0.9944295','326.92572','8.7391794e-10','2.9931114e-24','2.1644435e-22','5.696814e-23','182.72734','6.8298545e-09','186.3324'],
['Austria','201907','0.00050786784','1.4157495e-10','2.4213162e-10','9.93865e-11','97.83634','1.0000072','321.9145','1.5110575e-09','1.02226364e-22','2.8914408e-22','3.0284553e-23','232.02567','-5.1779914e-11','490.9262'],
['Bulgaria','201907','0.00057567086','2.2998267e-10','2.5658337e-10','1.1447365e-10','109.080315','1.0000072','325.9785','1.444619e-09','5.062172e-22','2.6922404e-22','4.683351e-23','336.81714','-5.1779914e-11','340.42087'],
['Cyprus','201907','0.00055587076','1.6693812e-10','2.7332206e-10','1.5548557e-10','88.28792','0.9167474','292.32788','6.0017197e-10','1.3505098e-24','3.0655054e-22','1.7887845e-23','100.272026','9.81727e-09','51.011814'],
['Czech Republic','201907','0.0005803104','4.0621406e-10','3.0132938e-10','1.2973905e-10','124.804634','1.0000072','330.96503','3.0524177e-09','7.063852e-22','6.886516e-22','5.3206024e-23','1561.9838','-5.1779914e-11','544.8912'],
['Estonia','201907','0.00056900457','1.8692867e-10','2.0300576e-10','8.1733904e-11','92.10484','1.0000072','342.5944','3.6947636e-09','2.3595824e-22','4.6732827e-22','4.22302e-23','314.28333','-5.1779914e-11','841.1803'],
['France','201907','0.00054762897','2.3835361e-10','2.0335424e-10','9.4050476e-11','97.701035','1.0000072','318.83197','8.3736224e-10','5.8196306e-22','1.4668707e-22','3.0322538e-23','400.07785','-5.1779914e-11','199.19449'],
['Germany','201907','0.0005668641','3.0549488e-10','2.6900387e-10','1.1897712e-10','109.223946','1.0000072','329.37283','2.7967608e-09','5.7174366e-22','5.3691105e-22','5.325341e-23','938.5759','-5.1779914e-11','495.09967'],
['Greece','201907','0.0005333022','1.9179963e-10','1.9181724e-10','8.859232e-11','98.92381','1.0000072','314.86377','1.5167301e-09','4.0246497e-23','1.790357e-22','1.9341692e-23','230.68015','-5.1779914e-11','244.58821'],
['Hungary','201907','0.0006150029','3.634715e-10','3.1873215e-10','1.3877903e-10','127.09878','1.0000072','329.4703','2.0211666e-09','1.04381305e-22','4.83721e-22','5.140598e-23','1106.5369','-5.1779914e-11','525.6404'],
['Latvia','201907','0.00056916865','2.7006858e-10','2.2781639e-10','9.293401e-11','95.5427','0.72886145','344.94125','3.8421715e-09','1.9337501e-22','6.1748533e-22','4.9296028e-23','375.18002','3.367239e-09','804.09216'],
['Lithuania','201907','0.0005708964','3.3743353e-10','2.6655197e-10','1.1007486e-10','103.06284','1.0000072','345.94022','3.1383536e-09','5.2339217e-22','6.300874e-22','5.272138e-23','687.7099','-5.1779914e-11','645.9264'],
['Luxembourg','201907','0.0005721545','3.747763e-10','2.6027397e-10','1.1567065e-10','128.88033','1.0000072','325.63058','2.5904583e-09','2.8810182e-22','4.7215064e-22','4.7272133e-23','2880.5635','-5.1779914e-11','414.73468'],
['Malta','201907','0.0005743828','1.767326e-11','2.5401692e-10','1.4138947e-10','82.965','0.0','308.51666','7.4689094e-10','1.0402128e-29','3.601927e-22','2.1813332e-23','53.340965','0.0','174.2609'],
['Netherlands','201907','0.0006036088','6.7987665e-10','2.808263e-10','1.2697958e-10','140.67888','1.0000072','330.21124','5.437804e-09','1.4813451e-22','1.1864017e-21','6.928267e-23','5108.1094','-5.1779914e-11','437.1754'],
['Poland','201907','0.00058281363','4.3266643e-10','3.1253608e-10','1.3442829e-10','113.11707','1.0000072','338.35025','2.9854177e-09','9.357637e-22','7.032219e-22','5.0365115e-23','1322.8209','-5.1779914e-11','609.8789'],
['Portugal','201907','0.000587042','3.5369938e-09','2.1915167e-10','1.0528526e-10','149.8064','1.0000072','319.42545','2.7348708e-08','3.886961e-16','7.485524e-21','2.5323863e-23','68776.88','-5.1779914e-11','261.4167'],
['Romania','201907','0.00055731356','2.1739689e-10','2.4196978e-10','1.0085537e-10','110.1101','1.0000072','330.01086','1.2959301e-09','3.3590635e-23','2.0849899e-22','4.4027507e-23','519.40216','-5.1779914e-11','446.6984'],
['Slovakia','201907','0.0005787788','2.2740249e-10','2.8861294e-10','1.2188675e-10','114.42174','1.0000072','331.30585','2.3079985e-09','2.6626623e-22','4.452474e-22','4.1960783e-23','758.99744','-5.1779914e-11','618.95355'],
['Slovenia','201907','0.00058503984','1.8325409e-10','2.6076685e-10','1.098646e-10','110.83391','1.0000072','323.91208','1.7514067e-09','5.1501833e-22','2.8265593e-22','3.6675642e-23','408.82462','-5.1779914e-11','401.18063'],
['Spain','201907','0.00054108334','1.4139122e-09','2.3344893e-10','1.1487572e-10','137.02304','1.0000072','313.1078','9.2424646e-10','3.3765608e-21','2.5270332e-22','2.6223556e-23','2369.622','-5.1779914e-11','173.73062'],
['Sweden','201907','0.00056657585','3.86031e-11','1.9560734e-10','8.3264284e-11','85.66004','0.77350706','336.3013','2.9401006e-09','1.4945723e-24','2.7771e-22','4.1767594e-23','346.16187','-3.3830039e-09','815.7268'],
['Ireland','201907','0.00055063545','4.6929745e-11','1.8250959e-10','8.6146985e-11','69.05646','1.0000072','326.5743','1.2686677e-09','7.083668e-24','1.1579982e-22','2.4441604e-23','29.199553','-5.1779914e-11','249.23538'],
['Italy','201907','0.0005974461','7.226895e-10','2.5737634e-10','1.2152758e-10','131.90991','1.0000072','319.72644','1.1080471e-09','4.819179e-22','2.5873186e-22','2.2786883e-23','1473.3099','-5.1779914e-11','226.54979'],
['Denmark','201907','0.0005593597','1.3180339e-10','2.2412462e-10','1.0161753e-10','82.255066','1.0000072','336.3828','3.0526495e-09','6.52326e-24','3.7092172e-22','5.259313e-23','328.5783','-5.1779914e-11','666.28394'],
['Finland','201907','0.00056061766','2.853833e-10','1.8470502e-10','7.4510474e-11','94.71006','1.0000072','335.21646','2.2695381e-09','1.5431753e-22','1.8975566e-22','3.1537924e-23','393.9101','-5.1779914e-11','766.53845'],
['Belgium','201907','0.0005890563','8.3648577e-10','2.7224373e-10','1.2444841e-10','137.51332','1.0000072','327.11743','3.333068e-09','2.558535e-22','6.817804e-22','5.4849764e-23','2595.219','-5.1779914e-11','377.6419'],
['Croatia','201907','0.00057332194','4.1297965e-11','2.508914e-10','1.0980603e-10','97.644684','0.994414','324.04837','1.4391724e-09','6.333489e-24','2.310069e-22','2.9345746e-23','168.6763','1.4108228e-08','363.70978'],
['Austria','201908','0.0005313656','9.57344e-11','2.1311175e-10','8.588091e-11','102.00284','0.99999434','300.4382','1.3949427e-09','8.331164e-23','4.1725739e-22','3.778431e-23','409.30716','-3.1901815e-11','275.4707'],
['Bulgaria','201908','0.0005901758','2.657912e-10','2.334264e-10','1.0614944e-10','113.213715','0.99999434','300.0276','1.081287e-09','1.29296375e-20','3.7079802e-22','4.0173603e-23','375.09216','-3.1901815e-11','271.51102'],
['Cyprus','201908','0.0005740164','1.6454572e-10','2.585269e-10','1.4191526e-10','97.74221','0.9167332','288.60175','5.7779637e-10','2.3596402e-22','4.0416886e-22','8.9749115e-23','229.4225','1.24341355e-08','88.89921'],
['Czech Republic','201908','0.00061732624','2.8194336e-10','2.6229055e-10','1.0924462e-10','124.30112','0.99999434','305.52872','1.4901211e-09','6.141474e-22','3.7743718e-22','5.510904e-23','1128.732','-3.1901815e-11','287.85168'],
['Estonia','201908','0.0006093328','1.09720635e-10','1.6662308e-10','6.5243665e-11','90.29402','0.99999434','312.46848','1.0416635e-09','4.3664423e-22','3.930405e-22','2.4552897e-23','138.229','-3.1901815e-11','482.52344'],
['France','201908','0.0005550371','1.3363231e-10','1.7677246e-10','7.892534e-11','88.026886','0.99999434','308.0209','6.7787953e-10','4.2347305e-22','1.8251197e-22','2.7173182e-23','118.20349','-3.1901815e-11','195.81198'],
['Germany','201908','0.000607478','2.0115053e-10','2.42866e-10','1.0046396e-10','115.02449','0.99999434','310.15186','1.2557368e-09','5.0284645e-22','3.751922e-22','4.335291e-23','632.50494','-3.1901815e-11','316.76205'],
['Greece','201908','0.0005495469','1.9171315e-10','1.940339e-10','8.970924e-11','104.99805','0.99999434','295.39847','6.62786e-10','5.3674293e-22','1.1864982e-22','9.232322e-24','231.08885','-3.1901815e-11','133.85504'],
['Hungary','201908','0.00064397126','3.133194e-10','2.8336142e-10','1.2378339e-10','127.68647','0.99999434','302.86597','1.5276032e-09','2.0876589e-22','5.831312e-22','7.4037524e-23','823.479','-3.1901815e-11','339.56537'],
['Latvia','201908','0.00061606476','2.0019886e-10','1.8370391e-10','7.3295814e-11','97.56866','0.7289238','310.45035','1.1029502e-09','3.2650764e-22','5.4341535e-22','3.597274e-23','182.34329','8.45941e-09','484.3226'],
['Lithuania','201908','0.0006246089','2.260296e-10','2.149412e-10','8.693942e-11','106.15512','0.99999434','309.59668','1.5771525e-09','7.50535e-22','8.60585e-22','5.1754005e-23','402.35852','-3.1901815e-11','452.46707'],
['Luxembourg','201908','0.0005919843','2.9166436e-10','2.2776024e-10','9.673051e-11','119.62584','0.99999434','312.06686','1.1190159e-09','2.315725e-22','3.3337608e-22','3.8553305e-23','1491.0822','-3.1901815e-11','339.0528'],
['Malta','201908','0.0005995645','1.7677157e-11','2.4823113e-10','1.3292628e-10','88.155304','0.0','296.2469','6.60763e-10','6.82968e-30','1.1874536e-22','5.895482e-23','60.66378','0.0','56.74168'],
['Netherlands','201908','0.0006391255','6.464674e-10','2.562297e-10','1.0654341e-10','147.08545','0.99999434','318.58362','2.713834e-09','9.408986e-23','5.8997006e-22','5.842809e-23','7304.9126','-3.1901815e-11','476.36813'],
['Poland','201908','0.00062906207','2.8672187e-10','2.6732425e-10','1.12565644e-10','119.02527','0.99999434','307.0752','1.1122518e-09','8.463788e-22','5.3043285e-22','6.230265e-23','984.0218','-3.1901815e-11','403.95908'],
['Portugal','201908','0.00056573295','2.839063e-10','1.9337867e-10','9.902732e-11','96.17223','0.99999434','307.5456','1.8275562e-09','2.1055275e-19','2.505322e-22','1.8566468e-23','407.77686','-3.1901815e-11','145.43887'],
['Romania','201908','0.00057540735','1.811256e-10','2.0325622e-10','8.749555e-11','110.070694','0.99999434','301.42957','1.2232199e-09','4.0119625e-22','5.1933114e-22','3.7396983e-23','383.55972','-3.1901815e-11','373.43704'],
['Slovakia','201908','0.00061037665','1.4767959e-10','2.5680066e-10','1.08583434e-10','113.78682','0.99999434','301.88364','1.2247235e-09','4.0328205e-22','4.478252e-22','5.457876e-23','465.39218','-3.1901815e-11','352.588'],
['Slovenia','201908','0.0006022546','1.7192017e-10','2.208433e-10','9.2429626e-11','116.50328','0.99999434','301.78326','1.7399068e-09','3.5185169e-25','4.1896521e-22','4.155721e-23','458.6191','-3.1901815e-11','276.80508'],
['Spain','201908','0.0005501744','1.5772539e-09','2.2206757e-10','1.0654173e-10','146.35178','0.99999434','304.3225','1.4782087e-09','2.0689437e-18','3.306667e-22','3.8798404e-23','4263.46','-3.1901815e-11','134.59819'],
['Sweden','201908','0.0006065263','3.1912997e-11','1.549947e-10','6.195844e-11','99.97793','0.77350014','315.95242','1.1571908e-09','4.7882827e-24','3.5245134e-22','3.9820263e-23','495.56717','-5.9452745e-09','503.85306'],
['Ireland','201908','0.00058832247','3.4700614e-11','1.7000755e-10','6.981857e-11','77.63474','0.99999434','330.59354','1.2592359e-09','6.4369563e-24','2.5303487e-22','3.5124814e-23','60.73979','-3.1901815e-11','377.52985'],
['Italy','201908','0.0006187774','6.8362643e-10','2.4575714e-10','1.1455738e-10','140.38358','0.99999434','299.9339','2.7059286e-09','5.119775e-22','4.047297e-22','3.865246e-23','1786.626','-3.1901815e-11','148.19788'],
['Denmark','201908','0.0006032143','1.196994e-10','2.0023977e-10','8.316128e-11','89.61528','0.99999434','319.64688','1.8184181e-09','7.615921e-24','3.6462696e-22','4.6820854e-23','343.9326','-3.1901815e-11','522.1021'],
['Finland','201908','0.0006007909','2.0809642e-10','1.4156411e-10','5.326879e-11','99.38632','0.99999434','315.0756','1.3243276e-09','1.0391831e-21','4.0659815e-22','3.1260245e-23','206.83948','-3.1901815e-11','524.8536'],
['Belgium','201908','0.00062029494','7.7216833e-10','2.454771e-10','1.0413326e-10','141.0103','0.99999434','315.69717','1.6011281e-09','4.704406e-22','4.449455e-22','4.543436e-23','3087.9834','-3.1901815e-11','404.3866'],
['Croatia','201908','0.0005926374','3.500363e-11','2.1707008e-10','9.452928e-11','106.56975','0.99441296','300.81094','1.7923824e-09','2.1889786e-24','3.5503136e-22','4.3731697e-23','360.49997','4.4172097e-09','253.64354'],
['Austria','201909','0.00051349535','8.065418e-11','1.5390444e-10','5.9458306e-11','96.97897','0.9999845','281.8336','1.528435e-09','2.4866723e-23','6.2797094e-22','6.597012e-23','217.30652','-2.3971736e-10','198.75737'],
['Bulgaria','201909','0.0006034375','2.438405e-10','2.0121112e-10','8.215965e-11','115.10801','0.9999845','295.8326','2.6053288e-09','1.105622e-19','4.598967e-22','4.9422836e-23','738.36694','-2.3971736e-10','108.45153'],
['Cyprus','201909','0.000559932','1.5876801e-10','2.0060013e-10','1.0622498e-10','87.7546','0.9167146','281.45612','4.4773996e-10','2.7996036e-23','2.689002e-22','7.9139744e-23','55.03219','1.2478478e-08','24.387651'],
['Czech Republic','201909','0.00060082576','2.4050262e-10','1.8050954e-10','7.031597e-11','121.30631','0.9999845','285.47195','2.0050028e-09','1.7418e-22','1.0855461e-21','1.3850042e-22','1010.4344','-2.3971736e-10','221.46584'],
['Estonia','201909','0.0005999294','5.386814e-11','1.0320343e-10','3.7511456e-11','87.73275','0.9999845','288.21384','1.29638e-09','1.4816495e-22','3.2817952e-22','8.053418e-23','81.14869','-2.3971736e-10','339.65167'],
['France','201909','0.0005347416','1.0990616e-10','1.3079314e-10','5.767589e-11','88.0431','0.9999845','286.94745','8.87739e-10','2.9150392e-22','2.297614e-22','3.9303565e-23','106.39496','-2.3971736e-10','393.74982'],
['Germany','201909','0.00058742217','1.7618997e-10','1.5810889e-10','6.237336e-11','109.0433','0.9999845','284.2804','1.7597851e-09','3.432312e-22','8.678487e-22','1.1497771e-22','495.85422','-2.3971736e-10','335.46866'],
['Greece','201909','0.00055778574','1.1241094e-10','1.8634806e-10','7.8407475e-11','100.96304','0.9999845','295.53906','1.3183075e-09','5.253053e-22','3.186818e-22','1.1738997e-23','170.766','-2.3971736e-10','89.34543'],
['Hungary','201909','0.00062819227','2.7085226e-10','2.0444393e-10','8.2698834e-11','122.17969','0.9999845','287.91864','1.9789714e-09','1.2342977e-22','9.305343e-22','1.5538154e-22','744.5247','-2.3971736e-10','142.79433'],
['Latvia','201909','0.0006035027','1.541082e-10','1.1379885e-10','4.223748e-11','93.325134','0.7289518','287.45264','1.4229825e-09','1.1134225e-22','3.951769e-22','9.5265203e-23','111.66043','6.591349e-09','340.71664'],
['Lithuania','201909','0.0006015139','1.6381137e-10','1.338021e-10','5.1156343e-11','98.05671','0.9999845','287.48962','1.7811538e-09','1.6285256e-22','5.8393716e-22','1.228713e-22','197.2136','-2.3971736e-10','343.4643'],
['Luxembourg','201909','0.0005678846','2.8091385e-10','1.52342e-10','6.317517e-11','116.092445','0.9999845','284.94296','1.310157e-09','1.537724e-22','6.0710437e-22','9.84016e-23','1253.8264','-2.3971736e-10','430.98065'],
['Malta','201909','0.0005795307','1.7670013e-11','2.1032596e-10','1.0770291e-10','83.06216','0.0','292.68332','4.141202e-10','7.012097e-30','3.0705862e-22','4.4236287e-23','42.759113','0.0','85.15443'],
['Netherlands','201909','0.0006077618','6.4019334e-10','1.5178668e-10','6.240417e-11','136.60202','0.9999845','285.13113','1.1247626e-09','5.2834783e-23','7.9689603e-22','1.3067158e-22','2947.7793','-2.3971736e-10','441.0909'],
['Poland','201909','0.0006138166','2.3797878e-10','1.7229874e-10','6.699329e-11','115.303085','0.9999845','288.78305','2.9764289e-09','1.668483e-22','1.2749149e-21','1.8784745e-22','907.2059','-2.3971736e-10','331.33197'],
['Portugal','201909','0.000572127','7.773137e-10','1.7098944e-10','8.35807e-11','105.08162','0.9999845','294.09192','1.6046108e-09','1.36737314e-17','3.8297016e-22','4.4800152e-23','3828.5085','-2.3971736e-10','168.4156'],
['Romania','201909','0.00057861744','1.13011114e-10','1.6656056e-10','6.55605e-11','105.17496','0.9999845','290.30988','1.4505759e-09','2.8000473e-22','2.5888095e-22','5.6686327e-23','304.8441','-2.3971736e-10','157.42761'],
['Slovakia','201909','0.0005891994','1.0267197e-10','1.7850652e-10','7.0576205e-11','104.46196','0.9999845','287.88647','1.8601044e-09','6.4086717e-22','8.498474e-22','1.3783604e-22','335.333','-2.3971736e-10','182.743'],
['Slovenia','201909','0.0005812231','1.7038475e-10','1.7750253e-10','7.158774e-11','114.7197','0.9999845','286.7336','1.4878228e-09','2.754012e-25','4.2191242e-22','4.8076153e-23','463.30038','-2.3971736e-10','219.89282'],
['Spain','201909','0.00054177677','1.3244075e-09','1.8437181e-10','8.354075e-11','142.88475','0.9999845','294.59296','1.049082e-09','9.320134e-22','3.7672766e-22','5.0134354e-23','2869.753','-2.3971736e-10','259.35016'],
['Sweden','201909','0.0006070881','2.5107265e-11','9.250256e-11','3.3870247e-11','95.12667','0.77349526','288.49036','1.5277596e-09','2.4252008e-24','3.4861566e-22','5.9883555e-23','427.39185','-4.511296e-09','368.16782'],
['Ireland','201909','0.00057487143','2.9393023e-11','1.0953034e-10','4.5630943e-11','81.6709','0.9999845','285.12955','1.3954276e-09','4.234626e-24','4.619557e-22','7.6196245e-23','74.68738','-2.3971736e-10','361.31827'],
['Italy','201909','0.0005989656','6.4487565e-10','2.0792172e-10','9.2203724e-11','128.5133','0.9999845','293.20648','1.3847333e-09','9.102521e-23','5.5032774e-22','4.7570594e-23','1111.1251','-2.3971736e-10','211.28421'],
['Denmark','201909','0.0005947461','1.13663155e-10','1.1807019e-10','4.57876e-11','87.25501','0.9999845','289.587','2.0668829e-09','3.646277e-24','6.320875e-22','8.4464643e-23','153.32343','-2.3971736e-10','358.54745'],
['Finland','201909','0.00059443','9.931777e-11','8.271434e-11','2.9047716e-11','93.66274','0.9999845','287.57925','1.2052993e-09','5.069019e-22','2.4113943e-22','5.631141e-23','104.75396','-2.3971736e-10','236.12146'],
['Belgium','201909','0.0005918922','7.6020873e-10','1.6125225e-10','6.771035e-11','136.97348','0.9999845','285.51108','1.3118394e-09','8.996169e-23','7.210305e-22','1.1232943e-22','2901.8594','-2.3971736e-10','432.29514'],
['Croatia','201909','0.0005741113','3.1289932e-11','1.756934e-10','7.2799315e-11','99.93069','0.9944042','288.59543','1.5045399e-09','8.76336e-25','4.442262e-22','4.8602888e-23','216.95546','1.5990477e-09','238.3108'],
['Austria','201910','0.000480587','6.216882e-11','8.1467895e-11','3.240393e-11','92.84564','0.9999807','270.06226','9.899578e-10','1.6659504e-23','3.4987713e-22','3.8718423e-23','393.77548','-3.7254133e-10','234.6857'],
['Bulgaria','201910','0.00056303723','1.241505e-10','1.422146e-10','6.009135e-11','110.62269','0.9999807','281.24298','1.9641089e-09','1.8915583e-21','4.2486685e-22','7.090317e-23','666.0137','-3.7254133e-10','255.59991'],
['Cyprus','201910','0.0005387126','1.642308e-10','1.6117975e-10','8.0254164e-11','85.812775','0.9166989','284.8737','2.6853486e-10','6.153345e-21','1.9517071e-22','1.191953e-22','157.88765','3.769844e-09','73.36355'],
['Czech Republic','201910','0.0005667774','1.922316e-10','8.918901e-11','3.4710207e-11','120.43534','0.9999807','271.11276','7.6008316e-10','9.677757e-23','4.874606e-22','5.533055e-23','731.98096','-3.7254133e-10','286.09967'],
['Estonia','201910','0.00059011806','2.2677086e-11','4.9880235e-11','1.6655548e-11','91.97474','0.9999807','285.17114','8.971082e-10','2.2577259e-23','1.4458898e-22','1.507301e-23','208.9518','-3.7254133e-10','711.4391'],
['France','201910','0.0005252554','5.997348e-11','8.998054e-11','3.7927314e-11','82.184685','0.9999807','276.18878','6.1889616e-10','1.147744e-22','1.7922284e-22','3.368694e-23','55.22475','-3.7254133e-10','371.5129'],
['Germany','201910','0.0005651724','1.2407543e-10','7.944252e-11','3.0146066e-11','110.61005','0.9999807','272.056','6.99477e-10','1.1013733e-22','3.7469924e-22','4.358222e-23','338.07404','-3.7254133e-10','333.1036'],
['Greece','201910','0.00050370884','7.127901e-11','1.3358323e-10','5.938915e-11','87.4836','0.9999807','286.02744','7.0172296e-10','2.6361962e-22','3.3185599e-22','5.1434065e-23','124.08621','-3.7254133e-10','144.09282'],
['Hungary','201910','0.0005985422','2.3840083e-10','1.14888155e-10','4.6115514e-11','138.24763','0.9999807','273.9069','1.2076184e-09','1.8714035e-22','4.9928534e-22','7.0198996e-23','4038.2458','-3.7254133e-10','253.70317'],
['Latvia','201910','0.00059235905','1.2604387e-10','5.6028418e-11','1.924243e-11','99.81208','0.7288993','283.96115','5.4293736e-10','2.038343e-23','1.7869795e-22','1.9208747e-23','260.06247','3.5531165e-09','600.2692'],
['Lithuania','201910','0.00059503753','1.297827e-10','6.7593854e-11','2.38965e-11','104.44655','0.9999807','278.31213','4.997749e-10','3.5619026e-23','2.6841827e-22','2.882441e-23','271.89993','-3.7254133e-10','552.47144'],
['Luxembourg','201910','0.00055892405','2.4841978e-10','8.174888e-11','3.1745228e-11','129.26312','0.9999807','272.8129','7.5137896e-10','4.6429797e-23','3.2152948e-22','4.6847802e-23','1456.2213','-3.7254133e-10','381.2528'],
['Malta','201910','0.0005602094','1.7664092e-11','1.4778281e-10','7.207755e-11','74.9296','0.0','289.34784','5.175313e-10','0.0','4.3037666e-22','9.696134e-23','33.534786','0.0','142.8895'],
['Netherlands','201910','0.0006141229','6.246322e-10','8.078355e-11','3.0090992e-11','155.30595','0.9999807','277.6103','1.2366397e-09','1.1347373e-23','3.2903711e-22','4.5081164e-23','3501.5916','-3.7254133e-10','459.04413'],
['Poland','201910','0.00059129077','1.9713664e-10','7.897887e-11','2.9520292e-11','124.93492','0.9999807','273.59222','6.1871047e-10','6.4292643e-23','4.1835652e-22','5.1724953e-23','1178.2203','-3.7254133e-10','444.05258'],
['Portugal','201910','0.0005511778','7.496893e-11','1.1650265e-10','5.6010502e-11','84.63802','0.9999807','283.00485','7.2284784e-10','6.665723e-22','2.5241078e-22','1.0902094e-22','112.57717','-3.7254133e-10','613.69104'],
['Romania','201910','0.00054745236','7.295589e-11','1.07767635e-10','4.330547e-11','106.19682','0.9999807','272.08658','1.2626674e-09','5.1648143e-23','3.777412e-22','5.0696563e-23','250.46112','-3.7254133e-10','270.8166'],
['Slovakia','201910','0.0005582216','5.8076e-11','9.6432154e-11','3.800253e-11','111.192184','0.9999807','272.05215','6.359981e-10','6.693133e-23','4.1564554e-22','5.0092226e-23','529.52496','-3.7254133e-10','264.7288'],
['Slovenia','201910','0.00055016804','1.6878095e-10','1.0893411e-10','4.4974663e-11','109.74003','0.9999807','274.18362','1.4953282e-09','1.1021633e-25','4.550415e-22','5.227574e-23','348.94696','-3.7254133e-10','229.0522'],
['Spain','201910','0.000520927','1.2818148e-09','1.2383186e-10','5.525007e-11','145.95323','0.9999807','281.49054','4.599322e-10','1.8406574e-22','2.7583942e-22','1.0984124e-22','2652.05','-3.7254133e-10','695.31635'],
['Sweden','201910','0.00059163786','2.1188842e-11','4.0463532e-11','1.350763e-11','96.72381','0.7735074','284.6416','7.9016665e-10','3.168047e-25','1.3275078e-22','1.2927607e-23','472.6384','3.1449487e-09','706.11285'],
['Ireland','201910','0.0005731896','2.1431116e-11','5.1401276e-11','1.9142864e-11','81.70812','0.9999807','287.1301','7.866587e-10','3.8746244e-24','1.7360156e-22','3.5604085e-23','109.522705','-3.7254133e-10','856.0068'],
['Italy','201910','0.00056959584','6.131629e-10','1.3819411e-10','6.148777e-11','126.54998','0.9999807','280.6766','9.031683e-10','8.16791e-23','4.954349e-22','7.830028e-23','898.21246','-3.7254133e-10','128.84158'],
['Denmark','201910','0.0005895696','1.0717918e-10','5.7369186e-11','2.0188398e-11','94.24406','0.9999807','285.99194','7.9873475e-10','1.3932867e-24','1.8863299e-22','2.2412105e-23','364.7354','-3.7254133e-10','443.31097'],
['Finland','201910','0.0005787967','5.0066215e-11','3.488344e-11','1.1070468e-11','90.64361','0.9999807','281.78436','7.836873e-10','4.550105e-23','1.1520587e-22','1.0799534e-23','144.22635','-3.7254133e-10','814.88'],
['Belgium','201910','0.0005905954','7.306176e-10','8.8121094e-11','3.3994113e-11','136.56361','0.9999807','275.32553','1.3725981e-09','3.6226487e-23','3.6144317e-22','5.570918e-23','2406.16','-3.7254133e-10','465.97748'],
['Croatia','201910','0.00054371665','2.8415931e-11','1.086065e-10','4.5714144e-11','95.55566','0.99439925','275.55753','1.2736082e-09','6.7481647e-25','4.3138226e-22','5.087064e-23','174.50496','9.015707e-09','202.31454'],
['Austria','201911','0.0005017847','5.333855e-11','4.114505e-11','1.2888805e-11','103.15931','0.9999681','290.10767','4.698612e-10','1.1721015e-24','9.44975e-23','1.1542327e-23','371.31','-1.0160097e-09','500.7261'],
['Bulgaria','201911','0.0005596251','8.4940374e-11','7.1621424e-11','2.742949e-11','97.417336','0.9999681','285.58344','5.9135175e-10','1.1179802e-22','2.781623e-22','5.563061e-23','108.77751','-1.0160097e-09','327.7305'],
['Cyprus','201911','0.0005549344','1.5260468e-10','1.01776185e-10','4.4931735e-11','92.742096','0.91665953','283.55334','2.1954338e-10','1.1011737e-22','2.8674786e-22','8.0592775e-23','105.9708','4.667812e-09','342.32156'],
['Czech Republic','201911','0.0005963228','1.7213893e-10','4.3299996e-11','1.3687031e-11','129.82626','0.9999681','287.1738','7.2610473e-10','5.348272e-24','1.1274724e-22','1.24580314e-23','479.79672','-1.0160097e-09','389.5999'],
['Estonia','201911','0.0006394468','1.6524145e-11','2.0696656e-11','5.9678863e-12','111.04735','0.9999681','264.48352','9.498977e-10','2.6231306e-25','5.0149052e-23','3.671142e-24','266.64572','-1.0160097e-09','411.89066'],
['France','201911','0.0005470881','3.5146285e-11','5.0531447e-11','1.6375033e-11','88.22827','0.9999681','316.09482','8.356255e-10','1.4194258e-23','1.187293e-22','1.5858673e-23','76.0512','-1.0160097e-09','1833.0665'],
['Germany','201911','0.00059920904','1.02746374e-10','3.7844915e-11','1.1430741e-11','125.86367','0.9999681','294.88046','1.2981515e-09','6.312957e-24','9.110762e-23','9.184453e-24','730.52893','-1.0160097e-09','520.5342'],
['Greece','201911','0.00051268033','4.7679173e-11','7.7567126e-11','3.0161162e-11','84.23834','0.9999681','289.7121','5.446315e-10','7.90927e-24','2.699808e-22','4.5813498e-23','81.610756','-1.0160097e-09','343.08252'],
['Hungary','201911','0.0006083968','2.2813167e-10','5.7414296e-11','1.9772736e-11','122.36508','0.9999681','285.09915','6.697451e-10','1.5208371e-21','1.7598555e-22','2.5055537e-23','382.86914','-1.0160097e-09','352.09195'],
['Latvia','201911','0.00063876004','1.1944887e-10','2.3999781e-11','7.1278153e-12','118.69616','0.7289144','267.54953','1.315756e-09','3.6845797e-25','6.317458e-23','4.9386215e-24','251.67865','7.964909e-10','339.9688'],
['Lithuania','201911','0.000628282','1.191097e-10','2.987576e-11','9.226137e-12','119.47827','0.9999681','270.71698','9.793601e-10','1.26438e-24','7.7357594e-23','7.220748e-24','263.40433','-1.0160097e-09','276.53207'],
['Luxembourg','201911','0.00058906846','2.3546295e-10','4.112248e-11','1.2553912e-11','150.12202','0.9999681','305.30615','1.0120362e-09','4.7411414e-23','8.864995e-23','9.6074344e-24','2735.1987','-1.0160097e-09','824.908'],
['Malta','201911','0.0005763601','1.7656987e-11','9.55855e-11','3.9927797e-11','80.50558','0.0','301.7279','4.4549459e-10','0.0','2.1787654e-22','5.2504875e-23','37.111294','0.0','290.68164'],
['Netherlands','201911','0.00064036023','6.168117e-10','3.6823964e-11','1.1264139e-11','184.96213','0.9999681','302.00662','1.3433293e-09','1.0225446e-24','9.634408e-23','1.0011696e-23','4646.4556','-1.0160097e-09','694.78265'],
['Poland','201911','0.0006199228','1.8280086e-10','3.6881637e-11','1.1611195e-11','134.55028','0.9999681','279.54977','8.3732465e-10','2.369255e-24','8.58287e-23','8.7085795e-24','555.8637','-1.0160097e-09','283.69913'],
['Portugal','201911','0.0005667187','4.725739e-11','7.354976e-11','2.9209152e-11','85.89559','0.9999681','301.14856','8.738038e-10','7.0389686e-23','1.4698002e-22','3.1824788e-23','93.314995','-1.0160097e-09','1324.384'],
['Romania','201911','0.0005483601','5.4454337e-11','5.1256832e-11','1.815353e-11','98.856895','0.9999681','278.64056','6.298349e-10','6.919193e-24','1.7192207e-22','2.7939885e-23','116.39597','-1.0160097e-09','315.6243'],
['Slovakia','201911','0.0005834754','4.0796363e-11','5.0293318e-11','1.6521725e-11','122.6214','0.9999681','281.40164','6.787723e-10','3.9566277e-24','1.312196e-22','1.6033653e-23','430.22455','-1.0160097e-09','324.44257'],
['Slovenia','201911','0.00057058735','1.6807272e-10','5.727598e-11','1.9149841e-11','122.31284','0.9999681','292.4658','5.9553773e-10','5.5324884e-27','1.6059495e-22','2.3463644e-23','472.77667','-1.0160097e-09','524.1495'],
['Spain','201911','0.0005312586','1.276515e-09','7.6527895e-11','2.7427002e-11','130.70549','0.9999681','309.06406','5.592611e-10','4.86476e-21','1.6894081e-22','3.2313794e-23','1054.1875','-1.0160097e-09','1870.3607'],
['Sweden','201911','0.00064475543','2.0606213e-11','1.6566444e-11','4.702162e-12','115.268555','0.7735036','269.5761','1.1906881e-09','1.3393666e-27','2.3104893e-23','2.01597e-24','750.4108','1.01427755e-08','548.7025'],
['Ireland','201911','0.00060329045','1.6733755e-11','2.1818011e-11','6.7233784e-12','95.06222','0.9999681','309.1367','1.1046509e-09','3.4557723e-25','3.027284e-23','4.04378e-24','266.14462','-1.0160097e-09','1269.5023'],
['Italy','201911','0.00057867076','5.9216443e-10','7.7460975e-11','2.8414054e-11','109.60629','0.9999681','303.43048','6.3860217e-10','1.1482244e-23','2.4359603e-22','4.2874407e-23','387.57874','-1.0160097e-09','645.6116'],
['Denmark','201911','0.0006420353','1.0523801e-10','2.5161291e-11','7.326328e-12','123.84401','0.9999681','282.94272','1.0404821e-09','3.9416736e-26','5.147471e-23','4.7695327e-24','817.68335','-1.0160097e-09','603.4937'],
['Finland','201911','0.00063265784','4.3020046e-11','1.2034982e-11','3.2982373e-12','111.395485','0.9999681','264.45898','1.2274601e-09','1.7502104e-25','1.8606419e-23','1.4542156e-24','435.71884','-1.0160097e-09','715.1465'],
['Belgium','201911','0.00061453966','7.1819767e-10','4.374905e-11','1.3591265e-11','149.43951','0.9999681','307.59128','1.1945838e-09','2.3999726e-24','1.0179511e-22','1.1142697e-23','3169.1707','-1.0160097e-09','844.6008'],
['Croatia','201911','0.0005638423','2.7133771e-11','5.964554e-11','2.038732e-11','99.449066','0.99438506','293.92102','6.749424e-10','3.2619564e-24','1.5312532e-22','2.3108347e-23','317.60864','8.1332825e-09','513.33887'],
['Austria','201912','0.00050676154','5.22854e-11','2.1955087e-11','6.5111193e-12','94.09773','0.9999901','303.15875','9.925424e-10','5.5108167e-27','1.6781714e-23','1.2222162e-24','199.79745','-9.795731e-11','1131.6163'],
['Bulgaria','201912','0.00059067813','8.2691055e-11','4.323098e-11','1.3757787e-11','109.087845','0.9999901','317.1947','2.03489e-09','1.4176215e-22','4.6549196e-23','4.1548444e-24','266.39346','-9.795731e-11','1704.4373'],
['Cyprus','201912','0.0005898614','1.4961149e-10','6.78706e-11','2.4689552e-11','96.49905','0.91668594','313.56216','1.6854587e-09','8.765546e-24','6.482309e-23','1.2251479e-23','139.13211','7.0914132e-09','634.27374'],
['Czech Republic','201912','0.0005993628','1.7018736e-10','2.2469847e-11','6.7280834e-12','128.0687','0.9999901','305.05118','1.2081305e-09','8.4393834e-27','1.6646634e-23','1.2873935e-24','478.68774','-9.795731e-11','1556.0161'],
['Estonia','201912','0.00062795315','1.6196936e-11','8.425436e-12','2.3025791e-12','111.22042','0.9999901','295.54395','1.0284684e-09','2.563757e-29','2.5111622e-24','1.842997e-25','949.85223','-9.795731e-11','2080.397'],
['France','201912','0.00054652616','2.8621808e-11','3.025098e-11','1.0270268e-11','90.778076','0.9999901','301.65536','8.0829365e-10','4.4642354e-25','1.7667365e-23','1.3310418e-24','195.90414','-9.795731e-11','680.5837'],
['Germany','201912','0.00059206714','1.0050925e-10','2.0617575e-11','6.2955617e-12','123.865524','0.9999901','302.89954','1.1854926e-09','1.3213855e-26','8.757583e-24','7.522411e-25','830.19794','-9.795731e-11','1532.7576'],
['Greece','201912','0.0005379474','4.2115013e-11','4.883105e-11','1.6483592e-11','95.19104','0.9999901','319.10626','1.3560651e-09','7.026993e-25','4.2000592e-23','7.500828e-24','217.36913','-9.795731e-11','1603.5747'],
['Hungary','201912','0.000630474','2.1655142e-10','3.0883362e-11','9.361621e-12','132.45305','0.9999901','310.82642','1.7772098e-09','1.5775932e-26','3.8387502e-23','2.5283954e-24','537.4171','-9.795731e-11','1550.4447'],
['Latvia','201912','0.00062829343','1.1904112e-10','1.0102145e-11','2.7987675e-12','117.184494','0.7289044','296.19656','8.841993e-10','5.0434563e-28','3.7373985e-24','2.835834e-25','675.4648','1.1683833e-09','2012.9633'],
['Lithuania','201912','0.0006267884','1.1838651e-10','1.36754635e-11','3.844796e-12','122.42697','0.9999901','297.48724','7.775032e-10','7.0608386e-28','6.5397855e-24','4.934051e-25','656.2761','-9.795731e-11','1954.1519'],
['Luxembourg','201912','0.00058415096','2.3181113e-10','2.3045474e-11','7.272556e-12','139.23773','0.9999901','301.62286','8.2921336e-10','1.311971e-26','9.926072e-24','1.0446253e-24','1546.9924','-9.795731e-11','1417.5002'],
['Malta','201912','0.00059093355','1.7656987e-11','6.040211e-11','2.39393e-11','84.15202','0.0','310.78055','1.9659743e-09','0.0','5.070045e-23','1.2837375e-23','79.85384','0.0','1093.5322'],
['Netherlands','201912','0.0006343172','6.157573e-10','1.9390666e-11','6.0478077e-12','175.62459','0.9999901','305.23593','1.2785997e-09','5.8100615e-27','7.359097e-24','8.868344e-25','5673.321','-9.795731e-11','1896.7396'],
['Poland','201912','0.0006237558','1.8160372e-10','1.8845297e-11','5.499563e-12','133.3851','0.9999901','304.90283','1.0084861e-09','3.2143628e-27','1.0873516e-23','8.1673176e-25','488.76068','-9.795731e-11','1938.6039'],
['Portugal','201912','0.0005633856','3.6577498e-11','5.1009016e-11','2.0144763e-11','91.44776','0.9999901','299.29315','9.298314e-10','1.2593627e-23','3.6694512e-23','2.1646732e-24','136.39409','-9.795731e-11','435.52393'],
['Romania','201912','0.000575244','5.2474976e-11','2.904806e-11','8.61871e-12','110.96011','0.9999901','310.66568','2.0796533e-09','5.7303136e-24','2.5887461e-23','1.6876263e-24','319.9565','-9.795731e-11','1711.986'],
['Slovakia','201912','0.00059666374','3.907985e-11','2.6338218e-11','7.7586566e-12','125.177605','0.9999901','307.47635','1.3629244e-09','8.6381794e-27','2.9900763e-23','1.856588e-24','499.7114','-9.795731e-11','1730.0962'],
['Slovenia','201912','0.0005806958','1.6801631e-10','3.1780356e-11','9.943409e-12','118.7915','0.9999901','307.53033','1.2605025e-09','5.6486707e-28','3.2366884e-23','2.3964036e-24','463.48434','-9.795731e-11','1124.8247'],
['Spain','201912','0.00053751','1.2573049e-09','4.7712806e-11','1.7026613e-11','162.71191','0.9999901','299.7341','8.170523e-10','1.0885819e-23','5.4803265e-23','2.4293833e-24','3585.084','-9.795731e-11','341.57428'],
['Sweden','201912','0.0006338083','2.0584423e-11','7.193869e-12','2.0053271e-12','118.454926','0.7734981','300.024','1.2228455e-09','0.0','1.3414702e-24','1.2303477e-25','870.8996','7.499044e-09','2193.2512'],
['Ireland','201912','0.00059528','1.6393368e-11','1.2706236e-11','3.9388497e-12','87.18663','0.9999901','317.7729','9.775818e-10','1.7240845e-26','2.2610505e-24','3.6754472e-25','40.018753','-9.795731e-11','3453.5203'],
['Italy','201912','0.0005937425','5.8536986e-10','4.5315716e-11','1.585457e-11','122.35782','0.9999901','308.5909','1.2509411e-09','8.067378e-25','4.266293e-23','4.813596e-24','573.0915','-9.795731e-11','832.6062'],
['Denmark','201912','0.00062424614','1.0510348e-10','1.2075224e-11','3.5790442e-12','109.668594','0.9999901','304.63553','1.1891731e-09','0.0','2.5034883e-24','3.4158634e-25','693.89874','-9.795731e-11','2538.0256'],
['Finland','201912','0.0006243573','4.4222345e-11','4.8677538e-12','1.2987358e-12','108.377495','0.9999901','293.16922','1.1763791e-09','1.742325e-25','5.667671e-25','3.5471485e-26','378.60477','-9.795731e-11','1424.2008'],
['Belgium','201912','0.0006121784','7.1655715e-10','2.468977e-11','7.922648e-12','148.9037','0.9999901','303.07193','1.1989069e-09','1.9474466e-26','1.1849963e-23','1.2608282e-24','3179.4727','-9.795731e-11','1585.1617'],
['Croatia','201912','0.00057853863','2.6613493e-11','3.4366968e-11','1.0911432e-11','102.87275','0.99440145','307.8928','1.150772e-09','7.5525757e-28','3.335941e-23','2.5684092e-24','211.49164','1.058622e-08','1117.0868'],
['Austria','202001','0.0005108134','5.2215263e-11','2.0894501e-11','6.1027945e-12','94.10666','0.99998736','329.84378','6.668164e-10','6.9557666e-28','4.825774e-24','2.3209792e-25','246.057','-1.5990198e-10','1020.7703'],
['Bulgaria','202001','0.0005994687','1.03055245e-10','3.888469e-11','1.2051489e-11','119.63242','0.99998736','345.57684','6.3870775e-10','2.4680638e-21','9.446011e-24','7.835629e-25','369.95065','-1.5990198e-10','865.55585'],
['Cyprus','202001','0.0006096688','1.4875685e-10','6.085203e-11','1.9983782e-11','96.10189','0.916715','340.32462','1.1790313e-09','1.5972471e-25','1.2558588e-23','1.0727208e-24','85.37678','7.0051076e-09','670.23004'],
['Czech Republic','202001','0.0006089574','1.7012135e-10','2.1476296e-11','6.282355e-12','137.77682','0.99998736','327.5953','7.270158e-10','5.0434563e-28','5.413099e-24','2.7197355e-25','695.09216','-1.5990198e-10','1241.62'],
['Estonia','202001','0.0006352504','1.6174933e-11','9.329702e-12','2.5017373e-12','101.10422','0.99998736','315.14902','1.2592865e-09','2.6961476e-28','3.720947e-24','1.0741582e-25','356.5671','-1.5990198e-10','2563.5242'],
['France','202001','0.00056304294','2.855465e-11','3.4737067e-11','1.109514e-11','91.47563','0.99998736','334.94977','8.7360563e-10','9.4196525e-26','1.3187629e-23','5.221316e-25','81.04796','-1.5990198e-10','1297.2703'],
['Germany','202001','0.0006045874','1.00464324e-10','2.1949905e-11','6.4752062e-12','128.83269','0.99998736','323.70914','8.381959e-10','8.708368e-28','7.954974e-24','4.544227e-25','920.7709','-1.5990198e-10','1505.544'],
['Greece','202001','0.0005563121','4.362381e-11','4.4110847e-11','1.4135499e-11','99.91334','0.99998736','342.1663','7.321291e-10','8.562415e-23','1.07198695e-23','1.5086279e-24','148.63614','-1.5990198e-10','839.5597'],
['Hungary','202001','0.0006492343','2.1664119e-10','2.819083e-11','8.323732e-12','152.43665','0.99998736','336.80402','1.027516e-09','6.600521e-25','4.926633e-24','2.6844513e-25','1081.5403','-1.5990198e-10','883.76337'],
['Latvia','202001','0.0006399848','1.1901728e-10','1.0692899e-11','2.8859473e-12','112.04935','0.7288971','316.61444','1.1673383e-09','5.6486707e-28','3.382065e-24','1.0742026e-25','563.2664','3.2854368e-09','2089.3743'],
['Lithuania','202001','0.0006375066','1.1830766e-10','1.3666415e-11','3.7621494e-12','120.727776','0.99998736','321.0387','1.0187164e-09','2.1249762e-27','3.063869e-24','1.1176302e-25','731.30865','-1.5990198e-10','1641.375'],
['Luxembourg','202001','0.0005984692','2.3314528e-10','2.5798488e-11','7.795856e-12','147.79573','0.99998736','324.6683','1.0322307e-09','1.168724e-23','1.0447551e-23','6.228497e-25','1980.5261','-1.5990198e-10','1528.4839'],
['Malta','202001','0.0006199527','1.7656987e-11','6.0389554e-11','2.188456e-11','88.46503','0.0','329.16245','6.554695e-10','0.0','8.16888e-24','1.5468323e-24','52.15674','0.0','542.0601'],
['Netherlands','202001','0.0006483474','6.1583794e-10','2.283266e-11','6.844691e-12','168.56808','0.99998736','321.8078','1.6601779e-09','2.2917465e-26','1.1555383e-23','6.9941848e-25','4468.5635','-1.5990198e-10','1708.3358'],
['Poland','202001','0.0006331562','1.8155788e-10','1.8339863e-11','5.322568e-12','135.17389','0.99998736','324.16287','7.521885e-10','0.0','3.2715452e-24','1.453895e-25','643.06134','-1.5990198e-10','1239.3175'],
['Portugal','202001','0.000589209','3.6625498e-11','5.513543e-11','2.0632754e-11','91.72387','0.99998736','324.02704','8.508269e-10','2.3957228e-24','1.7808016e-23','1.7328265e-24','75.02106','-1.5990198e-10','1173.627'],
['Romania','202001','0.000580745','5.171376e-11','2.5701868e-11','7.527923e-12','111.35303','0.99998736','342.57327','6.150831e-10','1.5828046e-27','4.5472763e-24','2.201954e-25','272.7181','-1.5990198e-10','1049.454'],
['Slovakia','202001','0.00060421304','3.9045947e-11','2.3480018e-11','6.907766e-12','124.17216','0.99998736','333.21304','8.6598895e-10','2.0657444e-26','3.7905496e-24','1.7750142e-25','514.0281','-1.5990198e-10','1037.3179'],
['Slovenia','202001','0.00059865025','1.7207122e-10','3.042035e-11','9.210374e-12','138.75339','0.99998736','335.87048','7.94549e-10','3.1225205e-22','8.285108e-24','4.199954e-25','760.988','-1.5990198e-10','839.96234'],
['Spain','202001','0.0005593216','1.2569877e-09','4.978604e-11','1.6868186e-11','170.98575','0.99998736','328.2438','6.079648e-10','8.823044e-25','1.20219736e-23','1.2844931e-24','4645.605','-1.5990198e-10','1377.6803'],
['Sweden','202001','0.000639222','2.0584423e-11','7.994722e-12','2.159809e-12','112.06925','0.77349937','311.2936','1.4357486e-09','0.0','4.109978e-24','1.30791e-25','626.36505','3.5709427e-09','2602.9392'],
['Ireland','202001','0.0006072005','1.6966385e-11','1.8094744e-11','5.3322854e-12','84.49642','0.99998736','326.28638','7.675939e-10','7.821728e-26','1.0543375e-23','1.0760092e-24','21.64451','-1.5990198e-10','2622.9385'],
['Italy','202001','0.0006164636','5.844656e-10','4.9787893e-11','1.676404e-11','136.17766','0.99998736','336.71255','7.4604867e-10','2.0581999e-25','2.2975489e-23','1.6402949e-24','612.83844','-1.5990198e-10','618.30927'],
['Denmark','202001','0.0006332072','1.0511265e-10','1.3861795e-11','3.9740576e-12','102.66082','0.99998736','314.46436','1.2315906e-09','1.7652096e-28','6.366177e-24','3.4892617e-25','686.4032','-1.5990198e-10','1790.437'],
['Finland','202001','0.00062741496','4.4745856e-11','5.778729e-12','1.4822786e-12','101.172874','0.99998736','309.63522','1.4813789e-09','1.9689569e-25','2.761342e-24','6.568236e-26','405.2848','-1.5990198e-10','2854.2612'],
['Belgium','202001','0.000622739','7.168065e-10','2.8054298e-11','8.6544695e-12','141.77582','0.99998736','324.47104','1.0935269e-09','3.6151495e-26','1.3057037e-23','8.052848e-25','1649.4236','-1.5990198e-10','1628.9193'],
['Croatia','202001','0.0005898557','2.655092e-11','3.3943837e-11','1.0642991e-11','112.930305','0.99441546','337.00037','7.596115e-10','1.5697758e-28','1.0955687e-23','5.43167e-25','385.77838','1.3019071e-08','751.61316'],
['Austria','202002','0.00051569636','5.2495876e-11','3.313183e-11','9.695279e-12','93.33686','0.999988','333.92508','5.3129057e-10','4.320207e-26','5.1240668e-23','5.3142005e-24','150.47755','-1.4423995e-10','1119.3727'],
['Bulgaria','202002','0.00060794223','9.696487e-11','5.4707523e-11','1.7065616e-11','108.65516','0.999988','340.6351','7.4191014e-10','1.02513e-21','8.25407e-23','8.966627e-24','186.76288','-1.4423995e-10','1075.8206'],
['Cyprus','202002','0.0005979345','1.5034023e-10','6.994464e-11','2.4423e-11','94.15116','0.9167097','329.12418','5.9836025e-10','1.8668404e-23','3.29331e-23','6.1044e-24','63.528175','7.851479e-09','576.8538'],
['Czech Republic','202002','0.0006068899','1.703068e-10','3.555802e-11','1.03349005e-11','116.05101','0.999988','349.95694','9.116039e-10','1.0226801e-25','6.978784e-23','6.011482e-24','480.52206','-1.4423995e-10','1368.6333'],
['Estonia','202002','0.0006340541','1.6163378e-11','1.8323728e-11','4.779374e-12','96.97127','0.999988','369.25977','5.9010197e-10','3.612019e-27','2.2867927e-23','1.3618992e-24','113.41789','-1.4423995e-10','1971.3716'],
['France','202002','0.00056829007','3.1200178e-11','4.806399e-11','1.6096608e-11','86.42817','0.999988','313.28586','1.0030673e-09','3.4830473e-24','5.999698e-23','7.8702665e-24','36.174427','-1.4423995e-10','662.04553'],
['Germany','202002','0.00060443074','1.0110288e-10','3.776616e-11','1.0999301e-11','111.479195','0.999988','349.2726','1.037916e-09','4.0382187e-25','7.3094514e-23','6.0886904e-24','542.60455','-1.4423995e-10','1319.2948'],
['Greece','202002','0.0005548763','4.6418307e-11','5.9955124e-11','2.01222e-11','96.635735','0.999988','321.9615','8.1825297e-10','6.351116e-24','6.5934774e-23','8.460721e-24','85.406555','-1.4423995e-10','969.7544'],
['Hungary','202002','0.0006329402','2.2458566e-10','4.061052e-11','1.2370353e-11','116.29459','0.999988','345.36746','6.3022154e-10','8.801509e-22','7.212106e-23','7.1366566e-24','331.4398','-1.4423995e-10','1182.9172'],
['Latvia','202002','0.0006355969','1.1899484e-10','2.0006437e-11','5.316126e-12','101.54706','0.72889227','369.06564','5.758143e-10','5.2175202e-27','2.6098901e-23','1.7076364e-24','108.49864','2.6618738e-09','1818.2355'],
['Lithuania','202002','0.0006333121','1.1824215e-10','2.3745327e-11','6.451093e-12','103.28179','0.999988','368.01917','5.2167376e-10','1.464594e-26','3.4994974e-23','2.4741112e-24','124.89293','-1.4423995e-10','1711.6516'],
['Luxembourg','202002','0.0005925411','2.3514848e-10','4.1229343e-11','1.2538928e-11','113.28469','0.999988','339.58667','9.232407e-10','6.4963857e-23','6.2543556e-23','6.2163687e-24','841.84607','-1.4423995e-10','1109.6488'],
['Malta','202002','0.0006185896','1.7662132e-11','7.131527e-11','2.7423553e-11','90.07351','0.0','309.84976','9.966201e-10','1.00854135e-29','5.5449866e-23','7.590239e-24','40.047554','0.0','486.1558'],
['Netherlands','202002','0.0006391723','6.1666877e-10','3.9003845e-11','1.16505e-11','122.990036','0.999988','351.70602','1.3307925e-09','1.6659182e-25','6.424457e-23','5.831985e-24','1241.1011','-1.4423995e-10','1318.7195'],
['Poland','202002','0.0006300066','1.8185582e-10','2.949136e-11','8.330496e-12','115.34892','0.999988','358.5367','7.3525613e-10','1.7997756e-25','5.0922852e-23','3.7006617e-24','375.0072','-1.4423995e-10','1488.2372'],
['Portugal','202002','0.0005794586','4.7056838e-11','6.9525996e-11','2.9133602e-11','88.905174','0.999988','297.42282','6.8903255e-10','1.0184044e-22','8.4337416e-23','1.18197435e-23','85.70042','-1.4423995e-10','336.07867'],
['Romania','202002','0.000588228','9.4864276e-11','3.805672e-11','1.108127e-11','108.95084','0.999988','349.14374','4.746014e-10','2.0430408e-20','6.621579e-23','5.9048523e-24','546.4014','-1.4423995e-10','1134.1576'],
['Slovakia','202002','0.00060003897','3.9617986e-11','3.4496562e-11','1.00827445e-11','106.83383','0.999988','349.4082','6.169224e-10','8.700174e-25','5.472824e-23','4.8579518e-24','201.75333','-1.4423995e-10','1291.3329'],
['Slovenia','202002','0.0005940628','1.6799974e-10','4.63799e-11','1.44483635e-11','115.08473','0.999988','331.35495','6.605999e-10','1.4753519e-27','9.5320285e-23','1.10333115e-23','298.40063','-1.4423995e-10','1154.7236'],
['Spain','202002','0.0005575729','1.271309e-09','6.514852e-11','2.454756e-11','167.16133','0.999988','295.77658','6.246446e-10','1.6488492e-21','6.864519e-23','9.647551e-24','2911.1375','-1.4423995e-10','363.98505'],
['Sweden','202002','0.0006389175','2.0598145e-11','1.7607442e-11','4.5816064e-12','103.056175','0.77350825','361.93463','6.952454e-10','2.9920058e-28','2.4727604e-23','1.2927686e-24','296.6274','2.4605722e-09','2266.7083'],
['Ireland','202002','0.00061491854','1.8163188e-11','3.4889536e-11','1.0146908e-11','86.0009','0.999988','361.96317','9.788118e-10','1.7986866e-25','5.669649e-23','4.6928124e-24','31.040924','-1.4423995e-10','1618.3414'],
['Italy','202002','0.0006170282','5.8769983e-10','6.956644e-11','2.469562e-11','129.06483','0.999988','317.13934','8.479077e-10','2.0089195e-23','1.0339711e-22','1.3647137e-23','900.4781','-1.4423995e-10','828.60376'],
['Denmark','202002','0.0006282205','1.0513288e-10','2.7454166e-11','7.68577e-12','97.4721','0.999988','367.20108','8.8867363e-10','1.0853826e-27','5.329831e-23','3.6142735e-24','338.37674','-1.4423995e-10','1828.3018'],
['Finland','202002','0.00062539475','4.26948e-11','1.3148389e-11','3.2315203e-12','98.75297','0.999988','359.39053','6.2428357e-10','1.7006023e-25','1.3470332e-23','6.557815e-25','115.534454','-1.4423995e-10','2865.9834'],
['Belgium','202002','0.00062146276','7.180069e-10','4.4970107e-11','1.3848466e-11','118.51047','0.999988','342.24164','1.1973607e-09','3.8887815e-25','6.937795e-23','6.987994e-24','822.0768','-1.4423995e-10','1122.2325'],
['Croatia','202002','0.00059183035','7.496915e-11','5.01538e-11','1.6006519e-11','100.98191','0.9944145','328.05988','7.163756e-10','1.6113231e-20','8.969179e-23','1.0188841e-23','62.700043','1.2126867e-08','1162.3555'],
['Austria','202003','0.0005344829','5.4617356e-11','7.5286576e-11','2.1495137e-11','103.49644','1.0000052','368.80643','1.0625254e-09','3.1009834e-24','3.361721e-22','1.763671e-23','309.11826','-2.7424951e-11','1000.26074'],
['Bulgaria','202003','0.0006196222','1.0721414e-10','1.01270894e-10','3.1910814e-11','114.440384','1.0000052','357.67056','2.1725213e-09','2.1896065e-21','6.0142957e-22','2.2053992e-23','264.8557','-2.7424951e-11','678.9216'],
['Cyprus','202003','0.0005901816','1.5226702e-10','9.373295e-11','3.5142275e-11','99.361465','0.91663647','333.0212','8.425473e-10','5.6414086e-25','1.5079341e-22','1.2398577e-23','89.80399','-1.1927725e-09','627.65186'],
['Czech Republic','202003','0.00063208304','1.7636542e-10','8.345405e-11','2.4025587e-11','123.69528','1.0000052','377.7488','8.6980073e-10','3.3862978e-23','3.9156458e-22','2.5326965e-23','790.50775','-2.7424951e-11','1008.17773'],
['Estonia','202003','0.00064950815','1.7629941e-11','4.579821e-11','1.2757386e-11','99.62175','1.0000052','400.71527','7.69333e-10','2.4806834e-24','1.5734127e-22','1.2054174e-23','119.429016','-2.7424951e-11','1085.4487'],
['France','202003','0.0005849393','4.402496e-11','9.162027e-11','2.9567484e-11','96.99297','1.0000052','355.32196','1.1086252e-09','2.17117e-22','3.1170563e-22','2.2329416e-23','161.3972','-2.7424951e-11','764.9711'],
['Germany','202003','0.0006234188','1.08148275e-10','8.2775065e-11','2.444304e-11','114.72941','1.0000052','374.72095','7.759945e-10','3.9075464e-23','2.8623468e-22','2.4575849e-23','476.68552','-2.7424951e-11','1159.9879'],
['Greece','202003','0.00055689324','5.825016e-11','9.4220166e-11','3.171644e-11','99.121376','1.0000052','342.58182','9.555761e-10','1.522642e-22','2.5842278e-22','1.22901594e-23','135.46536','-2.7424951e-11','589.81525'],
['Hungary','202003','0.0006597979','2.4171495e-10','9.198492e-11','2.7672448e-11','125.49293','1.0000052','372.8745','1.1250294e-09','4.5842277e-21','5.488693e-22','3.2707678e-23','601.7064','-2.7424951e-11','912.7516'],
['Latvia','202003','0.00065203174','1.2116083e-10','5.0555227e-11','1.4227594e-11','104.1596','0.72889','396.22647','9.181951e-10','1.7359717e-23','2.118618e-22','1.532082e-23','157.40306','3.3437408e-10','1143.3752'],
['Lithuania','202003','0.0006552273','1.2587334e-10','6.059127e-11','1.7073447e-11','109.54144','1.0000052','390.8885','1.4610437e-09','2.764836e-22','3.1097952e-22','2.121897e-23','325.62173','-2.7424951e-11','1095.2524'],
['Luxembourg','202003','0.00061441556','2.3767624e-10','8.3865075e-11','2.5529835e-11','125.19392','1.0000052','368.5545','9.389209e-10','7.7537564e-24','2.7854593e-22','2.3710893e-23','1160.7385','-2.7424951e-11','1047.9547'],
['Malta','202003','0.00062272977','1.7665468e-11','1.0202708e-10','3.750693e-11','96.013535','0.0','339.42172','1.3816572e-09','7.8804005e-30','2.3847937e-22','1.398303e-23','104.767815','0.0','566.5366'],
['Netherlands','202003','0.00065937574','6.3346306e-10','8.205054e-11','2.5177133e-11','145.98163','1.0000052','372.82745','1.2002576e-09','2.0320158e-21','3.0872525e-22','2.8710987e-23','3217.6096','-2.7424951e-11','1410.1274'],
['Poland','202003','0.00065873016','1.8734857e-10','7.274581e-11','2.0490384e-11','126.30967','1.0000052','383.44977','1.9461448e-09','2.9057168e-23','4.5462935e-22','2.4721831e-23','1154.1914','-2.7424951e-11','1132.9135'],
['Portugal','202003','0.0005973568','6.5903887e-11','1.1329308e-10','4.3267945e-11','95.519646','1.0000052','331.66086','1.4359887e-09','1.2436926e-22','2.976464e-22','1.4916424e-23','148.83794','-2.7424951e-11','964.31024'],
['Romania','202003','0.00061329716','1.9616511e-10','8.524179e-11','2.5128724e-11','114.99646','1.0000052','361.17307','1.6412616e-09','1.2098315e-19','5.524353e-22','2.4387916e-23','433.18478','-2.7424951e-11','650.27313'],
['Slovakia','202003','0.00062192976','5.9148346e-11','7.982887e-11','2.3369357e-11','111.86549','1.0000052','378.10815','8.1182094e-10','1.2411893e-21','4.504557e-22','2.6403037e-23','332.81018','-2.7424951e-11','1080.5493'],
['Slovenia','202003','0.0006145928','1.6839917e-10','1.03128006e-10','3.082165e-11','116.54565','1.0000052','367.77014','1.3286946e-09','6.9881066e-25','5.195954e-22','2.953254e-23','257.79636','-2.7424951e-11','1167.6663'],
['Spain','202003','0.0005611335','1.2753841e-09','1.0659385e-10','3.717138e-11','144.84467','1.0000052','332.26334','1.2420501e-09','1.6398762e-22','3.333651e-22','1.5534276e-23','1866.0557','-2.7424951e-11','725.4726'],
['Sweden','202003','0.0006560432','2.0774894e-11','4.455829e-11','1.235041e-11','112.07341','0.7735182','402.5915','8.2645285e-10','2.1069144e-26','1.3653276e-22','1.2067488e-23','479.65222','5.657283e-09','1315.7181'],
['Ireland','202003','0.00063422165','2.0004987e-11','6.95179e-11','2.1619e-11','90.34649','1.0000052','374.3776','1.1035106e-09','5.5777115e-25','1.5619358e-22','1.7942371e-23','72.26943','-2.7424951e-11','1381.809'],
['Italy','202003','0.000637842','5.956848e-10','1.2193489e-10','3.9691927e-11','133.98419','1.0000052','355.60574','2.0025688e-09','8.6865576e-23','5.794232e-22','2.842509e-23','1024.3219','-2.7424951e-11','934.4797'],
['Denmark','202003','0.00064649584','1.0544913e-10','6.6831936e-11','1.9803846e-11','100.65015','1.0000052','389.87808','9.805764e-10','6.8328747e-26','2.6328267e-22','2.5308763e-23','182.44485','-2.7424951e-11','1654.911'],
['Finland','202003','0.0006450219','4.3503093e-11','3.483758e-11','9.221446e-12','103.45924','1.0000052','408.52353','8.3913165e-10','1.7777708e-24','1.1285284e-22','8.594371e-24','239.37318','-2.7424951e-11','1166.2125'],
['Belgium','202003','0.00064851856','7.244712e-10','9.034275e-11','2.7711163e-11','147.02972','1.0000052','369.8009','1.6461017e-09','4.1068742e-23','3.613069e-22','2.7757693e-23','4001.9814','-2.7424951e-11','1151.8047'],
['Croatia','202003','0.0006067695','4.0140613e-11','1.0122849e-10','3.1100404e-11','108.398926','0.99443','365.489','1.07603e-09','1.3964934e-21','4.05854e-22','2.53259e-23','182.02867','5.540997e-09','1164.7827'],
['Austria','202004','0.0005619976','6.4478436e-11','1.2943299e-10','4.1271323e-11','105.31211','1.0000144','328.71255','7.3581014e-10','1.5858575e-23','3.45673e-22','4.6788574e-23','185.3824','-2.0951307e-10','500.38773'],
['Bulgaria','202004','0.0006485659','1.5337215e-10','1.725211e-10','5.6246976e-11','109.518234','1.0000144','343.3135','9.415725e-10','2.359578e-20','5.6437053e-22','1.0803974e-22','193.72847','-2.0951307e-10','712.239'],
['Cyprus','202004','0.00061033416','1.5675307e-10','1.3895778e-10','5.3633067e-11','95.54319','0.91663367','346.92993','4.4471976e-10','1.3053475e-22','2.4214392e-22','5.784679e-23','84.4167','4.8323723e-10','247.44774'],
['Czech Republic','202004','0.0006446446','2.2269514e-10','1.4684685e-10','4.854357e-11','121.84662','1.0000144','332.79678','8.0101864e-10','7.1731245e-21','5.4146913e-22','8.074268e-23','714.40857','-2.0951307e-10','555.46643'],
['Estonia','202004','0.0006509135','3.0443204e-11','9.580731e-11','2.7447683e-11','93.0339','1.0000144','343.68628','1.1135728e-09','6.771589e-23','3.6871828e-22','3.573995e-23','75.21181','-2.0951307e-10','1153.6853'],
['France','202004','0.000609653','6.560536e-11','1.351195e-10','4.664766e-11','100.54879','1.0000144','336.8963','5.8023586e-10','1.1624914e-22','3.1414393e-22','3.7984776e-23','122.19516','-2.0951307e-10','426.00092'],
['Germany','202004','0.0006449913','1.3661558e-10','1.412824e-10','4.6784895e-11','115.56606','1.0000144','331.53427','9.617385e-10','1.8964914e-22','4.6502166e-22','6.94035e-23','592.16187','-2.0951307e-10','537.0692'],
['Greece','202004','0.00058656116','9.0724386e-11','1.412702e-10','4.8153995e-11','102.873985','1.0000144','341.7145','7.9479134e-10','9.471318e-22','2.0837093e-22','5.557944e-23','142.70279','-2.0951307e-10','726.1274'],
['Hungary','202004','0.00067603803','2.461189e-10','1.6882895e-10','5.6419206e-11','119.57969','1.0000144','338.3005','8.078185e-10','1.0969493e-22','6.4118645e-22','1.080521e-22','474.3056','-2.0951307e-10','528.68933'],
['Latvia','202004','0.00065170444','1.4559305e-10','1.0551916e-10','3.1240305e-11','95.74735','0.7288868','343.70032','8.724972e-10','1.0600412e-21','3.6454593e-22','4.2742854e-23','65.299736','-7.408717e-10','972.51196'],
['Lithuania','202004','0.0006547055','1.4052262e-10','1.2348733e-10','3.7710526e-11','97.741325','1.0000144','340.72037','9.210915e-10','1.0831766e-22','4.3431294e-22','5.6961784e-23','169.1458','-2.0951307e-10','649.87115'],
['Luxembourg','202004','0.0006460862','2.61352e-10','1.3963733e-10','4.6601733e-11','139.01411','1.0000144','332.8323','1.4294305e-09','4.31216e-22','3.9915976e-22','6.422558e-23','2461.2942','-2.0951307e-10','510.0238'],
['Malta','202004','0.0006496762','1.7672619e-11','1.4625699e-10','5.6552214e-11','94.438736','0.0','333.8557','6.5202116e-10','8.0779355e-30','1.8857446e-22','6.032434e-23','87.968475','0.0','610.97626'],
['Netherlands','202004','0.0006928891','6.518652e-10','1.4289284e-10','4.7898158e-11','160.2757','1.0000144','335.70078','1.8828994e-09','4.8787525e-21','5.854772e-22','7.2477877e-23','4835.863','-2.0951307e-10','581.78204'],
['Poland','202004','0.00066286774','2.1499129e-10','1.4240988e-10','4.573412e-11','112.391846','1.0000144','331.88693','9.098661e-10','1.7821866e-22','7.1388236e-22','9.493819e-23','491.84015','-2.0951307e-10','477.6215'],
['Portugal','202004','0.0006160196','1.07352904e-10','1.5226896e-10','5.621072e-11','95.56422','1.0000144','348.36932','4.5594228e-10','2.7673933e-21','1.6460473e-22','2.7216244e-23','165.83138','-2.0951307e-10','610.9275'],
['Romania','202004','0.00062734034','1.2655449e-10','1.6033837e-10','5.031967e-11','109.6581','1.0000144','341.83923','8.29932e-10','7.5845515e-21','4.635642e-22','7.778922e-23','341.30206','-2.0951307e-10','494.90433'],
['Slovakia','202004','0.0006360393','8.380781e-11','1.5211767e-10','4.97824e-11','105.024895','1.0000144','335.6629','8.473785e-10','1.5042641e-21','6.1971444e-22','8.849726e-23','250.10577','-2.0951307e-10','494.54666'],
['Slovenia','202004','0.0006406029','1.6908162e-10','1.6350525e-10','5.369753e-11','118.30551','1.0000144','333.15814','9.705158e-10','1.1754219e-25','3.7765084e-22','6.281042e-23','355.48083','-2.0951307e-10','531.0593'],
['Spain','202004','0.0005927104','1.3033129e-09','1.552549e-10','5.3194466e-11','153.70204','1.0000144','341.4453','6.314971e-10','1.697742e-21','2.8425799e-22','3.4492978e-23','2483.1497','-2.0951307e-10','457.28577'],
['Sweden','202004','0.0006503865','2.1809636e-11','8.9627104e-11','2.6658151e-11','93.324936','0.7735157','332.82388','1.0235467e-09','3.6250303e-25','2.4197136e-22','3.2016838e-23','188.54568','5.8300618e-09','1314.1282'],
['Ireland','202004','0.00065617595','2.4917549e-11','1.0408047e-10','3.4768702e-11','99.81415','1.0000144','339.0047','1.0605403e-09','4.1307953e-23','2.4735926e-22','2.8492088e-23','328.8519','-2.0951307e-10','795.07806'],
['Italy','202004','0.0006580679','6.1426103e-10','1.7732144e-10','6.199134e-11','139.25534','1.0000144','338.36008','1.0724434e-09','1.05566363e-22','2.647077e-22','6.3367183e-23','1233.3251','-2.0951307e-10','653.6595'],
['Denmark','202004','0.0006586149','1.0797976e-10','1.2062945e-10','3.972297e-11','94.18659','1.0000144','332.0222','1.076572e-09','4.258763e-24','3.057136e-22','5.263926e-23','189.94774','-2.0951307e-10','835.6405'],
['Finland','202004','0.00063963665','5.7049e-11','8.1398666e-11','2.2261136e-11','92.177414','1.0000144','335.8902','1.1376444e-09','1.0586253e-22','2.929662e-22','2.7346523e-23','35.53486','-2.0951307e-10','1865.1051'],
['Belgium','202004','0.0006803319','7.419279e-10','1.4827774e-10','4.9796986e-11','165.96056','1.0000144','334.301','1.474019e-09','1.2507574e-21','4.841833e-22','7.092406e-23','4234.0654','-2.0951307e-10','537.74243'],
['Croatia','202004','0.0006312597','6.1584196e-11','1.5465283e-10','5.1647773e-11','109.2211','0.9944304','334.2974','6.915001e-10','7.161654e-21','2.8211875e-22','5.2737403e-23','295.14835','4.775975e-09','601.1451'],]

In [69]:
# Create the DataFrame that will be written in S3
columns = ["Country","Date","COCL","COEM","COLS","COPD","COSC","LWI","TO3","Var_COCL","Var_COEM","Var_COLS","Var_COPD","Var_COSC","Var_LWI","Var_TO3"]

df_pollution = spark.createDataFrame(info, columns)
 
# Writting the results to s3

# df_pollution.repartition(1).write.option("delimiter", ",").mode('overwrite').option("header", "false").csv("D://NASA/Output/CO.csv")



In [71]:
df_pollution.show(10, False)

+--------------+------+-------------+--------------+-------------+--------------+----------+---------+---------+-------------+-------------+-------------+-------------+---------+-------------+---------+
|Country       |Date  |COCL         |COEM          |COLS         |COPD          |COSC      |LWI      |TO3      |Var_COCL     |Var_COEM     |Var_COLS     |Var_COPD     |Var_COSC |Var_LWI      |Var_TO3  |
+--------------+------+-------------+--------------+-------------+--------------+----------+---------+---------+-------------+-------------+-------------+-------------+---------+-------------+---------+
|Country       |Date  |COCL         |COEM          |COLS         |COPD          |COSC      |LWI      |TO3      |Var_COCL     |Var_COEM     |Var_COLS     |Var_COPD     |Var_COSC |Var_LWI      |Var_TO3  |
|Austria       |201901|0.00052217644|5.2215263e-11 |2.130699e-11 |5.2717713e-12 |107.294075|0.9999939|332.40884|5.4041827e-10|6.9557666e-28|4.0904974e-24|2.1660355e-25|527.91626|-3.722289e

In [112]:
info_so2 = [['Country','Date','BCANGSTR','BCCMASS','BCEXTTAU','BCFLUXU','BCFLUXV','BCSCATAU','BCSMASS','DMSCMASS','DMSSMASS','DUANGSTR','DUCMASS','DUCMASS25','DUEXTT25','DUEXTTAU','DUFLUXU','DUFLUXV','DUSCAT25','DUSCATAU','DUSMASS','DUSMASS25','OCANGSTR','OCCMASS','OCEXTTAU','OCFLUXU','OCFLUXV','OCSCATAU','OCSMASS','SO2CMASS','SO2SMASS','SO4CMASS','SO4SMASS','SSANGSTR','SSCMASS','SSCMASS25','SSEXTT25','SSEXTTAU','SSFLUXU','SSFLUXV','SSSCAT25','SSSCATAU','SSSMASS','SSSMASS25','SUANGSTR','SUEXTTAU','SUFLUXU','SUFLUXV','SUSCATAU','TOTANGSTR','TOTEXTTAU','TOTSCATAU','Var_BCANGSTR','Var_BCCMASS','Var_BCEXTTAU','Var_BCFLUXU','Var_BCFLUXV','Var_BCSCATAU','Var_BCSMASS','Var_DMSCMASS','Var_DMSSMASS','Var_DUANGSTR','Var_DUCMASS','Var_DUCMASS25','Var_DUEXTT25','Var_DUEXTTAU','Var_DUFLUXU','Var_DUFLUXV','Var_DUSCAT25','Var_DUSCATAU','Var_DUSMASS','Var_DUSMASS25','Var_OCANGSTR','Var_OCCMASS','Var_OCEXTTAU','Var_OCFLUXU','Var_OCFLUXV','Var_OCSCATAU','Var_OCSMASS','Var_SO2CMASS','Var_SO2SMASS','Var_SO4CMASS','Var_SO4SMASS','Var_SSANGSTR','Var_SSCMASS','Var_SSCMASS25','Var_SSEXTT25','Var_SSEXTTAU','Var_SSFLUXU','Var_SSFLUXV','Var_SSSCAT25','Var_SSSCATAU','Var_SSSMASS','Var_SSSMASS25','Var_SUANGSTR','Var_SUEXTTAU','Var_SUFLUXU','Var_SUFLUXV','Var_SUSCATAU','Var_TOTANGSTR','Var_TOTEXTTAU','Var_TOTSCATAU'],
['Austria','201911','1.4720345','4.4505222e-07','0.0047698026','2.8888405e-06','3.3422186e-06','0.0013005987','4.1118714e-10','1.3056937e-09','5.433376e-14','-0.046603817','2.4898498e-05','7.5029166e-06','0.009909179','0.017011354','8.116303e-05','0.00049105013','0.009438009','0.015826132','3.8813006e-09','1.1374189e-09','1.8026178','1.2590015e-06','0.010667545','9.367653e-06','1.0338003e-05','0.010535646','9.832644e-10','2.0018188e-06','1.8291287e-09','3.1554466e-06','1.8417786e-09','0.027183779','1.3113045e-06','3.7207187e-07','0.0015613572','0.0027969652','5.522216e-06','1.0580139e-05','0.0015613572','0.0027969652','9.091491e-10','2.266146e-10','1.4864446','0.05093391','1.6963626e-05','2.5260639e-05','0.05093391','1.1694051','0.08617963','0.08139294','0.0012193462','1.414177e-14','1.496255e-06','5.7073643e-12','9.810546e-12','1.3388534e-07','2.775374e-20','9.974427e-18','7.9339525e-26','0.000117143725','2.0516862e-09','1.4386016e-10','0.00024220791','0.0008005601','1.3387182e-07','1.5312663e-06','0.00021944752','0.00068782654','3.849493e-17','2.598604e-18','0.0266441','5.7649496e-14','1.6217786e-05','5.3636297e-11','9.987874e-11','1.614021e-05','2.0208895e-19','5.7970456e-12','2.5593092e-18','1.7554664e-12','6.570793e-19','0.00051588594','1.0766945e-12','5.526007e-14','1.2930326e-06','5.115197e-06','8.914211e-11','2.9291392e-10','1.2930326e-06','5.115197e-06','1.3062987e-18','5.909362e-20','0.019235808','0.000892177','2.0776937e-10','8.644665e-10','0.000892177','0.076360896','0.0021834096','0.0020387392'],
['Bulgaria','201911','1.4742433','5.8094764e-07','0.00644846','1.8639818e-06','3.0764245e-06','0.001819343','3.8952716e-10','2.6729439e-09','2.2583055e-12','-0.0547822','6.135957e-05','1.9732794e-05','0.025675353','0.043892812','-8.937352e-06','0.00057509437','0.024441853','0.04088069','2.057284e-08','6.124429e-09','1.8559653','1.9064272e-06','0.017154763','8.137589e-06','1.0422836e-05','0.016954001','8.499936e-10','8.910509e-06','6.1376e-09','6.37124e-06','3.0707679e-09','0.015115915','7.1353684e-06','1.776718e-06','0.0076756985','0.014815089','-1.6892147e-05','3.1456042e-05','0.0076756985','0.014815089','8.443652e-09','1.7446165e-09','1.5475509','0.10292749','1.057372e-05','3.367859e-05','0.10292749','1.0395709','0.18523948','0.17739561','0.0022204167','4.037598e-14','6.5854256e-06','1.0580322e-11','1.4526239e-11','8.3969553e-07','4.0630042e-20','5.0332458e-17','4.4525013e-23','6.864976e-05','5.8029226e-09','5.6076405e-10','0.0009312949','0.0028259316','1.0440303e-07','1.5855263e-06','0.00084333646','0.0024461146','4.0258346e-16','3.4613224e-17','0.033635467','6.750963e-13','0.00013542722','1.1930464e-10','2.0689536e-10','0.00013384261','1.7493132e-19','3.163768e-11','1.9082407e-17','7.217074e-12','4.522559e-18','0.0002557196','3.647843e-11','1.4567891e-12','6.3952284e-05','0.00029246756','5.8260032e-09','1.8072718e-09','6.3952284e-05','0.00029246756','1.7616001e-16','4.4133456e-18','0.031118702','0.0036810802','1.2042927e-09','1.6536552e-09','0.0036810802','0.08288891','0.012752534','0.011872605'],
['Cyprus','201911','1.4223132','7.3651506e-07','0.007160162','1.0544369e-06','-2.2334757e-07','0.0016173067','3.536269e-10','5.2328475e-09','7.703372e-12','-0.06311839','0.00011930018','3.569902e-05','0.045665134','0.08099946','-0.00016647043','0.00012610352','0.04344491','0.07527543','4.4758355e-08','1.2091193e-08','2.0783658','2.6969124e-06','0.013231682','4.732477e-06','-1.0789691e-06','0.012951974','1.1198962e-09','9.727335e-06','6.8688633e-09','7.734871e-06','3.243182e-09','0.011897412','1.4912539e-05','2.8886427e-06','0.006821673','0.014757117','-2.1410471e-05','-1.3403651e-06','0.006821673','0.014757117','1.7605052e-08','2.9712162e-09','1.8140287','0.06946368','-2.1733244e-06','-8.523414e-07','0.06946368','0.756653','0.18561187','0.17406547','0.00044105962','5.6383653e-14','5.144991e-06','1.0612893e-11','6.3969767e-12','2.6531606e-07','3.600384e-20','6.06957e-17','8.3173247e-23','2.1663896e-05','6.069425e-09','4.966076e-10','0.0007975834','0.002619825','3.1700537e-07','4.511137e-07','0.0007214195','0.0022566516','1.1133234e-15','7.94919e-17','0.01917917','8.505145e-13','2.4286115e-05','1.6470375e-10','8.76846e-11','2.3609098e-05','4.2206531e-19','1.7793721e-11','7.918535e-18','1.2072997e-11','3.0517698e-18','6.41196e-05','6.665669e-11','1.996273e-12','2.3358158e-05','0.00012429318','9.354617e-09','4.2778714e-09','2.3358158e-05','0.00012429318','9.010328e-17','2.8237008e-18','0.012399578','0.0013320107','1.2300602e-09','6.4244726e-10','0.0013320107','0.04834333','0.0070205918','0.006282095'],
['Czech Republic','201911','1.4683495','7.5924174e-07','0.008157189','3.0793233e-06','5.5522087e-06','0.0021996868','1.1287604e-09','1.3036947e-09','1.1130552e-14','-0.04774247','2.7056078e-05','8.4218655e-06','0.011129764','0.018918967','9.505383e-05','0.00046528742','0.010601045','0.017614497','5.115335e-09','1.5696141e-09','1.834583','1.8470328e-06','0.015576684','8.593545e-06','1.5552954e-05','0.0153829185','2.1011444e-09','6.4968362e-06','7.0510144e-09','4.762449e-06','3.0538367e-09','0.027110677','2.901714e-06','8.1715035e-07','0.0029711665','0.0052857255','1.4360463e-05','1.5610227e-05','0.0029711665','0.0052857255','1.7032247e-09','4.494091e-10','1.5189631','0.080056004','1.5254804e-05','3.958931e-05','0.080056004','1.203533','0.12799384','0.120538555','0.0010138756','4.0573146e-14','4.8682714e-06','1.9244101e-11','1.4681454e-11','4.6569318e-07','2.2697448e-19','1.0596165e-17','2.7278953e-27','0.00010617392','1.7476217e-09','1.297197e-10','0.00021871501','0.0007090767','1.4602783e-07','1.0987834e-06','0.00019819381','0.00061011844','3.721965e-17','3.249686e-18','0.025567532','1.3725018e-13','3.991524e-05','1.0589829e-10','1.3494855e-10','3.959742e-05','6.4675686e-19','1.639897e-11','1.1702515e-17','4.948929e-12','3.4601155e-18','0.00024289016','8.385845e-12','4.413099e-13','5.527955e-06','2.2291959e-05','1.1665413e-09','9.403449e-10','5.527955e-06','2.2291959e-05','5.7719302e-18','2.7990295e-19','0.017810116','0.0035431855','6.333519e-10','1.2964325e-09','0.0035431855','0.052059107','0.0060409512','0.005756434'],
['Estonia','201911','1.5098501','6.076654e-07','0.0071778577','3.1523955e-06','4.207771e-06','0.0022371104','3.824374e-10','1.058072e-09','2.2297767e-13','-0.048884857','3.5542616e-05','1.1198466e-05','0.014741123','0.025058562','0.0003601175','0.0005794221','0.014038468','0.023335451','5.605995e-09','1.7525399e-09','1.7380431','1.912829e-06','0.021627916','1.0238762e-05','1.3317894e-05','0.021425456','1.4041267e-09','2.9181342e-06','2.1856823e-09','4.7463636e-06','2.7564437e-09','0.022019912','3.8529824e-06','9.775551e-07','0.004594508','0.008463843','1.23987165e-05','4.7891076e-06','0.004594508','0.008463843','6.4567436e-09','1.2516086e-09','1.421511','0.09366513','1.7683677e-05','3.5428962e-05','0.09366513','1.0872544','0.1559934','0.1491276','0.0021574972','3.199837e-14','5.0794993e-06','1.2801402e-11','1.9280343e-11','7.0602067e-07','2.6895298e-20','8.331646e-18','3.663875e-25','8.633725e-05','3.462434e-09','2.7101654e-10','0.00045189628','0.0014556133','9.752366e-07','2.3611358e-06','0.00040924843','0.0012538377','7.868973e-17','6.7853977e-18','0.024963662','2.6487105e-13','0.000112925874','1.1034944e-10','2.0328214e-10','0.000112085494','5.163141e-19','4.1531114e-12','2.3099606e-18','4.6944137e-12','3.900529e-18','0.00039234202','3.0452318e-11','1.1079329e-12','2.1034164e-05','9.793988e-05','2.5705749e-09','2.498724e-09','2.1034164e-05','9.793988e-05','6.167828e-17','1.6469102e-18','0.019530714','0.00356806','8.362708e-10','1.6482101e-09','0.00356806','0.041049503','0.0076335045','0.007210694'],
['France','201911','1.462593','4.6439553e-07','0.004893197','4.051597e-06','5.9093367e-07','0.0012949485','3.0911818e-10','2.4241793e-09','8.5378335e-13','-0.042331487','9.360419e-06','3.2399369e-06','0.004393839','0.0070565296','6.0429575e-05','1.8447165e-06','0.004188783','0.00659152','1.5559368e-09','4.861026e-10','1.870439','1.3303595e-06','0.00944042','1.4015559e-05','9.2777003e-07','0.009301741','4.681051e-10','6.3270267e-07','6.197533e-10','2.2482482e-06','1.0690355e-09','0.011700491','7.997262e-06','1.7536443e-06','0.008450974','0.017415782','8.343123e-05','2.3946473e-05','0.008450974','0.017415782','8.5402325e-09','1.6526535e-09','1.5099308','0.031902153','2.056815e-05','2.8586485e-06','0.031902153','0.9703306','0.07070798','0.06650644','0.0013157009','2.4640905e-14','2.7963156e-06','1.1644514e-11','1.2289771e-11','2.405863e-07','3.3496882e-20','2.5976975e-17','1.3003903e-23','9.332806e-05','1.3859114e-10','1.0181377e-11','1.7340615e-05','5.5214623e-05','1.2206606e-08','8.8417265e-09','1.5718551e-05','4.7523677e-05','6.4772667e-18','6.4001833e-19','0.029389657','9.160394e-14','1.3753232e-05','1.6777166e-10','1.217193e-10','1.3663196e-05','6.5306103e-20','7.209751e-13','1.630234e-18','5.6704066e-13','5.031637e-19','0.00023877883','1.003058e-10','3.0874233e-12','8.459849e-05','0.00044445464','2.3199542e-08','6.5028547e-09','8.459849e-05','0.00044445464','1.6013655e-16','3.682908e-18','0.016045311','0.00030198565','2.7459515e-10','2.9185174e-10','0.00030198565','0.073696055','0.00065903313','0.0006296701'],
['Germany','201911','1.4765294','7.7941e-07','0.0085338075','3.7728423e-06','4.7364765e-06','0.0023913372','1.014615e-09','1.8065466e-09','1.0054227e-13','-0.046066742','2.1287105e-05','6.5201557e-06','0.008647125','0.014707107','1.1222391e-05','0.00030351165','0.00823719','0.013689386','2.7799834e-09','8.2396356e-10','1.8189923','1.5519639e-06','0.013188572','9.60205e-06','1.1333291e-05','0.01302609','1.0480102e-09','4.6251016e-06','7.2787847e-09','3.7481525e-06','2.4797848e-09','0.026844893','3.842731e-06','1.0385847e-06','0.004674125','0.008580673','2.750635e-05','1.125946e-05','0.004674125','0.008580673','3.3138343e-09','7.9823126e-10','1.4687163','0.06461677','1.7105302e-05','2.307049e-05','0.06461677','1.1389309','0.10962748','0.10230464','0.0012053994','6.630676e-14','8.865941e-06','2.1606071e-11','1.5410189e-11','9.364101e-07','1.7697916e-19','3.3935793e-17','2.081922e-25','0.00011584593','2.165317e-09','1.4203051e-10','0.00023990535','0.00080758386','1.4577445e-07','8.270856e-07','0.00021740067','0.0006927378','1.5832981e-17','1.101414e-18','0.02658245','1.05508965e-13','3.500307e-05','1.0448564e-10','9.9626654e-11','3.479205e-05','1.631696e-19','1.314746e-11','1.5670927e-17','3.1411007e-12','2.2795108e-18','0.0004127081','3.037906e-11','1.438833e-12','3.2544e-05','0.00014073413','2.8115217e-09','2.1386495e-09','3.2544e-05','0.00014073413','3.4778958e-17','1.3068536e-18','0.015393596','0.0018668373','4.8381626e-10','3.4819495e-10','0.0018668373','0.056276273','0.004088914','0.003741548'],
['Greece','201911','1.4712616','3.8326058e-07','0.004154852','3.0737258e-06','2.177644e-06','0.0011428494','2.4396385e-10','3.1119074e-09','2.1800179e-12','-0.055740196','5.8997637e-05','1.744402e-05','0.022672018','0.03974692','0.000195242','0.00077719457','0.02158191','0.036943745','2.3483741e-08','6.425653e-09','1.8592491','1.2957794e-06','0.010621226','1.144697e-05','7.790576e-06','0.010485373','5.2197313e-10','4.9698406e-06','3.391853e-09','3.7202387e-06','1.9655846e-09','0.006007719','8.73429e-06','1.7516851e-06','0.0072740065','0.015661508','3.2535758e-05','4.2263837e-05','0.0072740065','0.015661508','1.0412403e-08','1.8411952e-09','1.5514911','0.053746432','2.2022563e-05','1.6420176e-05','0.053746432','0.8486218','0.12393072','0.11798047','0.0017116901','5.0503937e-15','8.222473e-07','4.7904376e-12','6.479075e-12','1.2893626e-07','1.8021364e-20','6.7851356e-17','4.7343948e-23','7.042466e-05','9.7333235e-09','7.5507955e-10','0.0012536965','0.0040625394','1.9230045e-07','4.9160344e-06','0.0011353488','0.0034979898','1.0531776e-15','7.472567e-17','0.033121143','9.383469e-14','3.0194782e-05','6.81073e-11','9.229722e-11','2.9946188e-05','1.1305039e-19','4.5381605e-11','1.6115306e-17','4.1601336e-12','2.2965437e-18','8.7098546e-05','4.1438776e-11','1.092175e-12','2.2099128e-05','0.00012383217','3.4177636e-09','2.3389364e-09','2.2099128e-05','0.00012383217','5.494967e-17','1.1669159e-18','0.013902041','0.0014502354','3.66259e-10','3.6958658e-10','0.0014502354','0.10810393','0.006038413','0.0054216334'],
['Hungary','201911','1.4639584','6.5136663e-07','0.0070092827','2.843134e-06','4.6131217e-06','0.0018821046','6.853874e-10','1.5623434e-09','1.980802e-14','-0.051882945','4.318806e-05','1.3134132e-05','0.017225077','0.02971817','0.00010513538','0.00069719966','0.016402189','0.027648216','1.0286515e-08','3.134052e-09','1.8721141','2.036314e-06','0.01635836','8.623624e-06','1.5607548e-05','0.016144741','2.0628892e-09','7.631513e-06','6.072501e-09','6.191642e-06','3.1233272e-09','0.023370512','4.9416376e-06','1.2321615e-06','0.004363797','0.008321789','1.8591603e-05','3.8028655e-05','0.004363797','0.008321789','2.8383533e-09','6.7953415e-10','1.566521','0.097819544','1.6630567e-05','4.62787e-05','0.097819544','1.173387','0.15922567','0.15181601','0.0013511541','2.2538839e-14','3.132462e-06','1.5541375e-11','1.5010741e-11','3.7934814e-07','7.516116e-20','3.4750403e-17','1.1859832e-26','6.25288e-05','7.707836e-09','5.815994e-10','0.000967504','0.0031671235','2.1571246e-07','4.5010047e-06','0.00087622926','0.002725171','1.330671e-16','1.1677381e-17','0.027052525','4.5391279e-13','6.0814447e-05','1.381469e-10','1.8490795e-10','6.010342e-05','7.466873e-19','2.3819363e-11','1.0093905e-17','8.161016e-12','4.6311903e-18','0.00041832164','2.3139721e-11','9.900509e-13','1.564367e-05','7.247648e-05','3.36304e-09','3.8912606e-09','1.564367e-05','7.247648e-05','1.0603005e-17','5.1793035e-19','0.021356016','0.0044681192','1.0872636e-09','1.1157646e-09','0.0044681192','0.07852463','0.008347415','0.007823977'],
['Latvia','201911','1.504985','6.5453975e-07','0.007638852','3.3339397e-06','5.134881e-06','0.0023370062','4.1717935e-10','9.939196e-10','1.2249196e-13','-0.04994849','4.235452e-05','1.3346765e-05','0.017515581','0.029871242','0.0003263395','0.0006849199','0.016679423','0.02781466','6.7033716e-09','2.0785607e-09','1.7467867','2.0412658e-06','0.022057433','1.0384311e-05','1.6461452e-05','0.021841513','1.4623736e-09','3.4120726e-06','2.2718671e-09','5.1824363e-06','2.9786151e-09','0.021830102','3.9532024e-06','1.0014334e-06','0.00478549','0.008783052','1.7914912e-05','6.7899155e-06','0.00478549','0.008783052','5.1063314e-09','1.0026106e-09','1.4128127','0.10142549','1.8876346e-05','4.3076318e-05','0.10142549','1.0734121','0.16977455','0.16219999','0.0020238785','4.149493e-14','5.8204514e-06','1.1549917e-11','1.9137318e-11','7.1781704e-07','3.2048874e-20','7.428452e-18','9.460404e-26','8.301067e-05','4.6560475e-09','3.7611883e-10','0.00062636286','0.0020016045','7.8943555e-07','2.9988046e-06','0.0005673479','0.0017253304','9.672894e-17','8.260199e-18','0.021301901','3.4172722e-13','0.000103065584','9.7329804e-11','2.1214866e-10','0.000102174636','5.444127e-19','5.2903645e-12','2.4519538e-18','5.010713e-12','2.8585666e-18','0.00031619304','4.499388e-11','1.37073e-12','2.8779446e-05','0.00014581242','3.4256744e-09','3.0572738e-09','2.8779446e-05','0.00014581242','3.641813e-17','1.0175285e-18','0.01691658','0.0033143358','6.7358913e-10','1.6270277e-09','0.0033143358','0.04721044','0.0076706214','0.0071823546'],
['Lithuania','201911','1.499229','6.8675405e-07','0.007920243','3.6752099e-06','5.5119567e-06','0.002364567','5.051847e-10','9.084349e-10','9.065444e-14','-0.049927436','4.5836605e-05','1.4448524e-05','0.018919818','0.032346234','0.00025068712','0.0007482075','0.018015303','0.030117527','7.0561526e-09','2.179019e-09','1.7710035','2.071886e-06','0.020901341','1.0024228e-05','1.7507287e-05','0.020682499','1.4782024e-09','4.0143404e-06','3.187055e-09','5.5056157e-06','3.0073362e-09','0.026103256','3.3596434e-06','9.646476e-07','0.004383848','0.0076285102','1.8857621e-05','1.2551581e-05','0.004383848','0.0076285102','2.6045155e-09','6.452988e-10','1.4335475','0.10251385','2.0895106e-05','4.667214e-05','0.10251385','1.1040304','0.17130968','0.16330566','0.0014073811','4.3185277e-14','5.761525e-06','1.7287298e-11','1.5732231e-11','6.19979e-07','4.949175e-20','6.200771e-18','6.293385e-26','8.776325e-05','5.2538485e-09','4.4215934e-10','0.00073527655','0.002327018','4.977283e-07','3.266267e-06','0.0006659539','0.002007728','5.5853505e-17','5.00963e-18','0.018461948','3.660233e-13','7.332503e-05','1.1882005e-10','1.8851336e-10','7.24994e-05','5.3663523e-19','5.946407e-12','3.5878173e-18','4.445298e-12','2.2562631e-18','0.0003226442','2.1727993e-11','8.906123e-13','1.7531633e-05','7.845271e-05','2.4276987e-09','2.0638222e-09','1.7531633e-05','7.845271e-05','2.358812e-17','7.263034e-19','0.013040379','0.002389743','8.0836265e-10','1.251635e-09','0.002389743','0.056754258','0.0067465743','0.0062389607'],
['Luxembourg','201911','1.4790783','7.183266e-07','0.007844888','3.0045103e-06','2.9735372e-06','0.002205631','1.1831739e-09','1.4544311e-09','2.0527455e-13','-0.044015035','1.2744328e-05','4.223077e-06','0.0056613823','0.009319693','2.2999751e-05','7.1624636e-05','0.005395036','0.0086936345','1.7343393e-09','5.1242705e-10','1.7941661','1.5398848e-06','0.013210944','8.57043e-06','8.106584e-06','0.013049667','1.2292092e-09','2.927425e-06','5.0013482e-09','2.9747807e-06','1.8345984e-09','0.015626816','5.9290965e-06','1.5044336e-06','0.0075768535','0.014464976','4.0387855e-05','2.6934273e-05','0.0075768535','0.014464976','5.6058944e-09','1.3011912e-09','1.4458904','0.051138468','1.42801355e-05','1.3695513e-05','0.051138468','1.0462998','0.0959799','0.08955263','0.0012995094','4.8882027e-14','6.155945e-06','1.7860947e-11','1.5849506e-11','6.399237e-07','3.1265822e-19','5.4489625e-18','3.9974e-25','0.00011527514','2.817309e-10','2.038704e-11','3.4226665e-05','0.00011247109','2.2819211e-08','4.0016676e-08','3.10088e-05','9.670366e-05','4.7097782e-18','3.5602254e-19','0.028781872','1.1449954e-13','2.917301e-05','1.2252609e-10','1.2220149e-10','2.9041219e-05','2.9146621e-19','7.647985e-12','1.9215207e-17','1.4230856e-12','1.2680821e-18','0.00027941648','5.417428e-11','2.6059268e-12','8.584071e-05','0.00036864335','6.3358865e-09','3.1975051e-09','8.584071e-05','0.00036864335','5.09434e-17','2.1539219e-18','0.022485202','0.0009320619','3.0895514e-10','3.710588e-10','0.0009320619','0.05026313','0.0016123537','0.0014994988'],
['Malta','201911','1.4391145','4.730665e-07','0.0048102685','5.338852e-06','1.7697265e-06','0.0011778701','1.5609802e-10','3.1925776e-08','6.505578e-11','-0.06043117','0.00010636085','2.7336459e-05','0.035302024','0.06485999','0.00063759583','0.0010379243','0.033596843','0.060065836','4.7811604e-08','1.1160363e-08','2.003918','1.4813955e-06','0.0088658985','1.8677152e-05','6.4800024e-06','0.008711763','3.7341094e-10','1.8765436e-06','1.545828e-09','2.7525894e-06','9.187964e-10','0.0054391837','3.467305e-05','5.3994436e-06','0.01608423','0.038821675','0.00025600035','0.00010120221','0.01608423','0.038821675','4.11718e-08','5.754699e-09','1.6933647','0.031390175','2.9084735e-05','9.175531e-06','0.031390175','0.53744','0.14874871','0.1401676','0.0012746793','6.717071e-15','8.7714426e-07','4.6023524e-12','1.2216469e-11','1.1319704e-07','2.8182167e-21','1.5953366e-15','3.527767e-21','9.8152486e-05','2.7356524e-08','1.5231876e-09','0.0025432229','0.009017493','1.9059983e-06','5.8422193e-06','0.0023034853','0.0077003865','4.8385063e-15','2.4573098e-16','0.03512105','5.101426e-14','1.5497493e-05','5.39729e-11','1.5277866e-10','1.5432439e-05','2.1310278e-20','1.3490879e-12','4.727121e-19','8.334551e-13','2.4918534e-19','4.2408148e-05','7.000639e-10','1.1490487e-11','0.00018910048','0.0012907085','1.8976392e-07','1.152487e-07','0.00018910048','0.0012907085','8.5286403e-16','1.3710368e-17','0.019159855','0.00038078803','1.4398957e-10','3.8498227e-10','0.00038078803','0.0739441','0.012144721','0.010745923'],
['Netherlands','201911','1.4640416','9.686058e-07','0.01038471','2.4454737e-06','4.452524e-06','0.0027848007','1.8850097e-09','3.0118248e-09','2.366786e-12','-0.044226073','1.2477708e-05','4.1711637e-06','0.0056059887','0.009181455','2.1072715e-06','4.9011494e-05','0.005342692','0.008567098','1.3651807e-09','4.2885373e-10','1.8538884','1.8156666e-06','0.014367908','6.8720665e-06','8.477841e-06','0.014177747','1.8337669e-09','5.77294e-06','1.1850419e-08','3.466445e-06','2.094068e-09','0.015692946','1.522692e-05','2.9662483e-06','0.013580684','0.029209897','0.00010352402','4.7959606e-05','0.013580684','0.029209897','1.4389058e-08','2.4420317e-09','1.5100504','0.05481899','1.1662733e-05','1.4421953e-05','0.05481899','1.0028005','0.11796332','0.10955881','0.00089023577','7.0446294e-14','9.0136145e-06','3.056287e-11','2.6710873e-11','8.76309e-07','9.905231e-19','2.2109826e-17','2.937424e-23','0.000100894016','2.8561248e-10','2.0494387e-11','3.467938e-05','0.00011338789','1.8724355e-08','3.4718077e-08','3.142666e-05','9.749873e-05','2.3262813e-18','1.8434156e-19','0.026465401','1.11439016e-13','2.655553e-05','1.185913e-10','1.3790379e-10','2.6385449e-05','8.4481047e-19','9.49799e-12','3.8366787e-17','1.3157376e-12','1.0893547e-18','0.0003797451','3.2454547e-10','8.8955865e-12','0.00022570728','0.0012039997','3.7760525e-08','3.796082e-08','0.00022570728','0.0012039997','3.2987838e-16','7.0328765e-18','0.01740884','0.00072860485','3.772948e-10','5.0615484e-10','0.00072860485','0.091666035','0.002206736','0.0021526199'],
['Poland','201911','1.474393','9.2552193e-07','0.010077976','3.4268157e-06','6.6653665e-06','0.0027709387','1.6253024e-09','1.3309565e-09','1.0215755e-14','-0.049299557','3.2959302e-05','1.0098276e-05','0.013274182','0.022785379','0.00015817629','0.0005501777','0.012641011','0.021203373','7.0475226e-09','2.1040776e-09','1.8077443','2.311336e-06','0.020665243','9.263425e-06','1.8401657e-05','0.020422282','3.433392e-09','1.05787685e-05','1.2587158e-08','6.1550522e-06','3.6356897e-09','0.028141908','3.2650198e-06','9.130704e-07','0.0034826936','0.006181698','2.1770502e-05','1.22134325e-05','0.0034826936','0.006181698','2.01003e-09','5.1990845e-10','1.5071331','0.104753956','1.8740468e-05','5.1343246e-05','0.104753956','1.2248985','0.1644648','0.15533192','0.0011080271','5.551573e-14','6.565498e-06','3.2354983e-11','1.8042986e-11','6.3457395e-07','4.8189397e-19','1.7005712e-17','2.0201892e-27','9.5217925e-05','3.6362455e-09','2.6262267e-10','0.00043830686','0.0014488788','3.0067832e-07','2.2595427e-06','0.00039700093','0.0012454248','6.916679e-17','5.8410217e-18','0.022452747','3.006524e-13','7.116314e-05','1.8516269e-10','1.8127999e-10','7.057363e-05','2.1646386e-18','2.8965372e-11','3.859341e-17','5.817288e-12','4.1777915e-18','0.0002383849','2.0370414e-11','9.417238e-13','1.4299075e-05','6.2699e-05','3.0538367e-09','1.8134273e-09','1.4299075e-05','6.2699e-05','1.6774538e-17','5.24393e-19','0.016778087','0.003658936','1.3223531e-09','1.5481402e-09','0.003658936','0.056952465','0.007088262','0.006686347'],
['Portugal','201911','1.4539635','3.7109163e-07','0.0038711021','5.538149e-06','-3.6957688e-06','0.0009984288','2.5351227e-10','8.150365e-09','7.377618e-12','-0.042180713','1.1491581e-05','4.2952674e-06','0.0057791313','0.009172603','0.00017496187','-7.019868e-05','0.00550776','0.008581','4.0467643e-09','1.303254e-09','1.9394196','1.3866398e-06','0.009495167','2.2553808e-05','-1.5420845e-05','0.009350505','6.233371e-10','2.0729851e-06','3.2062852e-09','2.011782e-06','6.7547534e-10','0.0015634827','3.1523028e-05','5.1576117e-06','0.0212945','0.049841784','0.00030188027','-0.00021961289','0.0212945','0.049841784','3.5600944e-08','5.3068616e-09','1.5765407','0.02577933','2.9170147e-05','-1.7875065e-05','0.02577933','0.6466243','0.09816014','0.09455144','0.0016604878','3.497015e-15','4.2436943e-07','4.993582e-12','1.216544e-11','7.338624e-08','1.6174888e-20','2.615131e-16','1.6057131e-22','4.521553e-05','6.993356e-10','8.4315346e-11','0.00014494485','0.00040122343','1.672915e-07','1.3299509e-08','0.00013140017','0.0003494725','1.6237165e-16','1.7245056e-17','0.03389833','1.04113095e-13','2.3183902e-05','9.886795e-11','2.399276e-10','2.3077731e-05','9.018754e-20','1.3449752e-12','4.2067254e-18','2.6696847e-13','2.2404515e-19','0.00010394775','1.1720857e-09','2.0629897e-11','0.00030717306','0.0021600612','2.0659255e-07','1.6822243e-07','0.00030717306','0.0021600612','1.4778379e-15','2.2145839e-17','0.023171475','0.00014434132','1.6337774e-10','2.8598315e-10','0.00014434132','0.071049415','0.0023490456','0.0023120767'],
['Romania','201911','1.4665841','5.359646e-07','0.0058380724','2.593585e-06','3.190669e-06','0.0016047264','5.489176e-10','8.568744e-10','1.5313843e-13','-0.05133014','4.7539426e-05','1.535122e-05','0.020050682','0.034107354','3.0353327e-05','0.00061032735','0.019089827','0.031772938','1.019031e-08','3.0663883e-09','1.8847979','2.1496849e-06','0.017192954','9.315858e-06','1.2636779e-05','0.01696759','2.5655775e-09','4.784384e-06','3.3076883e-09','5.5629303e-06','2.4836262e-09','0.03979474','2.6150085e-06','7.7743636e-07','0.002628193','0.0045152213','5.1380744e-06','1.836275e-05','0.002628193','0.0045152213','2.0617923e-09','5.579221e-10','1.5848745','0.08644612','2.4670742e-05','3.5632438e-05','0.08644612','1.1769865','0.14810061','0.1413059','0.0017139269','2.4541524e-14','4.21514e-06','9.062841e-12','1.0278814e-11','5.602196e-07','7.019856e-20','2.2290081e-18','4.39294e-25','7.765971e-05','7.3082647e-09','6.679954e-10','0.0011143301','0.0034298934','7.54955e-08','2.832418e-06','0.0010092066','0.0029654051','1.3197902e-16','1.1113724e-17','0.030435856','7.740894e-13','0.00013629474','1.2392494e-10','1.6953913e-10','0.00013451675','1.8228147e-18','2.0968501e-11','5.928463e-18','8.12016e-12','1.4904594e-18','0.0013773446','5.37714e-12','3.962174e-13','5.445645e-06','1.6358576e-05','5.981362e-10','6.973601e-10','5.445645e-06','1.6358576e-05','4.576676e-18','2.919432e-19','0.022217177','0.0044592116','1.4830315e-09','1.4151286e-09','0.0044592116','0.09083215','0.011503344','0.010699915'],
['Slovakia','201911','1.4818405','6.1738274e-07','0.0068455553','2.7308915e-06','3.988794e-06','0.0019482432','6.702336e-10','1.2957972e-09','1.6674208e-15','-0.050490208','3.6077847e-05','1.1115762e-05','0.014601909','0.025052302','0.00012575243','0.0006200304','0.013905291','0.023315735','6.922714e-09','2.1226494e-09','1.7975351','1.8780348e-06','0.017800942','8.590054e-06','1.3069749e-05','0.017603457','1.7966202e-09','6.0277634e-06','5.3721716e-09','5.7221164e-06','3.2280982e-09','0.021598626','2.9886273e-06','8.101413e-07','0.0034475003','0.0063294717','9.193172e-06','2.2760212e-05','0.0034475003','0.0063294717','1.6227407e-09','3.9814277e-10','1.4686326','0.105693445','1.7772245e-05','3.9589915e-05','0.105693445','1.1789685','0.16172132','0.1548908','0.001859637','2.469811e-14','3.1789873e-06','1.159553e-11','1.354027e-11','4.1279918e-07','5.074215e-20','1.7778171e-17','3.2472313e-29','6.736714e-05','5.826125e-09','4.5360005e-10','0.0007547703','0.002447109','2.1834032e-07','3.8719377e-06','0.0006835975','0.0021074668','7.0303735e-17','5.910214e-18','0.031164605','3.187478e-13','9.118941e-05','1.0125217e-10','1.7933574e-10','9.046699e-05','6.076705e-19','1.330322e-11','6.310326e-18','7.780237e-12','7.6276475e-18','0.00042921677','8.046588e-12','4.2587456e-13','9.672906e-06','4.0140396e-05','4.5705015e-10','1.7742452e-09','9.672906e-06','4.0140396e-05','3.2129194e-18','1.7494034e-19','0.028110428','0.0049905567','6.1306327e-10','9.819193e-10','0.0049905567','0.06270215','0.00870103','0.008243577'],
['Slovenia','201911','1.4818463','5.539742e-07','0.0060950136','2.9305022e-06','3.4575312e-06','0.0017146115','5.565627e-10','1.8734236e-09','3.0170593e-13','-0.049935915','3.8539536e-05','1.12849175e-05','0.014801956','0.025813606','7.204065e-05','0.00059413235','0.014094997','0.023993867','7.519355e-09','2.0284738e-09','1.7844528','1.526662e-06','0.014345694','9.719956e-06','1.07550595e-05','0.014185188','1.3270182e-09','5.5569194e-06','5.4590443e-09','4.3364653e-06','2.7735083e-09','0.011415831','5.6338836e-06','1.2554192e-06','0.005756832','0.011811249','1.4254534e-05','4.2974563e-05','0.005756832','0.011811249','3.7746144e-09','7.4854806e-10','1.4518454','0.077717915','1.7578688e-05','2.677776e-05','0.077717915','1.0788034','0.13578404','0.1294225','0.0013925263','1.4270411e-14','1.4731747e-06','1.0507653e-11','1.1034884e-11','1.5647207e-07','3.9270254e-20','1.926453e-17','1.4828392e-24','9.767985e-05','5.3188507e-09','3.719946e-10','0.0006222582','0.002072862','2.2275582e-07','2.313223e-06','0.00056367577','0.0017805565','1.9728795e-16','1.14301645e-17','0.029534437','9.383923e-14','3.194256e-05','9.229869e-11','1.1808746e-10','3.1793683e-05','2.79219e-19','1.9528698e-11','6.273597e-18','2.9739189e-12','3.7812903e-18','0.00019502995','2.862821e-11','9.973098e-13','2.8165407e-05','0.00014494194','2.4636222e-09','3.6597794e-09','2.8165407e-05','0.00014494194','1.8558576e-17','6.4328564e-19','0.015718315','0.0016690837','6.520638e-10','7.1618583e-10','0.0016690837','0.08818805','0.004655853','0.004316247'],
['Spain','201911','1.4472014','4.3022794e-07','0.004419481','5.7304464e-06','-2.9654152e-06','0.0011108556','6.025858e-10','2.1729096e-09','3.3240362e-13','-0.041543934','7.843177e-06','3.027832e-06','0.0041240733','0.006407956','0.00011789497','-4.8527054e-05','0.0039321114','0.006001079','2.7666973e-09','9.2234725e-10','1.9521974','1.3316117e-06','0.008876345','2.1235832e-05','-1.1665817e-05','0.008737474','9.371581e-10','2.5849315e-06','5.2159024e-09','2.3758118e-06','1.6244048e-09','0.016405543','6.848402e-06','1.4201437e-06','0.004996913','0.0103514725','6.4659136e-05','-2.4442228e-05','0.004996913','0.0103514725','7.039562e-09','1.3222891e-09','1.5798161','0.03266005','3.1267242e-05','-1.4423904e-05','0.03266005','1.0907508','0.06271525','0.05886172','0.0013266965','8.463082e-15','9.803437e-07','5.744842e-12','1.220063e-11','1.0886468e-07','1.4110227e-19','2.085401e-17','1.9859609e-24','4.6459398e-05','1.0014874e-10','1.31310796e-11','2.2982758e-05','6.0775157e-05','2.0617978e-08','4.9283586e-09','2.0851081e-05','5.306534e-05','5.5673404e-17','6.4250613e-18','0.03742819','9.668686e-14','2.068461e-05','9.454725e-11','1.5962524e-10','2.0549176e-05','3.0971982e-19','1.3361773e-12','6.2888503e-18','3.518919e-13','7.836088e-19','0.0003794642','4.2481716e-11','1.1697825e-12','1.5325759e-05','8.3869396e-05','7.036486e-09','2.9734675e-09','1.5325759e-05','8.3869396e-05','4.8285315e-17','1.177697e-18','0.025196968','0.00029534663','1.8739144e-10','3.2568592e-10','0.00029534663','0.04406723','0.00050391135','0.00049126236'],
['Sweden','201911','1.5099983','6.042558e-07','0.0071798475','2.6383705e-06','3.6153283e-06','0.0022698056','3.2708142e-10','2.585902e-09','4.298784e-12','-0.047612835','1.9125364e-05','6.476921e-06','0.008636861','0.014200226','0.000110182766','0.00021409977','0.008228961','0.013250925','3.7363153e-09','1.2240267e-09','1.7417822','1.6507429e-06','0.01849149','8.981455e-06','9.9006575e-06','0.018316917','7.874331e-10','3.3908414e-06','2.454677e-09','4.572366e-06','2.7025806e-09','0.0053050052','1.0998015e-05','2.2903498e-06','0.011506705','0.023687316','-1.4330669e-05','3.22465e-06','0.011506705','0.023687316','2.4636375e-08','4.1939843e-09','1.4407054','0.09320765','1.2499886e-05','2.5642954e-05','0.09320765','0.9835417','0.15676573','0.15073203','0.0027304285','5.1976703e-14','8.709273e-06','8.941921e-12','2.212766e-11','1.2578972e-06','4.1029483e-20','6.6644097e-18','1.9163945e-23','8.1269434e-05','4.379233e-10','3.695987e-11','6.257203e-05','0.00019407227','1.4229452e-07','2.038062e-07','5.6704335e-05','0.00016754781','2.4490805e-17','2.5527995e-18','0.032134477','1.860238e-13','8.9261564e-05','8.782671e-11','1.4949916e-10','8.863538e-05','1.7011773e-19','9.415116e-12','5.743525e-18','5.6093576e-12','5.274455e-18','0.00017658895','7.475099e-11','2.0709998e-12','5.627e-05','0.00031853674','5.3586002e-09','1.902543e-08','5.627e-05','0.00031853674','3.5096853e-16','6.6110297e-18','0.035189945','0.004870157','5.2634486e-10','1.2282269e-09','0.004870157','0.029297242','0.008058942','0.0077812234'],
['Ireland','201911','1.4688839','5.060522e-07','0.0054654786','4.454126e-07','-1.2297584e-06','0.0014964513','3.9768605e-10','1.1232039e-08','9.961328e-12','-0.038754147','5.3050876e-06','2.2094266e-06','0.0030716346','0.004588621','1.6668486e-05','-1.221093e-05','0.0029307154','0.0043070656','7.446198e-10','2.6971758e-10','1.8629929','1.3101737e-06','0.0106283','4.2573784e-06','-2.749003e-06','0.010491318','5.6505783e-10','1.914824e-06','3.2159022e-09','2.3768214e-06','9.348413e-10','0.0047176834','2.2515658e-05','4.0786344e-06','0.019037493','0.04192954','0.00012545173','-0.00010571834','0.019037493','0.04192954','2.6618498e-08','4.423763e-09','1.5083221','0.03645092','4.7195767e-06','-6.1443493e-06','0.03645092','0.8152834','0.09906271','0.094674736','0.00164913','4.351839e-14','5.751625e-06','1.7506765e-11','2.3094162e-11','5.73667e-07','6.083987e-20','3.2397282e-16','2.4514604e-22','2.4997333e-05','3.3192151e-12','4.8562814e-13','9.211038e-07','2.143692e-06','2.2832891e-09','4.1494617e-09','8.380818e-07','1.8820853e-06','2.7106336e-19','2.971398e-20','0.041053377','8.360077e-14','3.7725582e-05','1.11502876e-10','2.5714605e-10','3.74865e-05','8.4557e-20','2.0892563e-12','4.138399e-18','6.1481625e-13','4.071343e-19','0.0001846461','1.0727791e-09','1.9977447e-11','0.00034850487','0.0021771342','3.8324833e-07','1.5577055e-07','0.00034850487','0.0021771342','1.0971423e-15','1.9287293e-17','0.028333066','0.00043546065','3.3611738e-10','5.6806715e-10','0.00043546065','0.09590566','0.002271512','0.0022438096'],
['Italy','201911','1.4648374','5.2042475e-07','0.0055870987','3.0379906e-06','3.6371694e-06','0.001496997','3.9376835e-10','1.4862821e-08','1.4602156e-11','-0.05456795','7.4107695e-05','1.9873676e-05','0.025837358','0.046631563','-9.5884625e-06','0.0012553716','0.024595808','0.04323827','2.3205432e-08','5.937117e-09','1.8777251','1.3795402e-06','0.01111361','1.1570435e-05','1.066508e-05','0.010968967','5.550481e-10','4.7515628e-06','5.163828e-09','3.5909763e-06','1.5336648e-09','0.0038543777','2.6788039e-05','4.4301787e-06','0.017650416','0.0412886','3.0199271e-05','0.00021935008','0.017650416','0.0412886','2.996986e-08','4.4957305e-09','1.5394435','0.056111075','1.3195131e-05','2.2173317e-05','0.056111075','0.7323546','0.16073053','0.15310487','0.0016512867','1.1126104e-14','1.4552496e-06','1.34389375e-11','1.6135501e-11','2.0177595e-07','3.1400028e-20','8.4239174e-16','7.5634035e-22','9.365507e-05','2.5869195e-08','1.5200952e-09','0.0025531955','0.008917885','6.6237254e-07','1.6466427e-05','0.002313269','0.0076259277','1.9213437e-15','1.1108126e-16','0.034955345','8.6949504e-14','2.981269e-05','1.4412939e-10','2.0253836e-10','2.9662655e-05','4.6382056e-20','1.6395657e-11','7.22036e-18','3.6906016e-12','7.628741e-19','0.00020703074','4.876468e-10','1.0980644e-11','0.00021656993','0.0012806904','7.1280255e-08','1.1268718e-07','0.00021656993','0.0012806904','7.452258e-16','1.5358926e-17','0.017882619','0.0016052803','6.568551e-10','5.346981e-10','0.0016052803','0.08672374','0.016084239','0.014448226'],
['Denmark','201911','1.4954996','8.0964e-07','0.009252991','-5.7997255e-07','4.009781e-06','0.0027659538','8.1175633e-10','2.0742381e-09','2.080033e-12','-0.045267917','1.7948689e-05','5.9045838e-06','0.007858568','0.013059956','-1.5498303e-05','0.00017396601','0.0074869795','0.012177972','4.034718e-09','1.3314667e-09','1.7628324','1.7500979e-06','0.017850947','1.1167659e-06','8.52507e-06','0.017666502','1.2356257e-09','5.2132177e-06','4.9831184e-09','4.493735e-06','2.846692e-09','0.0059163477','9.537142e-06','2.2017352e-06','0.011247563','0.022188673','-1.5529373e-05','1.3218637e-05','0.011247563','0.022188673','1.46013175e-08','2.855467e-09','1.4235004','0.09062933','-6.264961e-06','2.2072902e-05','0.09062933','1.0289931','0.15298241','0.14542854','0.0018094425','5.6483694e-14','8.199026e-06','1.5267473e-11','2.2914993e-11','1.0124006e-06','1.3202714e-19','8.33346e-18','6.8972476e-24','0.000103589075','9.1871893e-10','7.0849826e-11','0.00011977835','0.00038443666','2.8500166e-08','3.3554582e-07','0.00010854017','0.00033112033','3.1549727e-17','3.3756513e-18','0.031010158','1.2381152e-13','6.294095e-05','8.726588e-11','1.2068194e-10','6.253263e-05','1.9561055e-19','1.6669595e-11','1.1408151e-17','4.4206943e-12','5.1066512e-18','0.00014359655','6.042879e-11','2.0946944e-12','6.636621e-05','0.00032523475','7.0798123e-09','1.1983886e-08','6.636621e-05','0.00032523475','1.1934167e-16','3.1378724e-18','0.031954188','0.0037098948','8.1483104e-10','8.184426e-10','0.0037098948','0.035155293','0.006892841','0.006586265'],
['Finland','201911','1.5089076','5.381191e-07','0.0063324505','4.6052346e-06','3.2335731e-06','0.0019797692','3.2385428e-10','9.016273e-10','2.5281475e-13','-0.047366735','2.24067e-05','7.3198444e-06','0.009721884','0.016205477','0.0002686242','0.0002610782','0.0092614405','0.015107999','3.4317813e-09','1.0483817e-09','1.7192072','1.6439352e-06','0.01863955','1.5721023e-05','8.959057e-06','0.018465659','1.0688244e-09','2.5640381e-06','2.386708e-09','4.0156556e-06','1.9610071e-09','0.02713559','3.0087808e-06','9.77627e-07','0.0044956496','0.0072556455','1.1226756e-05','3.1729414e-06','0.0044956496','0.0072556455','2.853657e-09','8.900214e-10','1.4078931','0.08042002','2.5710611e-05','2.4405173e-05','0.08042002','1.1158555','0.12885422','0.12322958','0.0026830174','3.0243186e-14','4.6039654e-06','8.255579e-12','1.5071803e-11','6.4415894e-07','3.1951173e-20','1.4624669e-18','1.3651535e-25','8.952024e-05','9.2965863e-10','7.232343e-11','0.00012082158','0.00038946143','3.5834472e-07','5.43887e-07','0.00010943208','0.00033545232','3.4245936e-17','2.8587784e-18','0.022729427','1.4470661e-13','6.587957e-05','8.711574e-11','1.3474165e-10','6.5463275e-05','3.3140803e-19','3.5144988e-12','2.462678e-18','3.0436977e-12','1.0715465e-18','0.0003876097','2.0050609e-11','1.0652422e-12','1.5432974e-05','5.69962e-05','2.5774987e-09','1.8078623e-09','1.5432974e-05','5.69962e-05','1.1033668e-17','7.081703e-19','0.022904193','0.002721167','3.9107248e-10','1.0930388e-09','0.002721167','0.024128914','0.004945081','0.0046991324'],
['Belgium','201911','1.4654762','8.449661e-07','0.009046108','3.1424338e-06','3.6969243e-06','0.0024338958','1.6044776e-09','2.224191e-09','8.4067063e-13','-0.043441094','1.0923154e-05','3.703531e-06','0.0049874154','0.008118369','1.9183199e-05','4.5794804e-05','0.004753516','0.0075780065','1.40957e-09','4.3030327e-10','1.839328','1.7402673e-06','0.013777813','8.157864e-06','8.24412e-06','0.013595763','1.6232974e-09','3.3436702e-06','5.3727707e-09','3.12152e-06','1.6947367e-09','0.016993316','1.1333042e-05','2.4932087e-06','0.0116153825','0.023857791','8.011402e-05','5.4022923e-05','0.0116153825','0.023857791','1.0277519e-08','2.0229807e-09','1.5033429','0.049894817','1.421765e-05','1.3985943e-05','0.049894817','1.0234635','0.10469498','0.09736068','0.00092115806','5.791845e-14','7.63675e-06','2.698955e-11','2.4358255e-11','7.925698e-07','6.380616e-19','1.06301344e-17','4.4588777e-24','9.863774e-05','1.9770725e-10','1.3978854e-11','2.3435998e-05','7.756818e-05','1.2252043e-08','3.653414e-08','2.1233127e-05','6.664996e-05','2.5228742e-18','2.0797584e-19','0.024931407','1.5910666e-13','2.7138647e-05','1.7884381e-10','1.8792214e-10','2.692626e-05','5.108006e-19','4.7650117e-12','2.0564627e-17','1.1717939e-12','7.772696e-19','0.00042356816','1.8954892e-10','6.824733e-12','0.00019251491','0.0009339793','2.3573099e-08','1.3887885e-08','0.00019251491','0.0009339793','1.7136034e-16','4.7699664e-18','0.020901661','0.00071699265','3.9213388e-10','5.3771354e-10','0.00071699265','0.06862748','0.0020944395','0.0020012227'],
['Croatia','201911','1.4824971','4.9928326e-07','0.0055226','2.6997934e-06','3.6628614e-06','0.0015613992','3.270318e-10','3.066752e-09','1.4056767e-12','-0.05103282','4.8411068e-05','1.3824317e-05','0.018084692','0.0318548','4.938916e-05','0.0007745736','0.01721921','0.029588314','1.4171427e-08','3.751532e-09','1.804509','1.4015324e-06','0.013025189','9.466533e-06','1.1150678e-05','0.012877757','7.4568496e-10','4.106114e-06','2.709291e-09','3.890393e-06','2.1044921e-09','0.0036828662','1.2384082e-05','2.3186096e-06','0.010792965','0.023987973','5.2748364e-06','0.000108412874','0.010792965','0.023987973','1.8518703e-08','2.9397282e-09','1.4848068','0.0667179','1.26353025e-05','2.74744e-05','0.0667179','0.9489197','0.14110826','0.13473256','0.001744814','1.3648759e-14','1.4799106e-06','1.1593649e-11','1.1916547e-11','1.7533516e-07','1.8006504e-20','7.066572e-17','3.168613e-23','7.794997e-05','1.07068745e-08','7.176461e-10','0.0011996862','0.0040496136','5.2318484e-07','4.3582004e-06','0.0010867243','0.0034746772','1.0404937e-15','6.3331304e-17','0.030754402','8.8759586e-14','3.2322387e-05','1.1564684e-10','1.2833067e-10','3.2163916e-05','1.403438e-19','2.2935721e-11','6.7514784e-18','2.408877e-12','2.8991818e-18','7.4898446e-05','1.6643564e-10','4.1647606e-12','0.00012392549','0.00072544423','1.3994747e-08','2.8915517e-08','0.00012392549','0.00072544423','3.183659e-16','6.7269946e-18','0.017574087','0.001365531','6.8186695e-10','7.5340084e-10','0.001365531','0.0992018','0.008727987','0.0079706395'],
['Austria','201912','1.4631764','3.8212988e-07','0.004056534','3.0020663e-06','-1.3314828e-06','0.0010936296','4.115361e-10','1.5111872e-09','9.237643e-14','-0.039139297','1.093769e-05','3.7574048e-06','0.005065341','0.008225216','6.714009e-05','8.755409e-05','0.0048279595','0.0076801977','3.752834e-09','1.1312141e-09','1.870964','1.1617514e-06','0.008690577','9.9445015e-06','-3.7711425e-06','0.008569274','9.749376e-10','1.7833665e-06','2.1562265e-09','1.8754549e-06','1.2028464e-09','0.039719403','1.5420377e-06','4.6106965e-07','0.0018367274','0.0031858631','1.0885621e-05','-4.6589907e-06','0.0018367274','0.0031858631','1.7396526e-09','4.5080195e-10','1.5521387','0.027321506','1.5113881e-05','-5.6776353e-06','0.027321506','1.2054449','0.051479876','0.047851063','0.0017658697','1.546859e-14','1.918026e-06','8.569821e-12','1.4716922e-11','1.9896856e-07','2.651379e-20','4.1209725e-18','9.8977336e-26','8.454704e-05','9.155816e-10','8.313475e-11','0.00014486087','0.00043147214','3.7002348e-08','2.4085065e-07','0.00013141155','0.00037343675','1.4131014e-16','1.0182813e-17','0.036113575','4.5094215e-14','1.7123783e-05','1.0553999e-10','1.5980334e-10','1.7009295e-05','1.9289542e-19','3.9712326e-12','2.9713322e-18','4.5307825e-13','5.912817e-19','0.00079745403','2.6133826e-12','1.4677749e-13','4.994027e-06','1.8088336e-05','3.159883e-10','2.3870086e-10','4.994027e-06','1.8088336e-05','6.310224e-18','3.3221468e-19','0.046079297','0.0002886099','1.860793e-10','3.5948783e-10','0.0002886099','0.05631822','0.0009367871','0.0008515398'],
['Bulgaria','201912','1.453297','5.186422e-07','0.0054315277','2.064532e-06','-1.3172008e-06','0.0014026402','4.310642e-10','2.0371695e-09','1.4494663e-12','-0.04331004','1.520266e-05','5.483245e-06','0.007382553','0.011816932','7.293568e-05','3.8700146e-05','0.00703627','0.011045767','7.090571e-09','2.4895372e-09','1.9448651','1.643266e-06','0.010833514','7.5516086e-06','-3.771809e-06','0.01066224','1.0169346e-09','1.0106874e-05','9.042928e-09','4.536247e-06','2.769288e-09','0.043661457','2.96489e-06','7.576539e-07','0.0027467948','0.005201215','2.5262366e-06','6.8341874e-06','0.0027467948','0.005201215','2.7649611e-09','6.1573296e-10','1.6243075','0.06538872','1.5131147e-05','-6.7333e-06','0.06538872','1.3014781','0.0986711','0.09370012','0.0014345673','2.403715e-14','2.8628103e-06','7.1264062e-12','9.245948e-12','2.596258e-07','2.8111488e-20','3.0708564e-17','4.4533937e-23','0.000103005266','7.17935e-10','7.422993e-11','0.00013044926','0.00036872455','6.376754e-08','6.739381e-08','0.000118374286','0.00032017333','2.0654553e-16','2.5271865e-17','0.033683132','2.2013528e-13','2.1244083e-05','8.576754e-11','9.6195205e-11','2.1028405e-05','5.343528e-19','5.1753178e-11','3.1400878e-17','2.968393e-12','1.9731273e-18','0.0015207306','1.1118253e-11','4.403441e-13','1.0940474e-05','4.9800663e-05','2.9045968e-10','6.6224115e-10','1.0940474e-05','4.9800663e-05','1.4348988e-17','4.745334e-19','0.037851408','0.0018416729','7.7983114e-10','6.903409e-10','0.0018416729','0.07080371','0.002956556','0.0028647608'],
['Cyprus','201912','1.4394568','5.0669644e-07','0.005138564','4.7696926e-06','1.422584e-07','0.0012514909','2.0333524e-10','9.771884e-09','8.2407406e-12','-0.0566443','3.626562e-05','1.0356128e-05','0.013495868','0.02377157','0.00015423202','0.00021031307','0.012848386','0.022078432','1.870077e-08','4.8098094e-09','2.0355768','1.5755693e-06','0.008625043','1.8827732e-05','-2.832186e-07','0.008461562','4.4308104e-10','5.403222e-06','4.4621435e-09','4.875928e-06','2.3868383e-09','0.0064084413','2.0908363e-05','3.2939593e-06','0.011795154','0.028421191','0.00011684055','9.5465e-05','0.011795154','0.028421191','1.9282444e-08','2.7171805e-09','1.6602087','0.062375087','2.8418155e-05','2.2495994e-06','0.062375087','0.9549287','0.12833025','0.122587174','0.0014074033','2.82327e-14','2.8615154e-06','1.0358204e-11','1.1610562e-11','2.2352097e-07','1.5722513e-20','4.8176476e-16','2.3512404e-22','7.152298e-05','2.8756366e-09','1.6207302e-10','0.00026982435','0.0009518565','1.4909128e-07','7.2354715e-07','0.00024439392','0.0008131986','1.048753e-15','5.568984e-17','0.033835284','3.7902217e-13','1.9523302e-05','1.5040665e-10','1.3332953e-10','1.9220824e-05','1.5830877e-19','1.4920494e-11','1.29722715e-17','5.07855e-12','2.4164113e-18','0.00013027732','4.9999116e-10','8.663123e-12','0.00019603084','0.0012837274','7.178675e-08','9.857544e-08','0.00019603084','0.0012837274','4.373395e-16','6.667585e-18','0.029294712','0.0019590457','6.798743e-10','7.281236e-10','0.0019590457','0.09882095','0.0058669313','0.005608032'],
['Czech Republic','201912','1.4577429','6.34315e-07','0.0067004813','4.522586e-06','-1.6681796e-07','0.0017570226','1.1915424e-09','1.5595459e-09','6.586102e-14','-0.039560378','1.3922648e-05','4.737583e-06','0.0063467068','0.010394796','0.00010161528','8.26596e-05','0.006047939','0.009702227','2.4709341e-09','7.305367e-10','1.8929533','1.5393532e-06','0.011463177','1.292322e-05','-9.2963495e-07','0.011302305','2.0664932e-09','5.5600585e-06','7.280917e-09','2.7124447e-06','1.7261672e-09','0.03424608','3.841819e-06','1.1320583e-06','0.004410368','0.007712809','3.086388e-05','-7.228203e-06','0.004410368','0.007712809','3.4127867e-09','8.9198543e-10','1.5582694','0.039389964','2.0966365e-05','-2.1499393e-06','0.039389964','1.1848829','0.07566167','0.06986468','0.00088925124','2.590137e-14','3.458946e-06','1.7398258e-11','2.593956e-11','3.5778913e-07','1.7515427e-19','4.7864442e-18','1.9657463e-26','7.522117e-05','1.419268e-09','1.2595874e-10','0.00021669215','0.00065853633','7.3910634e-08','2.3279719e-07','0.00019648504','0.0005694479','3.9953585e-17','2.9077293e-18','0.027280418','1.0533531e-13','2.6139136e-05','1.5328103e-10','2.1557146e-10','2.5909028e-05','7.0079636e-19','8.466944e-12','1.0996504e-17','6.7612685e-13','6.235741e-19','0.00041979417','1.9825846e-11','1.1481211e-12','2.429696e-05','9.0162815e-05','3.6273224e-09','1.7751314e-09','2.429696e-05','9.0162815e-05','2.7797514e-17','1.407443e-18','0.020479264','0.00040851132','3.5587633e-10','5.8585065e-10','0.00040851132','0.06787282','0.0013645394','0.0012263926'],
['Estonia','201912','1.5065668','4.853416e-07','0.005782781','5.1726247e-06','1.0966132e-06','0.0018296292','4.4928355e-10','2.2157887e-09','9.66247e-13','-0.04005073','1.0668324e-05','4.0448886e-06','0.0054711173','0.008598463','9.058608e-05','4.2585776e-05','0.0052152663','0.00804766','2.3838103e-09','8.079606e-10','1.7601573','1.3448973e-06','0.01491603','1.5403666e-05','1.4889605e-06','0.014773705','1.2639403e-09','2.1631477e-06','1.7245433e-09','3.4226703e-06','2.5770706e-09','0.014961002','1.2797306e-05','2.9450453e-06','0.01554381','0.030786606','0.00015563294','3.765889e-05','0.01554381','0.030786606','2.5390753e-08','4.7158215e-09','1.4275919','0.06970331','2.4725075e-05','8.814237e-06','0.06970331','0.90667963','0.12978694','0.1251413','0.0029129407','3.952001e-14','6.9517446e-06','2.9116466e-11','2.1348378e-11','9.987291e-07','1.2449756e-19','3.214073e-18','1.929535e-24','9.034708e-05','2.1002772e-10','2.3702762e-11','4.1227475e-05','0.00011453473','3.1775027e-08','2.5065207e-08','3.7397334e-05','9.963897e-05','1.2613811e-17','1.3488168e-18','0.03471536','1.1918251e-13','5.632699e-05','1.7038546e-10','1.8492641e-10','5.589002e-05','1.2028718e-18','4.5749363e-12','2.6914437e-18','4.320209e-12','4.6563705e-18','0.00036221833','1.760567e-10','7.335855e-12','0.00030218455','0.0013168481','5.324004e-08','2.1716364e-08','0.00030218455','0.0013168481','1.1332373e-15','3.2575557e-17','0.030095382','0.003160073','6.835189e-10','1.1789945e-09','0.003160073','0.077811144','0.0050360416','0.004805537'],
['France','201912','1.4586688','3.6868656e-07','0.0038721112','3.4903546e-06','-4.721155e-07','0.0010184178','3.5858175e-10','2.6974043e-09','1.1492925e-12','-0.041281305','1.7479113e-05','5.598642e-06','0.007349876','0.012456087','0.00010896524','0.00015267647','0.0069989166','0.011603018','3.1636205e-09','8.772425e-10','1.9031631','1.2215226e-06','0.008236153','1.4893838e-05','-3.9415663e-06','0.008108916','5.6067884e-10','1.0675244e-06','1.7488041e-09','1.8921255e-06','1.092932e-09','0.02939346','1.00655525e-05','2.0798311e-06','0.011080297','0.023749374','0.00014966023','-4.3449036e-06','0.011080297','0.023749374','1.3961847e-08','2.3812647e-09','1.5607511','0.024420438','1.8005196e-05','-1.4411453e-06','0.024420438','0.90411615','0.07273287','0.06890001','0.0014591237','1.0512408e-14','1.3082606e-06','1.6737013e-11','1.6968262e-11','1.4260839e-07','5.7369215e-20','1.9017872e-17','1.2832418e-23','0.00014045063','1.657383e-09','1.3366154e-10','0.00022059095','0.0007130783','1.4252186e-07','3.9311408e-07','0.00019971491','0.0006144726','7.064659e-17','4.8329065e-18','0.029706037','9.057536e-14','1.1621354e-05','3.1929692e-10','2.7198233e-10','1.1522131e-05','1.1626667e-19','7.632827e-13','2.9306517e-18','5.311782e-13','6.507276e-19','0.001339423','3.3765032e-10','8.994114e-12','0.00032835174','0.0018682416','1.3663363e-07','2.5894456e-08','0.00032835174','0.0018682416','7.9113433e-16','1.3035759e-17','0.033528745','0.00016051592','4.021076e-10','3.9296727e-10','0.00016051592','0.14803348','0.0027562508','0.0026376918'],
['Germany','201912','1.4666412','5.699722e-07','0.006109291','5.4184216e-06','9.818717e-07','0.0016563871','9.822592e-10','1.9824153e-09','3.2602848e-13','-0.039001923','1.1585808e-05','4.0223904e-06','0.0053599123','0.008751821','0.00015321407','7.7270095e-05','0.00510666','0.008171156','1.8410184e-09','5.470836e-10','1.8520706','1.3027912e-06','0.010207643','1.5627711e-05','5.242985e-07','0.01007151','1.0257001e-09','3.8351777e-06','7.12305e-09','2.17459e-06','1.4256792e-09','0.0309333','6.1987103e-06','1.6001152e-06','0.0076149106','0.014523641','6.994004e-05','1.0809113e-05','0.0076149106','0.014523641','6.740926e-09','1.4768962e-09','1.5153831','0.03277436','2.3090104e-05','1.7819284e-06','0.03277436','1.0650389','0.07236684','0.06719722','0.00092442683','3.1835093e-14','4.1026065e-06','2.0417909e-11','2.0499097e-11','4.0255054e-07','1.7042502e-19','1.2538714e-17','6.1476616e-25','0.000102562444','7.646778e-10','7.286176e-11','0.00012181313','0.00036929155','3.41288e-07','1.4502382e-07','0.00011034394','0.00031959856','2.5244108e-17','1.8458626e-18','0.025024906','9.3507436e-14','2.1983837e-05','1.9332128e-10','1.8956926e-10','2.1795418e-05','2.0947382e-19','4.253728e-12','9.021588e-18','3.3923773e-13','2.74386e-19','0.0008179044','8.960763e-11','3.8074877e-12','0.000119534074','0.00054427556','1.7484519e-08','7.115752e-09','0.000119534074','0.00054427556','1.4370586e-16','4.5110956e-18','0.027456844','0.00026037465','3.587582e-10','3.786417e-10','0.00026037465','0.08391158','0.0014250877','0.0013200351'],
['Greece','201912','1.4578618','4.1114797e-07','0.00430305','2.5470265e-06','-1.4046203e-06','0.0011242654','2.959674e-10','3.0161769e-09','1.7809955e-12','-0.048686586','2.154035e-05','7.172459e-06','0.009501434','0.01579266','0.00015278773','8.843893e-05','0.009050557','0.014728375','1.2236127e-08','3.821505e-09','1.919566','1.2776853e-06','0.0086403405','1.0650063e-05','-4.226957e-06','0.008507384','5.2496857e-10','6.536343e-06','7.919572e-09','3.7185282e-06','2.2584612e-09','0.016055593','6.4745905e-06','1.2827196e-06','0.0056688017','0.012407831','3.0216006e-05','2.1643502e-06','0.0056688017','0.012407831','8.826766e-09','1.4444066e-09','1.5469894','0.057226803','1.497891e-05','-9.551585e-06','0.057226803','1.0781722','0.09837036','0.09399461','0.0015930221','1.1499896e-14','1.3489756e-06','8.245019e-12','8.67257e-12','1.4325887e-07','2.238772e-20','1.1930716e-16','6.744237e-23','0.00010645327','8.1293783e-10','8.474875e-11','0.00014656287','0.0004194788','1.1867261e-07','1.0500238e-07','0.00013291546','0.0003642009','6.6036695e-16','6.997526e-17','0.039408118','5.7342226e-14','1.4896805e-05','9.7735285e-11','8.85695e-11','1.4827753e-05','7.8371667e-20','4.0249606e-11','1.06372045e-16','2.5806874e-12','2.1161168e-18','0.00060959766','7.9056255e-11','2.4381205e-12','6.7057765e-05','0.00035853052','1.7520456e-08','3.0953755e-09','6.7057765e-05','0.00035853052','1.6577789e-16','3.4685315e-18','0.031990204','0.001622176','6.493124e-10','5.357145e-10','0.001622176','0.110144146','0.0026646508','0.002595253'],
['Hungary','201912','1.4632158','6.567662e-07','0.0070158974','3.0848696e-06','-1.4045e-06','0.0018789845','8.191677e-10','1.4482753e-09','2.08818e-13','-0.043521192','1.0652049e-05','3.8386434e-06','0.0051688827','0.008276756','5.5943885e-05','5.6851743e-05','0.0049264263','0.0077367416','2.419579e-09','7.9453016e-10','1.8856685','1.8509619e-06','0.014492813','1.0280654e-05','-2.0024268e-06','0.0142990155','1.9641098e-09','6.7787673e-06','6.270963e-09','4.0906593e-06','2.5397497e-09','0.03542488','3.2539965e-06','9.504296e-07','0.0032861165','0.005774631','1.5548494e-05','5.1864968e-06','0.0032861165','0.005774631','2.2454039e-09','6.062686e-10','1.5531927','0.06581533','1.728405e-05','-2.8374247e-06','0.06581533','1.277833','0.10137596','0.09550506','0.0015822694','4.25976e-14','5.3221843e-06','1.4822005e-11','3.508325e-11','5.395147e-07','8.9768406e-20','6.198653e-18','1.1342828e-24','9.385683e-05','2.3915614e-10','2.482451e-11','4.3224714e-05','0.00012326823','1.1379341e-08','6.2627784e-08','3.921212e-05','0.00010703574','3.0362902e-17','3.0213562e-18','0.036708027','1.4345538e-13','5.084987e-05','1.7125965e-10','2.5453786e-10','5.043935e-05','7.270284e-19','2.5944908e-11','8.416315e-18','2.9933848e-12','2.811256e-18','0.00072155293','7.17315e-12','3.6647532e-13','5.747935e-06','2.3243936e-05','9.043601e-10','9.470856e-10','5.747935e-06','2.3243936e-05','5.646144e-18','2.857838e-19','0.02725419','0.0023917465','5.1336935e-10','1.2767413e-09','0.0023917465','0.03441016','0.0034249069','0.0032987488'],
['Latvia','201912','1.5004262','5.117048e-07','0.0060025784','5.3471135e-06','1.3990395e-06','0.0018549152','4.8675386e-10','1.9202888e-09','6.181645e-13','-0.040434357','1.08857585e-05','4.1445096e-06','0.0056105554','0.008800789','8.750241e-05','4.0791103e-05','0.0053483215','0.008238029','2.6595086e-09','9.0117747e-10','1.7676605','1.4204459e-06','0.015131961','1.5788637e-05','2.0736295e-06','0.014982023','1.3066981e-09','2.648033e-06','2.2382063e-09','3.389471e-06','2.609365e-09','0.015907317','1.0388727e-05','2.412944e-06','0.012178417','0.024039123','0.0001246757','1.529911e-05','0.012178417','0.024039123','1.6495942e-08','3.041785e-09','1.4288284','0.066225424','2.4911133e-05','9.042307e-06','0.066225424','0.962396','0.120200455','0.115339905','0.0023205758','3.643846e-14','6.3411117e-06','2.8516979e-11','2.1755691e-11','8.8505215e-07','1.0201643e-19','2.4546777e-18','9.119537e-25','7.818485e-05','1.8988958e-10','2.1686663e-11','3.793219e-05','0.0001044983','3.1353036e-08','2.8377938e-08','3.4413693e-05','9.095006e-05','1.5189207e-17','1.5680524e-18','0.029427256','1.0985533e-13','5.2799744e-05','1.8762959e-10','1.9306698e-10','5.2388954e-05','6.106827e-19','5.379741e-12','3.8271087e-18','3.4087292e-12','4.3130103e-18','0.00029889226','1.2237553e-10','4.685164e-12','0.00016458191','0.0007631386','3.8081417e-08','1.484054e-08','0.00016458191','0.0007631386','4.0736025e-16','1.1044305e-17','0.023352636','0.0022740876','7.0489253e-10','1.2095699e-09','0.0022740876','0.061796356','0.0033722112','0.0032073709'],
['Lithuania','201912','1.4965333','5.882611e-07','0.006799553','5.4712095e-06','1.2702649e-06','0.0020568676','6.625061e-10','1.6899799e-09','3.703718e-13','-0.04148536','1.1149768e-05','4.2827114e-06','0.0058131553','0.009072475','0.00011163237','2.760549e-05','0.0055420217','0.008494526','2.309028e-09','7.906562e-10','1.7672155','1.552167e-06','0.016121767','1.6021271e-05','1.9114173e-06','0.01595807','1.4239988e-09','3.701512e-06','3.6968657e-09','3.4427487e-06','2.736705e-09','0.023708154','6.577396e-06','1.7224982e-06','0.00824426','0.015266865','7.735878e-05','-6.0512257e-06','0.00824426','0.015266865','7.014461e-09','1.5750954e-09','1.4255283','0.06701594','2.4296474e-05','6.338804e-06','0.06701594','1.0615213','0.11427601','0.10879164','0.0020353044','4.722209e-14','7.236519e-06','3.0659954e-11','2.0193985e-11','8.8156105e-07','1.6689868e-19','2.5927745e-18','4.7274595e-25','8.310408e-05','1.9770424e-10','2.6844133e-11','4.8784616e-05','0.00012257465','8.843579e-08','3.1895272e-08','4.4321714e-05','0.000107228145','9.7929904e-18','1.0626523e-18','0.025465228','1.5133423e-13','5.677048e-05','2.3494925e-10','1.9297146e-10','5.6286062e-05','4.961328e-19','8.79025e-12','7.962869e-18','2.394737e-12','2.8154509e-18','0.00036892534','8.0017284e-11','3.227268e-12','9.2565475e-05','0.00042379875','2.2499508e-08','5.761616e-09','9.2565475e-05','0.00042379875','1.372597e-16','4.3821216e-18','0.022131095','0.0018188218','6.625169e-10','1.0208852e-09','0.0018188218','0.062840074','0.0023157035','0.0022063968'],
['Luxembourg','201912','1.4767162','5.465398e-07','0.0059411437','4.5044894e-06','9.56923e-07','0.0016586985','1.1015565e-09','2.5679596e-09','8.990659e-13','-0.039682932','1.1275192e-05','3.8713238e-06','0.0051387777','0.008444498','0.00011475463','9.908914e-05','0.004895325','0.007881314','1.8485459e-09','5.567156e-10','1.8148681','1.3058171e-06','0.010897624','1.5185629e-05','6.9989966e-07','0.010760911','1.158954e-09','2.646655e-06','5.4946145e-09','2.017105e-06','1.2173629e-09','0.023614677','8.446428e-06','2.0150255e-06','0.010502085','0.020916859','9.8923534e-05','2.3808492e-05','0.010502085','0.020916859','1.0153764e-08','2.074801e-09','1.4885688','0.030627787','2.0592843e-05','3.2271957e-06','0.030627787','0.97115195','0.07682852','0.07184554','0.0014827799','1.7911208e-14','2.3864804e-06','2.039343e-11','2.2071742e-11','2.8604558e-07','3.2668573e-19','1.1492487e-17','7.155655e-24','0.00011707311','4.692746e-10','4.199885e-11','6.9168695e-05','0.00021644247','8.298322e-08','1.12624555e-07','6.2623105e-05','0.00018696688','2.2794666e-17','1.685538e-18','0.027579242','6.1633356e-14','2.5896512e-05','2.0600661e-10','2.0032839e-10','2.5747087e-05','2.9305187e-19','2.814596e-12','2.135716e-17','3.1059871e-13','4.2750584e-19','0.0011641154','1.3664597e-10','5.2424887e-12','0.00018745402','0.0008915694','3.0867408e-08','1.8414925e-08','0.00018745402','0.0008915694','2.771435e-16','8.240467e-18','0.030574724','0.00020015483','3.5301106e-10','4.6998133e-10','0.00020015483','0.1014063','0.0014537825','0.0014042825'],
['Malta','201912','1.4375528','4.438534e-07','0.0044918805','4.263199e-06','-1.577385e-06','0.0010906929','1.8670791e-10','3.0288984e-08','7.02236e-11','-0.05505895','0.000105970605','2.9364297e-05','0.03818424','0.068194814','0.0003354197','0.00086207653','0.03634877','0.06328562','5.4934972e-08','1.3748679e-08','2.023897','1.5849823e-06','0.009047485','1.8133329e-05','-5.4968596e-06','0.008882801','4.4782797e-10','5.215389e-06','2.458361e-09','3.016725e-06','1.1112558e-09','0.009321892','3.670203e-05','6.177927e-06','0.018703321','0.043113783','0.000284593','-0.0001439055','0.018703321','0.043113783','4.7859746e-08','7.289908e-09','1.6887065','0.0353915','1.9571527e-05','-1.1110294e-05','0.0353915','0.5572464','0.16023967','0.15176547','0.0010679346','6.752573e-15','7.737633e-07','1.4122842e-11','1.8694553e-11','8.466323e-08','5.722616e-21','1.5478998e-15','4.589329e-21','0.00011470393','6.1048176e-08','3.844572e-09','0.0065565077','0.022022','1.3550717e-06','1.4127072e-05','0.005942964','0.018876389','1.4096171e-14','8.156654e-16','0.03192218','9.300372e-14','1.9067335e-05','2.0299457e-10','3.2509703e-10','1.8917855e-05','3.466552e-20','1.12448856e-10','2.8165268e-18','3.33588e-12','3.464824e-19','0.00010471222','1.219173e-09','2.7525515e-11','0.00023806343','0.0014320959','5.807353e-07','2.5881192e-07','0.00023806343','0.0014320959','1.6971801e-15','3.5558883e-17','0.016986726','0.0010719525','6.0925426e-10','9.761878e-10','0.0010719525','0.099979565','0.022931062','0.019918224'],
['Netherlands','201912','1.4653468','7.7630676e-07','0.008357768','5.8761575e-06','3.2813434e-06','0.0022667393','1.8101705e-09','4.934812e-09','4.818269e-12','-0.03929921','8.510536e-06','3.0717724e-06','0.0041220705','0.00661311','8.751961e-05','7.780232e-05','0.003928253','0.0061811246','1.6980927e-09','5.133731e-10','1.8533964','1.5959034e-06','0.013008347','1.8249693e-05','4.620122e-06','0.012841253','1.9057274e-09','5.432981e-06','1.1645057e-08','2.2849558e-06','1.4644131e-09','0.015816187','2.0522462e-05','4.1633957e-06','0.018421544','0.03896344','0.0002330346','7.0010596e-05','0.018421544','0.03896344','2.463151e-08','4.3644603e-09','1.5339596','0.033526387','2.3677941e-05','6.6323378e-06','0.033526387','0.8847638','0.10046915','0.09377855','0.0010195203','6.366274e-14','8.56783e-06','2.6840847e-11','3.48937e-11','8.756463e-07','1.2139159e-18','4.541422e-17','6.2130035e-23','9.885813e-05','2.2501201e-10','2.1487203e-11','3.575051e-05','0.00010863512','3.126263e-08','8.6028436e-08','3.2378062e-05','9.402209e-05','2.1115648e-17','1.5575891e-18','0.02823558','1.569444e-13','3.8244525e-05','2.3129354e-10','2.473861e-10','3.789351e-05','1.5780145e-18','6.9981998e-12','3.8974904e-17','3.7061435e-13','6.2852673e-19','0.0004944431','7.1853995e-10','2.0081829e-11','0.00047869643','0.0025412599','1.8521753e-07','8.863429e-08','0.00047869643','0.0025412599','1.1985584e-15','2.789293e-17','0.021634243','0.00027865064','3.293925e-10','5.113467e-10','0.00027865064','0.11661365','0.002936113','0.0029225906'],
['Poland','201912','1.469063','7.777158e-07','0.008413376','5.8581745e-06','1.1042195e-06','0.0023063486','1.7320324e-09','1.3749004e-09','1.4464191e-13','-0.040429424','1.29391365e-05','4.5497873e-06','0.0061017126','0.009879358','0.00014964082','6.586381e-05','0.0058147144','0.00922842','1.9721482e-09','6.350443e-10','1.8388954','1.9061428e-06','0.016031144','1.6941216e-05','1.5738152e-06','0.015831282','3.4505865e-09','6.9308553e-06','1.0930304e-08','3.3243393e-06','2.4629756e-09','0.031006895','4.5048646e-06','1.3260337e-06','0.0054113767','0.009448601','4.193744e-05','-6.4427745e-06','0.0054113767','0.009448601','3.7211614e-09','9.826365e-10','1.5123123','0.055575766','2.5720909e-05','1.0068568e-06','0.055575766','1.1798754','0.09934814','0.0923906','0.0014491966','4.13792e-14','5.869928e-06','2.8288582e-11','3.1026275e-11','7.0811217e-07','4.2866958e-19','2.293513e-18','1.7910246e-25','0.00010503567','7.5528084e-10','7.696409e-11','0.00013156745','0.00038399678','2.7093748e-07','1.0794695e-07','0.00011926479','0.00033308973','1.4708211e-17','1.360658e-18','0.03378558','1.7768412e-13','4.775955e-05','2.5291458e-10','2.4948926e-10','4.7375073e-05','1.8700064e-18','1.6922887e-11','2.25479e-17','1.912118e-12','1.3627327e-18','0.00039900662','3.1308574e-11','1.6876338e-12','3.5999543e-05','0.00014014676','7.591891e-09','2.922239e-09','3.5999543e-05','0.00014014676','2.9585134e-17','1.3545776e-18','0.031062178','0.0016196453','4.7074705e-10','7.3271617e-10','0.0016196453','0.055673424','0.0025047713','0.0023713117'],
['Portugal','201912','1.4546297','4.138298e-07','0.0042567905','2.0873158e-06','-1.0984421e-06','0.0010751331','4.2384007e-10','4.150963e-09','4.019249e-12','-0.043652404','1.12465905e-05','3.7733691e-06','0.005022624','0.008274953','-3.563295e-05','-4.1217634e-05','0.0047851875','0.0077195764','3.2121692e-09','9.788007e-10','1.9389586','1.4066807e-06','0.009408884','1.06427005e-05','-4.3663954e-06','0.009262269','1.0350389e-09','2.5205254e-06','3.9052956e-09','2.3080358e-06','1.224131e-09','0.020545965','1.6898548e-05','3.3651247e-06','0.014261264','0.031168735','0.00015628527','1.5877029e-05','0.014261264','0.031168735','2.2117753e-08','3.951887e-09','1.6352308','0.026144','9.834554e-06','-6.145867e-06','0.026144','0.8876273','0.07925347','0.075370476','0.0021506902','1.5589873e-14','1.1959672e-06','2.0107473e-11','8.7076535e-12','6.965272e-08','7.310465e-20','1.8340826e-16','1.8067135e-22','9.566039e-05','3.825487e-10','3.0139932e-11','5.0362276e-05','0.00016149705','7.097521e-08','3.999357e-08','4.5620418e-05','0.00013911606','1.8364725e-17','1.5351361e-18','0.043960515','1.2952974e-13','1.4753048e-05','2.9678063e-10','1.3980986e-10','1.4705924e-05','4.635348e-19','2.3637485e-12','5.9379533e-18','9.331748e-13','5.6091117e-19','0.0009701954','5.710635e-10','1.564361e-11','0.00042178683','0.0023387307','2.1030746e-07','3.424637e-08','0.00042178683','0.0023387307','1.3212883e-15','3.1322934e-17','0.03699709','0.00013838593','6.305809e-10','2.9484762e-10','0.00013838593','0.11141418','0.0023626422','0.0023756465'],
['Romania','201912','1.4592042','5.7486085e-07','0.0061193933','2.667152e-06','-8.3273915e-07','0.0016342478','7.881038e-10','1.059266e-09','2.0107935e-13','-0.041152194','1.1749623e-05','4.532886e-06','0.006171402','0.009589003','4.864282e-05','1.2428384e-05','0.005884091','0.008979874','4.3152024e-09','1.5557562e-09','1.9076936','2.104645e-06','0.015136883','1.0101998e-05','-2.1264998e-06','0.014917027','3.4441638e-09','5.429001e-06','5.4375753e-09','3.708441e-06','2.195048e-09','0.049706466','1.279146e-06','4.2484646e-07','0.0014665052','0.002370834','6.2814443e-06','-4.0492935e-07','0.0014665052','0.002370834','7.7983586e-10','2.1408547e-10','1.5878285','0.056945834','1.5979496e-05','-4.4128483e-06','0.056945834','1.3423901','0.090162285','0.084847465','0.0014830512','3.9039623e-14','5.431883e-06','1.06120035e-11','9.375248e-12','6.0668725e-07','9.75071e-20','2.2436515e-18','9.30173e-25','0.00010944471','1.9207538e-10','2.650366e-11','4.8963353e-05','0.0001201313','1.4787491e-08','1.4876347e-08','4.4507557e-05','0.00010519266','6.722137e-17','7.452402e-18','0.032077633','3.1023174e-13','4.7242953e-05','1.3669538e-10','1.2292711e-10','4.6792185e-05','2.3232972e-18','2.2766681e-11','1.1373508e-17','2.4335557e-12','1.6926391e-18','0.000867189','1.5538017e-12','9.572102e-14','2.0971136e-06','7.325746e-06','1.1978926e-10','7.538621e-11','2.0971136e-06','7.325746e-06','1.2687514e-18','6.5266586e-20','0.03878328','0.0019120035','4.970973e-10','5.4268734e-10','0.0019120035','0.046715222','0.0030307055','0.0028836168'],
['Slovakia','201912','1.4634863','5.8882785e-07','0.0062423265','3.0574292e-06','-9.254656e-07','0.0016629533','7.9297297e-10','1.3914829e-09','1.0382652e-13','-0.041861936','9.371417e-06','3.4439524e-06','0.004663513','0.0073926067','6.668156e-05','5.13048e-05','0.0044456683','0.006914416','1.7858659e-09','5.9628236e-10','1.8738632','1.6194923e-06','0.0127076125','1.0693986e-05','-1.673031e-06','0.012538312','1.672116e-09','5.288387e-06','6.405804e-09','3.131212e-06','2.1463147e-09','0.03989771','2.169002e-06','6.902732e-07','0.002352328','0.003951942','1.3148547e-05','3.30695e-06','0.002352328','0.003951942','1.4693526e-09','4.3588813e-10','1.5328503','0.049625985','1.6442951e-05','-1.0615815e-06','0.049625985','1.2715604','0.07992061','0.07469332','0.0017343346','3.546998e-14','3.8776466e-06','1.2671291e-11','1.961297e-11','3.4723624e-07','8.1959815e-20','4.9678685e-18','2.74406e-25','9.101888e-05','1.4680843e-10','1.5812334e-11','2.7729388e-05','7.776489e-05','2.1186517e-08','4.1096442e-08','2.5162411e-05','6.7589426e-05','1.6675988e-17','1.75983e-18','0.04206582','9.832583e-14','3.7852486e-05','1.6082237e-10','1.826756e-10','3.7574635e-05','3.874467e-19','1.3544955e-11','1.181565e-17','1.4597153e-12','2.200033e-18','0.00075864757','4.291977e-12','2.6270267e-13','2.9100952e-06','1.0858791e-05','8.450809e-10','2.9083236e-10','2.9100952e-06','1.0858791e-05','2.7526768e-18','1.9587726e-19','0.032945894','0.0010579744','3.2288913e-10','5.620922e-10','0.0010579744','0.037103273','0.0015781785','0.0015173509'],
['Slovenia','201912','1.4560629','5.2608044e-07','0.005506504','2.5529762e-06','-1.3063544e-06','0.0014354023','5.9017613e-10','1.948771e-09','4.5233332e-13','-0.042895436','1.6257973e-05','5.386613e-06','0.0071842354','0.011899852','9.630938e-05','0.00011220783','0.00684507','0.01109904','3.9271604e-09','1.1733132e-09','1.9184147','1.5374067e-06','0.011047178','9.459546e-06','-3.668999e-06','0.0108868275','1.4368395e-09','5.1507577e-06','6.2985137e-09','2.9886862e-06','1.6903255e-09','0.034361165','3.291099e-06','8.9463e-07','0.003272352','0.0060519134','1.6453483e-05','5.9274985e-06','0.003272352','0.0060519134','3.5434773e-09','8.1195756e-10','1.5978646','0.042327292','1.5250266e-05','-4.162772e-06','0.042327292','1.2220176','0.07683302','0.071800366','0.0017142585','2.0534418e-14','2.5133477e-06','1.392346e-11','1.9526129e-11','2.6962215e-07','7.671525e-20','1.7155656e-17','4.424047e-24','9.998146e-05','1.6342525e-09','1.3795834e-10','0.00023782202','0.0007308728','5.9147244e-08','2.715421e-07','0.00021566468','0.0006313439','1.2405165e-16','1.0747968e-17','0.046153866','8.064297e-14','2.8109003e-05','1.6106971e-10','2.0893524e-10','2.793688e-05','6.0365294e-19','1.2522391e-11','1.0032778e-17','1.7414292e-12','1.3469793e-18','0.0010409092','8.600049e-12','3.5095895e-13','7.6124325e-06','3.404652e-05','1.2070619e-09','9.754395e-10','7.6124325e-06','3.404652e-05','1.7975689e-17','6.1084667e-19','0.03679338','0.00092598127','4.8024434e-10','6.195011e-10','0.00092598127','0.07600059','0.002375996','0.0022286696'],
['Spain','201912','1.4459865','4.6191394e-07','0.004707835','2.5343393e-06','-9.214212e-07','0.0011656333','8.1098306e-10','1.1909013e-09','1.9873338e-13','-0.04300743','1.53635e-05','4.807495e-06','0.006306898','0.01075415','-5.2978496e-05','1.5405061e-05','0.0060056606','0.010012047','3.829427e-09','1.07164e-09','1.9528868','1.3170135e-06','0.008444532','1.15319135e-05','-4.140718e-06','0.008307382','1.1994351e-09','5.332382e-06','9.581898e-09','2.3927478e-06','1.8126276e-09','0.031184582','5.452018e-06','1.2353574e-06','0.0039896015','0.007941737','4.130143e-05','1.2156946e-06','0.0039896015','0.007941737','5.92342e-09','1.256214e-09','1.6311365','0.030197807','1.1988923e-05','-4.2398365e-06','0.030197807','1.1333791','0.06204603','0.057624966','0.0012835406','1.6509556e-14','1.5461118e-06','1.8679325e-11','8.42849e-12','1.0749711e-07','3.3109651e-19','7.049794e-18','7.668392e-25','0.00014627914','8.389644e-10','6.32474e-11','0.000103832805','0.000342716','9.041085e-08','1.3204216e-07','9.39949e-05','0.0002947944','5.9871994e-17','3.7818334e-18','0.03763144','7.5200005e-14','1.1076733e-05','2.633354e-10','1.1573163e-10','1.1024921e-05','6.964488e-19','1.6495832e-11','4.4286995e-17','7.444997e-13','8.218249e-19','0.00091019546','6.155403e-11','1.7795853e-12','2.7790873e-05','0.00014927586','3.6821703e-08','2.492016e-09','2.7790873e-05','0.00014927586','6.8043804e-17','1.9504842e-18','0.039562132','0.00027630336','5.465456e-10','2.701669e-10','0.00027630336','0.07149103','0.0011015324','0.0010210123'],
['Sweden','201912','1.4968399','4.7257151e-07','0.0054419637','4.4437515e-06','1.0665443e-06','0.0016477067','3.9612066e-10','3.2313618e-09','1.8215542e-12','-0.03839129','8.759089e-06','3.2621722e-06','0.0044107367','0.0069719823','7.7167e-05','3.1994063e-05','0.0042043924','0.0065227007','1.3458271e-09','4.3157242e-10','1.7584151','1.2323848e-06','0.0128421625','1.4155481e-05','1.1778325e-06','0.012712051','8.7318186e-10','3.1669215e-06','3.1416907e-09','2.9242972e-06','2.0636888e-09','0.013007325','1.1567198e-05','2.7711428e-06','0.013574005','0.025958048','6.4467575e-05','2.4910418e-05','0.013574005','0.025958048','2.1801492e-08','4.1162083e-09','1.4406515','0.056822576','2.2100765e-05','6.217522e-06','0.056822576','0.9502765','0.10803686','0.10366243','0.002096381','4.6981136e-14','6.8586282e-06','1.3953321e-11','2.1692126e-11','7.995605e-07','6.430235e-20','8.731918e-18','2.7292714e-24','0.00010490943','1.7922103e-10','1.9281677e-11','3.2947886e-05','9.44048e-05','3.039264e-08','1.7365414e-08','2.9866713e-05','8.1998754e-05','5.4151375e-18','4.3010044e-19','0.030549152','1.1146358e-13','3.9447168e-05','1.1201868e-10','1.7678903e-10','3.9107195e-05','3.3139358e-19','9.757732e-12','6.960264e-18','2.0811556e-12','2.2125712e-18','0.00019063467','1.12874474e-10','4.271718e-12','0.00013544693','0.0006092846','1.7094711e-08','3.0099073e-08','0.00013544693','0.0006092846','4.016474e-16','9.041196e-18','0.029868796','0.001691949','4.836129e-10','8.021597e-10','0.001691949','0.05719078','0.0037717505','0.0035822915'],
['Ireland','201912','1.4766015','3.1618012e-07','0.0034700637','4.1940366e-06','1.0466534e-06','0.0009745755','2.36803e-10','1.4481397e-08','1.3465432e-11','-0.035768893','1.2839982e-05','5.1298775e-06','0.0069733863','0.0107513415','0.00016394582','0.00010736411','0.006648312','0.01007577','1.0276501e-09','3.2329228e-10','1.8671255','1.0677065e-06','0.008799325','1.6535649e-05','3.634835e-06','0.008687536','4.4493917e-10','1.4253384e-06','2.3904383e-09','1.5922499e-06','5.416405e-10','0.004854927','4.2508855e-05','7.2161547e-06','0.03478536','0.07916771','0.00057998195','7.150446e-05','0.03478536','0.07916771','4.9125763e-08','7.63402e-09','1.541514','0.021330731','2.220002e-05','5.467675e-06','0.021330731','0.5028162','0.12351957','0.120236576','0.0026919234','8.484839e-15','1.1420867e-06','8.435977e-12','1.1848098e-11','1.4922485e-07','2.7257197e-20','5.992137e-16','3.9885595e-22','8.693669e-05','1.3254068e-09','1.998642e-10','0.00035931976','0.00088896684','3.6921162e-07','1.4510577e-07','0.00032627722','0.00077916344','2.7440278e-18','1.6968023e-19','0.041872207','5.803771e-14','2.5286308e-05','1.2415831e-10','1.556891e-10','2.5136796e-05','7.06639e-20','1.5104552e-12','3.2611505e-18','1.5320872e-13','1.4105593e-19','0.00022493154','2.6587854e-09','4.3194788e-11','0.001248333','0.008502484','1.2509513e-06','3.5129057e-07','0.001248333','0.008502484','4.655297e-15','5.1513715e-17','0.023901429','9.5761345e-05','2.2724594e-10','2.869661e-10','9.5761345e-05','0.07783077','0.008627212','0.008577487'],
['Italy','201912','1.4424833','5.2019396e-07','0.005317147','2.4256653e-06','-9.892808e-07','0.0013191912','4.8164356e-10','8.312513e-09','1.0877078e-11','-0.0483321','4.231858e-05','1.2397027e-05','0.016294094','0.028385911','-8.391689e-05','0.00032377391','0.015517138','0.026385896','1.9513227e-08','5.236627e-09','2.0020387','1.5119026e-06','0.009466395','1.0279119e-05','-4.565302e-06','0.0093090935','6.1016947e-10','6.0075845e-06','7.0449575e-09','2.92654e-06','1.3818979e-09','0.014116809','1.6400474e-05','3.1546185e-06','0.011075459','0.024075067','9.451413e-05','1.4016461e-05','0.011075459','0.024075067','1.9732939e-08','3.4384118e-09','1.6897374','0.03556245','1.1346991e-05','-2.4462809e-06','0.03556245','0.9440398','0.102807514','0.0966521','0.0016115229','1.654293e-14','1.9556626e-06','1.503514e-11','2.215844e-11','2.0302205e-07','4.5171713e-20','5.311708e-16','8.193079e-22','0.00011932447','1.615772e-08','1.1304143e-09','0.0019412043','0.006323676','6.2921504e-07','2.05372e-06','0.0017602104','0.0054356316','5.1121174e-15','3.4456482e-16','0.042042207','8.816955e-14','2.65013e-05','2.0106136e-10','3.0687025e-10','2.6338399e-05','4.4208212e-20','1.2922275e-11','1.6038537e-17','1.5592046e-12','7.6957177e-19','0.00024496598','5.254913e-10','1.1812291e-11','0.00020657408','0.0011919178','2.0710564e-07','3.0089492e-08','0.00020657408','0.0011919178','6.732234e-16','1.4570793e-17','0.038357165','0.0006998516','4.1558035e-10','6.235372e-10','0.0006998516','0.12366676','0.009354702','0.008408291'],
['Denmark','201912','1.4813671','5.4109944e-07','0.0060552447','5.1431084e-06','1.997093e-06','0.0017573804','6.340343e-10','7.5265145e-09','9.129277e-12','-0.037422203','8.908116e-06','3.4737616e-06','0.004732898','0.0073251235','0.00012147325','4.7623624e-05','0.004512719','0.0068616886','1.1990533e-09','3.7568157e-10','1.8175001','1.300262e-06','0.011833378','1.6350767e-05','2.4227736e-06','0.011696831','9.073703e-10','2.923463e-06','3.5594097e-09','2.2544666e-06','1.168784e-09','0.005660131','2.0776608e-05','4.0947652e-06','0.020785669','0.044357106','0.00024518534','9.983332e-05','0.020785669','0.044357106','3.23708e-08','5.594128e-09','1.497061','0.036482707','2.3795688e-05','5.4114134e-06','0.036482707','0.7931538','0.10605461','0.10115506','0.0018714508','6.8357205e-14','1.0116151e-05','2.1375273e-11','2.9934225e-11','1.1742633e-06','1.3710003e-19','6.8631136e-17','1.3214344e-22','8.564968e-05','3.2946001e-10','5.0006464e-11','9.19128e-05','0.00022041256','1.6178316e-07','4.684536e-08','8.353302e-05','0.00019337634','5.4868917e-18','3.675583e-19','0.03474936','8.108978e-14','3.737611e-05','1.2970618e-10','2.2095009e-10','3.711245e-05','2.0143081e-19','6.3347374e-12','4.514459e-18','6.9363e-13','8.077383e-19','0.00017226061','5.614929e-10','1.517084e-11','0.0005204868','0.0028170024','1.7266052e-07','7.277341e-08','0.0005204868','0.0028170024','1.6064012e-15','3.6238693e-17','0.030241597','0.00056848006','3.4248052e-10','5.487596e-10','0.00056848006','0.09894191','0.004042049','0.003922521'],
['Finland','201912','1.5047202','4.1934842e-07','0.004902441','4.1549793e-06','7.5244674e-07','0.0015203719','3.5691108e-10','2.6154292e-09','1.07292e-12','-0.040088106','8.197287e-06','3.1174586e-06','0.004224804','0.00662395','6.655105e-05','3.010889e-05','0.0040275096','0.00620044','1.4103714e-09','4.8981963e-10','1.7112081','1.1605521e-06','0.013174365','1.3607051e-05','7.9848655e-07','0.013051507','9.490914e-10','1.7107259e-06','1.5021221e-09','3.2767261e-06','2.8836393e-09','0.022469442','6.977503e-06','2.0346597e-06','0.010339658','0.018044872','6.684355e-05','7.0779074e-06','0.010339658','0.018044872','7.624602e-09','2.0198745e-09','1.370952','0.06939998','2.148777e-05','7.973286e-06','0.06939998','1.0183086','0.11214558','0.10821779','0.0024436386','2.283552e-14','3.7789357e-06','1.00943715e-11','1.6076855e-11','5.2969216e-07','4.0901832e-20','7.468222e-18','1.4938048e-24','0.00010144246','9.711686e-11','1.0488388e-11','1.793035e-05','5.1375864e-05','1.155325e-08','1.199045e-08','1.6255543e-05','4.4627894e-05','4.1741213e-18','4.814713e-19','0.019348908','7.452369e-14','3.4656034e-05','9.330005e-11','1.4630398e-10','3.442263e-05','2.617989e-19','2.8002763e-12','1.6783877e-18','2.9143318e-12','2.8475267e-18','0.00041168582','6.747046e-11','3.5482385e-12','0.0001060394','0.0004240355','1.4241685e-08','7.799231e-09','0.0001060394','0.0004240355','1.1845942e-16','4.8387307e-18','0.01803749','0.0023761299','4.81504e-10','9.472106e-10','0.0023761299','0.050445877','0.0033142366','0.0031880715'],
['Belgium','201912','1.4668244','6.971398e-07','0.007482608','5.3474446e-06','2.2146182e-06','0.0020241828','1.6412901e-09','3.516626e-09','1.8681727e-12','-0.04011955','9.505883e-06','3.3693195e-06','0.0045027984','0.007288271','8.665279e-05','9.2824615e-05','0.0042904806','0.0068085254','2.0511155e-09','6.1601685e-10','1.841692','1.527527e-06','0.012328864','1.7634815e-05','2.9082823e-06','0.012168863','1.751895e-09','3.7429577e-06','6.268066e-09','2.1566593e-06','1.2241518e-09','0.020973966','1.7578448e-05','3.6513181e-06','0.017681645','0.03748656','0.00021315973','5.2378826e-05','0.017681645','0.03748656','2.1622363e-08','3.8682497e-09','1.5330019','0.031318646','2.293053e-05','4.967709e-06','0.031318646','0.903002','0.095905356','0.08980697','0.0010474197','4.8057332e-14','6.221515e-06','2.4453398e-11','2.8602072e-11','5.9830523e-07','8.710481e-19','3.3171756e-17','2.2158252e-23','0.0001017126','2.773192e-10','2.5374642e-11','4.192557e-05','0.0001299333','2.9854384e-08','1.07890266e-07','3.7963313e-05','0.00011231089','3.153421e-17','2.3884735e-18','0.02555238','1.2488323e-13','2.9428686e-05','2.5425348e-10','2.350896e-10','2.9172947e-05','1.0514621e-18','4.325921e-12','2.0280573e-17','3.962774e-13','5.900662e-19','0.00082603947','6.9407347e-10','1.8642069e-11','0.000606622','0.0033907662','1.6826368e-07','9.070512e-08','0.000606622','0.0033907662','1.8266011e-15','3.4982173e-17','0.026181534','0.0002587997','3.9517045e-10','4.9482807e-10','0.0002587997','0.12353946','0.0037462309','0.003741019'],
['Croatia','201912','1.4624166','4.954646e-07','0.005258199','1.7725273e-06','-1.0987818e-06','0.0014004993','3.714109e-10','2.4859923e-09','9.193838e-13','-0.044235483','1.7356135e-05','5.673608e-06','0.0075584594','0.012583133','6.209741e-05','0.00013149646','0.00720118','0.011731992','4.868205e-09','1.4154294e-09','1.9022846','1.44119e-06','0.010496555','7.701486e-06','-3.3670403e-06','0.010346097','8.026847e-10','4.232364e-06','4.0508956e-09','2.9135879e-06','1.6429634e-09','0.015863841','6.0746866e-06','1.3053589e-06','0.0056781657','0.011856316','1.0370781e-05','2.3959388e-05','0.0056781657','0.011856316','1.1838571e-08','1.959456e-09','1.5669106','0.04231622','9.056567e-06','-3.81129e-06','0.04231622','1.1085559','0.0825109','0.07765124','0.0017826018','1.7328078e-14','2.1889368e-06','1.5818109e-11','2.1054786e-11','2.5331906e-07','2.828849e-20','3.7693598e-17','1.7341917e-23','9.3869596e-05','1.8587468e-09','1.576829e-10','0.00027189276','0.00083420234','2.369707e-08','3.456775e-07','0.00024654638','0.00072063814','1.6540575e-16','1.2028429e-17','0.03822136','6.922754e-14','2.2467426e-05','1.6786848e-10','2.2652612e-10','2.2350308e-05','1.3173622e-19','1.4898903e-11','8.715124e-18','1.370412e-12','1.5729636e-18','0.00045879948','2.8494238e-11','8.711714e-13','2.7983398e-05','0.00014639631','2.6324485e-09','5.1060085e-09','2.7983398e-05','0.00014639631','1.158516e-16','2.5160868e-18','0.031765647','0.00075902336','5.130567e-10','6.6963407e-10','0.00075902336','0.08465516','0.002303489','0.0021545927'],
['Austria','202001','1.4542285','4.1528344e-07','0.0043652686','2.5335928e-06','-9.892293e-07','0.0011467963','5.0824334e-10','1.0479425e-09','7.361899e-15','-0.03902687','2.3880424e-05','8.244709e-06','0.011058787','0.018078893','6.160428e-05','2.7550484e-05','0.010538682','0.016881192','4.8821476e-09','1.4728141e-09','1.9439243','1.6215971e-06','0.011396675','1.0393464e-05','-2.1177345e-06','0.011227657','1.2639656e-09','1.9976762e-06','2.486626e-09','2.0438563e-06','1.2143786e-09','0.058368478','1.5986643e-06','5.212356e-07','0.0018966494','0.003159648','1.2531502e-05','-5.9946715e-06','0.0018966494','0.003159648','1.5703663e-09','4.4802914e-10','1.645301','0.026536422','1.2396305e-05','-4.4950357e-06','0.026536422','1.2408917','0.06353632','0.058951043','0.0017515571','1.4315972e-14','2.0455552e-06','6.877921e-12','7.7521375e-12','2.439445e-07','4.0970663e-20','4.7552914e-18','5.015287e-28','7.9674275e-05','4.081016e-09','4.5815793e-10','0.00080887805','0.002245476','5.2583253e-08','3.014199e-08','0.0007340944','0.0019541082','1.1970432e-16','9.40698e-18','0.044958916','4.2158359e-13','5.394803e-05','1.3799599e-10','1.5599444e-10','5.3355103e-05','3.7239682e-19','2.7374422e-12','4.5580816e-18','9.346456e-13','5.2517997e-19','0.0012634741','3.2209877e-12','2.2076987e-13','7.116342e-06','2.3311477e-05','7.925199e-10','2.51639e-10','7.116342e-06','2.3311477e-05','4.56978e-18','3.1903723e-19','0.058355004','0.0004529192','1.7374661e-10','1.5063488e-10','0.0004529192','0.106113','0.004078505','0.003639482'],
['Bulgaria','202001','1.439648','5.398376e-07','0.0054999604','9.1199144e-07','-3.0469746e-06','0.0013571307','5.146144e-10','9.694505e-10','1.3101088e-13','-0.042807903','1.617925e-05','5.775486e-06','0.0077981898','0.012524158','7.519417e-05','-1.2204797e-05','0.0074329535','0.011705192','3.0113512e-09','9.540542e-10','2.0277653','2.6875705e-06','0.015793521','2.8658637e-06','-1.4479229e-05','0.0155140115','1.6471171e-09','7.891891e-06','7.768989e-09','3.5178798e-06','2.1160262e-09','0.044207893','3.6876827e-06','1.0791043e-06','0.0029950724','0.005198313','1.54786e-06','-1.9840689e-05','0.0029950724','0.005198313','3.574701e-09','9.179272e-10','1.7029043','0.042330723','7.3198576e-06','-1.5793627e-05','0.042330723','1.2972257','0.0813469','0.07610564','0.0018371032','1.5397997e-14','2.0040952e-06','1.21656955e-11','9.8817985e-12','2.611965e-07','4.6990974e-20','1.4758436e-18','3.0244262e-25','8.9066976e-05','1.0594728e-09','1.13663426e-10','0.00019594686','0.00056387344','5.2996025e-08','5.4494905e-08','0.00017766577','0.00048982596','2.2589633e-17','1.922458e-18','0.03691956','1.7035771e-12','0.00011054207','3.644934e-10','3.9273246e-10','0.00010914608','6.8493315e-19','2.4557658e-11','1.601269e-17','2.5652566e-12','1.2756105e-18','0.0007013514','7.741986e-12','3.8409426e-13','4.903061e-06','1.7481238e-05','4.0196382e-10','7.806993e-10','4.903061e-06','1.7481238e-05','9.610276e-18','4.6834785e-19','0.030559264','0.0010286402','5.0357757e-10','3.6455985e-10','0.0010286402','0.043856185','0.0033680375','0.0031645203'],
['Cyprus','202001','1.4544888','5.285916e-07','0.0055902787','3.6950764e-06','-8.978924e-07','0.0014638634','1.770086e-10','5.5004876e-09','5.499381e-12','-0.054587055','7.564692e-05','2.3959961e-05','0.031824593','0.053466633','0.000325791','0.00022630102','0.03031777','0.049810156','2.3586109e-08','5.9033085e-09','1.9694635','2.3743517e-06','0.015858363','1.974495e-05','-6.291532e-06','0.015610652','4.5700635e-10','5.6369568e-06','4.1319446e-09','4.2861748e-06','1.884083e-09','0.006336047','1.685025e-05','2.760963e-06','0.010257806','0.02410753','2.7805358e-05','1.3457884e-05','0.010257806','0.02410753','1.8066958e-08','2.5042919e-09','1.6174582','0.061440006','1.7657938e-05','-6.2483564e-06','0.061440006','0.8200841','0.16046381','0.15243395','0.0019380072','2.1402959e-14','3.0092228e-06','1.0263494e-11','1.496397e-11','3.777441e-07','6.817365e-21','9.703987e-17','7.304203e-23','5.5749813e-05','1.0351551e-08','8.4193047e-10','0.0014851379','0.004361848','7.004573e-07','1.8645535e-06','0.0013478232','0.0037698199','3.4516472e-15','1.7141406e-16','0.038963888','1.4927387e-12','0.00012766127','3.1978478e-10','3.8927137e-10','0.0001257837','6.528457e-20','1.3763044e-11','7.423978e-18','2.805701e-12','1.2766659e-18','0.00012326307','2.1033449e-10','3.263457e-12','9.055881e-05','0.0006232826','2.1274335e-08','4.0550262e-08','9.055881e-05','0.0006232826','3.2017967e-16','4.2059657e-18','0.037952725','0.0017008486','6.329977e-10','7.981281e-10','0.0017008486','0.070308596','0.0106942095','0.009784665'],
['Czech Republic','202001','1.4716147','6.780429e-07','0.007424644','4.678301e-06','9.223929e-09','0.0020619805','1.4298313e-09','1.2377127e-09','9.2843085e-14','-0.039639235','2.2245758e-05','7.846051e-06','0.010545017','0.017083291','0.00010097675','2.489714e-06','0.010049691','0.015959555','4.6990016e-09','1.4072284e-09','1.862919','1.8790133e-06','0.016264329','1.4973018e-05','-9.971287e-07','0.016067138','2.3796984e-09','6.1055216e-06','7.861731e-09','2.933701e-06','2.3758264e-09','0.04525288','4.0580258e-06','1.2768927e-06','0.0048822337','0.00818693','3.3451608e-05','-1.6377686e-05','0.0048822337','0.00818693','3.5824328e-09','1.0607782e-09','1.541833','0.044308536','2.124052e-05','-1.939277e-06','0.044308536','1.1982443','0.09326791','0.0865845','0.0016090858','2.7735487e-14','4.516618e-06','1.5979882e-11','1.6411093e-11','6.3949915e-07','2.7364918e-19','4.061847e-18','8.058201e-26','6.491255e-05','3.9366146e-09','4.5039106e-10','0.0007964175','0.002190578','5.3997372e-08','1.4702665e-08','0.0007228255','0.0019070619','1.4296104e-16','1.11144065e-17','0.032752197','2.8105518e-13','0.00010343557','2.145135e-10','1.9904711e-10','0.000102622485','1.1304832e-18','6.94258e-12','1.0244009e-17','1.1137354e-12','2.022461e-18','0.00063726906','3.6830365e-11','2.3216082e-12','3.957552e-05','0.00013884105','4.932254e-09','5.048985e-09','3.957552e-05','0.00013884105','3.3442988e-17','1.8283037e-18','0.023259226','0.00063494017','3.3678693e-10','3.3567404e-10','0.00063494017','0.08100101','0.0050443984','0.004554208'],
['Estonia','202001','1.5042936','4.5831817e-07','0.005491725','6.9249386e-06','-3.764447e-07','0.0017582722','3.6991443e-10','3.178962e-09','2.3370305e-12','-0.038460862','1.2240478e-05','4.5452293e-06','0.00617628','0.009744724','0.0001078182','1.5209316e-05','0.005888514','0.009117404','2.0003388e-09','6.4665606e-10','1.7757593','1.5306305e-06','0.017310621','2.5857638e-05','-4.756263e-06','0.017148614','9.556422e-10','1.3917014e-06','1.1613567e-09','2.6946543e-06','2.0173314e-09','0.011567138','1.883115e-05','4.500228e-06','0.02053003','0.039795183','0.00023900619','-2.491312e-05','0.02053003','0.039795183','3.2712922e-08','6.778131e-09','1.4466796','0.050849624','3.4918514e-05','-9.733045e-07','0.050849624','0.7947359','0.123192616','0.11866899','0.0035931964','4.178599e-14','8.385577e-06','2.7672063e-11','2.2797814e-11','1.3126773e-06','6.0017136e-20','7.136273e-18','3.4153417e-24','6.651426e-05','5.995417e-10','6.8831454e-11','0.0001229082','0.00033294','3.864108e-08','3.9850487e-08','0.00011159895','0.0002899781','1.3484874e-17','1.0706489e-18','0.038690098','2.8251305e-13','0.00015564793','3.016197e-10','3.2039288e-10','0.00015438593','3.1808747e-19','2.418962e-12','2.290893e-18','2.0127398e-12','2.7080148e-18','0.00024856412','3.6259004e-10','1.394777e-11','0.00025806593','0.0012023778','8.4861526e-08','4.9810804e-08','0.00025806593','0.0012023778','9.053108e-16','2.7159094e-17','0.03316856','0.0016595578','6.1875527e-10','7.4400675e-10','0.0016595578','0.0729164','0.00414957','0.0038983803'],
['France','202001','1.4727678','4.0670048e-07','0.004391861','2.1245305e-06','3.857408e-08','0.0012073829','4.1463916e-10','1.3489966e-09','2.5054177e-13','-0.042304453','4.3017644e-05','1.3145117e-05','0.017461546','0.029815191','-0.00011978731','0.00023094371','0.016635017','0.027751042','5.1308517e-09','1.5879629e-09','1.8659693','1.5493307e-06','0.011980355','1.0422157e-05','-1.0005197e-06','0.011818237','6.9870004e-10','1.3960669e-06','1.9323678e-09','2.2907236e-06','1.308051e-09','0.03483121','4.4838744e-06','1.158945e-06','0.0053878115','0.010221074','3.893907e-05','5.98601e-06','0.0053878115','0.010221074','5.282539e-09','1.216252e-09','1.5371171','0.030653102','9.8052815e-06','1.9640001e-06','0.030653102','1.0331627','0.08706196','0.081651','0.0018329083','1.1042041e-14','1.2859335e-06','2.0599038e-11','1.2056369e-11','1.509878e-07','5.094854e-20','7.259913e-18','5.743366e-25','0.000108173655','1.2581038e-08','1.0243231e-09','0.0017685563','0.0055051623','8.103179e-07','5.9184015e-07','0.0016039481','0.0047506276','9.156073e-17','7.901417e-18','0.031982012','2.9608908e-13','4.435983e-05','4.0722342e-10','2.3062231e-10','4.390553e-05','1.461177e-19','1.6994995e-12','4.13315e-18','9.983309e-13','3.784631e-19','0.0010383936','6.344443e-11','1.9816913e-12','6.395436e-05','0.00033085153','2.4731447e-08','1.7225616e-09','6.395436e-05','0.00033085153','1.0897121e-16','2.9449764e-18','0.025223548','0.00022071943','6.4892713e-10','3.5336525e-10','0.00022071943','0.14365603','0.006754012','0.0059159244'],
['Germany','202001','1.4809915','6.014299e-07','0.006678508','5.6502386e-06','1.2412827e-06','0.0019136176','1.1237802e-09','1.7316473e-09','3.7415283e-13','-0.040028006','2.1006565e-05','7.392778e-06','0.009944218','0.016103586','0.00014258418','7.302511e-05','0.009477494','0.015044055','4.1707686e-09','1.3444861e-09','1.8372419','1.5454702e-06','0.013500951','1.6970178e-05','6.7049854e-07','0.013338601','1.1610767e-09','3.7851469e-06','7.865913e-09','2.550583e-06','2.2861022e-09','0.0372469','5.563797e-06','1.567503e-06','0.0072677424','0.013108004','5.2637173e-05','-1.028426e-05','0.0072677424','0.013108004','4.960482e-09','1.3111828e-09','1.4898988','0.040726334','2.5104027e-05','3.474641e-06','0.040726334','1.110354','0.09011748','0.08413037','0.0016640691','2.0234057e-14','3.3597078e-06','1.738298e-11','1.513305e-11','4.9534134e-07','2.0496164e-19','7.915043e-18','9.587084e-25','6.388445e-05','3.624914e-09','4.090501e-10','0.0007241689','0.001996699','1.054041e-07','8.415383e-08','0.0006572948','0.0017378159','1.02447384e-16','9.388812e-18','0.030607015','2.1431245e-13','6.131994e-05','2.4236932e-10','1.7207784e-10','6.0758462e-05','2.3605046e-19','2.0524795e-12','1.0983363e-17','8.860801e-13','2.7592458e-18','0.0007349105','8.330077e-11','4.4655443e-12','0.00012565969','0.00049649714','1.131566e-08','3.87473e-09','0.00012565969','0.00049649714','7.1325237e-17','3.4640442e-18','0.026787424','0.00048661284','3.3183833e-10','2.9444347e-10','0.00048661284','0.09603669','0.0047942414','0.0043496573'],
['Greece','202001','1.4439723','4.8514823e-07','0.004982701','8.3087815e-07','-2.5757195e-06','0.0012554474','3.6771788e-10','1.1316194e-09','4.5704483e-13','-0.046382286','2.689069e-05','9.567398e-06','0.012819215','0.020748945','0.0001342586','1.4254433e-05','0.012215861','0.01938689','3.9426236e-09','1.2010427e-09','2.0102525','2.581964e-06','0.0144568905','3.958192e-06','-1.4159516e-05','0.014189064','1.0511785e-09','9.662909e-06','1.2872237e-08','3.4985242e-06','2.2565496e-09','0.026773341','6.0737543e-06','1.3775363e-06','0.00480786','0.009641504','-5.9061467e-06','-2.3989882e-05','0.00480786','0.009641504','7.3392448e-09','1.4171576e-09','1.6593164','0.044562962','4.081749e-06','-1.5887468e-05','0.044562962','1.1251925','0.09439409','0.0890363','0.0015144915','1.9271414e-14','2.7109659e-06','1.2297914e-11','7.641439e-12','3.0948684e-07','2.8266568e-20','4.010454e-18','3.9974406e-24','6.5632914e-05','2.7464457e-09','3.2690328e-10','0.00056668813','0.0015666668','3.5352602e-07','6.75329e-08','0.00051399117','0.0013645168','3.436653e-17','3.1566515e-18','0.03272735','2.1981006e-12','8.190789e-05','3.716221e-10','3.2150246e-10','7.992759e-05','5.676032e-19','7.229083e-11','1.9045503e-16','1.8429871e-12','1.2944219e-18','0.00062686054','3.318033e-11','1.0160903e-12','2.0938613e-05','0.000102383885','2.8439038e-09','1.4471253e-09','2.0938613e-05','0.000102383885','5.868249e-17','1.399722e-18','0.04611508','0.0011029382','4.6553572e-10','2.8122848e-10','0.0011029382','0.09526628','0.0036302397','0.0033484166'],
['Hungary','202001','1.4745928','7.9515644e-07','0.008836866','2.9263597e-06','-1.2824612e-06','0.0025040342','1.053741e-09','1.0894723e-09','2.4597332e-14','-0.04205997','2.7123328e-05','9.2234695e-06','0.012371403','0.020308245','0.00015204345','7.26002e-06','0.011789703','0.018955557','2.9495901e-09','7.8068846e-10','1.874609','2.8718139e-06','0.024718972','1.110475e-05','-2.928544e-06','0.024417091','2.6285785e-09','8.490251e-06','8.629895e-09','4.606803e-06','3.6879944e-09','0.03732589','6.331406e-06','1.77701e-06','0.0056247073','0.009986379','1.7924309e-05','-7.900364e-06','0.0056247073','0.009986379','4.684297e-09','1.2738409e-09','1.5596466','0.07198702','1.6505863e-05','-4.560572e-06','0.07198702','1.2187072','0.13583875','0.12784895','0.0024781239','5.495165e-14','8.822679e-06','1.5411249e-11','1.8685022e-11','1.2170068e-06','1.4787884e-19','2.4808493e-18','5.4645523e-27','8.7444205e-05','4.7897215e-09','5.064191e-10','0.0008902825','0.002526009','1.3779417e-07','2.492788e-08','0.00080791203','0.0021947324','2.1050775e-17','1.0171038e-18','0.041441526','2.2596927e-12','0.00034000163','2.5567906e-10','4.8036763e-10','0.00033558992','1.3679977e-18','3.5340425e-11','2.3810485e-17','7.111951e-12','7.005346e-18','0.00064568163','5.117431e-11','2.2133955e-12','2.3053602e-05','0.00010580878','1.7256829e-09','7.0589627e-09','2.3053602e-05','0.00010580878','2.5686013e-17','1.0997772e-18','0.028236818','0.0036419637','5.418205e-10','5.151606e-10','0.0036419637','0.051283725','0.012366829','0.011530158'],
['Latvia','202001','1.5038971','5.3782685e-07','0.0063890694','6.8982818e-06','-1.5507541e-07','0.0020185246','4.9486126e-10','2.5553781e-09','1.6305604e-12','-0.038165778','1.0797825e-05','4.0633677e-06','0.0055456767','0.008679741','9.595833e-05','1.0165036e-06','0.0052879225','0.008124308','2.0266702e-09','6.828061e-10','1.7686175','1.6936057e-06','0.019021248','2.6419832e-05','-4.0991836e-06','0.01884206','1.2171384e-09','2.0021382e-06','1.9934072e-09','2.9483238e-06','2.4242763e-09','0.015607781','1.3362233e-05','3.361219e-06','0.015334273','0.028365549','0.00014822853','-4.5255587e-05','0.015334273','0.028365549','2.0303407e-08','4.4040736e-09','1.4224365','0.05715252','3.4442626e-05','-1.0720127e-06','0.05715252','0.90934956','0.11960724','0.11450312','0.0029394086','7.0226234e-14','1.1821604e-05','2.3914922e-11','2.3722775e-11','1.5704496e-06','1.1466269e-19','5.7467207e-18','2.3945443e-24','5.735542e-05','5.5114024e-10','6.317992e-11','0.00011303076','0.00030652693','1.9224055e-08','3.255087e-08','0.00010262612','0.00026694709','1.5808917e-17','1.3775846e-18','0.03333889','3.9136893e-13','0.00018833978','3.3327727e-10','3.6716774e-10','0.00018677597','5.981251e-19','5.227404e-12','4.837898e-18','2.131393e-12','2.6495641e-18','0.00023209266','2.527143e-10','9.486924e-12','0.00014097923','0.0006408991','3.2911885e-08','3.7727798e-08','0.00014097923','0.0006408991','3.1557232e-16','1.0858896e-17','0.027219117','0.0017385026','5.301002e-10','6.610564e-10','0.0017385026','0.0817203','0.003177669','0.0029294933'],
['Lithuania','202001','1.5000054','6.4711696e-07','0.0075478875','7.150799e-06','-3.852942e-07','0.0023278953','8.2025753e-10','2.1419375e-09','9.885677e-13','-0.037210803','1.08379045e-05','4.100256e-06','0.005593322','0.008749007','9.970862e-05','-3.4692905e-06','0.0053331894','0.008190011','2.417458e-09','8.2813245e-10','1.7697659','1.8380626e-06','0.019335307','2.512047e-05','-3.8566204e-06','0.019141216','1.5290117e-09','3.2584499e-06','4.4903277e-09','3.033302e-06','2.8511318e-09','0.023789868','9.571804e-06','2.5849813e-06','0.012680702','0.022827882','9.116136e-05','-4.3465858e-05','0.012680702','0.022827882','1.2233099e-08','3.0587717e-09','1.408314','0.05826967','3.414354e-05','-3.21791e-06','0.05826967','1.0069298','0.11672929','0.11075679','0.0020845202','8.783439e-14','1.3633271e-05','2.0027785e-11','2.2761818e-11','1.6644418e-06','3.251571e-19','5.6702776e-18','1.5713864e-24','5.444262e-05','7.4164475e-10','8.5310516e-11','0.0001517678','0.0004145758','3.7009983e-08','2.4705015e-08','0.00013775729','0.0003610222','3.6965547e-17','3.5715206e-18','0.028331302','4.652451e-13','0.00018010763','2.6410193e-10','3.9498724e-10','0.00017848506','1.0986882e-18','9.005259e-12','1.5541933e-17','1.2338493e-12','2.6464122e-18','0.00046967185','1.9023569e-10','7.83765e-12','0.00017308928','0.0007733496','1.7995085e-08','2.5369442e-08','0.00017308928','0.0007733496','2.7587016e-16','1.2028021e-17','0.02383244','0.0010446614','3.5642655e-10','5.4569466e-10','0.0010446614','0.095589325','0.0026411253','0.0024235803'],
['Luxembourg','202001','1.4936289','5.6371243e-07','0.006348416','4.1329868e-06','1.0691288e-06','0.0018744968','1.1371104e-09','1.7403596e-09','4.604739e-13','-0.04086236','2.4667313e-05','8.522989e-06','0.011438224','0.018663045','0.0001220978','0.00012078133','0.010900523','0.01742675','4.1194337e-09','1.2068788e-09','1.7932105','1.6273384e-06','0.014668996','1.520033e-05','1.7076067e-06','0.0144983','1.2508514e-09','2.3839352e-06','5.0945803e-09','2.3979974e-06','1.878251e-09','0.032684095','6.2020135e-06','1.6227085e-06','0.007878111','0.014870357','4.9897197e-05','-9.828056e-06','0.007878111','0.014870357','5.825185e-09','1.3592544e-09','1.4426029','0.03891383','1.9176541e-05','4.304602e-06','0.03891383','1.0419666','0.0934651','0.08758328','0.0017344871','1.3967298e-14','2.1022931e-06','2.1483038e-11','1.7652471e-11','3.2659221e-07','3.0102279e-19','8.281569e-18','1.2683995e-24','6.535607e-05','4.345426e-09','4.713788e-10','0.00083204865','0.002327754','8.8075005e-08','1.4837599e-07','0.00075514533','0.0020240082','6.4521365e-17','3.9187822e-18','0.027238788','2.335846e-13','5.7032987e-05','3.285067e-10','2.3222484e-10','5.6561363e-05','3.1764198e-19','2.4730907e-12','2.3990685e-17','8.590215e-13','1.8463848e-18','0.001157253','1.0087186e-10','4.65697e-12','0.00012289456','0.0005473575','1.701087e-08','6.200594e-09','0.00012289456','0.0005473575','1.10738606e-16','3.7492586e-18','0.02332109','0.0004259602','4.0763132e-10','3.7897485e-10','0.0004259602','0.102265306','0.0047860993','0.0043268367'],
['Malta','202001','1.4318697','4.862174e-07','0.004858962','2.260244e-06','-1.5240325e-06','0.001148219','1.944556e-10','7.7553155e-09','1.7842578e-11','-0.050861828','4.703887e-05','1.59143e-05','0.021163644','0.03496759','0.0002030286','9.856104e-05','0.02016217','0.032626998','1.1636143e-08','3.1470828e-09','2.057804','2.0272705e-06','0.01049936','9.8296605e-06','-4.9841633e-06','0.010289184','4.9389204e-10','1.2434338e-05','2.7651204e-09','3.457924e-06','1.2678146e-09','0.011650859','1.5515623e-05','2.8365541e-06','0.008242627','0.018315427','7.5426487e-06','-2.5823168e-05','0.008242627','0.018315427','1.8455433e-08','2.8680471e-09','1.6794057','0.03835266','8.661352e-06','-8.9642635e-06','0.03835266','0.83123267','0.1069938','0.10073196','0.0007356415','1.0505747e-14','1.0545643e-06','8.383536e-12','1.0527984e-11','8.354536e-08','8.119597e-21','7.3740405e-17','2.918526e-22','7.854727e-05','5.1672107e-09','5.5012633e-10','0.00095238537','0.0027165613','4.3282955e-07','2.9486242e-07','0.0008637324','0.0023597623','5.2820736e-16','4.1267743e-17','0.023414485','6.3016275e-13','3.0710322e-05','2.1680352e-10','2.5156555e-10','3.0090061e-05','4.2810474e-20','4.806911e-10','2.8254414e-18','2.5837908e-12','2.4849465e-19','0.00011339412','1.2100244e-10','2.924866e-12','2.3602479e-05','0.00014220591','1.5088219e-08','2.2237593e-08','2.3602479e-05','0.00014220591','1.6709157e-16','3.4811917e-18','0.017783148','0.00034892897','2.945867e-10','4.742267e-10','0.00034892897','0.085993744','0.0030462209','0.0026752157'],
['Netherlands','202001','1.4765757','8.296011e-07','0.009166974','6.968547e-06','3.707726e-06','0.0025804879','1.8795114e-09','3.1387204e-09','3.9975783e-12','-0.040143244','1.557327e-05','5.4500315e-06','0.0073563973','0.011877851','0.00011498345','9.500426e-05','0.007011868','0.01109576','3.7009003e-09','1.1739498e-09','1.8357123','1.9671713e-06','0.01710087','2.1878908e-05','6.258078e-06','0.016894259','2.1176523e-09','5.5702376e-06','1.28170985e-08','2.73457e-06','2.2071442e-09','0.021374095','1.6018743e-05','3.5401683e-06','0.015208789','0.030594053','0.00015813853','-6.7549213e-06','0.015208789','0.030594053','1.7360948e-08','3.3745347e-09','1.4844109','0.04359458','2.7246328e-05','9.393131e-06','0.04359458','0.94450736','0.11233536','0.10476004','0.0009982841','8.70411e-14','1.21364255e-05','2.6866105e-11','3.6539306e-11','1.2407916e-06','1.2170907e-18','1.498093e-17','4.221207e-23','7.258342e-05','1.3677419e-09','1.354632e-10','0.00023781792','0.00068584783','6.155125e-08','9.420309e-08','0.00021579061','0.0005949055','4.628447e-17','3.8108893e-18','0.021902194','3.7886502e-13','7.942648e-05','3.6358325e-10','3.526483e-10','7.852602e-05','1.999697e-18','9.383376e-12','5.898971e-17','1.0138116e-12','2.3886937e-18','0.00063417066','4.3503462e-10','1.4865735e-11','0.0002828537','0.0013282865','6.709692e-08','3.6993157e-08','0.0002828537','0.0013282865','5.0720105e-16','1.33507075e-17','0.017930541','0.00046669878','4.104895e-10','5.926832e-10','0.00046669878','0.108564496','0.002437742','0.002250898'],
['Poland','202001','1.4830691','8.1060443e-07','0.009073945','7.904155e-06','2.1798328e-07','0.0025991662','1.8484079e-09','1.4828396e-09','2.1498589e-13','-0.03785292','1.545876e-05','5.7214534e-06','0.0077671427','0.0122819645','0.00013529281','7.1906566e-06','0.0074046883','0.011490301','3.80089e-09','1.2076071e-09','1.8133184','2.1787628e-06','0.02081869','2.3042603e-05','-7.552815e-07','0.020589286','3.515961e-09','6.681728e-06','1.1587135e-08','3.2480866e-06','2.8758835e-09','0.038718864','6.2564673e-06','1.8921436e-06','0.008368189','0.014144542','6.163032e-05','-3.169658e-05','0.008368189','0.014144542','5.6856133e-09','1.655287e-09','1.482853','0.055586074','3.409334e-05','-2.5136371e-06','0.055586074','1.1686252','0.11190547','0.1044089','0.0016246582','5.579686e-14','8.411457e-06','2.2554063e-11','2.5946821e-11','1.0035845e-06','6.673344e-19','4.501803e-18','2.4256795e-25','5.3486263e-05','2.753812e-09','3.3225178e-10','0.00059197086','0.0015897298','1.9672841e-07','1.3688834e-08','0.0005373738','0.0013861182','1.09360874e-16','9.627845e-18','0.029694097','4.252183e-13','0.00016479511','2.0249914e-10','3.8993078e-10','0.00016350416','2.7717286e-18','1.0511647e-11','2.6031511e-17','1.4659586e-12','2.242636e-18','0.0006642929','9.782757e-11','4.5641806e-12','9.6494856e-05','0.00038426497','1.1952455e-08','1.8351608e-08','9.6494856e-05','0.00038426497','8.2827494e-17','3.6278078e-18','0.02890573','0.0009932655','4.016909e-10','5.1902044e-10','0.0009932655','0.07822626','0.005110952','0.004662625'],
['Portugal','202001','1.4547662','4.6973375e-07','0.004905096','8.7322366e-07','-8.8107e-07','0.001265569','4.0546128e-10','1.9015804e-09','8.5603605e-13','-0.046085734','4.748549e-05','1.35119835e-05','0.01792515','0.031101676','-0.00029469916','4.3283894e-06','0.017076151','0.028898392','9.860639e-09','2.9383649e-09','1.950049','1.8321199e-06','0.012374302','6.2052677e-06','-5.9510453e-06','0.01218312','1.3625306e-09','2.8828235e-06','3.662993e-09','2.9840473e-06','1.8151024e-09','0.022375923','7.3003357e-06','1.6495151e-06','0.0064647757','0.013083803','1.8429993e-05','1.6366881e-06','0.0064647757','0.013083803','8.306626e-09','1.6355032e-09','1.6112751','0.04051344','3.3956765e-06','-3.025727e-06','0.04051344','1.0188093','0.1019782','0.095943354','0.0022572035','1.5417082e-14','1.7593603e-06','1.9532936e-11','1.1777278e-11','1.9734148e-07','5.0584127e-20','1.00758415e-17','7.053073e-24','0.0001272614','2.3447818e-08','1.2956108e-09','0.002221737','0.007702938','5.2292953e-06','5.923418e-07','0.0020147313','0.006581619','2.0370671e-16','1.6589694e-17','0.041000664','4.1462382e-13','3.5997677e-05','3.4634376e-10','3.614061e-10','3.569672e-05','3.7029435e-18','3.7618116e-12','5.291465e-18','1.621969e-12','2.065515e-18','0.00061024673','7.235726e-11','2.2596864e-12','5.03641e-05','0.00025914723','4.9831628e-09','1.08881375e-08','5.03641e-05','0.00025914723','1.2751607e-16','3.6524722e-18','0.043106873','0.0007487684','6.702624e-10','3.535084e-10','0.0007487684','0.12855332','0.010340674','0.009052138'],
['Romania','202001','1.4531058','5.8987285e-07','0.0062158876','2.399026e-06','-2.3039183e-06','0.0016259137','8.577834e-10','1.0128307e-09','2.2599315e-14','-0.03955279','1.577146e-05','5.598169e-06','0.007572938','0.012172693','8.9193025e-05','-2.4450554e-05','0.0072187977','0.011375995','2.7837972e-09','8.2987495e-10','1.9380045','2.6542255e-06','0.019013792','1.14470695e-05','-9.35469e-06','0.018736487','3.8958547e-09','4.417711e-06','4.788521e-09','2.9454927e-06','1.7956823e-09','0.051011175','2.6961923e-06','9.252221e-07','0.0031190154','0.0050177807','1.0874575e-05','-1.175742e-05','0.0031190154','0.0050177807','2.3102562e-09','7.444753e-10','1.6384039','0.040452458','1.3354532e-05','-1.143836e-05','0.040452458','1.3049086','0.0828732','0.07720876','0.0016693374','2.3724261e-14','3.5449716e-06','1.144611e-11','8.630429e-12','4.5647317e-07','1.162795e-19','7.9938435e-19','3.97979e-27','9.5545925e-05','1.2492385e-09','1.3017586e-10','0.00022963888','0.0006522247','5.4312036e-08','5.0383444e-09','0.00020839753','0.000566547','2.4230046e-17','1.6636397e-18','0.04186011','8.425156e-13','0.000106727384','2.4421956e-10','1.4896476e-10','0.00010552011','2.6841572e-18','1.7867572e-11','4.9234266e-18','3.4390407e-12','1.0910807e-18','0.00096210703','5.8911075e-12','5.1167225e-13','9.74094e-06','2.9360628e-05','3.7820838e-10','5.8638533e-10','9.74094e-06','2.9360628e-05','3.5200224e-18','3.0837229e-19','0.03983367','0.001585335','3.883381e-10','2.3220546e-10','0.001585335','0.044776633','0.0047214217','0.004431823'],
['Slovakia','202001','1.4663919','6.554726e-07','0.007132389','3.0890787e-06','-1.3375834e-06','0.0019704155','9.644073e-10','1.1053851e-09','3.0838345e-14','-0.039531294','1.770271e-05','6.3759335e-06','0.008619723','0.013800916','0.00010303543','6.5398413e-06','0.008216608','0.01290137','2.9837621e-09','8.9176344e-10','1.9103918','2.2894212e-06','0.019157596','1.2610533e-05','-2.393121e-06','0.018917803','2.174009e-09','5.47333e-06','7.0069306e-09','3.3516355e-06','3.1292058e-09','0.050957464','4.054421e-06','1.2046405e-06','0.0034531988','0.005902884','1.4605251e-05','-5.183388e-06','0.0034531988','0.005902884','3.0447638e-09','9.0043856e-10','1.597174','0.049459886','1.612903e-05','-5.271071e-06','0.049459886','1.2449371','0.09545451','0.08915286','0.0029605832','3.9286428e-14','5.977185e-06','1.2336523e-11','1.338926e-11','8.337783e-07','1.2542113e-19','2.379072e-18','7.162741e-27','7.677922e-05','1.6924399e-09','2.0621818e-10','0.0003697286','0.0009843386','6.411227e-08','2.1972582e-08','0.00033575873','0.0008584745','2.916573e-17','2.1051895e-18','0.050669484','1.2830143e-12','0.00027898615','2.2124523e-10','3.9861264e-10','0.00027585303','9.459408e-19','1.4264491e-11','1.3553997e-17','3.582252e-12','7.28822e-18','0.000896124','3.0638877e-11','1.4969448e-12','1.177645e-05','4.5764456e-05','1.0633685e-09','3.3561662e-09','1.177645e-05','4.5764456e-05','1.3727162e-17','7.5638057e-19','0.040669326','0.0017808194','3.663421e-10','3.1593358e-10','0.0017808194','0.057264578','0.005095004','0.004773905'],
['Slovenia','202001','1.4531189','6.837429e-07','0.0072357166','2.1504716e-06','-1.1420593e-06','0.0019123646','9.268787e-10','9.656449e-10','1.2193416e-13','-0.04069551','2.6157311e-05','8.827931e-06','0.011825179','0.019482281','0.00010146958','7.1124385e-05','0.011268526','0.018180331','3.118401e-09','9.2921065e-10','1.9519368','2.4753e-06','0.017432876','1.0162274e-05','-2.3421585e-06','0.017174654','2.8839635e-09','7.206692e-06','8.758026e-09','3.612724e-06','2.604057e-09','0.04786363','2.863702e-06','8.871916e-07','0.002580427','0.004357449','1.1360255e-05','-3.094013e-06','0.002580427','0.004357449','2.7252194e-09','7.864935e-10','1.6379131','0.050000966','1.313326e-05','-4.2420706e-06','0.050000966','1.3001778','0.09850802','0.09162577','0.0018469597','4.6966486e-14','7.0126507e-06','1.847976e-11','8.745896e-12','8.615569e-07','1.6148278e-19','3.8557994e-18','5.9876328e-25','9.077633e-05','6.223006e-09','6.343563e-10','0.0011107117','0.0032018588','7.083978e-08','1.5452099e-07','0.001007716','0.0027790724','4.1222456e-17','2.8500785e-18','0.042401582','1.3903419e-12','0.00013345497','2.4974464e-10','1.5789932e-10','0.0001315204','1.2120378e-17','1.6969922e-11','1.15598185e-17','3.6985163e-12','2.337929e-18','0.00084092404','8.9601755e-12','5.004704e-13','5.332424e-06','2.09846e-05','9.837295e-10','4.0466772e-10','5.332424e-06','2.09846e-05','8.956515e-18','5.1773736e-19','0.040345564','0.0016657811','6.1071154e-10','2.8145383e-10','0.0016657811','0.07039365','0.0077407635','0.0070638233'],
['Spain','202001','1.4536011','5.217875e-07','0.0054147616','1.2557495e-06','-8.770734e-07','0.0013899421','9.036371e-10','1.8307907e-09','2.043373e-13','-0.04446175','4.55829e-05','1.2604321e-05','0.016681725','0.029273968','-0.00032241215','5.9479225e-05','0.015890129','0.027179353','1.1260906e-08','3.3022494e-09','1.9225942','1.6022591e-06','0.011126214','6.837613e-06','-4.790098e-06','0.010959107','1.394862e-09','5.2745345e-06','9.730667e-09','2.7209455e-06','2.398175e-09','0.03171263','3.3848764e-06','8.8645123e-07','0.002860416','0.0052358424','3.4580753e-06','-2.315744e-06','0.002860416','0.0052358424','3.365953e-09','7.820969e-10','1.6112207','0.037578467','5.8442965e-06','-3.6767533e-06','0.037578467','1.1616429','0.08862921','0.082342885','0.0017662519','2.67549e-14','2.7891194e-06','1.8593562e-11','8.7447705e-12','2.4340596e-07','3.8947882e-19','2.7499062e-17','3.808834e-24','0.00013328115','2.2745358e-08','1.2155036e-09','0.0020874955','0.007283925','7.542888e-06','3.1102294e-07','0.0018928896','0.0062184935','5.2678657e-16','3.801469e-17','0.042419735','2.50108e-13','2.60676e-05','2.5954083e-10','1.719988e-10','2.5863666e-05','8.674097e-19','1.5948371e-11','5.0661907e-17','5.5636794e-13','1.2717285e-18','0.000602843','7.1503203e-12','2.986462e-13','5.5472806e-06','2.1621316e-05','6.63289e-10','6.1840044e-10','5.5472806e-06','2.1621316e-05','8.579493e-18','3.4615164e-19','0.047984526','0.00044954303','6.14087e-10','2.6988733e-10','0.00044954303','0.11293758','0.009721674','0.008521582'],
['Sweden','202001','1.4933015','4.629139e-07','0.005377762','6.7800506e-06','2.1852239e-07','0.0016562331','3.812798e-10','4.134415e-09','1.799647e-12','-0.037585903','1.06737525e-05','4.1014987e-06','0.0056091174','0.008704003','0.00012993325','1.6742617e-05','0.005348845','0.008151387','2.3786308e-09','8.2097623e-10','1.8102546','1.43726e-06','0.014835723','2.5930467e-05','-3.1858926e-06','0.014684099','7.7925455e-10','2.0347666e-06','2.9631357e-09','2.6166756e-06','1.9137487e-09','0.0147153875','1.7180968e-05','3.9871047e-06','0.014686091','0.028210029','0.0001843232','-3.9758736e-05','0.014686091','0.028210029','2.7840407e-08','5.4453597e-09','1.5016185','0.047115885','3.517601e-05','1.3429951e-06','0.047115885','0.8749991','0.10424368','0.09981787','0.0034727175','4.9102012e-14','8.334456e-06','2.0621114e-11','2.1925316e-11','1.1686701e-06','3.9797352e-20','2.4108869e-17','3.431312e-24','6.8763904e-05','5.024734e-10','6.232138e-11','0.000113138856','0.0002931139','8.1405254e-08','3.542479e-08','0.00010278608','0.00025588615','3.2455907e-17','3.367937e-18','0.048807092','1.7229508e-13','0.00010248569','2.2428312e-10','2.6360675e-10','0.000101782134','1.2445396e-19','2.5015003e-12','4.2464024e-18','1.631323e-12','2.3907331e-18','0.00016009474','3.5159584e-10','1.1456732e-11','0.00012542355','0.00060951983','8.0750596e-08','7.758184e-08','0.00012542355','0.00060951983','4.3674678e-16','9.138217e-18','0.04500649','0.0015667018','5.6101646e-10','6.6025657e-10','0.0015667018','0.067275904','0.0039476347','0.0036899836'],
['Ireland','202001','1.4750516','3.1245077e-07','0.0034332294','5.4547704e-06','1.9881231e-06','0.000965071','1.893526e-10','8.286719e-09','6.325021e-12','-0.03798862','8.083967e-06','3.252579e-06','0.004478995','0.006813392','9.607949e-05','6.9981994e-05','0.0042720856','0.006388564','1.3216505e-09','4.797045e-10','1.8795557','1.3851381e-06','0.011179028','2.5005509e-05','8.748846e-06','0.011033967','3.812205e-10','1.2170598e-06','1.9861837e-09','1.7119621e-06','6.103304e-10','0.0063544894','4.0212843e-05','6.6145885e-06','0.029228082','0.06692645','0.00053340395','0.00025480596','0.029228082','0.06692645','4.3876952e-08','6.8925265e-09','1.5462035','0.02307092','2.8440738e-05','1.0880941e-05','0.02307092','0.57424814','0.11142235','0.108383626','0.0023556373','4.31364e-15','8.186066e-07','8.908136e-12','1.203781e-11','1.4374422e-07','1.3191293e-20','1.6208521e-16','9.8445423e-23','6.244869e-05','2.2926762e-10','3.5033216e-11','6.508327e-05','0.00015426078','2.4784754e-08','3.702768e-08','5.9169324e-05','0.00013540406','4.5821496e-18','4.825113e-19','0.035511207','2.679558e-13','6.189345e-05','2.688469e-10','2.5736957e-10','6.1268925e-05','3.4841432e-20','7.17948e-13','1.4378337e-18','2.5881375e-13','1.3745936e-19','0.00023784144','2.179115e-09','3.6230276e-11','0.0007276112','0.004844878','8.5250383e-07','3.998255e-07','0.0007276112','0.004844878','2.2399638e-15','3.373591e-17','0.026629735','0.00015002549','3.3553949e-10','3.7314607e-10','0.00015002549','0.08359557','0.0047290665','0.004728878'],
['Italy','202001','1.4350433','6.079625e-07','0.006101803','1.1178396e-06','-9.86746e-07','0.0014593565','6.0337446e-10','1.6428546e-09','1.5271526e-12','-0.043604374','3.1764608e-05','1.0694289e-05','0.014285838','0.023597745','0.00012174829','0.0001728488','0.0136118205','0.022018526','5.850884e-09','1.8604858e-09','2.0214067','2.2679944e-06','0.013045338','5.4988495e-06','-4.1869303e-06','0.012809648','8.271321e-10','1.2478309e-05','1.0268614e-08','4.1006924e-06','1.9358997e-09','0.030010289','7.635394e-06','1.7385073e-06','0.004884384','0.009850057','1.4867565e-05','-1.1358173e-06','0.004884384','0.009850057','8.7195735e-09','1.7389791e-09','1.7269785','0.045955442','4.6033783e-06','-1.3040511e-06','0.045955442','1.1900743','0.09855004','0.0920929','0.0011305269','2.0277105e-14','1.9343706e-06','8.744245e-12','9.810386e-12','1.5046415e-07','5.915812e-20','1.4339585e-17','2.8365703e-23','6.85537e-05','6.7780377e-09','6.792789e-10','0.0011830588','0.0034405098','1.3504291e-07','8.809614e-07','0.0010731269','0.0029849103','2.3351295e-16','2.220815e-17','0.040489297','1.1858241e-12','5.539351e-05','2.0620809e-10','2.143212e-10','5.444535e-05','7.801012e-20','1.2234948e-10','2.8538095e-17','4.0714996e-12','8.6044e-19','0.0003950293','6.570576e-11','1.8144348e-12','2.5193467e-05','0.00013691446','6.075734e-09','1.9834767e-09','2.5193467e-05','0.00013691446','9.4068766e-17','2.461708e-18','0.03140356','0.0008009847','2.603169e-10','4.1779785e-10','0.0008009847','0.13727616','0.0055983816','0.005031455'],
['Denmark','202001','1.4944557','5.188725e-07','0.0059702857','7.424011e-06','2.0323953e-06','0.001817952','5.580129e-10','6.7841706e-09','1.0323388e-11','-0.038500287','9.193723e-06','3.4277325e-06','0.0046764016','0.0073366635','0.00011861464','5.986373e-05','0.0044590305','0.006865761','2.3713809e-09','8.105872e-10','1.781978','1.5246698e-06','0.015588738','2.6379259e-05','3.0720148e-06','0.015428106','8.310312e-10','2.006743e-06','3.002639e-09','2.4554784e-06','1.6281273e-09','0.0029812118','2.4700854e-05','5.123821e-06','0.025581976','0.05316747','0.000319973','3.2404143e-05','0.025581976','0.05316747','4.449698e-08','8.194872e-09','1.4495941','0.044764355','3.4874512e-05','6.279827e-06','0.044764355','0.7102586','0.12682869','0.122043364','0.0021726575','6.742269e-14','1.03857e-05','1.7057574e-11','3.4859213e-11','1.276571e-06','1.2056041e-19','3.102401e-17','6.9435183e-23','5.7205645e-05','4.4443976e-10','4.614479e-11','8.160186e-05','0.00023030466','4.5505793e-08','6.811445e-08','7.406572e-05','0.00020006014','2.3890066e-17','2.2453239e-18','0.03650796','2.743932e-13','9.914738e-05','2.126396e-10','3.0754843e-10','9.8253295e-05','1.5116988e-19','2.5211658e-12','4.346309e-18','1.2430184e-12','1.9283964e-18','0.00012105547','6.487863e-10','1.7109491e-11','0.00046501282','0.0025232635','2.0157542e-07','1.1331847e-07','0.00046501282','0.0025232635','1.9667931e-15','4.866619e-17','0.035553858','0.0010066929','3.1258982e-10','6.6274064e-10','0.0010066929','0.07209435','0.0043178275','0.004128372'],
['Finland','202001','1.5066913','3.9401536e-07','0.0047520464','6.009614e-06','-3.8781633e-07','0.0015425503','3.0942338e-10','5.1180042e-09','1.9868525e-12','-0.038277525','1.3333836e-05','5.0651083e-06','0.006888617','0.010791849','9.801265e-05','4.5644705e-05','0.0065677096','0.010102359','2.5120201e-09','8.056708e-10','1.7551321','1.2994186e-06','0.015486571','2.2346247e-05','-4.2004403e-06','0.015348838','7.369872e-10','1.2087937e-06','1.1846225e-09','2.9315631e-06','2.860379e-09','0.018609911','1.860113e-05','4.555967e-06','0.019134868','0.03635894','0.00024314277','-6.7971465e-05','0.019134868','0.03635894','1.6597172e-08','3.9434775e-09','1.4191699','0.060452607','3.0575913e-05','2.433397e-06','0.060452607','0.82518506','0.12784167','0.123805575','0.0044462867','3.388216e-14','6.7714213e-06','1.9735989e-11','1.7623885e-11','1.1184856e-06','3.981833e-20','2.3900234e-17','2.0157727e-24','9.784532e-05','5.763707e-10','8.061389e-11','0.00014766258','0.0003667265','9.7041266e-08','5.8928176e-08','0.00013418352','0.00032108917','2.5855767e-17','2.2198242e-18','0.038600743','1.8667816e-13','0.00013173076','1.9226915e-10','2.1564103e-10','0.00013085183','1.7080197e-19','1.4913025e-12','1.045769e-18','4.5330254e-12','9.254624e-18','0.00025895305','5.820675e-10','2.2073883e-11','0.00038390417','0.0018243365','1.559387e-07','5.6803525e-08','0.00038390417','0.0018243365','3.8289563e-16','1.2270942e-17','0.03180796','0.003922295','8.6274704e-10','9.441847e-10','0.003922295','0.087753594','0.0071917996','0.006884677'],
['Belgium','202001','1.4804107','7.105051e-07','0.007851965','5.196914e-06','2.6435775e-06','0.002222917','1.6438513e-09','2.0748745e-09','1.0066568e-12','-0.041389238','2.0103333e-05','6.832007e-06','0.0091834515','0.015018721','0.00011931217','0.0001149054','0.008752195','0.014018796','4.3004307e-09','1.2433233e-09','1.8279452','1.8780372e-06','0.016408399','1.8411623e-05','4.7856556e-06','0.016211206','1.8123981e-09','3.5712003e-06','5.8061325e-09','2.640552e-06','1.9809046e-09','0.028846513','1.132664e-05','2.6434768e-06','0.011446306','0.022670086','9.5034986e-05','-6.703437e-06','0.011446306','0.022670086','1.0456808e-08','2.2738296e-09','1.4918022','0.04124258','2.2431708e-05','8.440489e-06','0.04124258','0.9976321','0.103191234','0.09636508','0.0011830137','3.2593218e-14','4.825115e-06','2.5425608e-11','2.7157531e-11','5.931708e-07','5.473422e-19','8.167447e-18','5.2055065e-24','6.545504e-05','2.5109164e-09','2.4046562e-10','0.00041932747','0.0012314735','8.709382e-08','1.1702933e-07','0.00038040287','0.0010671777','6.886146e-17','4.1320625e-18','0.024953682','3.1631907e-13','8.2491664e-05','3.7866346e-10','3.1633204e-10','8.1729035e-05','5.697061e-19','2.9483406e-12','1.5822933e-17','1.2769597e-12','2.2798562e-18','0.00086662045','2.6678218e-10','9.573736e-12','0.0001980542','0.0009988825','4.0092292e-08','1.8758e-08','0.0001980542','0.0009988825','2.893429e-16','8.810901e-18','0.020361992','0.0006267161','4.4527274e-10','5.233662e-10','0.0006267161','0.10347873','0.0036823114','0.0033698394'],
['Croatia','202001','1.4553086','5.801663e-07','0.006163051','9.4933876e-07','-1.0764113e-06','0.0016335881','5.437629e-10','9.732644e-10','1.10512456e-13','-0.041695993','2.3295224e-05','7.884856e-06','0.010565129','0.017382668','9.419561e-05','8.088618e-05','0.010067859','0.016222471','2.845582e-09','8.909571e-10','1.9640343','2.2631293e-06','0.015345254','6.797337e-06','-2.4079488e-06','0.01510914','1.3018527e-09','5.746878e-06','6.087337e-09','3.3413276e-06','2.2067879e-09','0.028769532','4.570176e-06','1.1600455e-06','0.00387616','0.007450711','7.4114596e-06','1.0899493e-06','0.00387616','0.007450711','7.641035e-09','1.4868995e-09','1.6466004','0.044349033','4.668488e-06','-4.735341e-06','0.044349033','1.2253294','0.09069056','0.084764704','0.0018192925','3.0906247e-14','4.459337e-06','1.7728048e-11','9.611762e-12','5.2604656e-07','4.9332758e-20','4.0190067e-18','3.341826e-25','7.349362e-05','4.418137e-09','4.5121734e-10','0.00078851444','0.0022770152','6.374596e-08','2.3004169e-07','0.000715358','0.0019763482','2.5085465e-17','2.0453035e-18','0.0379847','1.2785791e-12','0.0001126455','2.4158037e-10','1.8083407e-10','0.00011093509','2.7744297e-19','1.9996726e-11','1.4814017e-17','2.3068823e-12','1.7755533e-18','0.0004982723','1.9371801e-11','7.582267e-13','1.5069903e-05','7.461318e-05','4.2623665e-09','1.7419863e-09','1.5069903e-05','7.461318e-05','4.725709e-17','1.2850143e-18','0.040370442','0.0010346561','5.8980243e-10','3.234692e-10','0.0010346561','0.06878942','0.00576304','0.0052580545'],
['Austria','202002','1.4605999','4.930605e-07','0.005278688','7.131129e-06','-2.872996e-06','0.0014321251','3.8581094e-10','1.2226431e-09','3.1751037e-14','-0.041015','1.8809662e-05','7.313454e-06','0.009976565','0.015484279','0.00027947602','-9.209504e-05','0.009512814','0.014504209','1.978871e-09','6.8736783e-10','1.8926473','1.6390495e-06','0.012880102','2.6948697e-05','-1.0369061e-05','0.012708542','9.0418445e-10','1.4655171e-06','1.8180656e-09','2.3681127e-06','1.4613306e-09','0.035134614','3.9654424e-06','1.0813409e-06','0.004555531','0.008447717','5.1175823e-05','-1.76487e-05','0.004555531','0.008447717','3.9983252e-09','9.781869e-10','1.5518725','0.036692787','3.4532364e-05','-1.376232e-05','0.036692787','1.08315','0.07878402','0.07378464','0.001683099','4.770126e-14','7.469113e-06','2.2889397e-11','1.7345556e-11','9.248577e-07','4.607868e-20','3.6040388e-18','4.833799e-27','3.2239717e-05','2.0205357e-09','3.011684e-10','0.0005573974','0.0013550392','7.0599106e-07','9.901325e-08','0.0005067042','0.001188418','3.0257026e-17','3.1950259e-18','0.034019604','3.5806015e-13','0.00010247428','4.230553e-10','2.0897338e-10','0.00010144614','2.802072e-19','1.9908153e-12','3.6127725e-18','1.8974473e-12','1.7191667e-18','0.0009291418','2.259212e-11','9.255607e-13','2.8696119e-05','0.00012878259','6.9168697e-09','1.3865162e-09','2.8696119e-05','0.00012878259','2.304342e-17','8.045046e-19','0.03658729','0.0014062073','5.626256e-10','4.036254e-10','0.0014062073','0.07518895','0.0040908754','0.0037422252'],
['Bulgaria','202002','1.444267','6.926682e-07','0.007243471','4.024145e-06','-3.8996077e-06','0.0018732569','4.5633733e-10','1.7665447e-09','3.4463553e-13','-0.04530669','1.4956761e-05','5.672886e-06','0.007709991','0.012078275','0.00013900538','-0.000104102786','0.007350677','0.011306166','2.236264e-09','7.2889283e-10','1.9805561','2.7234764e-06','0.018765911','1.7491813e-05','-1.60037e-05','0.018482035','1.5348232e-09','7.642811e-06','5.715657e-09','5.5526834e-06','2.7969373e-09','0.035609346','5.2722216e-06','1.3669796e-06','0.003663692','0.006541694','2.5388059e-05','-1.8938816e-05','0.003663692','0.006541694','4.2183577e-09','9.774033e-10','1.6656615','0.082484536','2.3974082e-05','-2.2815979e-05','0.082484536','1.263399','0.12711363','0.1206868','0.0016090336','8.295944e-14','1.4043476e-05','2.1240662e-11','2.2881388e-11','1.8418637e-06','5.0132328e-20','1.17435386e-17','1.2378701e-24','3.0334462e-05','3.0500097e-10','4.4939715e-11','8.314059e-05','0.00020192853','7.29346e-08','1.386035e-07','7.557828e-05','0.00017705341','4.7098833e-18','4.2265794e-19','0.03635945','1.7978557e-12','0.0003686317','4.036871e-10','4.0769282e-10','0.0003641405','1.269522e-18','2.3590782e-11','1.2656823e-17','1.4867534e-11','6.0586914e-18','0.00046808505','2.293145e-11','6.5412015e-13','4.9679143e-06','2.283866e-05','1.3446995e-09','1.6185558e-09','4.9679143e-06','2.283866e-05','1.7854009e-17','4.721346e-19','0.0377739','0.010095827','9.685335e-10','1.3551238e-09','0.010095827','0.054093022','0.014655961','0.014112932'],
['Cyprus','202002','1.441682','5.4890967e-07','0.0056340774','4.6430673e-06','-2.4273456e-07','0.0013998792','2.0907942e-10','7.960498e-09','8.919052e-12','-0.055319943','4.5133784e-05','1.29811115e-05','0.016935306','0.029757671','0.00022598625','0.00018895367','0.016123218','0.027643472','1.7863835e-08','4.4095634e-09','2.026248','1.8749572e-06','0.011499051','2.2340466e-05','-1.4127706e-06','0.011303818','4.3773518e-10','5.511034e-06','3.8315036e-09','6.0365383e-06','2.7717748e-09','0.009131353','1.6992726e-05','3.0069393e-06','0.009953209','0.022639943','8.1010396e-05','1.9262372e-05','0.009953209','0.022639943','1.8516028e-08','2.7838472e-09','1.6445736','0.081713244','2.3143559e-05','-2.627741e-06','0.081713244','0.9952376','0.1512433','0.14470011','0.001557977','2.1685251e-14','2.7364697e-06','1.584473e-11','1.663068e-11','3.084701e-07','1.7293227e-20','2.177055e-16','1.6071607e-22','6.562289e-05','5.861322e-09','3.543373e-10','0.0005925802','0.0020381133','6.385605e-07','1.5658743e-06','0.000536824','0.0017443127','9.578707e-16','4.8702786e-17','0.033324573','1.3129424e-13','5.7259313e-05','2.4145833e-10','2.7175548e-10','5.691432e-05','5.0845264e-20','1.3311136e-11','4.935511e-18','9.463237e-12','2.8448073e-18','0.00010269063','2.4678223e-10','4.530644e-12','7.7146484e-05','0.0005129197','6.349366e-08','2.5478618e-08','7.7146484e-05','0.0005129197','2.606968e-16','4.03664e-18','0.026506364','0.004434249','1.5759192e-09','1.1086951e-09','0.004434249','0.06451581','0.009153809','0.008640666'],
['Czech Republic','202002','1.4662989','6.8938164e-07','0.007441101','9.631188e-06','-1.5465222e-06','0.0020276604','8.8144153e-10','1.6622871e-09','1.2671144e-13','-0.041618865','2.1467818e-05','8.040581e-06','0.010888465','0.017207006','0.0002992282','-1.8524323e-05','0.0103799915','0.016100738','1.3632507e-09','4.807047e-10','1.8629211','1.8935659e-06','0.01638113','3.1220774e-05','-5.7716406e-06','0.01618226','1.4596915e-09','4.2208217e-06','5.2048854e-09','3.3689423e-06','1.9198965e-09','0.02799567','9.378655e-06','2.3967214e-06','0.0090230135','0.016801596','0.000146749','-3.1274394e-05','0.0090230135','0.016801596','7.033219e-09','1.6563215e-09','1.523407','0.05336153','4.9624028e-05','-9.350224e-06','0.05336153','1.0637724','0.11119276','0.10447328','0.0015302072','2.6537486e-14','4.2967363e-06','3.3231317e-11','2.9960593e-11','5.947333e-07','1.9124797e-19','7.397096e-18','6.54925e-26','4.0705858e-05','3.5244971e-09','4.744304e-10','0.0008585079','0.0021993758','7.912076e-07','1.6226062e-07','0.0007798851','0.0019232002','9.425025e-18','1.0006297e-18','0.035494484','5.2516834e-13','0.00013867352','4.871917e-10','3.3165193e-10','0.00013720448','8.128387e-19','3.039317e-12','5.6240576e-18','1.2977196e-12','1.0102013e-18','0.00044037434','1.7495731e-10','6.705654e-12','9.024675e-05','0.00041910767','8.530313e-08','6.7415855e-09','9.024675e-05','0.00041910767','8.881392e-17','2.70903e-18','0.021052232','0.00086771016','9.61321e-10','7.078812e-10','0.00086771016','0.07338592','0.0045280615','0.0041016755'],
['Estonia','202002','1.4884349','4.672985e-07','0.0054021236','6.2162308e-06','9.0757993e-07','0.0016457665','3.2512806e-10','4.668698e-09','2.8179223e-12','-0.039255116','1.14332925e-05','4.185522e-06','0.005671455','0.00901108','0.0001386328','-5.4470518e-05','0.00540654','0.008427396','1.036527e-09','3.4174139e-10','1.8280078','1.4086853e-06','0.01386253','2.1264752e-05','-5.1939264e-07','0.013714252','8.697029e-10','1.3213016e-06','1.3469917e-09','2.6797518e-06','1.8355921e-09','0.012075501','2.069731e-05','4.8390634e-06','0.023442484','0.046421792','0.00027599375','6.7989284e-05','0.023442484','0.046421792','3.2789885e-08','6.392194e-09','1.4710689','0.0495267','3.2621057e-05','5.373484e-06','0.0495267','0.77248573','0.124223605','0.119735904','0.0030459918','3.818379e-14','7.368375e-06','2.3323794e-11','3.5125007e-11','1.1101301e-06','5.1653646e-20','2.4717477e-17','1.2088546e-23','6.319009e-05','4.6503276e-10','4.9468762e-11','8.692206e-05','0.00024511808','8.752284e-08','1.7322371e-07','7.887015e-05','0.0002129913','2.9242603e-18','2.7395198e-19','0.04634437','2.0338322e-13','9.727755e-05','2.1205628e-10','3.6989065e-10','9.646722e-05','2.7267828e-19','1.9857041e-12','3.136719e-18','1.4452327e-12','2.388789e-18','0.00022123201','5.5156824e-10','1.9020398e-11','0.0006013802','0.0029703684','2.4905287e-07','5.153186e-08','0.0006013802','0.0029703684','1.4418518e-15','3.826444e-17','0.04524113','0.0013276613','6.9313316e-10','8.921072e-10','0.0013276613','0.090594','0.0047399946','0.004598946'],
['France','202002','1.4689558','4.172154e-07','0.004489716','7.5310622e-06','-7.8303503e-07','0.0012264879','2.5885813e-10','2.5380742e-09','8.2458384e-13','-0.043337177','3.230908e-05','1.2089029e-05','0.016333269','0.025862698','0.0006179272','3.0735206e-05','0.015568824','0.024198389','5.032107e-09','1.5920466e-09','1.8599273','1.8063826e-06','0.013908223','3.616744e-05','-4.109009e-06','0.013719172','5.695213e-10','7.6123973e-07','5.714901e-10','2.2421204e-06','1.1136041e-09','0.022637023','1.2386278e-05','2.6835382e-06','0.012529628','0.026282143','0.00017658038','-2.4978178e-06','0.012529628','0.026282143','1.4888273e-08','2.9267226e-09','1.5644553','0.028998641','3.9272283e-05','-1.6892914e-06','0.028998641','0.85289776','0.09954103','0.09442444','0.0017425937','1.30114205e-14','1.7754952e-06','2.0580414e-11','1.3612483e-11','2.2546e-07','2.0452115e-20','2.7736344e-17','5.9412515e-24','3.2988584e-05','6.101985e-09','8.252192e-10','0.0014888044','0.0038165683','2.4174499e-06','5.017407e-07','0.0013521941','0.0033376003','1.5576884e-16','1.5060739e-17','0.025247354','6.6513526e-13','7.73076e-05','7.141468e-10','3.0709543e-10','7.620998e-05','1.166087e-19','7.902274e-13','9.877595e-19','7.518775e-13','6.82661e-19','0.0007141499','2.9528258e-10','7.820383e-12','0.00025776512','0.0014415402','1.1891183e-07','9.709152e-09','0.00025776512','0.0014415402','4.608317e-16','9.661573e-18','0.023266874','0.0003102352','5.9489164e-10','4.488668e-10','0.0003102352','0.12411842','0.005944371','0.005372272'],
['Germany','202002','1.4772897','6.3632484e-07','0.0069933524','1.0944197e-05','1.9946098e-07','0.0019641307','7.077156e-10','2.288157e-09','4.835083e-13','-0.041200403','2.0418951e-05','7.5240814e-06','0.01017683','0.016177204','0.00035737333','5.2540825e-05','0.0097006885','0.015131077','2.3870417e-09','7.915917e-10','1.8223219','1.6995119e-06','0.015368943','3.503694e-05','-1.935636e-06','0.015190458','8.5483143e-10','3.1912393e-06','5.0913855e-09','3.1588495e-06','2.1243498e-09','0.019306451','1.4384355e-05','3.3692759e-06','0.014775462','0.029238546','0.00025966624','6.021197e-06','0.014775462','0.029238546','1.3741538e-08','2.9426992e-09','1.47526','0.052957736','5.524194e-05','1.0435275e-06','0.052957736','0.93980706','0.120735385','0.11448246','0.001423121','2.3068734e-14','3.3881809e-06','3.411044e-11','2.7871299e-11','4.3448392e-07','1.3670769e-19','1.0983683e-17','1.4217397e-24','4.9981827e-05','3.0450633e-09','3.8360168e-10','0.00068949','0.0018132065','1.276282e-06','2.9001532e-07','0.0006260922','0.0015828725','6.874506e-17','7.265344e-18','0.032545578','3.220747e-13','8.5089705e-05','4.616866e-10','3.3256312e-10','8.427734e-05','1.9203541e-19','2.177617e-12','8.877402e-18','1.4446709e-12','1.8165598e-18','0.00052923063','4.2734147e-10','1.3862579e-11','0.00026220488','0.0013731444','2.4541666e-07','7.850798e-09','0.00026220488','0.0013731444','3.5819792e-16','9.336146e-18','0.018249227','0.0007895349','1.022282e-09','7.584562e-10','0.0007895349','0.08492083','0.0043307715','0.0039936607'],
['Greece','202002','1.4551672','5.310754e-07','0.0055455407','3.3973986e-06','-3.3561882e-06','0.0014407738','3.2241815e-10','2.319386e-09','1.2061248e-12','-0.045053836','1.4525037e-05','5.524364e-06','0.0075174808','0.011744413','0.000160758','-6.7884255e-05','0.007167374','0.010994806','3.2786571e-09','1.0241612e-09','1.9480746','2.0981545e-06','0.014132977','1.6237049e-05','-1.4104702e-05','0.0139145','8.1930845e-10','8.124857e-06','8.282906e-09','4.5758766e-06','2.9977114e-09','0.026180835','5.8030914e-06','1.4067045e-06','0.0049555856','0.009425146','3.251395e-05','-8.421226e-06','0.0049555856','0.009425146','6.4467516e-09','1.3937408e-09','1.5938426','0.06680652','1.983731e-05','-2.3429797e-05','0.06680652','1.169649','0.10765502','0.10258199','0.0018997098','3.1376008e-14','3.841427e-06','2.1343332e-11','1.7040844e-11','3.8903661e-07','2.0249971e-20','2.7068594e-17','1.573648e-23','2.3788174e-05','2.2538249e-10','3.2443465e-11','6.0099297e-05','0.0001465521','9.480736e-08','7.683139e-08','5.463066e-05','0.00012844082','2.2696896e-17','1.8934849e-18','0.03980213','5.0579403e-13','6.550157e-05','5.011119e-10','3.0358152e-10','6.474087e-05','1.8529165e-19','5.8636346e-11','8.69125e-17','6.1797737e-12','3.0880193e-18','0.000611002','3.3110417e-11','1.098617e-12','1.78956e-05','7.7304416e-05','9.628431e-09','1.7161348e-09','1.78956e-05','7.7304416e-05','3.877319e-17','1.3499387e-18','0.03832997','0.0036455179','1.2243041e-09','1.1025956e-09','0.0036455179','0.061636433','0.0044397675','0.004313725'],
['Hungary','202002','1.4442508','7.1352946e-07','0.0073627657','7.620427e-06','-3.3772114e-06','0.0018626859','6.2724165e-10','1.0541231e-09','6.513454e-14','-0.04268607','1.7289682e-05','6.573796e-06','0.00892913','0.0140027115','0.00023353966','-7.695023e-05','0.008512768','0.013108112','1.1611668e-09','4.0939804e-10','1.9572904','2.1720286e-06','0.015503818','2.7288866e-05','-1.0636084e-05','0.015277051','1.5487366e-09','4.167653e-06','4.2553614e-09','3.811461e-06','1.822527e-09','0.035759736','6.91333e-06','1.917266e-06','0.0051710475','0.0092931185','6.923997e-05','-2.7098287e-05','0.0051710475','0.0092931185','4.349791e-09','1.1376439e-09','1.6606858','0.049348276','3.982697e-05','-1.5967455e-05','0.049348276','1.1911908','0.09551091','0.08888837','0.0015960272','5.316142e-14','7.249934e-06','3.47995e-11','3.110541e-11','8.003122e-07','8.175321e-20','1.9496105e-18','4.964812e-26','3.420077e-05','1.0816461e-09','1.4966119e-10','0.0002708496','0.0006874513','4.37487e-07','1.08372795e-07','0.0002460147','0.0006015363','1.3464335e-17','1.5800542e-18','0.041896503','5.572923e-13','0.00014450455','4.6966825e-10','2.9552966e-10','0.00014306503','7.609167e-19','6.474809e-12','6.2870313e-18','3.227809e-12','1.3199471e-18','0.0004897379','3.6900788e-11','1.7335435e-12','1.5752245e-05','6.567588e-05','1.1113934e-08','5.789559e-09','1.5752245e-05','6.567588e-05','1.3304214e-17','6.156758e-19','0.027081938','0.0015654417','1.066817e-09','9.607845e-10','0.0015654417','0.07011662','0.0039517004','0.0036629087'],
['Latvia','202002','1.4783435','4.9660053e-07','0.0055664927','6.981313e-06','9.53545e-07','0.0016129124','3.5577988e-10','4.253116e-09','2.3962958e-12','-0.03967717','1.2760462e-05','4.7410485e-06','0.006425698','0.010164244','0.00015374264','-4.7637113e-05','0.006125915','0.009509287','9.440433e-10','3.2124342e-10','1.8411186','1.5111489e-06','0.013633163','2.3579723e-05','-5.855195e-07','0.013474508','9.459075e-10','1.5726077e-06','1.6081807e-09','2.7988572e-06','1.850184e-09','0.013525589','1.9481164e-05','4.596907e-06','0.021355174','0.04210911','0.00025455235','3.89391e-05','0.021355174','0.04210911','2.5876396e-08','5.0164126e-09','1.4839009','0.04841812','3.8019796e-05','6.0138577e-06','0.04841812','0.80229086','0.1198902','0.11512336','0.0020659906','3.9541656e-14','6.6465054e-06','2.2488332e-11','3.484654e-11','8.527091e-07','6.6348984e-20','1.8727215e-17','7.6809935e-24','4.9889484e-05','5.7398825e-10','6.769936e-11','0.000120163684','0.00032552693','1.00522215e-07','1.4457652e-07','0.000109093824','0.00028363196','2.8411123e-18','2.9041264e-19','0.0362367','2.5097085e-13','8.306469e-05','2.4926752e-10','3.7533182e-10','8.226724e-05','3.4171857e-19','3.0807102e-12','4.490243e-18','1.2924304e-12','1.6758577e-18','0.00022468016','4.4228632e-10','1.6344133e-11','0.00043831888','0.002091944','1.954032e-07','2.5755272e-08','0.00043831888','0.002091944','7.493309e-16','2.1017951e-17','0.034767736','0.0010497692','5.7355803e-10','8.8346075e-10','0.0010497692','0.09607116','0.0033473545','0.003233229'],
['Lithuania','202002','1.4770157','5.681448e-07','0.006283375','7.9605015e-06','4.7290806e-07','0.0017833717','4.664384e-10','3.134236e-09','1.3886481e-12','-0.040251076','1.376377e-05','5.126911e-06','0.0069544287','0.010981369','0.00015852104','-5.7790094e-05','0.006629951','0.010274991','9.2660085e-10','3.1909592e-10','1.8371516','1.6528227e-06','0.014384042','2.611347e-05','-1.9121198e-06','0.014210512','9.874355e-10','2.4788833e-06','2.8894056e-09','3.0401786e-06','2.0347977e-09','0.017096153','1.7142844e-05','4.164219e-06','0.020004194','0.03938084','0.00022692699','7.1010436e-06','0.020004194','0.03938084','1.980697e-08','4.1298827e-09','1.4933664','0.051195554','4.2257354e-05','3.0447043e-06','0.051195554','0.87512296','0.1222243','0.11684437','0.0017428068','4.7229453e-14','6.878977e-06','2.9946157e-11','2.910785e-11','7.6843986e-07','9.5369743e-20','1.0938675e-17','3.5106827e-24','4.2092655e-05','7.2930356e-10','8.9442356e-11','0.0001603997','0.00042535976','1.0923965e-07','1.2323422e-07','0.00014565224','0.00037112413','2.5284988e-18','2.5940733e-19','0.034614354','3.322351e-13','7.1318675e-05','3.5012027e-10','3.4098202e-10','7.052163e-05','4.0881323e-19','4.6420077e-12','7.042328e-18','1.6823092e-12','2.2467818e-18','0.00038379183','4.182182e-10','1.6621796e-11','0.00057339243','0.0026501748','1.4166159e-07','8.06777e-09','0.00057339243','0.0026501748','8.7591296e-16','2.8285673e-17','0.034416903','0.0011298538','8.142843e-10','7.219616e-10','0.0011298538','0.10716953','0.004231322','0.0041232836'],
['Luxembourg','202002','1.4868455','5.752936e-07','0.006401211','1.0258291e-05','3.0182437e-07','0.0018429952','7.3549067e-10','2.541813e-09','8.5534075e-13','-0.041747876','2.238249e-05','8.24138e-06','0.011138915','0.017719764','0.00043900692','0.00012178709','0.010617863','0.016573349','2.719019e-09','8.726761e-10','1.7904028','1.7398288e-06','0.016622376','3.8316306e-05','-4.8659865e-07','0.016439192','9.2694924e-10','1.5972754e-06','2.4942703e-09','2.700328e-06','1.6677416e-09','0.012609278','1.7601249e-05','3.7660998e-06','0.018380161','0.03830293','0.00031762224','2.9865123e-05','0.018380161','0.03830293','1.9316046e-08','3.6485008e-09','1.4513507','0.045384534','5.2666568e-05','1.5044319e-06','0.045384534','0.8661247','0.12443179','0.11854327','0.0019788833','1.9755273e-14','2.6187388e-06','3.2982405e-11','2.7829484e-11','3.4654025e-07','1.8783478e-19','1.6378102e-17','3.3530186e-24','4.0212122e-05','3.951905e-09','5.111185e-10','0.0009205122','0.002395165','1.512311e-06','6.8832503e-07','0.00083599024','0.0020922925','5.594693e-17','4.998301e-18','0.032749187','4.1195503e-13','9.850875e-05','5.5489346e-10','4.2728515e-10','9.756759e-05','2.2178007e-19','1.3929817e-12','7.439495e-18','1.3292187e-12','9.30134e-19','0.0003566825','6.7036365e-10','1.6301922e-11','0.00046321005','0.0027657228','4.1389399e-07','1.7493765e-08','0.00046321005','0.0027657228','9.935253e-16','1.6012905e-17','0.026398951','0.0006632078','1.0017744e-09','7.692854e-10','0.0006632078','0.092193305','0.006295218','0.005911447'],
['Malta','202002','1.4265587','6.0499656e-07','0.0060114227','4.604607e-06','-3.9098654e-06','0.0014005533','3.3958744e-10','1.46734145e-08','4.4275077e-11','-0.046588767','2.454661e-05','8.9426e-06','0.012036795','0.019181952','0.00023309382','-0.00014883702','0.011472062','0.017935006','9.453589e-09','2.6775444e-09','2.0828557','2.4661767e-06','0.012933997','2.1246431e-05','-1.711019e-05','0.012677864','1.0490889e-09','8.258836e-06','3.339749e-09','3.307042e-06','1.4569207e-09','0.016612457','2.1448694e-05','4.258954e-06','0.011302501','0.02378483','0.00013883019','-0.00014274534','0.011302501','0.02378483','3.143226e-08','5.68475e-09','1.7571168','0.031587888','2.2806902e-05','-2.0441174e-05','0.031587888','0.901647','0.093500316','0.08738623','0.0006281971','1.9659662e-14','2.2990666e-06','1.7997975e-11','8.114328e-12','2.0212575e-07','2.7287535e-20','2.538065e-16','2.2110976e-21','4.9152302e-05','8.77558e-10','1.1826534e-10','0.00021385477','0.0005397904','1.3636253e-07','9.842329e-08','0.0001942605','0.00047216224','1.963477e-16','1.4058995e-17','0.018923577','7.896245e-13','5.4956294e-05','4.9989e-10','1.7968974e-10','5.389526e-05','3.612998e-19','1.6707025e-10','1.9537586e-18','1.0222313e-12','3.1582464e-19','9.7569035e-05','3.3317654e-10','6.424861e-12','4.9732014e-05','0.00026744755','6.6373744e-08','8.123252e-08','4.9732014e-05','0.00026744755','5.2625e-16','1.4815786e-17','0.008193253','0.00013524813','4.726511e-10','2.2252428e-10','0.00013524813','0.07122111','0.0012066615','0.0010869098'],
['Netherlands','202002','1.4699042','6.8307884e-07','0.0074329716','1.1269915e-05','2.3463858e-06','0.0020512633','9.978824e-10','8.498465e-09','1.1116241e-11','-0.041333277','1.9931656e-05','7.213565e-06','0.009742382','0.015594598','0.00038940657','0.00016345842','0.0092862155','0.014579985','4.067252e-09','1.3108142e-09','1.8485444','1.8089708e-06','0.016323209','3.905884e-05','4.376273e-06','0.016133219','1.1793505e-09','3.624996e-06','6.815548e-09','2.7705555e-06','1.6728064e-09','0.010052399','4.2646105e-05','8.204399e-06','0.0331767','0.07170811','0.0007466575','0.00013015357','0.0331767','0.07170811','5.0752572e-08','8.691736e-09','1.5256152','0.043637067','5.233896e-05','6.910298e-06','0.043637067','0.68114835','0.1546966','0.14811018','0.0018790631','5.242284e-14','6.908234e-06','3.2311483e-11','4.442626e-11','7.6718726e-07','5.1841074e-19','1.0006001e-16','2.1517472e-22','3.898415e-05','4.4140016e-09','5.0555204e-10','0.00089514465','0.0024641224','1.7386783e-06','1.1976047e-06','0.0008124094','0.0021455544','3.079552e-16','3.125521e-17','0.04470509','5.5072635e-13','0.0001200772','4.957025e-10','5.8703153e-10','0.00011892051','5.2486047e-19','4.455019e-12','1.7504055e-17','1.0211996e-12','1.5145112e-18','0.00029017217','1.820753e-09','4.7910044e-11','0.00085505197','0.0045731333','1.0374837e-06','1.3054454e-07','0.00085505197','0.0045731333','2.2504107e-15','5.1265137e-17','0.028628746','0.00071104214','8.380395e-10','8.624941e-10','0.00071104214','0.098038316','0.0076860082','0.0073175626'],
['Poland','202002','1.4704133','7.84689e-07','0.008520761','1.046394e-05','-1.3883691e-07','0.0023484682','1.1645288e-09','1.7049242e-09','2.0807213e-13','-0.04135737','1.8899074e-05','7.005173e-06','0.009488977','0.01503178','0.00023319553','-1.4613237e-05','0.009045943','0.014062331','1.0307355e-09','3.6150447e-10','1.8296052','2.056936e-06','0.01792332','3.1337066e-05','-2.8903185e-06','0.017707266','2.251037e-09','5.306456e-06','8.558744e-09','3.6917058e-06','2.3253643e-09','0.023062188','1.2768948e-05','3.204687e-06','0.0139298625','0.026667528','0.00019360932','-2.1864178e-05','0.0139298625','0.026667528','1.0960558e-08','2.500735e-09','1.502742','0.062325623','5.0725004e-05','-3.4057157e-06','0.062325623','1.0003157','0.13046883','0.123110496','0.0014472419','5.6031307e-14','7.936276e-06','4.5223554e-11','3.6552174e-11','9.012754e-07','3.749071e-19','3.9224412e-18','1.2806264e-25','4.591849e-05','2.5252556e-09','3.3744885e-10','0.0006123696','0.0015667181','5.8579604e-07','1.3152793e-07','0.0005563222','0.0013699812','2.8609036e-18','2.834803e-19','0.03352229','4.952502e-13','9.7671866e-05','4.9502596e-10','3.5352374e-10','9.6564916e-05','1.682852e-18','7.0552258e-12','2.1988957e-17','2.7738367e-12','2.2790673e-18','0.00047192172','3.0442415e-10','1.2807865e-11','0.00030072127','0.0013516068','1.2696154e-07','8.356646e-09','0.00030072127','0.0013516068','2.2823832e-16','7.769095e-18','0.032874554','0.0017150008','1.1515437e-09','7.944522e-10','0.0017150008','0.0993896','0.004631693','0.0043269084'],
['Portugal','202002','1.4423823','5.4930257e-07','0.0055507845','2.436451e-06','-3.265002e-07','0.0013562263','3.948622e-10','8.83663e-10','5.59466e-13','-0.046214834','6.2460764e-05','2.1514155e-05','0.028778967','0.047195885','0.00020856883','4.8029666e-05','0.027423117','0.044062816','1.45023735e-08','3.916032e-09','1.9907458','2.3624634e-06','0.013719173','1.2655339e-05','-1.4882014e-06','0.013473612','1.2434616e-09','3.1669003e-06','3.7662717e-09','3.3220067e-06','1.5234908e-09','0.02886345','8.361217e-06','2.192339e-06','0.006965856','0.012783157','1.5345238e-05','-4.044947e-06','0.006965856','0.012783157','7.430706e-09','1.6560027e-09','1.6983933','0.036257684','1.3983482e-05','-1.6763591e-06','0.036257684','0.9392791','0.11550588','0.10793387','0.0013339483','2.2581925e-14','1.9311476e-06','1.1560543e-11','7.660922e-12','1.3303392e-07','3.746379e-20','6.9797144e-18','6.174131e-24','6.245748e-05','2.0713447e-08','2.2794764e-09','0.004006276','0.011270216','7.202288e-07','4.904252e-07','0.003635459','0.0098021235','1.9113409e-15','1.282463e-16','0.035588223','8.4060547e-13','3.332593e-05','2.705206e-10','1.9047648e-10','3.277825e-05','5.1148527e-19','4.291448e-12','6.5513854e-18','9.505509e-13','5.5716244e-19','0.00031787745','4.2806984e-11','1.917484e-12','3.7941736e-05','0.00014272558','2.9368774e-09','2.4230784e-09','3.7941736e-05','0.00014272558','4.0482692e-17','1.2679104e-18','0.041658964','0.00039671853','3.5294107e-10','2.242956e-10','0.00039671853','0.10847433','0.012679486','0.011068715'],
['Romania','202002','1.4579622','6.9140253e-07','0.007350967','6.400532e-06','-2.9737635e-06','0.0019603209','7.043131e-10','1.6729005e-09','4.7451853e-14','-0.043703947','1.5096299e-05','5.630821e-06','0.0076174336','0.012058741','0.00015943925','-0.00012174901','0.0072613647','0.011282323','1.7263355e-09','5.603923e-10','1.9109851','2.7846436e-06','0.021638673','2.6096297e-05','-1.0675551e-05','0.021347284','3.3691314e-09','3.5761025e-06','3.7267336e-09','3.9385886e-06','2.1000715e-09','0.037643347','3.849298e-06','1.1366017e-06','0.0036297115','0.006272178','3.9800587e-05','-1.5489633e-05','0.0036297115','0.006272178','2.5238067e-09','6.9387907e-10','1.5759934','0.062255207','3.3943874e-05','-1.56872e-05','0.062255207','1.2216989','0.109575905','0.10311609','0.0017600372','5.1946288e-14','7.360936e-06','2.6795247e-11','1.7702477e-11','8.396915e-07','1.9581478e-19','1.03601386e-17','9.253438e-26','3.780884e-05','6.091208e-10','8.0363396e-11','0.00014349048','0.00037411333','1.8930668e-07','1.4202656e-07','0.0001302851','0.00032691477','9.02374e-18','7.9064756e-19','0.038879454','1.9065181e-12','0.00028679462','4.821529e-10','3.5715123e-10','0.0002828158','8.567136e-18','6.1262926e-12','6.6473607e-18','3.758477e-12','2.5187674e-18','0.0008008208','1.5247227e-11','8.3109485e-13','8.735576e-06','3.428243e-05','4.7850457e-09','1.2822007e-09','8.735576e-06','3.428243e-05','6.3131553e-18','3.7005695e-19','0.039786506','0.0025173605','8.2673374e-10','5.3913757e-10','0.0025173605','0.07004988','0.0045780386','0.004345452'],
['Slovakia','202002','1.4514207','6.602989e-07','0.0069159265','7.285708e-06','-2.648974e-06','0.0017990782','6.242311e-10','1.2715758e-09','5.686857e-14','-0.041696776','1.5409792e-05','5.8635524e-06','0.007980758','0.012487938','0.00019042299','-7.5249984e-05','0.0076092035','0.0116910655','8.807881e-10','3.1616582e-10','1.9269532','1.9391534e-06','0.014719818','2.5502497e-05','-8.915927e-06','0.014517162','1.3134567e-09','3.415895e-06','3.8146397e-09','3.2867993e-06','1.8204963e-09','0.037849918','5.6442614e-06','1.5767125e-06','0.004681949','0.00844101','6.490459e-05','-1.66845e-05','0.004681949','0.00844101','4.104779e-09','1.0766311e-09','1.6149355','0.04430784','3.6495392e-05','-1.1650098e-05','0.04430784','1.17923','0.08687309','0.08075581','0.001906217','3.1596365e-14','4.516024e-06','2.7357542e-11','2.2890056e-11','5.7027137e-07','6.031952e-20','3.1493498e-18','9.682795e-27','3.698524e-05','8.003971e-10','1.1009725e-10','0.00020149956','0.0005064995','2.3619315e-07','1.0021946e-07','0.00018310345','0.00044329453','2.282496e-18','2.418589e-19','0.048412584','3.0851228e-13','9.396246e-05','3.1614203e-10','2.272726e-10','9.318587e-05','3.464367e-19','5.0051686e-12','4.812116e-18','1.4763374e-12','1.1910044e-18','0.0005061332','4.762245e-11','2.1400613e-12','2.0414416e-05','9.162868e-05','1.6676157e-08','3.8911216e-09','2.0414416e-05','9.162868e-05','2.0532725e-17','8.429493e-19','0.029476553','0.0006073397','6.4836914e-10','6.0159766e-10','0.0006073397','0.07666859','0.0019280886','0.001782952'],
['Slovenia','202002','1.442473','6.4241607e-07','0.0065853097','7.00435e-06','-2.4588621e-06','0.0016479972','5.217623e-10','1.3133398e-09','2.7984012e-13','-0.04261058','1.9783667e-05','7.677934e-06','0.0104592135','0.016256263','0.00028883488','-7.286149e-05','0.0099724615','0.015225868','1.5454351e-09','5.2114824e-10','1.9796642','2.0288514e-06','0.0136516625','2.6624806e-05','-1.0447647e-05','0.013440028','1.2105964e-09','4.800708e-06','5.63618e-09','3.3844876e-06','2.0083581e-09','0.03322081','6.2979007e-06','1.6126428e-06','0.0045240247','0.0084488625','5.6142406e-05','-2.1115004e-06','0.0045240247','0.0084488625','4.067827e-09','9.468661e-10','1.6513677','0.043966148','3.6773174e-05','-1.0966041e-05','0.043966148','1.1826112','0.088907555','0.0827294','0.0014698623','2.890101e-14','3.8085977e-06','2.8840616e-11','1.8337349e-11','4.15752e-07','6.0588014e-20','4.210321e-18','1.2319879e-24','3.0282916e-05','1.4798667e-09','2.2435938e-10','0.00041484885','0.0010034156','5.4729225e-07','8.76039e-08','0.0003770707','0.000880297','9.957479e-18','1.1077896e-18','0.040181044','3.9351617e-13','8.7565226e-05','4.6500054e-10','2.2813788e-10','8.664449e-05','4.6567373e-19','5.537526e-12','1.0613725e-17','1.6341085e-12','1.3968628e-18','0.00055373914','4.7532565e-11','1.9687223e-12','1.6695689e-05','7.2938514e-05','6.7769297e-09','1.7342687e-09','1.6695689e-05','7.2938514e-05','2.3606032e-17','9.937872e-19','0.03269437','0.0008291049','8.7514657e-10','4.9820265e-10','0.0008291049','0.06753969','0.0031334918','0.0028754452'],
['Spain','202002','1.4420339','5.774941e-07','0.005872951','3.4773739e-06','-9.292767e-07','0.0014538968','8.5698865e-10','5.6706045e-10','1.01137445e-14','-0.043312985','4.6694822e-05','1.7192724e-05','0.023270749','0.036966957','0.00034360838','-7.704031e-05','0.02218275','0.034576148','1.588144e-08','4.806185e-09','1.9836621','2.2298289e-06','0.013349339','1.7512088e-05','-4.7307394e-06','0.013117549','1.6053101e-09','5.9449794e-06','9.5236885e-09','3.3491056e-06','2.698188e-09','0.04208696','5.325559e-06','1.3060861e-06','0.0034947707','0.0065904767','1.6824948e-05','-1.6390486e-06','0.0034947707','0.0065904767','4.1919797e-09','9.106199e-10','1.6992737','0.039347917','1.8924928e-05','-4.301908e-06','0.039347917','1.1043632','0.10212815','0.09508626','0.0012636945','1.2887885e-14','1.6394606e-06','1.0422934e-11','7.232171e-12','1.981516e-07','2.2526155e-19','1.0443901e-18','5.3722036e-27','6.197186e-05','1.2226482e-08','1.5580416e-09','0.002823335','0.007347199','1.1075721e-06','9.633915e-07','0.0025645066','0.006416446','1.5338532e-15','1.3604763e-16','0.034349397','5.2012176e-13','3.6448826e-05','2.648586e-10','2.0682679e-10','3.6023484e-05','7.264857e-19','9.2912865e-12','2.106746e-17','1.0490182e-12','2.1521434e-18','0.0007351104','6.401996e-11','1.7256878e-12','1.7260307e-05','8.911812e-05','1.1123715e-09','4.249516e-10','1.7260307e-05','8.911812e-05','3.408503e-17','7.573371e-19','0.04184036','0.0006704937','3.2929445e-10','1.951841e-10','0.0006704937','0.12485575','0.008933993','0.007885567'],
['Sweden','202002','1.4825611','4.1402117e-07','0.0046883523','5.356718e-06','3.5414038e-07','0.0013846152','2.627313e-10','5.5392837e-09','2.610303e-12','-0.03928175','1.1548329e-05','4.336427e-06','0.005886243','0.009265686','0.00013383693','-4.9138285e-05','0.0056115976','0.008671102','7.106506e-10','2.4617725e-10','1.850305','1.2520007e-06','0.011588449','1.9228135e-05','-1.1336349e-06','0.011456916','5.283795e-10','1.6913501e-06','2.2503226e-09','2.31127e-06','1.4456728e-09','0.01368473','1.9902389e-05','4.482228e-06','0.01859044','0.037162364','0.00018445296','7.494953e-05','0.01859044','0.037162364','3.357377e-08','5.9867795e-09','1.527156','0.037566546','2.8688699e-05','3.2423989e-06','0.037566546','0.77085423','0.10027168','0.09624203','0.0031195309','2.4122562e-14','4.410545e-06','1.5596552e-11','2.5245465e-11','6.701579e-07','2.9401805e-20','3.7191605e-17','1.2343955e-23','4.7469228e-05','5.3789945e-10','6.268406e-11','0.00011126098','0.00030218068','8.165754e-08','1.416901e-07','0.00010099484','0.00026322805','4.708844e-19','3.9198183e-20','0.048067063','1.4063331e-13','6.1615305e-05','1.5267027e-10','2.352958e-10','6.115088e-05','8.423427e-20','1.4979422e-12','3.892821e-18','9.493805e-13','1.9145641e-18','0.00012255149','3.016446e-10','9.897968e-12','0.0002835367','0.0013891254','1.1726937e-07','7.085983e-08','0.0002835367','0.0013891254','1.1391604e-15','2.6344993e-17','0.03277524','0.0007386234','4.546009e-10','6.557274e-10','0.0007386234','0.05038848','0.003805912','0.003654157'],
['Ireland','202002','1.4884765','3.2860402e-07','0.0037157822','8.296866e-06','1.5053889e-06','0.0010922233','1.4190456e-10','1.7625984e-08','1.1473496e-11','-0.0392379','1.0109423e-05','3.939523e-06','0.0054012844','0.00833092','0.00018277326','8.5653526e-05','0.0051510483','0.0078050415','2.156459e-09','7.2459944e-10','1.8373075','1.3633155e-06','0.01191376','3.5937133e-05','6.7483243e-06','0.011770635','3.2443123e-10','8.0063427e-07','1.2165576e-09','1.8963557e-06','7.240564e-10','-0.0021734554','9.238452e-05','1.3628946e-05','0.063207954','0.15463537','0.0017381503','0.00041834154','0.063207954','0.15463537','9.8782365e-08','1.3421359e-08','1.5212791','0.027169723','4.433801e-05','7.661593e-06','0.027169723','0.36938816','0.20576434','0.20247185','0.0033314028','1.2718684e-14','1.8339447e-06','1.887545e-11','1.8322842e-11','2.4739256e-07','1.5004152e-20','4.9118017e-16','1.997688e-22','4.6363715e-05','2.4166671e-10','2.9988095e-11','5.4300057e-05','0.0001416303','4.7635073e-08','8.82371e-08','4.932211e-05','0.00012361951','7.40512e-17','7.067399e-18','0.046329927','3.4793492e-13','5.0091265e-05','4.4166504e-10','3.7691092e-10','4.9521717e-05','5.2457297e-20','4.3014862e-13','1.7623017e-18','4.1743933e-13','3.4361708e-19','5.6560348e-05','5.688782e-09','9.4811575e-11','0.0024198866','0.015933238','3.783955e-06','1.0044297e-06','0.0024198866','0.015933238','5.960076e-15','9.946404e-17','0.027277274','0.00018800833','5.687486e-10','5.0668114e-10','0.00018800833','0.06391012','0.014767781','0.014839968'],
['Italy','202002','1.4365351','6.762759e-07','0.00681404','4.636416e-06','-3.0144734e-06','0.0016477194','6.196715e-10','5.069299e-09','7.536571e-12','-0.04347902','1.8010636e-05','6.960933e-06','0.009471614','0.014745889','0.000194543','-7.0280366e-05','0.009030608','0.013809324','3.4524834e-09','1.1318457e-09','2.0281558','2.7319375e-06','0.01609878','2.5629977e-05','-1.3742727e-05','0.01581463','1.5162237e-09','6.800777e-06','7.885382e-09','3.1799393e-06','1.5739094e-09','0.022139702','1.1758753e-05','2.5766517e-06','0.007929219','0.015987162','6.420305e-05','-1.766678e-05','0.007929219','0.015987162','1.2770701e-08','2.5309213e-09','1.7287557','0.034655903','2.3181543e-05','-1.401937e-05','0.034655903','1.1129395','0.088301815','0.08191447','0.0011464328','3.3160086e-14','3.6156341e-06','2.144359e-11','1.7740767e-11','2.9974734e-07','1.050314e-19','1.3100443e-16','3.5600023e-22','2.678355e-05','9.546077e-10','1.4576763e-10','0.0002686068','0.000648922','1.8248437e-07','4.301897e-08','0.00024414275','0.0005693279','4.5931738e-17','4.8489373e-18','0.031265438','6.0766535e-12','0.00024746175','1.1751669e-09','3.1036826e-10','0.00024011123','6.6043945e-18','1.4036693e-11','2.1359216e-17','8.4764736e-13','6.2712946e-19','0.00030538673','1.623488e-10','4.4562604e-12','7.1614806e-05','0.00036181087','4.1861583e-08','1.1539093e-08','7.1614806e-05','0.00036181087','2.0458708e-16','6.26948e-18','0.04077223','0.0003847488','4.990364e-10','4.1452974e-10','0.0003847488','0.083476834','0.0026532079','0.0024678744'],
['Denmark','202002','1.4855185','4.779963e-07','0.005438567','7.280497e-06','1.4762069e-06','0.0016242705','4.321183e-10','1.4415534e-08','1.7756362e-11','-0.039232608','1.6738843e-05','6.23204e-06','0.0084514385','0.013357859','0.0001637017','0.00013832969','0.008057252','0.01249813','1.7461553e-09','5.8776445e-10','1.8237116','1.452321e-06','0.013978607','2.6497028e-05','2.351613e-06','0.013825847','6.6509553e-10','1.5041246e-06','2.2227085e-09','2.4121748e-06','1.5792181e-09','0.0032068363','4.6608257e-05','8.634038e-06','0.043448523','0.09619307','0.00076043705','0.00018555438','0.043448523','0.09619307','7.1415435e-08','1.154985e-08','1.472709','0.042855263','3.6859044e-05','6.2555705e-06','0.042855263','0.564274','0.1718233','0.16699712','0.0026310326','4.5887092e-14','7.827827e-06','1.8191037e-11','3.6269307e-11','1.086741e-06','1.0726943e-19','2.795458e-16','4.5814386e-22','4.5525034e-05','2.862535e-09','3.443645e-10','0.00061187445','0.0016510967','1.9570997e-07','8.726025e-07','0.00055557484','0.0014393929','2.7258786e-17','2.6855097e-18','0.04434001','6.228821e-13','0.00014995424','2.401436e-10','4.5961562e-10','0.00014826353','1.4444622e-19','1.2129803e-12','2.0411716e-18','1.1766519e-12','2.4171105e-18','0.00016035372','2.3891116e-09','5.953355e-11','0.0016020865','0.009038314','1.2031797e-06','2.6280972e-07','0.0016020865','0.009038314','4.6530412e-15','9.518968e-17','0.03255407','0.00096741354','5.099455e-10','7.200246e-10','0.00096741354','0.08948844','0.01157794','0.011273747'],
['Finland','202002','1.4828063','3.8848557e-07','0.0044208653','4.6495834e-06','7.058733e-07','0.0013312276','2.7407357e-10','6.5767156e-09','2.5361863e-12','-0.0384211','8.9837495e-06','3.44401e-06','0.0047096135','0.007314883','0.00010385543','-5.7039404e-05','0.0044910344','0.0068498584','1.3155119e-09','4.2077114e-10','1.8357048','1.1984339e-06','0.011774511','1.6376485e-05','-9.2547623e-07','0.011648532','6.056122e-10','1.0834983e-06','1.2513574e-09','2.4801227e-06','2.3189315e-09','0.023330674','1.4036926e-05','3.7920308e-06','0.017548272','0.03218948','0.00012152831','9.951681e-06','0.017548272','0.03218948','1.608872e-08','3.8597108e-09','1.4736369','0.04557474','2.449365e-05','5.326861e-06','0.04557474','0.83836913','0.10127459','0.09759394','0.0039483095','2.3967563e-14','4.7881595e-06','1.1291233e-11','2.962223e-11','7.9320773e-07','3.1569765e-20','4.1986808e-17','8.044007e-24','5.5181987e-05','2.9935993e-10','3.4311384e-11','6.1238694e-05','0.00016564666','6.36625e-08','1.3686973e-07','5.5599456e-05','0.00014422994','4.4391177e-18','3.5540828e-19','0.055521388','1.3499943e-13','8.329794e-05','1.1248084e-10','2.9708586e-10','8.2750856e-05','1.2229466e-19','7.900566e-13','1.5959675e-18','8.9536874e-13','3.122395e-18','0.00075605017','1.8228351e-10','8.360155e-12','0.00031489666','0.0013429675','3.5710208e-08','5.4052975e-08','0.00031489666','0.0013429675','5.4162114e-16','2.088297e-17','0.059066053','0.001098419','3.0646696e-10','9.092737e-10','0.001098419','0.062062338','0.004034233','0.0039039073'],
['Belgium','202002','1.4781356','6.6677546e-07','0.0073151095','1.1249362e-05','1.7982553e-06','0.0020504857','1.0509295e-09','4.3085078e-09','2.6067703e-12','-0.041854784','2.3781096e-05','8.702534e-06','0.011753842','0.018743817','0.0004526719','0.00017597192','0.011203811','0.017528132','4.014336e-09','1.2831318e-09','1.8150955','1.9001872e-06','0.017577369','4.1033363e-05','3.1573077e-06','0.017377634','1.2733383e-09','2.668885e-06','3.816616e-09','2.7950434e-06','1.5635271e-09','0.011472363','3.16345e-05','6.2025724e-06','0.026429998','0.056812044','0.00056135375','8.7475986e-05','0.026429998','0.056812044','3.2514407e-08','5.8147376e-09','1.5000175','0.044356104','5.491065e-05','5.7892316e-06','0.044356104','0.76664364','0.14480445','0.13812435','0.0017979247','2.7181297e-14','3.4735863e-06','3.2107858e-11','3.6884457e-11','4.277827e-07','3.965866e-19','4.5377143e-17','2.2028714e-23','4.0591935e-05','4.81508e-09','5.957603e-10','0.0010647335','0.0028308749','1.6689606e-06','1.0838546e-06','0.000966756','0.0024697594','1.6205707e-16','1.5742154e-17','0.0369962','4.7110704e-13','0.00010222395','5.434713e-10','5.610072e-10','0.000101266865','3.4803624e-19','1.86865e-12','5.9904015e-18','1.1191422e-12','9.207056e-19','0.00027506432','1.5529925e-09','3.5546545e-11','0.0007019285','0.0042194976','9.446593e-07','7.303542e-08','0.0007019285','0.0042194976','1.7039243e-15','3.1547937e-17','0.027728217','0.00059091434','1.0128632e-09','9.084989e-10','0.00059091434','0.106666304','0.0073919827','0.007009926'],
['Croatia','202002','1.4526703','6.1185904e-07','0.006407929','5.4919924e-06','-2.297364e-06','0.0016686858','3.8956183e-10','1.4675068e-09','4.4685935e-13','-0.043242555','1.8797142e-05','7.2809676e-06','0.009910001','0.015421426','0.00024763372','-5.1790343e-05','0.009448566','0.014442719','2.1096194e-09','6.958336e-10','1.9425272','2.126973e-06','0.01546296','2.3667993e-05','-1.0468068e-05','0.015240829','1.1477774e-09','3.9913016e-06','3.5904457e-09','3.2598864e-06','1.7663849e-09','0.022600494','8.422158e-06','1.9535396e-06','0.0069455625','0.013839452','5.2792697e-05','1.2162705e-05','0.0069455625','0.013839452','1.1146436e-08','1.997798e-09','1.6161336','0.044393487','2.7559994e-05','-1.0182186e-05','0.044393487','1.1184369','0.09552549','0.08958603','0.001924418','1.8474661e-14','2.9054268e-06','2.7263167e-11','2.112565e-11','4.0643675e-07','1.7162586e-20','5.8271532e-18','2.2992227e-24','3.0186273e-05','1.1320912e-09','1.7111207e-10','0.0003152169','0.0007654209','3.6233604e-07','8.339034e-08','0.00028648935','0.0006713276','2.2033944e-17','2.2267737e-18','0.044270337','5.121414e-13','9.8721895e-05','4.7720083e-10','2.9032216e-10','9.761288e-05','4.9307874e-19','3.941458e-12','4.8085374e-18','1.1016688e-12','9.777371e-19','0.00045234573','6.0797575e-11','2.2269495e-12','4.793854e-05','0.00022153145','8.427283e-09','4.6646456e-09','4.793854e-05','0.00022153145','1.2179624e-16','3.213181e-18','0.042178582','0.0006820287','7.3182216e-10','5.664275e-10','0.0006820287','0.07342933','0.0030199622','0.0028234522'],
['Austria','202003','1.4667755','6.619644e-07','0.0071720257','2.2061383e-06','-1.5285855e-06','0.0019635833','4.6174675e-10','7.269712e-10','1.2822613e-14','-0.047467887','5.8609614e-05','1.9272004e-05','0.025773117','0.042762842','0.00010638345','6.994181e-05','0.024558429','0.039880067','1.425618e-08','4.6295785e-09','1.8826195','2.3391906e-06','0.01886681','6.5850613e-06','-5.711651e-06','0.01862177','1.4463102e-09','1.5744755e-06','1.5106711e-09','4.2158595e-06','2.4764948e-09','0.04047033','1.8018529e-06','5.740979e-07','0.0023181618','0.0039135194','5.4915226e-06','-2.766515e-06','0.0023181618','0.0039135194','1.7618159e-09','4.5707277e-10','1.5420319','0.06837387','9.992204e-06','-8.235046e-06','0.06837387','1.0475534','0.14108816','0.13275096','0.0016818739','5.9508096e-14','8.4357e-06','3.1633883e-11','1.569831e-11','8.883797e-07','5.2068822e-20','5.538188e-18','5.709631e-27','5.2882937e-05','8.364729e-09','8.0562057e-10','0.0014155487','0.004109428','1.6128863e-06','7.735105e-07','0.0012844378','0.0035627463','2.0230617e-15','2.1281152e-16','0.03704857','1.2838334e-12','0.00018690599','6.0665245e-10','2.8636676e-10','0.00018409533','8.423345e-19','3.5071772e-12','1.9332321e-18','5.6369254e-12','4.038589e-18','0.0007281069','2.2948622e-12','1.2600682e-13','2.8885997e-06','1.1374407e-05','2.4803354e-10','2.1121982e-10','2.8885997e-06','1.1374407e-05','5.0623185e-18','2.0313909e-19','0.036718342','0.0031311985','1.1725604e-09','7.866984e-10','0.0031311985','0.07118427','0.011728551','0.0106681185'],
['Bulgaria','202003','1.4705552','9.4898184e-07','0.010377149','1.0926901e-06','-1.2112592e-06','0.0028759304','5.424418e-10','2.3235889e-09','2.0412508e-12','-0.053122234','0.000120458535','3.7043563e-05','0.048983954','0.08382565','0.00010111583','0.00034508161','0.046657298','0.078025505','3.618005e-08','9.977205e-09','1.9176842','4.6671776e-06','0.038219318','4.130599e-06','-8.606675e-06','0.037729405','2.4392262e-09','6.246749e-06','3.924326e-09','8.866417e-06','4.9122635e-09','0.030663414','6.5432714e-06','1.6336193e-06','0.0064023198','0.012114287','-2.2282067e-05','2.2472598e-06','0.0064023198','0.012114287','6.814714e-09','1.3723117e-09','1.6054873','0.14133865','1.0713424e-06','-8.508211e-06','0.14133865','1.0109978','0.28587347','0.27208248','0.003560174','2.3625364e-13','2.931782e-05','2.4590817e-11','3.4318624e-11','3.05165e-06','1.1251202e-19','3.9015003e-17','3.6641097e-23','8.640078e-05','2.0862181e-08','1.8254633e-09','0.0031677291','0.009593416','1.9784043e-06','2.1492842e-06','0.0028731194','0.008295352','7.787924e-15','5.634971e-16','0.049677677','1.1222817e-11','0.0013633753','7.5895756e-10','7.1491657e-10','0.0013432138','4.730187e-18','1.5436609e-11','6.4440124e-18','2.9659265e-11','1.4140424e-17','0.00060962775','5.593805e-11','1.9799297e-12','5.480294e-05','0.0002421099','7.456655e-09','5.0837325e-09','5.480294e-05','0.0002421099','1.0426291e-16','2.617595e-18','0.04938252','0.016704986','2.114062e-09','1.978351e-09','0.016704986','0.1380482','0.039166696','0.036900084'],
['Cyprus','202003','1.4352984','6.278058e-07','0.006362933','4.216244e-06','2.2473031e-07','0.0015505264','2.2080916e-10','6.2064287e-09','9.571751e-12','-0.05809873','0.00022755176','6.231292e-05','0.08157254','0.14542338','0.0007806483','0.0002722403','0.07767015','0.13494225','5.1010506e-08','1.2051122e-08','2.0369053','2.4290323e-06','0.014729626','2.1345068e-05','1.9984484e-06','0.014476702','5.50134e-10','6.2901076e-06','4.2753903e-09','6.260117e-06','2.6682225e-09','0.020657687','1.0036175e-05','2.1395929e-06','0.0061897356','0.012802706','2.5486728e-05','-3.0137116e-06','0.0061897356','0.012802706','1.2308865e-08','2.2727957e-09','1.7397293','0.06972956','1.8845478e-05','-2.2790518e-06','0.06972956','0.6239015','0.24905057','0.23350234','0.001207203','2.966791e-14','3.961962e-06','1.9879955e-11','2.3358572e-11','4.1047522e-07','2.35186e-20','1.731142e-16','2.731011e-22','4.359775e-05','6.434764e-08','4.1553614e-09','0.0070475917','0.023630524','7.0724077e-06','7.183274e-06','0.006386934','0.020267807','9.402425e-15','4.823277e-16','0.033043765','7.310346e-13','0.00012264898','4.5979842e-10','4.601459e-10','0.00012098761','2.0371462e-19','1.818613e-11','7.3714415e-18','9.6667405e-12','3.178147e-18','0.00020745074','4.9204692e-11','1.3788355e-12','2.343065e-05','0.00012240873','1.16330625e-08','2.447039e-09','2.343065e-05','0.00012240873','1.1857916e-16','3.1242345e-18','0.017499','0.0035152908','1.888476e-09','1.887691e-09','0.0035152908','0.09174243','0.039575975','0.035058185'],
['Czech Republic','202003','1.4429692','1.002593e-06','0.010334345','3.2670089e-06','-1.1960334e-06','0.002588315','1.0974003e-09','1.0555463e-09','1.1777472e-14','-0.04725478','4.675454e-05','1.5611142e-05','0.02092049','0.034464903','6.058536e-05','-5.66908e-05','0.01993604','0.032154925','6.3753145e-09','2.0728792e-09','1.9806702','3.1090674e-06','0.019352864','7.2632542e-06','-6.010631e-06','0.019029003','2.4535682e-09','5.321869e-06','5.2779905e-09','6.044158e-06','3.272774e-09','0.043554492','4.1810868e-06','1.2151141e-06','0.0039046442','0.00681751','1.784016e-05','-6.1078626e-06','0.0039046442','0.00681751','2.8924192e-09','7.3524686e-10','1.6341658','0.08449988','1.8263183e-05','-7.3155215e-06','0.08449988','1.1148534','0.15546897','0.14508933','0.0010974791','1.6771613e-13','2.0639709e-05','6.8557895e-11','2.8246834e-11','1.7874281e-06','4.567203e-19','6.018362e-18','2.9129717e-27','4.517845e-05','4.3552113e-09','4.1054415e-10','0.0007201626','0.0021041841','5.928423e-07','3.6888395e-07','0.00065345265','0.0018231437','1.3951805e-16','1.4151e-17','0.03213224','4.8142644e-12','0.00023823857','1.0167143e-09','5.173394e-10','0.0002322477','4.1059244e-18','1.1977336e-11','1.0105421e-17','1.2692032e-11','9.339703e-18','0.0009996572','1.9633017e-11','8.976724e-13','1.0499401e-05','4.4862685e-05','3.811578e-09','1.5201908e-09','1.0499401e-05','4.4862685e-05','1.4324381e-17','4.997638e-19','0.032557778','0.005147689','2.0883768e-09','9.778651e-10','0.005147689','0.057286076','0.010900388','0.00991998'],
['Estonia','202003','1.4701535','5.813878e-07','0.006389672','5.489462e-06','8.709418e-07','0.0018024805','3.3317135e-10','3.7819787e-09','2.212286e-12','-0.046992507','2.8998635e-05','1.0030158e-05','0.01349075','0.021907557','0.00027309562','3.5154462e-05','0.01285744','0.020458307','3.2601652e-09','1.0814712e-09','1.9055469','2.0581635e-06','0.015905647','1.9885136e-05','1.3213859e-06','0.015690424','1.1477967e-09','1.4814112e-06','9.79749e-10','3.8043236e-06','1.8934267e-09','0.023653507','1.2001536e-05','2.9805058e-06','0.011567396','0.021770822','0.00010483271','1.2076526e-05','0.011567396','0.021770822','1.718405e-08','3.5364127e-09','1.5862725','0.057062645','2.9089239e-05','6.892179e-06','0.057062645','0.87613356','0.12303626','0.116785556','0.0032981993','4.401244e-14','7.04193e-06','2.7266488e-11','2.4267735e-11','1.0164498e-06','3.1640305e-20','3.9839788e-17','2.3317236e-23','5.4733882e-05','9.1681157e-10','8.651027e-11','0.00015172214','0.00044100816','1.9508144e-07','1.593446e-07','0.00013766302','0.00038217698','1.118754e-17','1.1427129e-18','0.051711887','1.5560922e-12','0.00012234438','3.6857514e-10','2.5465857e-10','0.00012071325','6.488168e-19','1.2612281e-12','6.7842154e-19','2.6370848e-12','2.7069845e-18','0.00042922876','1.11172786e-10','5.1524e-12','0.00012791238','0.00052161247','3.082838e-08','1.5215715e-08','0.00012791238','0.00052161247','3.6495365e-16','1.2145545e-17','0.054332957','0.0020155192','9.400727e-10','1.126276e-09','0.0020155192','0.04870533','0.004133515','0.0038921675'],
['France','202003','1.4665442','6.6827636e-07','0.007244782','1.3778047e-06','-1.3653678e-06','0.0019707356','4.3054624e-10','1.1140625e-09','2.5902118e-13','-0.050354905','6.78846e-05','2.093438e-05','0.027783627','0.047302824','-5.7133902e-06','3.9633054e-05','0.026467223','0.04403729','5.376e-09','1.4742643e-09','1.9060283','2.3463224e-06','0.018109296','7.1178724e-06','-6.1321166e-06','0.017863797','1.0129189e-09','1.9874415e-06','1.2198583e-09','5.0763606e-06','2.3567406e-09','0.024524268','6.092584e-06','1.5822939e-06','0.007010346','0.013202794','3.270717e-05','-2.9067744e-06','0.007010346','0.013202794','6.313267e-09','1.4088292e-09','1.5638599','0.077676825','4.1862704e-06','-4.513909e-06','0.077676825','0.88421696','0.16353613','0.15475209','0.0023904806','8.0216995e-14','1.0985118e-05','3.731898e-11','1.8880786e-11','1.2182601e-06','8.2108856e-20','1.1139463e-17','3.0305411e-24','6.794358e-05','7.121892e-09','5.5541965e-10','0.0009517563','0.0030071298','4.6062985e-07','2.6994806e-07','0.0008628697','0.002591927','5.215988e-17','3.1903074e-18','0.03474101','7.488367e-13','0.00014005763','6.160123e-10','2.9998237e-10','0.00013873892','5.041388e-19','8.50399e-12','2.5853915e-18','1.6287287e-11','6.0245227e-18','0.00054969796','3.4968722e-11','1.4124666e-12','4.227408e-05','0.00019166282','8.976776e-09','3.6028367e-09','4.227408e-05','0.00019166282','7.1139584e-17','2.0335036e-18','0.033649478','0.0069228294','1.3657755e-09','8.759975e-10','0.0069228294','0.060640156','0.015641494','0.014503397'],
['Germany','202003','1.4493595','8.3770726e-07','0.008764563','3.3822487e-06','-4.491455e-07','0.0022632321','7.8116524e-10','1.5761121e-09','2.966926e-13','-0.046776596','3.8700084e-05','1.2993603e-05','0.017431295','0.028634276','2.1761016e-05','-3.666393e-05','0.016611675','0.026719522','5.618732e-09','1.8070727e-09','1.9615052','2.45286e-06','0.015853502','7.887917e-06','-4.2878996e-06','0.0155978','1.3346395e-09','3.7441807e-06','4.9929434e-09','5.0786775e-06','2.9600218e-09','0.036171414','6.3123075e-06','1.6995995e-06','0.006629884','0.012118769','3.514342e-05','2.2602683e-06','0.006629884','0.012118769','5.0630775e-09','1.2218279e-09','1.6065202','0.076579735','1.7880757e-05','-3.2989494e-06','0.076579735','1.0332193','0.141952','0.1332799','0.0015494039','8.8274284e-14','1.2223871e-05','6.936975e-11','1.7431605e-11','1.2958608e-06','1.8825766e-19','9.107518e-18','1.7410501e-24','4.021908e-05','3.3751981e-09','3.1065153e-10','0.0005424251','0.0016024493','4.248022e-07','1.2024967e-07','0.0004921241','0.0013875571','1.7912318e-16','1.7387257e-17','0.03943329','1.4797489e-12','7.8210316e-05','9.597639e-10','2.3212164e-10','7.6770455e-05','1.1163147e-18','5.5937932e-12','1.0413305e-17','8.547255e-12','5.342173e-18','0.0012372832','5.764309e-11','2.176504e-12','4.6194906e-05','0.00021149805','1.5096205e-08','6.1043464e-09','4.6194906e-05','0.00021149805','4.8066278e-17','1.3799782e-18','0.050266948','0.005010418','2.083508e-09','6.587132e-10','0.005010418','0.05492756','0.009214279','0.008577612'],
['Greece','202003','1.4631276','6.1516687e-07','0.0066031027','2.3983782e-06','-1.18594926e-07','0.0017919219','3.3787945e-10','1.3659911e-09','1.1621239e-12','-0.05440513','9.66689e-05','2.8362467e-05','0.03745151','0.064870216','0.00035185195','0.0005321787','0.03567061','0.060310967','1.918206e-08','5.129759e-09','1.9263706','2.5243132e-06','0.019441672','1.1110563e-05','-6.8104754e-07','0.019177562','1.102927e-09','7.1241334e-06','5.226607e-09','6.999308e-06','4.2003645e-09','0.024083786','5.2957407e-06','1.2765593e-06','0.005018932','0.009807915','7.635561e-07','5.5039063e-06','0.005018932','0.009807915','6.8777966e-09','1.3434975e-09','1.6125835','0.107879505','1.622604e-05','-3.929207e-06','0.107879505','0.9622076','0.20860377','0.19896953','0.0032885566','2.7732043e-14','4.66776e-06','1.3462392e-11','2.0290193e-11','8.065237e-07','2.9918266e-20','1.748349e-17','1.9736109e-23','6.0887687e-05','1.4876304e-08','1.0439846e-09','0.0017940524','0.005806829','1.130226e-06','1.9767301e-06','0.0016266776','0.00499239','6.225053e-16','4.388979e-17','0.049477644','8.265926e-13','0.00021283247','2.6530683e-10','3.790499e-10','0.00021111974','5.5722117e-19','5.765623e-11','5.4114285e-17','1.8048639e-11','1.7356698e-17','0.00051152124','1.6784333e-11','5.792182e-13','1.8056868e-05','8.527891e-05','2.9021079e-09','1.9115738e-09','1.8056868e-05','8.527891e-05','6.369214e-17','1.5327968e-18','0.053862873','0.011687153','1.1360644e-09','1.0363035e-09','0.011687153','0.10924696','0.029164968','0.027552618'],
['Hungary','202003','1.4451178','1.0927267e-06','0.011209695','2.3664347e-06','-1.4194546e-06','0.0027951372','7.596496e-10','4.4270415e-10','3.2522995e-15','-0.049529843','6.40093e-05','2.1176016e-05','0.028313315','0.04691569','0.00020869191','4.9174196e-05','0.026978685','0.043758918','8.7938e-09','2.7493874e-09','1.9801693','4.3234163e-06','0.028190557','6.02403e-06','-5.4886173e-06','0.027739353','2.6384317e-09','5.0482295e-06','3.787204e-09','7.097087e-06','2.8967266e-09','0.04030152','3.8645603e-06','1.1110449e-06','0.0032389131','0.005599289','4.083248e-06','1.2459941e-06','0.0032389131','0.005599289','2.3484703e-09','5.9824384e-10','1.6628902','0.09191563','1.0978557e-05','-5.7682514e-06','0.09191563','1.1506201','0.18383133','0.1718091','0.0016826235','2.2542311e-13','2.4396422e-05','4.8887346e-11','2.7474419e-11','1.8654846e-06','1.8423224e-19','4.700683e-19','1.3399678e-28','4.208576e-05','7.624137e-09','7.535084e-10','0.0013261074','0.0038228463','1.3704038e-06','4.913204e-07','0.0012033443','0.003316276','2.7458685e-16','2.503191e-17','0.037664883','8.6416525e-12','0.00054795895','1.1095185e-09','6.2786587e-10','0.00053575874','3.545755e-18','1.086093e-11','3.982218e-18','1.6390446e-11','4.541269e-18','0.000529314','8.77961e-12','4.3493827e-13','5.92126e-06','2.1375306e-05','1.4525819e-09','1.324294e-09','5.92126e-06','2.1375306e-05','4.8417843e-18','2.1395073e-19','0.03776809','0.0054029333','1.8129828e-09','1.4491165e-09','0.0054029333','0.08620235','0.016320484','0.014911702'],
['Latvia','202003','1.4619809','6.422796e-07','0.006910455','4.997752e-06','8.882248e-07','0.0018795497','3.5890443e-10','3.1176588e-09','1.3924822e-12','-0.04751481','3.375073e-05','1.16940455e-05','0.015728394','0.025538774','0.0002113236','4.218773e-05','0.014990269','0.02384991','4.0631822e-09','1.3244271e-09','1.9300786','2.5222052e-06','0.01761359','1.8200906e-05','9.1196824e-07','0.017350309','1.3526605e-09','1.6406239e-06','1.1269186e-09','4.082501e-06','1.9729014e-09','0.030414963','1.1455418e-05','2.7756055e-06','0.01082111','0.020756328','0.00011619804','-5.476301e-06','0.01082111','0.020756328','1.41523495e-08','2.763646e-09','1.6043221','0.058481857','2.6142268e-05','6.897195e-06','0.058481857','0.9094585','0.1293004','0.122318774','0.0027287763','6.914129e-14','9.0378735e-06','3.0014505e-11','2.4902006e-11','1.0466741e-06','4.762403e-20','3.5561818e-17','1.6062978e-23','5.3719486e-05','1.2820267e-09','1.3267773e-10','0.0002356109','0.0006606901','1.6504801e-07','1.6345741e-07','0.00021389664','0.00057400233','2.291384e-17','2.259101e-18','0.047957785','4.5183874e-12','0.00020062686','3.9644568e-10','2.8319294e-10','0.00019611961','1.2223456e-18','1.6793171e-12','1.024519e-18','2.7918106e-12','2.6042045e-18','0.0005872334','2.3289795e-10','7.613497e-12','0.00018409084','0.00092559797','1.00687e-07','2.1458456e-08','0.00018409084','0.00092559797','6.6394704e-16','1.4310168e-17','0.048914555','0.0019190103','9.815203e-10','1.1401886e-09','0.0019190103','0.05426593','0.0046520447','0.0043519037'],
['Lithuania','202003','1.4588647','7.8487693e-07','0.008528613','4.537338e-06','6.943972e-07','0.0023470437','4.7362825e-10','1.9029152e-09','7.572127e-13','-0.047488447','3.8905964e-05','1.35488135e-05','0.018226447','0.02956465','0.00012582284','4.19142e-05','0.017370617','0.027612904','4.980037e-09','1.6198008e-09','1.9382745','3.1479008e-06','0.0235057','1.4853278e-05','-7.828013e-07','0.02317647','1.6276783e-09','2.141491e-06','1.983986e-09','4.755509e-06','2.412887e-09','0.040168434','9.341779e-06','2.342462e-06','0.008841975','0.016818434','0.00010551212','-1.5103728e-05','0.008841975','0.016818434','7.918594e-09','1.7542704e-09','1.6089407','0.07356796','2.5005013e-05','5.2622577e-06','0.07356796','0.95349216','0.15198304','0.1435221','0.0023448009','1.9203354e-13','3.069654e-05','3.9457472e-11','2.4765873e-11','3.6601639e-06','9.637395e-20','1.2088868e-17','5.612328e-24','5.3418218e-05','2.10351e-09','2.3350913e-10','0.0004163583','0.0011420082','2.5857167e-07','1.8465447e-07','0.00037796743','0.0009938006','3.7186136e-17','3.92155e-18','0.045879487','1.3114561e-11','0.0016625479','5.193173e-10','3.7309497e-10','0.0016358256','2.5036e-18','2.8450883e-12','2.972032e-18','6.91997e-12','5.2805858e-18','0.0012087509','2.7086608e-10','8.124898e-12','0.00017711606','0.0009084186','1.5793347e-07','9.070446e-09','0.00017711606','0.0009084186','2.7644162e-16','7.433029e-18','0.050986666','0.0051697087','1.275585e-09','1.1056162e-09','0.0051697087','0.064644925','0.013178347','0.012172057'],
['Luxembourg','202003','1.4536774','8.178071e-07','0.008549165','2.4044127e-06','-1.0747517e-06','0.0022047465','9.478337e-10','1.232512e-09','5.611374e-13','-0.047955718','5.2575568e-05','1.7021297e-05','0.022714488','0.037934512','0.00010968615','-9.531527e-05','0.021642253','0.03536179','5.8534066e-09','1.7928814e-09','1.946916','2.3438904e-06','0.015142303','8.001537e-06','-5.179501e-06','0.014898164','1.3321572e-09','2.360494e-06','3.323189e-09','4.4944695e-06','2.791792e-09','0.032522358','6.7790647e-06','1.7364844e-06','0.0077036778','0.014625169','4.928906e-05','3.7545065e-06','0.0077036778','0.014625169','6.2693806e-09','1.4046849e-09','1.5778047','0.06633392','1.11146755e-05','-6.9427792e-06','0.06633392','0.93573654','0.14258578','0.13342488','0.0017754429','1.00615e-13','1.2017958e-05','6.869768e-11','1.6600096e-11','1.0551244e-06','3.3671033e-19','5.7219268e-18','4.6065852e-24','5.1033025e-05','7.234049e-09','6.665859e-10','0.0011614492','0.0034475173','1.0037982e-06','1.2327445e-07','0.0010535978','0.002984787','1.1660314e-16','1.0567426e-17','0.04379473','8.920192e-13','5.3049935e-05','9.902237e-10','2.130058e-10','5.2287818e-05','6.638944e-19','4.221933e-12','9.300251e-18','6.2188735e-12','4.2916752e-18','0.0012434371','6.3823974e-11','2.8219133e-12','7.459392e-05','0.00033046','1.7317298e-08','4.5501247e-09','7.459392e-05','0.00033046','7.971978e-17','2.682279e-18','0.055541586','0.003111811','2.0121695e-09','5.476823e-10','0.003111811','0.06175546','0.010956371','0.009922031'],
['Malta','202003','1.4477067','6.919836e-07','0.0071874717','4.9209457e-06','-3.4269218e-07','0.0018144419','2.8801916e-10','1.824236e-08','4.6496702e-11','-0.057916027','0.00021762644','5.796371e-05','0.07573934','0.1363851','0.0010468551','0.0006811952','0.07211187','0.12645973','2.5330717e-08','5.6974376e-09','1.9842085','2.6995979e-06','0.017719846','2.1970536e-05','2.5237972e-07','0.01743799','8.6864926e-10','7.098115e-06','2.3085198e-09','6.1925725e-06','2.3659719e-09','0.014890282','1.9837718e-05','3.7547645e-06','0.012944306','0.028284566','4.281598e-05','-6.2330616e-07','0.012944306','0.028284566','2.7640853e-08','4.514678e-09','1.6635529','0.07793582','2.2548975e-05','-6.0794723e-06','0.07793582','0.71130943','0.2675135','0.25193313','0.0014885986','1.9170253e-14','2.2772624e-06','2.3587816e-11','1.9984766e-11','2.826957e-07','1.875726e-20','8.6276907e-16','3.2474709e-21','9.156789e-05','8.6218314e-08','5.521747e-09','0.0093908','0.031486884','2.1795465e-06','7.0347755e-06','0.008511769','0.027001137','1.8347394e-15','9.0614466e-17','0.030440701','6.631977e-13','0.00010445719','4.3569243e-10','3.795102e-10','0.000103234954','1.6669137e-19','1.14062225e-10','1.1515703e-18','1.7416717e-11','2.3006833e-18','0.00019941172','3.0069425e-10','7.399123e-12','0.0001657571','0.00090935297','7.531344e-08','2.7275846e-08','0.0001657571','0.00090935297','6.141252e-16','1.1064923e-17','0.024330754','0.004606265','1.6419391e-09','1.8347619e-09','0.004606265','0.14478494','0.048860714','0.043436904'],
['Netherlands','202003','1.4457536','8.840163e-07','0.0091171535','2.6056757e-06','9.8636114e-08','0.0022965355','1.2231802e-09','1.1090127e-08','2.1650477e-11','-0.045609485','3.600785e-05','1.2420477e-05','0.016768798','0.027155623','-2.2787812e-05','-6.9165755e-05','0.015983928','0.025360595','3.698915e-09','1.1986718e-09','1.977483','2.5540767e-06','0.015851041','8.818718e-06','-3.493144e-06','0.015585057','1.7051288e-09','4.4243775e-06','7.519012e-09','4.319595e-06','2.1527278e-09','0.025912061','1.8335848e-05','3.9787187e-06','0.015597386','0.03165668','0.00015174238','3.3716435e-05','0.015597386','0.03165668','2.017348e-08','3.9457877e-09','1.6473607','0.057418935','1.3305233e-05','-1.3417794e-06','0.057418935','0.8603496','0.1411993','0.13231717','0.0015137655','8.044161e-14','9.685046e-06','8.6569814e-11','2.7668326e-11','9.4273315e-07','6.762703e-19','4.817587e-16','1.2990762e-21','3.975668e-05','2.8380334e-09','2.6147667e-10','0.00045889727','0.0013485452','3.9578785e-07','5.7609068e-08','0.00041641283','0.0011678914','6.8644715e-17','6.1942994e-18','0.038351815','1.4787363e-12','6.6888846e-05','1.1703414e-09','3.1959535e-10','6.562778e-05','1.4241825e-18','6.1308623e-12','1.811655e-17','3.3384237e-12','1.9317806e-18','0.00091013813','4.711798e-10','1.3241295e-11','0.0002748632','0.0014112828','1.6034865e-07','6.1577275e-08','0.0002748632','0.0014112828','5.21197e-16','1.2940424e-17','0.05256485','0.0019066836','2.116474e-09','7.80174e-10','0.0019066836','0.06498367','0.004988082','0.004647294'],
['Poland','202003','1.4457979','1.1179432e-06','0.01169381','4.4674134e-06','9.584508e-08','0.0030145417','1.3530914e-09','1.0215471e-09','1.07528935e-13','-0.047932614','4.8981703e-05','1.6612727e-05','0.022309596','0.03653806','8.8043365e-05','-4.968609e-05','0.021261124','0.03410377','9.793469e-09','3.1508622e-09','1.9846203','4.184619e-06','0.027366312','9.066946e-06','-2.4398516e-06','0.026929708','3.8791668e-09','5.830156e-06','7.551949e-09','5.8617247e-06','3.0657699e-09','0.038488254','6.2471913e-06','1.7521834e-06','0.0060965153','0.010813067','3.818119e-05','-8.874776e-06','0.0060965153','0.010813067','4.5518775e-09','1.1263334e-09','1.6348493','0.087222666','2.7271228e-05','8.636679e-07','0.087222666','1.0665699','0.17363389','0.1620833','0.001698369','3.680627e-13','4.7253816e-05','8.141807e-11','3.92686e-11','4.4320136e-06','7.1751776e-19','2.4164973e-18','1.0430135e-25','4.49841e-05','4.6816018e-09','4.823439e-10','0.00085481367','0.002415388','4.7751536e-07','1.7045221e-07','0.000775898','0.0020979794','4.1227698e-16','4.2945335e-17','0.03943147','2.5225369e-11','0.0018402813','1.4035355e-09','9.0357255e-10','0.0018012665','1.706027e-17','1.3095191e-11','2.160944e-17','8.44945e-12','9.457144e-18','0.0007469799','4.6097518e-11','2.0454068e-12','3.8358234e-05','0.0001559405','1.1356361e-08','3.3544718e-09','3.8358234e-05','0.0001559405','3.0051306e-17','1.0627585e-18','0.04724091','0.0061669075','2.3536249e-09','1.2996633e-09','0.0061669075','0.056531787','0.01816233','0.01642128'],
['Portugal','202003','1.4418653','6.428529e-07','0.006589127','1.4477714e-06','-1.8760946e-06','0.0016421336','3.5085776e-10','2.2907187e-09','1.6149801e-12','-0.050614443','0.00013141161','3.8983646e-05','0.051531576','0.089123905','-0.00042086584','7.493784e-05','0.04908474','0.08288377','2.5647205e-08','6.6630275e-09','1.9948341','2.8954328e-06','0.017802857','7.932271e-06','-8.683338e-06','0.017501218','1.3594594e-09','2.3932052e-06','2.3943847e-09','4.885712e-06','2.3598217e-09','0.02469764','1.0870182e-05','2.3052667e-06','0.007918475','0.01625516','4.4557622e-05','-4.715972e-05','0.007918475','0.01625516','1.1226753e-08','2.124432e-09','1.6576912','0.060682427','5.617486e-06','-9.608941e-06','0.060682427','0.8402899','0.19045219','0.17896391','0.0014167634','4.449157e-14','5.492721e-06','2.8505179e-11','1.8918013e-11','5.3143907e-07','4.6129546e-20','2.812829e-17','2.1129041e-23','7.758209e-05','7.598529e-08','5.844975e-09','0.009995787','0.03190138','3.047374e-06','1.7198781e-06','0.009062448','0.027482878','3.4628433e-15','2.0733491e-16','0.03057948','6.4698377e-12','0.00023497255','5.065988e-10','3.7836465e-10','0.0002278356','1.3987881e-18','3.5934012e-12','3.8555823e-18','4.8399704e-12','3.716477e-18','0.00046972558','1.602594e-10','3.1067943e-12','5.5397308e-05','0.00032191517','1.771473e-08','1.9483856e-08','5.5397308e-05','0.00032191517','2.0095293e-16','4.6987246e-18','0.025036078','0.0019800202','1.3445269e-09','8.1324586e-10','0.0019800202','0.12689528','0.039334614','0.034366995'],
['Romania','202003','1.4690392','1.0123412e-06','0.011108428','3.500369e-06','3.8575988e-07','0.0030960597','8.153998e-10','5.680006e-10','4.5193735e-15','-0.05277602','9.112994e-05','2.9339379e-05','0.03903157','0.06550633','0.00031208523','0.00032553577','0.03718608','0.061050944','1.875864e-08','5.3848126e-09','1.8980583','5.1728675e-06','0.04221894','1.33966e-05','6.3441917e-07','0.041676592','4.6950315e-09','3.9175775e-06','3.0298e-09','7.2310436e-06','3.885021e-09','0.036988586','2.9401695e-06','9.0351455e-07','0.0030964254','0.0050744233','2.3982564e-06','3.6343415e-06','0.0030964254','0.0050744233','1.9478161e-09','5.102329e-10','1.5632904','0.12103041','2.2432429e-05','5.3294652e-06','0.12103041','1.0650102','0.24494013','0.23192723','0.002428971','1.8220963e-13','2.9424846e-05','3.9917725e-11','2.3442396e-11','3.7919308e-06','2.588902e-19','1.8493161e-18','3.0986138e-28','5.5392702e-05','1.1429991e-08','1.171236e-09','0.0020861984','0.0058854227','2.943299e-06','4.5543038e-07','0.00189413','0.0051121064','1.1920188e-15','8.53607e-17','0.04095977','9.84256e-12','0.0018057034','1.1953438e-09','4.944829e-10','0.0017830175','1.4656894e-17','1.0511866e-11','5.9397263e-18','1.5964764e-11','1.4305814e-17','0.0003962913','7.2519313e-12','3.311308e-13','5.216535e-06','1.513118e-05','1.2601576e-09','3.6251765e-10','5.216535e-06','1.513118e-05','7.579302e-18','2.256581e-19','0.044290803','0.010085573','1.909837e-09','1.1560727e-09','0.010085573','0.092947766','0.029281085','0.027327778'],
['Slovakia','202003','1.4467402','9.465778e-07','0.009820145','2.3447808e-06','-9.767567e-07','0.0024981527','6.5694833e-10','4.8985244e-10','2.1610127e-15','-0.048577636','4.8990885e-05','1.6633487e-05','0.022341304','0.036544006','0.00018757557','1.9034025e-05','0.021291826','0.034109782','5.9926513e-09','1.9753659e-09','1.9764463','3.650314e-06','0.024322825','5.3316867e-06','-4.0973023e-06','0.023941854','2.0760804e-09','3.9204006e-06','3.1931686e-09','6.0941147e-06','2.831412e-09','0.04811677','2.9252287e-06','8.882727e-07','0.002807264','0.0047903196','5.768262e-06','6.7462145e-07','0.002807264','0.0047903196','1.933469e-09','5.0287025e-10','1.6405463','0.083884016','1.1655037e-05','-3.971654e-06','0.083884016','1.1486747','0.15936041','0.1492241','0.0019391956','1.4608392e-13','1.9311115e-05','4.6701868e-11','1.7024245e-11','1.9123263e-06','9.623661e-20','5.8040803e-19','4.7928338e-29','3.6839494e-05','3.7430357e-09','3.7365877e-10','0.0006628362','0.0018804382','6.065431e-07','2.7294104e-07','0.0006017159','0.0016319988','7.312005e-17','8.0226486e-18','0.042049047','6.6916086e-12','0.00051214313','1.0074004e-09','3.2893196e-10','0.00050139666','2.3129252e-18','6.985928e-12','5.2462656e-18','1.3843329e-11','4.278212e-18','0.0009635853','6.710104e-12','3.722722e-13','7.970157e-06','2.8443224e-05','6.596618e-10','7.338984e-10','7.970157e-06','2.8443224e-05','4.6840384e-18','2.0055221e-19','0.040249057','0.0059482916','1.4670816e-09','9.432506e-10','0.0059482916','0.06386532','0.015034115','0.01393041'],
['Slovenia','202003','1.4561338','9.266009e-07','0.00986195','7.167013e-07','-1.908212e-06','0.0026186819','6.2584277e-10','6.1161193e-10','7.791929e-14','-0.049212262','8.464278e-05','2.7336004e-05','0.036484607','0.06105531','-0.00021451118','4.80904e-05','0.034762774','0.05691141','2.3976446e-08','7.323325e-09','1.9371312','3.318973e-06','0.026380831','-1.8085665e-08','-7.870277e-06','0.0260333','2.0630009e-09','4.713395e-06','4.6852175e-09','6.932618e-06','3.7805785e-09','0.033664435','4.5761376e-06','1.1937035e-06','0.0041208565','0.007580817','2.2075342e-06','2.070035e-06','0.0041208565','0.007580817','3.3806902e-09','7.675306e-10','1.602427','0.107433096','-1.7030231e-06','-1.2646211e-05','0.107433096','1.108331','0.21231177','0.20057823','0.002494851','1.0791185e-13','1.5818034e-05','5.8212817e-11','2.2876262e-11','1.8758099e-06','1.1299038e-19','2.0907348e-18','1.5223793e-25','4.31014e-05','2.6286312e-08','2.5413263e-09','0.0044890638','0.013023553','4.2846336e-06','5.091886e-07','0.004074384','0.0112923775','7.703716e-15','7.0020005e-16','0.04515429','2.7487452e-12','0.0005174432','1.0819775e-09','3.814415e-10','0.0005110029','1.4558131e-18','1.3763075e-11','7.956075e-18','1.513836e-11','8.680859e-18','0.0006109119','2.5524492e-11','8.6749884e-13','1.2122303e-05','5.544937e-05','3.0733356e-09','1.6274992e-09','1.2122303e-05','5.544937e-05','1.2417265e-17','3.707648e-19','0.040676143','0.008426316','4.125295e-09','1.1644496e-09','0.008426316','0.09363531','0.031057205','0.028411798'],
['Spain','202003','1.4501013','6.5065603e-07','0.006767445','1.8596454e-06','-1.5205758e-06','0.0017274432','5.991339e-10','7.810368e-10','1.0677122e-13','-0.051919106','0.00017159182','5.0137536e-05','0.06607795','0.11526192','-0.00060402823','0.00036233073','0.06293191','0.10714437','3.1842536e-08','8.332768e-09','1.9514337','2.2964787e-06','0.01553786','9.365075e-06','-7.022892e-06','0.015297986','1.1923385e-09','4.3722193e-06','6.2405987e-09','4.4254957e-06','2.6464573e-09','0.036291216','4.0427512e-06','1.097181e-06','0.003534461','0.0062840623','1.4684407e-05','-1.32763425e-05','0.003534461','0.0062840623','3.1413454e-09','7.2615e-10','1.6261586','0.060132686','8.4905105e-06','-7.4893906e-06','0.060132686','0.8820651','0.20398451','0.19058588','0.0014766479','3.276424e-14','3.9591996e-06','2.626178e-11','2.2562596e-11','3.9488202e-07','1.3906856e-19','2.636797e-18','3.6481585e-25','7.00239e-05','1.0590091e-07','8.3998986e-09','0.0144342','0.045480713','3.8999888e-06','2.52875e-06','0.013087254','0.03921907','3.899138e-15','2.4293363e-16','0.032761075','5.532447e-13','5.6227815e-05','4.0525358e-10','4.015765e-10','5.548017e-05','4.9436413e-19','7.0137576e-12','1.4017652e-17','2.792768e-12','3.2089936e-18','0.0006861104','1.5932845e-11','5.616057e-13','6.982828e-06','2.7522188e-05','2.6630171e-09','2.6675206e-09','6.982828e-06','2.7522188e-05','1.5215425e-17','5.2275866e-19','0.03355812','0.0014412629','1.1866786e-09','8.2937834e-10','0.0014412629','0.16259721','0.052208576','0.045550127'],
['Sweden','202003','1.4639086','5.9410803e-07','0.00642691','6.076089e-06','5.386185e-07','0.0017687356','3.2961955e-10','6.25949e-09','5.1791384e-12','-0.04515848','2.5237738e-05','9.251223e-06','0.0125183975','0.01989326','0.00028257066','-1.6617956e-05','0.011933189','0.018604882','2.9147906e-09','9.664979e-10','1.9148479','1.907528e-06','0.014613252','2.2078808e-05','-2.4032565e-07','0.014413906','7.919203e-10','2.1624667e-06','2.5242053e-09','4.051186e-06','2.192107e-09','0.01815289','1.5132203e-05','3.4921e-06','0.012724416','0.024216376','8.5986234e-05','-1.5092455e-05','0.012724416','0.024216376','2.391655e-08','4.4152726e-09','1.6192436','0.05723488','3.6321082e-05','3.9259216e-06','0.05723488','0.8734569','0.12238396','0.11623907','0.0028641648','6.906971e-14','9.269598e-06','2.420424e-11','2.13499e-11','1.0449588e-06','4.763317e-20','6.355005e-17','4.6735506e-23','4.1938238e-05','8.8974067e-10','1.10510726e-10','0.00019869965','0.0005228831','1.6139519e-07','1.4015644e-07','0.00018044689','0.00045628904','2.4707731e-17','2.6270892e-18','0.05370441','9.596744e-13','0.0001221244','2.9106753e-10','2.5592978e-10','0.00012072587','5.40172e-19','2.962807e-12','4.6739696e-18','4.3440347e-12','4.029188e-18','0.00022780267','1.2622912e-10','4.6195265e-12','0.0001061328','0.0003938864','1.3172783e-08','3.9370903e-08','0.0001061328','0.0003938864','3.1355418e-16','8.742042e-18','0.054517914','0.0023064527','1.3368657e-09','1.0040453e-09','0.0023064527','0.042489808','0.005634151','0.005284085'],
['Ireland','202003','1.4700955','4.919062e-07','0.0054274485','4.636274e-06','9.933235e-07','0.0015266433','2.3408198e-10','1.3279267e-08','1.09053235e-11','-0.046156667','3.6639034e-05','1.2554368e-05','0.016826613','0.02753788','0.00021681128','0.00020046505','0.01603486','0.025707418','3.154461e-09','1.0002054e-09','1.9109771','1.780389e-06','0.014420883','2.083735e-05','2.9225007e-06','0.014234512','4.658533e-10','1.2789473e-06','1.589762e-09','3.4903435e-06','1.5122323e-09','0.007037036','4.027821e-05','6.749166e-06','0.029860098','0.068228915','0.0004988969','6.089868e-05','0.029860098','0.068228915','4.5439656e-08','6.9402635e-09','1.5831156','0.05013931','2.2056505e-05','7.1108675e-06','0.05013931','0.5931608','0.1657534','0.15983585','0.0031721748','6.2869875e-14','9.5016685e-06','2.8168495e-11','2.4818305e-11','1.1372006e-06','4.1764805e-20','5.706842e-16','4.871852e-22','4.4794386e-05','3.9139323e-09','3.7707917e-10','0.0006572988','0.0019282864','1.1898071e-07','5.819187e-07','0.0005962508','0.0016712244','1.8502187e-17','1.5723223e-18','0.052315235','4.598459e-13','0.000118025695','4.797262e-10','4.1178225e-10','0.00011686834','1.3838761e-19','1.7582378e-12','1.922294e-18','6.591848e-12','2.9047911e-18','0.00015186533','1.7528508e-09','2.8914947e-11','0.00079221855','0.0051363297','8.457536e-07','1.3557836e-07','0.00079221855','0.0051363297','2.2802822e-15','3.8230455e-17','0.03289074','0.003143263','1.0543695e-09','1.0331712e-09','0.003143263','0.08441664','0.009971674','0.00943478'],
['Italy','202003','1.4395869','8.5179823e-07','0.008617318','1.7432418e-06','-6.1881104e-07','0.0020969897','6.0797173e-10','4.3573403e-09','3.893831e-12','-0.05162641','7.514366e-05','2.364934e-05','0.031492595','0.053103574','2.7464097e-05','0.00012172827','0.030003639','0.049467955','1.2567503e-08','3.838211e-09','2.0082707','3.0307956e-06','0.017993122','8.073581e-06','-2.299322e-06','0.01767786','1.2282936e-09','9.032156e-06','6.7533277e-09','7.0170204e-06','3.1828649e-09','0.02586115','1.0377265e-05','2.3085656e-06','0.0077024153','0.01519078','2.5258785e-05','1.9459945e-05','0.0077024153','0.01519078','1.0712325e-08','2.1309008e-09','1.6916608','0.08355529','7.24082e-06','-4.35445e-06','0.08355529','0.99391276','0.17846134','0.16798985','0.0014937957','5.829243e-14','5.8417727e-06','1.7593151e-11','2.2351359e-11','4.7388008e-07','1.1510527e-19','1.9541455e-16','1.8457088e-22','5.4131346e-05','7.538171e-09','6.75679e-10','0.0011844597','0.0035027403','1.0706915e-06','6.5809195e-07','0.0010746808','0.0030318769','4.857722e-16','5.171877e-17','0.037530635','1.844087e-12','8.7632754e-05','2.7357047e-10','3.4191408e-10','8.599473e-05','5.0563943e-19','1.10769546e-10','2.3412912e-17','1.5338945e-11','2.6963898e-18','0.0002922945','1.8108358e-10','5.451438e-12','8.828746e-05','0.00040220964','1.6168444e-08','8.324745e-09','8.828746e-05','0.00040220964','2.0689964e-16','6.9285848e-18','0.033816766','0.004007557','7.6510853e-10','9.393972e-10','0.004007557','0.106059805','0.012545547','0.011602491'],
['Denmark','202003','1.4523392','6.015114e-07','0.0063119577','4.944892e-06','7.952116e-07','0.0016497066','5.101104e-10','1.9135214e-08','3.928236e-11','-0.04482258','2.1821623e-05','8.02769e-06','0.010885485','0.017242445','0.00017092541','-3.3637913e-05','0.010377273','0.016128108','2.1420878e-09','7.2417095e-10','1.9574094','1.9475842e-06','0.012660378','1.875996e-05','-8.491893e-07','0.012457835','9.801592e-10','2.1210503e-06','2.8142257e-09','3.6246834e-06','1.8728454e-09','0.017459717','1.9425706e-05','4.3145956e-06','0.019166924','0.038792472','0.00018536522','7.167513e-05','0.019166924','0.038792472','2.6503933e-08','5.1317244e-09','1.6430728','0.046920236','2.640136e-05','3.8721487e-06','0.046920236','0.81406385','0.121927366','0.115948476','0.002051182','3.8091353e-14','5.3494005e-06','2.7844631e-11','2.6972459e-11','6.544377e-07','8.472401e-20','9.658484e-16','4.3056403e-21','2.8536548e-05','3.7993056e-10','4.0303847e-11','7.116318e-05','0.00019974278','4.06125e-08','4.8570683e-08','6.458405e-05','0.00017358601','9.447712e-18','8.949183e-19','0.046322227','6.8673767e-13','5.421788e-05','3.6202444e-10','2.7676753e-10','5.3592146e-05','6.0911854e-19','1.3343505e-12','2.8045521e-18','2.0688156e-12','2.1834582e-18','0.00034158232','5.4922417e-10','1.6179811e-11','0.0005834023','0.0029390596','1.6470207e-07','9.64213e-08','0.0005834023','0.0029390596','1.2712509e-15','3.076249e-17','0.058374785','0.0012019722','9.559449e-10','9.853312e-10','0.0012019722','0.069548614','0.0045277947','0.004496906'],
['Finland','202003','1.4844917','5.292058e-07','0.0060025007','6.29371e-06','7.611748e-07','0.0017799849','3.1379083e-10','6.562583e-09','2.3216427e-12','-0.045083363','2.1387274e-05','7.959606e-06','0.01080734','0.017041607','0.000291345','-7.6280385e-06','0.01030333','0.015944708','2.5775226e-09','8.529704e-10','1.8419093','1.6554093e-06','0.015091351','2.3081044e-05','6.588066e-07','0.0149175385','7.3393924e-10','1.2556934e-06','1.3322278e-09','3.7174232e-06','2.499978e-09','0.022488438','1.0923323e-05','2.9280534e-06','0.011479555','0.020284442','8.605479e-05','-1.2903649e-06','0.011479555','0.020284442','9.776023e-09','2.377529e-09','1.5084362','0.063663565','3.661356e-05','5.424623e-06','0.063663565','0.914368','0.12208352','0.11659106','0.0033534507','3.5700216e-14','6.1551705e-06','2.3781518e-11','1.6813165e-11','9.044536e-07','3.7143046e-20','9.1809934e-17','8.311206e-24','3.3826156e-05','4.9892407e-10','6.274608e-11','0.00011362144','0.00029549917','1.918452e-07','6.901615e-08','0.00010320939','0.0002579876','7.448136e-18','7.125072e-19','0.051691145','4.0834816e-13','9.9272715e-05','3.2162115e-10','1.7646193e-10','9.82996e-05','4.0617013e-19','1.0818988e-12','2.5195001e-18','2.083846e-12','5.3189484e-18','0.0002538774','8.888063e-11','3.833301e-12','6.8894384e-05','0.0002653682','1.2651361e-08','1.4143406e-08','6.8894384e-05','0.0002653682','1.10695526e-16','3.929859e-18','0.05428311','0.0020501448','1.3359969e-09','8.0049967e-10','0.0020501448','0.037337713','0.0039794436','0.0037577068'],
['Belgium','202003','1.4493687','9.636158e-07','0.010001979','2.5162806e-06','-6.83649e-07','0.0025485174','1.4329243e-09','2.9567577e-09','3.289574e-12','-0.04761577','4.929046e-05','1.6024396e-05','0.021408355','0.03566274','8.565387e-05','-0.000100859834','0.020398736','0.03324866','5.9150236e-09','1.8241667e-09','1.9587919','2.5999807e-06','0.016742637','8.797259e-06','-4.9498835e-06','0.016471835','1.8254658e-09','4.1039566e-06','5.059308e-09','4.5765437e-06','2.5025406e-09','0.03150429','1.2670055e-05','2.8787892e-06','0.011796632','0.023512308','0.0001050634','1.5983513e-05','0.011796632','0.023512308','1.1370187e-08','2.340002e-09','1.6151255','0.06442166','1.220556e-05','-4.4255175e-06','0.06442166','0.9019191','0.1503415','0.14020188','0.0014930745','1.5587317e-13','1.8514169e-05','8.6826546e-11','2.7914553e-11','1.5754418e-06','8.928363e-19','3.9056153e-17','7.834545e-23','4.407635e-05','7.0920065e-09','6.4404626e-10','0.0011209238','0.0033464327','8.956156e-07','1.1003099e-07','0.0010168108','0.0028962123','1.3788732e-16','1.2438232e-17','0.039349813','1.4559548e-12','8.399118e-05','1.1244614e-09','3.153725e-10','8.2441096e-05','1.5127125e-18','1.2660634e-11','1.3143837e-17','4.995362e-12','2.4730353e-18','0.0011900344','2.6049574e-10','8.550594e-12','0.00016572596','0.00083330745','7.4946485e-08','2.2054486e-08','0.00016572596','0.00083330745','2.1845094e-16','5.997781e-18','0.054280404','0.0024025445','2.1552726e-09','7.210707e-10','0.0024025445','0.06364071','0.009869497','0.008827904'],
['Croatia','202003','1.4534776','8.750487e-07','0.0091974195','-3.567161e-07','-1.5929296e-06','0.0023850547','5.170114e-10','6.1747785e-10','2.2160609e-13','-0.049713153','9.4420175e-05','3.0003119e-05','0.040002134','0.067322254','-0.00040171645','6.3084335e-05','0.038112924','0.06272897','3.556155e-08','1.0831711e-08','1.9482131','3.472565e-06','0.024847675','-2.791992e-06','-6.729792e-06','0.024484131','2.1126445e-09','3.7105467e-06','3.1039142e-09','6.902862e-06','3.6449281e-09','0.02485922','6.223064e-06','1.4167289e-06','0.0050559947','0.009974172','-1.220153e-05','6.1692535e-06','0.0050559947','0.009974172','1.0560143e-08','1.7410503e-09','1.618238','0.09964545','-1.4362783e-05','-1.17402515e-05','0.09964545','1.0790805','0.21098669','0.19921558','0.0020973368','7.4082206e-14','9.358532e-06','4.959363e-11','2.6542225e-11','9.384929e-07','5.6564594e-20','2.1052526e-18','1.6319111e-24','4.3714208e-05','3.858934e-08','3.5375975e-09','0.0062199538','0.01841623','6.0022153e-06','6.553318e-07','0.0056440737','0.015947713','1.7774292e-14','1.5956721e-15','0.042690385','3.0899513e-12','0.00026651638','1.0185703e-09','4.2634285e-10','0.00026187702','2.2268012e-18','1.0183068e-11','3.7278846e-18','1.3598756e-11','5.2074523e-18','0.0004052287','3.2760204e-11','1.0587378e-12','2.0887494e-05','0.00010096936','3.4625567e-09','4.6410267e-09','2.0887494e-05','0.00010096936','7.995604e-17','1.92661e-18','0.03783381','0.005821577','4.001744e-09','1.3886401e-09','0.005821577','0.0987626','0.03284892','0.02961849'],
['Austria','202004','1.4500461','8.9882525e-07','0.009486248','3.6348774e-06','-1.7657801e-06','0.0024788508','5.726503e-10','1.3655747e-10','1.2091628e-15','-0.048419535','3.1271324e-05','1.0899154e-05','0.014685102','0.023732973','0.00022466433','-0.0001268613','0.013996497','0.02216747','4.2222683e-09','1.3555234e-09','1.9778941','3.8443295e-06','0.026902646','1.5816786e-05','-8.3418845e-06','0.026500829','2.0180395e-09','2.4043743e-06','2.1327327e-09','5.391726e-06','2.9531244e-09','0.0421844','2.3536263e-06','6.5079126e-07','0.002026974','0.0036450054','6.3183998e-06','-2.7537983e-06','0.002026974','0.0036450054','1.8448394e-09','4.2653275e-10','1.6871582','0.075197026','1.9464995e-05','-7.250152e-06','0.075197026','1.1647763','0.13896362','0.12998961','0.0023201816','1.4025197e-13','1.9796777e-05','3.1006586e-11','1.4341211e-11','2.1568078e-06','1.0256351e-19','1.0278718e-19','3.8891158e-29','3.3928925e-05','4.5023088e-10','4.032286e-11','6.877855e-05','0.00020879733','2.433511e-07','1.03750246e-07','6.234583e-05','0.00018054657','4.638691e-18','3.1468504e-19','0.05022233','1.8350146e-12','0.0003117933','6.484152e-10','3.7910233e-10','0.00030897124','1.4296343e-18','3.2798527e-12','2.4153533e-18','1.2883464e-11','1.0113159e-17','0.0007636793','3.4136559e-12','1.7029728e-13','3.7955213e-06','1.49047855e-05','1.69265e-10','2.272308e-10','3.7955213e-06','1.49047855e-05','3.5460617e-18','1.4602628e-19','0.04702663','0.005989741','1.0574823e-09','4.819754e-10','0.005989741','0.03926562','0.009589478','0.009080516'],
['Bulgaria','202004','1.447657','1.1103839e-06','0.011555288','1.4869062e-06','-3.1706943e-06','0.002950364','4.9760734e-10','1.0474455e-09','8.4393717e-13','-0.05120204','7.796368e-05','2.2572558e-05','0.029867072','0.0516802','-5.4529028e-05','-6.9543385e-05','0.02844879','0.04803699','6.673452e-09','2.0554178e-09','2.0107913','7.515988e-06','0.04834244','7.985347e-06','-2.2488177e-05','0.047558293','3.0737275e-09','7.796891e-06','4.531272e-09','8.497617e-06','3.2299425e-09','0.02881789','6.1304486e-06','1.4272516e-06','0.003918255','0.007484895','-1.9977231e-05','-1.7098993e-05','0.003918255','0.007484895','4.705326e-09','9.681687e-10','1.7200193','0.113317296','8.239369e-06','-1.6193982e-05','0.113317296','1.1859943','0.23238158','0.21934916','0.0026288317','2.4868166e-13','2.710387e-05','5.7249098e-11','1.3897972e-11','2.426899e-06','5.903113e-20','1.8111096e-17','1.1196317e-23','6.014767e-05','1.3806084e-08','7.315933e-10','0.001218815','0.0044076536','2.7809344e-06','6.743018e-07','0.0011038402','0.003758992','2.7488967e-17','2.0595962e-18','0.04094819','2.7092549e-11','0.0014225828','3.163395e-09','8.4765167e-10','0.0013943755','5.6479053e-18','3.002182e-11','6.0343115e-18','2.1654012e-11','5.892669e-18','0.0006083217','1.694798e-11','5.980399e-13','7.724776e-06','3.0546085e-05','3.6051961e-09','1.2720829e-09','7.724776e-06','3.0546085e-05','2.0652867e-17','6.297029e-19','0.04402939','0.011325878','3.0866794e-09','5.8493665e-10','0.011325878','0.09766347','0.025840668','0.024511002'],
['Cyprus','202004','1.4402256','8.298582e-07','0.008368449','6.421872e-06','-5.455348e-07','0.0020317351','2.506405e-10','8.875942e-09','1.43517455e-11','-0.059498455','0.00017711414','4.7392405e-05','0.06179976','0.11131061','0.0016011057','0.00038427836','0.05883533','0.103215784','2.8885212e-08','6.9393487e-09','2.0246897','4.690307e-06','0.027059559','3.9765222e-05','-3.0259464e-06','0.026572173','1.085761e-09','4.6744426e-06','2.9492084e-09','6.3623e-06','2.6962623e-09','0.027013905','8.783978e-06','2.112863e-06','0.005708558','0.011087363','3.5630383e-05','-7.805356e-07','0.005708558','0.011087363','1.1164371e-08','2.3360285e-09','1.74361','0.06428039','3.294626e-05','-3.0009714e-06','0.06428039','0.75676674','0.22210686','0.2071877','0.0019288603','1.3308563e-13','1.170731e-05','1.5097614e-11','1.40930245e-11','7.099075e-07','2.604806e-20','3.7681145e-16','7.911855e-22','5.1144314e-05','3.7596024e-08','2.077101e-09','0.00346053','0.0123759415','6.8539684e-06','5.1320217e-06','0.0031342658','0.010565741','1.2525395e-15','6.172002e-17','0.039896775','6.904825e-12','0.00028030126','5.9493654e-10','5.554445e-10','0.0002750267','1.0169747e-18','6.473224e-12','2.0304831e-18','5.987472e-12','1.712985e-18','0.00030555073','3.439562e-11','1.08523e-12','1.3965931e-05','7.0462695e-05','4.261467e-09','1.1339996e-09','1.3965931e-05','7.0462695e-05','9.54916e-17','3.1120751e-18','0.024678275','0.0010860087','9.379968e-10','5.564877e-10','0.0010860087','0.12004834','0.0158201','0.013876133'],
['Czech Republic','202004','1.4293034','1.1788427e-06','0.011756338','4.307915e-06','-2.2239394e-06','0.0027881453','1.1483755e-09','2.1911863e-10','1.1004947e-14','-0.047182173','2.9445591e-05','1.0804656e-05','0.014646583','0.023196792','0.00017305934','-0.000118050535','0.013962906','0.02169624','4.1154555e-09','1.3223777e-09','2.039219','4.569613e-06','0.025193399','1.8310186e-05','-1.0261957e-05','0.02471875','3.1426521e-09','7.136941e-06','5.9251186e-09','6.2281906e-06','2.3688609e-09','0.039973535','5.1583766e-06','1.3512179e-06','0.0034767678','0.006475906','1.2984171e-05','-1.1128079e-05','0.0034767678','0.006475906','3.227999e-09','7.608682e-10','1.7555988','0.070641525','2.3033856e-05','-1.1461293e-05','0.070641525','1.2231916','0.13726375','0.12631978','0.0009318655','1.2669373e-13','1.5991149e-05','5.3772348e-11','3.030315e-11','1.4640727e-06','4.6666976e-19','1.6375598e-19','2.4468476e-27','1.7389184e-05','1.6665692e-10','1.6703715e-11','2.9126313e-05','8.346902e-05','1.0039627e-07','4.1507647e-08','2.6422373e-05','7.242926e-05','5.8219627e-18','3.6129915e-19','0.033619642','2.812868e-12','0.00012840732','8.7976926e-10','1.0090717e-09','0.00012672212','5.5861616e-18','1.18028165e-11','1.0532157e-17','8.714825e-12','1.3393078e-18','0.00062570034','2.9868975e-11','1.1531477e-12','1.3941851e-05','6.258068e-05','1.3438143e-09','2.0992128e-09','1.3941851e-05','6.258068e-05','1.0561729e-17','3.8486248e-19','0.031875733','0.0031555037','1.7965235e-09','8.014281e-10','0.0031555037','0.022928054','0.0046383296','0.004348735'],
['Estonia','202004','1.4609777','6.3821517e-07','0.0068688835','7.3470037e-06','-5.5401083e-06','0.0018552822','2.4412708e-10','8.609858e-09','5.9184025e-12','-0.04563685','2.7291386e-05','1.03240045e-05','0.014054216','0.021989904','0.00032751155','-0.00028793068','0.013399971','0.020583875','2.5627407e-09','8.2909696e-10','1.952756','3.3385236e-06','0.023748005','3.8225506e-05','-3.3918703e-05','0.02339889','1.0249187e-09','1.3449065e-06','8.664784e-10','3.6802205e-06','1.3451175e-09','0.019651813','1.50936785e-05','3.1899242e-06','0.012162251','0.025190914','0.0001699341','-2.397854e-05','0.012162251','0.025190914','2.0655284e-08','3.8445096e-09','1.6179668','0.049608596','3.754333e-05','-2.4833924e-05','0.049608596','0.92600375','0.12740535','0.12063694','0.0030707626','6.108353e-14','8.7413155e-06','3.5626502e-11','4.073931e-11','1.1489656e-06','4.1763257e-20','2.1414003e-16','7.803481e-23','2.5762653e-05','9.038335e-11','8.908121e-12','1.5514477e-05','4.4568762e-05','7.122458e-08','1.0520794e-07','1.40756465e-05','3.8655446e-05','2.747232e-18','2.764379e-19','0.043479834','2.627439e-12','0.00022175681','8.718001e-10','1.3619004e-09','0.00021917796','1.1041212e-18','1.6125287e-12','8.236942e-19','2.7468659e-12','3.745476e-18','0.00033446017','3.0074646e-10','8.1345885e-12','0.00020746818','0.0011525049','1.0139432e-07','4.609445e-08','0.00020746818','0.0011525049','7.5774244e-16','1.6018255e-17','0.033189435','0.0015514538','1.2402431e-09','1.1598112e-09','0.0015514538','0.06142632','0.0039015512','0.00375737'],
['France','202004','1.4655766','9.125136e-07','0.009845469','4.882682e-07','1.5464234e-06','0.002662063','5.2499355e-10','1.4998425e-09','6.859372e-13','-0.05082717','4.6272755e-05','1.5075323e-05','0.020096952','0.033464745','-4.2981133e-05','0.00011885704','0.019147756','0.031198833','7.0149846e-09','2.0188426e-09','1.926005','3.6880574e-06','0.028899632','4.6357195e-06','8.09252e-06','0.028513296','1.3967427e-09','2.8555253e-06','2.007683e-09','5.728327e-06','2.6526472e-09','0.027301865','4.4875587e-06','1.069205e-06','0.0033823976','0.006592126','6.9794114e-06','9.4526695e-06','0.0033823976','0.006592126','3.665179e-09','7.664252e-10','1.6359441','0.07740069','-1.145951e-06','9.902902e-06','0.07740069','1.1152414','0.15620314','0.14636713','0.002946892','9.382456e-14','1.2314603e-05','1.8493686e-11','1.4806645e-11','1.3894222e-06','6.6285013e-20','7.106031e-17','2.6974206e-23','4.099218e-05','2.2348592e-09','1.7903984e-10','0.0003009743','0.0009604087','2.4338934e-07','1.1700136e-07','0.000272695','0.0008278539','2.747514e-17','1.5341803e-18','0.050433293','1.4375055e-12','0.00027190516','4.8192556e-10','3.584102e-10','0.00027010348','4.756832e-19','8.023257e-12','3.2274387e-18','6.719592e-12','1.8056749e-18','0.0003901136','2.4274287e-11','6.3026743e-13','1.0024492e-05','5.822871e-05','6.5757337e-09','1.0076139e-09','1.0024492e-05','5.822871e-05','2.393862e-17','5.8884527e-19','0.043308936','0.0025548514','5.410763e-10','5.9706645e-10','0.0025548514','0.04624475','0.0061174138','0.005648329'],
['Germany','202004','1.4340396','1.1194745e-06','0.011257974','3.8048752e-06','-1.3905159e-06','0.002708203','8.043868e-10','7.414546e-10','5.4602335e-13','-0.04733791','3.169545e-05','1.1520507e-05','0.015586708','0.024804126','0.00020456284','-0.00010087359','0.0148578705','0.023193052','3.8435575e-09','1.2764494e-09','2.0166247','3.8129258e-06','0.022828043','1.5440482e-05','-8.890045e-06','0.022431262','1.6246186e-09','5.9344084e-06','5.666344e-09','6.1226465e-06','2.6143898e-09','0.04207748','6.3843772e-06','1.6252951e-06','0.004649056','0.008842395','1.5448599e-05','-1.5345264e-05','0.004649056','0.008842395','4.9686038e-09','1.1227262e-09','1.7405305','0.06884354','1.9352374e-05','-6.0529687e-06','0.06884354','1.1584483','0.13657641','0.12601818','0.001137883','2.4125233e-13','2.7137557e-05','5.733412e-11','2.6375058e-11','2.1227843e-06','2.0239778e-19','6.6761205e-18','4.7654283e-24','2.0643418e-05','3.10059e-10','2.8965936e-11','4.993666e-05','0.00014796636','1.7350261e-07','1.04600716e-07','4.5280318e-05','0.00012811755','3.028226e-18','2.505284e-19','0.036755998','1.1302133e-12','0.00013921192','7.7146706e-10','5.3321264e-10','0.00013786972','8.268176e-19','1.40307965e-11','1.0247386e-17','1.0478826e-11','2.016964e-18','0.0009467071','5.668049e-11','2.3395988e-12','3.2056374e-05','0.00014416975','2.701491e-09','3.5917649e-09','3.2056374e-05','0.00014416975','4.3251398e-17','1.5763837e-18','0.029952839','0.0031394102','1.6741722e-09','6.74589e-10','0.0031394102','0.041197985','0.005222957','0.004807499'],
['Greece','202004','1.4702742','8.197335e-07','0.008801405','2.8856182e-06','-2.45095e-06','0.002379141','3.9856204e-10','2.5589022e-09','1.4122683e-12','-0.05494454','0.0001272428','3.4326014e-05','0.044963837','0.08030365','0.00035686838','0.00014245603','0.042813946','0.07448945','1.8007153e-08','4.6640105e-09','1.9042758','5.0563463e-06','0.037447143','1.9384015e-05','-1.544745e-05','0.036917627','2.0579016e-09','6.758578e-06','4.678939e-09','7.5175444e-06','4.2276125e-09','0.026162792','6.469377e-06','1.4555326e-06','0.0056624142','0.0112848515','-2.185012e-05','-2.3656174e-05','0.0056624142','0.0112848515','7.1126265e-09','1.3323933e-09','1.6015109','0.11202292','1.4603405e-05','-1.4789282e-05','0.11202292','1.0102822','0.24985924','0.23709387','0.0035238133','1.5603317e-13','1.620417e-05','3.3021003e-11','1.0763935e-11','1.256584e-06','4.987962e-20','5.220138e-17','1.5762921e-23','8.268912e-05','3.5033818e-08','1.9134372e-09','0.0031947216','0.011423457','2.8282857e-06','5.33336e-06','0.0028937773','0.009750571','9.150991e-16','4.9957523e-17','0.049693134','1.3120385e-11','0.00064518687','1.2924094e-09','4.8397697e-10','0.000630978','2.9849563e-18','5.776783e-11','2.9879385e-17','1.9673891e-11','6.6323e-18','0.0009957028','4.4972366e-11','1.5052253e-12','3.642471e-05','0.00017124579','8.0322025e-09','5.5122995e-09','3.642471e-05','0.00017124579','8.266924e-17','2.128368e-18','0.05086689','0.008135208','1.6847506e-09','8.784541e-10','0.008135208','0.14110065','0.02818584','0.025962947'],
['Hungary','202004','1.4276346','1.298899e-06','0.012901148','4.318252e-06','-3.6295144e-06','0.0030484241','6.897298e-10','1.4318804e-10','5.438868e-16','-0.049579255','3.6626243e-05','1.2673184e-05','0.017063228','0.027629402','0.00020314907','-0.00016516473','0.016262949','0.025802819','5.1383156e-09','1.5900739e-09','2.0660524','6.547966e-06','0.034018166','1.8323499e-05','-1.5757347e-05','0.033339128','3.036035e-09','5.2001865e-06','3.3537155e-09','7.70691e-06','2.6379867e-09','0.035154525','6.282556e-06','1.5555308e-06','0.0032544176','0.0060595446','3.7897864e-06','-1.2024891e-05','0.0032544176','0.0060595446','3.4340124e-09','7.60735e-10','1.8083774','0.08124074','2.4402078e-05','-1.8365183e-05','0.08124074','1.2406756','0.16184811','0.14949133','0.0011811571','2.8845248e-13','3.2643664e-05','1.0137371e-10','2.9896616e-11','2.7514134e-06','1.1676462e-19','5.864615e-20','9.9456106e-30','2.7437221e-05','3.0713965e-10','2.5492082e-11','4.362392e-05','0.000133466','2.3163729e-07','7.1109675e-08','3.955008e-05','0.00011524896','8.725285e-18','5.4570527e-19','0.03384132','1.3878019e-11','0.0005031056','2.3049698e-09','9.016666e-10','0.000492906','4.9284935e-18','9.593328e-12','3.8496725e-18','1.6563103e-11','3.3923155e-18','0.000485203','3.741919e-11','1.2888498e-12','7.0685446e-06','2.9559793e-05','1.1432439e-09','1.5905167e-09','7.0685446e-06','2.9559793e-05','1.21836326e-17','3.567302e-19','0.03435566','0.0058507356','3.478163e-09','1.15707e-09','0.0058507356','0.035293035','0.009984694','0.00938511'],
['Latvia','202004','1.4634185','6.621706e-07','0.007225298','8.239448e-06','-5.0144727e-06','0.00200201','2.3273379e-10','6.3637766e-09','3.7170995e-12','-0.045925092','2.7829376e-05','1.0519148e-05','0.0143104745','0.02241088','0.0003539932','-0.00027619896','0.013643889','0.02097729','2.5188958e-09','8.207184e-10','1.9446647','3.3650324e-06','0.024847297','4.1914765e-05','-3.0823412e-05','0.024495542','1.1808197e-09','1.2488362e-06','7.483428e-10','3.898613e-06','1.2762085e-09','0.019008597','1.5105907e-05','3.1986376e-06','0.012164019','0.025000485','0.00016781871','-4.9409096e-05','0.012164019','0.025000485','1.8979614e-08','3.4800545e-09','1.6161517','0.05380299','4.4502336e-05','-2.1418238e-05','0.05380299','0.92244536','0.1332869','0.12627843','0.003687523','1.03696396e-13','1.7181805e-05','5.7108436e-11','4.0031853e-11','2.3768757e-06','3.3263718e-20','1.5069485e-16','4.4489366e-23','2.525073e-05','8.7550175e-11','8.91192e-12','1.5648631e-05','4.4034106e-05','8.758549e-08','9.483579e-08','1.4199657e-05','3.823485e-05','2.3690105e-18','2.2502528e-19','0.04502581','2.692622e-12','0.0003084575','1.3910012e-09','1.3589649e-09','0.00030542593','1.4177662e-18','1.948583e-12','6.8608045e-19','5.3441106e-12','1.4704384e-18','0.00028334663','2.554978e-10','6.901891e-12','0.0001462654','0.000787849','7.737499e-08','2.5481448e-08','0.0001462654','0.000787849','3.6773197e-16','8.7474325e-18','0.034239754','0.0027186933','2.227533e-09','1.0923845e-09','0.0027186933','0.062218614','0.0056155217','0.0053046667'],
['Lithuania','202004','1.4523559','7.3955306e-07','0.0078025926','8.780097e-06','-4.7050717e-06','0.002045949','3.1553365e-10','2.9044191e-09','1.4064281e-12','-0.046397682','2.8800268e-05','1.0825994e-05','0.01470187','0.023101563','0.00036452862','-0.00027649288','0.014016251','0.021620052','2.6766804e-09','8.808677e-10','1.9637341','3.5899259e-06','0.024276873','4.2203777e-05','-2.8660897e-05','0.023902368','1.5426131e-09','1.8166728e-06','1.1873538e-09','4.2734687e-06','1.6441437e-09','0.024929151','1.23405125e-05','2.7745787e-06','0.0106405085','0.021448586','0.00013308055','-3.7380687e-05','0.0106405085','0.021448586','1.0557616e-08','2.143171e-09','1.6299168','0.055722024','4.644721e-05','-2.0796699e-05','0.055722024','0.97894686','0.1323515','0.124739096','0.002396649','1.4107503e-13','1.7958604e-05','6.175548e-11','3.363921e-11','1.7360514e-06','1.0491517e-19','3.1298685e-17','8.716568e-24','1.8988558e-05','9.572217e-11','9.238775e-12','1.5971293e-05','4.6764046e-05','9.650968e-08','7.573053e-08','1.4484819e-05','4.0529223e-05','2.9773706e-18','2.588449e-19','0.040058866','3.790465e-12','0.00024697193','1.3174661e-09','1.2654823e-09','0.00024336108','1.0850387e-17','4.8771074e-12','1.619325e-18','5.421489e-12','1.2797647e-18','0.00043937107','2.2285643e-10','7.0185316e-12','0.00019195703','0.0009633114','5.0614613e-08','7.2441764e-09','0.00019195703','0.0009633114','2.2952965e-16','6.1147677e-18','0.031869043','0.0020790745','2.230595e-09','8.2956064e-10','0.0020790745','0.0633329','0.004845671','0.0045580035'],
['Luxembourg','202004','1.4312444','1.2315643e-06','0.012252975','1.9483086e-06','-1.4162664e-07','0.0028824345','1.1770238e-09','1.0731798e-09','9.894824e-13','-0.04875754','3.6375925e-05','1.2641754e-05','0.017002046','0.027567541','0.00016572393','-4.7969243e-05','0.016203882','0.025746351','4.186105e-09','1.358403e-09','2.0438619','3.840417e-06','0.02143629','1.0181698e-05','-1.6922811e-06','0.02103717','1.7018498e-09','5.4347106e-06','6.0044623e-09','6.2406875e-06','2.7676728e-09','0.043596942','5.3123117e-06','1.3741517e-06','0.003918167','0.0073398724','4.094662e-06','-4.3536916e-06','0.003918167','0.0073398724','4.64506e-09','1.0367093e-09','1.7381939','0.06919937','1.1110662e-05','5.1041894e-07','0.06919937','1.1312245','0.13779593','0.12620354','0.0011723095','2.9515924e-13','2.889867e-05','3.8964214e-11','3.4254943e-11','1.6833541e-06','4.4649746e-19','1.761041e-17','2.0008946e-23','2.655524e-05','7.3533235e-10','6.385925e-11','0.000108424545','0.0003350904','1.036312e-07','1.1643123e-07','9.826524e-05','0.00028951096','3.512084e-18','2.690958e-19','0.03333291','1.088922e-12','8.549552e-05','4.816257e-10','5.4428917e-10','8.458487e-05','6.990197e-19','1.5322672e-11','2.1522976e-17','1.4709378e-11','2.872407e-18','0.00096171664','3.3617074e-11','1.184381e-12','1.7496668e-05','7.469286e-05','2.481736e-09','3.00813e-09','1.7496668e-05','7.469286e-05','3.5788128e-17','1.2181814e-18','0.034131613','0.002699711','1.0562774e-09','8.47768e-10','0.002699711','0.055648074','0.0039287116','0.0035004118'],
['Malta','202004','1.4402026','8.0919676e-07','0.008242189','7.640618e-06','-1.8715973e-06','0.0020238084','3.4201308e-10','2.392035e-08','9.490716e-11','-0.061017912','0.00020197939','5.3482177e-05','0.069591306','0.1260719','0.0015330714','0.00034537716','0.06624872','0.11686533','2.4901917e-08','5.2187166e-09','2.0160868','4.3554505e-06','0.02659122','4.6566718e-05','-9.909584e-06','0.026137251','1.2246221e-09','1.2619146e-05','3.6194334e-09','7.3890237e-06','3.0553546e-09','0.02372534','1.25451415e-05','3.01663e-06','0.008168479','0.015887054','1.1902788e-06','-1.4137185e-05','0.008168479','0.015887054','2.1097035e-08','4.2337716e-09','1.7275175','0.08108889','2.7699896e-05','-1.5925527e-05','0.08108889','0.7563011','0.25788102','0.24199988','0.001716038','1.9730337e-14','2.0817738e-06','2.5635173e-11','1.5443457e-11','2.975169e-07','2.4301313e-20','1.7194197e-15','2.2197227e-20','4.4693188e-05','3.9657202e-08','2.6074696e-09','0.0043600816','0.014753234','3.7452628e-06','9.564649e-06','0.003949535','0.012654897','3.625578e-15','2.0190707e-16','0.037324093','1.861662e-12','0.00016521793','1.0785357e-09','5.1403654e-10','0.00016358847','3.7607498e-19','3.209037e-10','2.4165275e-18','1.9448063e-11','2.5709977e-18','0.0001344966','5.1844813e-11','2.0589643e-12','2.7194274e-05','0.00012576113','7.8451325e-09','7.758229e-09','2.7194274e-05','0.00012576113','2.701332e-16','9.4188e-18','0.015894469','0.004738039','7.368758e-10','1.8327616e-09','0.004738039','0.12727705','0.020646673','0.018463595'],
['Netherlands','202004','1.4334886','1.3492751e-06','0.013621397','1.7389618e-06','6.4233245e-07','0.0032976584','1.402274e-09','1.0450471e-08','2.1760888e-11','-0.04868386','3.8668153e-05','1.3625223e-05','0.018332938','0.029595932','0.00020166834','-6.687101e-05','0.017472554','0.02764989','3.7406536e-09','1.2215771e-09','2.0080075','4.2441243e-06','0.02590098','9.608361e-06','-1.9274291e-06','0.025458984','2.1238902e-09','6.909228e-06','8.069721e-09','6.335613e-06','2.556683e-09','0.038374998','1.0750572e-05','2.4067901e-06','0.007123301','0.014319107','1.288148e-05','-1.6349768e-05','0.007123301','0.014319107','9.992554e-09','2.097666e-09','1.741181','0.07284487','1.1447119e-05','2.8973827e-06','0.07284487','1.0790391','0.15628265','0.14357129','0.0010847328','4.4082465e-13','5.0239694e-05','5.741631e-11','4.561195e-11','3.938422e-06','1.020206e-18','6.606197e-16','2.6324062e-21','2.3608593e-05','7.0015854e-10','6.4542295e-11','0.00011054121','0.00033188862','1.9142328e-07','1.3319936e-07','0.00010021449','0.0002872228','4.3092127e-18','3.6833177e-19','0.033505507','2.1266034e-12','0.0001949443','8.0756885e-10','7.141083e-10','0.00019230989','2.2458611e-18','1.703627e-11','2.5352255e-17','1.2857451e-11','3.4064774e-18','0.0009746397','1.7887776e-10','4.8974136e-12','7.017417e-05','0.00035707498','1.0959009e-08','1.6307126e-08','7.017417e-05','0.00035707498','1.6761064e-16','5.6237425e-18','0.03352128','0.004352399','1.1774093e-09','1.0132264e-09','0.004352399','0.065571405','0.007285638','0.0066147526'],
['Poland','202004','1.4333855','1.2182803e-06','0.012296161','7.1066697e-06','-3.1304958e-06','0.0029820404','1.2301636e-09','6.989378e-10','3.131319e-13','-0.047315624','3.0416464e-05','1.1227461e-05','0.015213829','0.024063593','0.00026636609','-0.00017799664','0.014503382','0.022509877','3.8551975e-09','1.2401723e-09','2.0192876','4.8380707e-06','0.028819645','3.167768e-05','-2.0210782e-05','0.028315866','3.3227217e-09','5.8114606e-06','6.174507e-09','6.149107e-06','2.4454982e-09','0.03483632','8.7666385e-06','2.1221197e-06','0.005499692','0.010570616','5.757305e-05','-2.503374e-05','0.005499692','0.010570616','5.582775e-09','1.2439735e-09','1.7250689','0.07151067','3.7899325e-05','-1.205981e-05','0.07151067','1.1555222','0.14726034','0.135889','0.0010884682','3.10999e-13','3.7292895e-05','5.8905485e-11','3.5854302e-11','3.2318253e-06','6.584933e-19','2.6383116e-18','8.2104076e-25','1.5056406e-05','1.2953902e-10','1.3200958e-11','2.306881e-05','6.556793e-05','7.212701e-08','3.9072532e-08','2.0930582e-05','5.6927638e-05','7.730308e-18','5.5841547e-19','0.032293186','3.5494515e-12','0.0002707409','1.1985893e-09','9.665689e-10','0.00026716854','5.004229e-18','1.819699e-11','1.4301612e-17','1.28902e-11','3.482893e-18','0.0004935111','1.0432065e-10','3.4668906e-12','3.0529958e-05','0.00014754973','1.2461113e-08','4.4621262e-09','3.0529958e-05','0.00014754973','4.0669827e-17','1.2521209e-18','0.027031269','0.0043758047','2.1330835e-09','8.640261e-10','0.0043758047','0.039765332','0.0070769573','0.0064888597'],
['Portugal','202004','1.4661419','6.881869e-07','0.007441055','4.576884e-06','1.6153432e-06','0.0020201595','3.3410133e-10','1.012805e-08','6.6941314e-12','-0.050832286','5.0298364e-05','1.5505007e-05','0.020688387','0.034949318','0.00020390168','0.00018172461','0.019711612','0.032541517','1.167053e-08','2.875614e-09','1.9036775','3.5529613e-06','0.02794576','2.8921366e-05','9.360359e-06','0.027573299','1.2217553e-09','2.3221266e-06','2.8579856e-09','4.3587083e-06','1.900734e-09','0.015827019','1.026398e-05','2.1744847e-06','0.009311776','0.019146033','2.4462533e-05','4.2588665e-05','0.009311776','0.019146033','9.660513e-09','1.8177653e-09','1.5345536','0.06463297','1.8945275e-05','7.790989e-06','0.06463297','0.9642728','0.15411514','0.14591412','0.0020993946','2.3626466e-14','3.7976022e-06','1.4880807e-11','1.9858754e-11','5.7850474e-07','3.3615728e-20','4.941727e-16','2.0028508e-22','4.943268e-05','4.352177e-09','2.6116057e-10','0.0004380151','0.0015401255','5.249775e-08','1.1977875e-07','0.0003967318','0.0013178964','2.0792776e-16','1.047866e-17','0.032616287','8.2899166e-13','0.00026457603','5.6621063e-10','6.7728867e-10','0.0002627063','5.0578134e-19','1.8034024e-12','4.5728397e-18','2.989704e-12','1.776866e-18','0.00030782915','1.1518122e-10','2.7919238e-12','6.376765e-05','0.00036251434','5.4459637e-09','1.11089244e-08','6.376765e-05','0.00036251434','1.0230627e-16','2.2091688e-18','0.0141252875','0.001117854','3.5265732e-10','5.5925914e-10','0.001117854','0.035867717','0.005915275','0.0054057017'],
['Romania','202004','1.4330695','1.2200169e-06','0.012281408','5.737714e-06','-2.994834e-06','0.0029659374','8.028703e-10','1.15645805e-10','5.5222557e-16','-0.049043715','3.3833214e-05','1.1788981e-05','0.015902475','0.025655301','0.0002458144','-0.00013208634','0.015157632','0.023964474','4.8351785e-09','1.551912e-09','2.0583615','7.99591e-06','0.043085936','3.4657118e-05','-1.7910506e-05','0.042255983','5.5879936e-09','3.8321446e-06','2.5302458e-09','6.590063e-06','2.6261295e-09','0.036778532','4.008268e-06','1.093528e-06','0.002610559','0.004645147','7.977025e-06','-6.7208207e-06','0.002610559','0.004645147','2.3213957e-09','5.578662e-10','1.7557433','0.07976524','3.319082e-05','-1.3319114e-05','0.07976524','1.2838738','0.16543452','0.15359789','0.0015665413','2.0963433e-13','2.3825705e-05','7.433158e-11','1.4813368e-11','2.2285217e-06','2.4345807e-19','2.4169264e-20','1.3498134e-29','2.694861e-05','2.5077163e-10','1.9016675e-11','3.2563672e-05','0.000102244005','1.7216367e-07','4.0882796e-08','2.9523178e-05','8.8095396e-05','4.3031805e-18','3.2176447e-19','0.032361437','2.7414186e-11','0.001042955','3.929013e-09','7.4456386e-10','0.0010176101','2.8860692e-17','5.4374435e-12','2.3741475e-18','1.3846964e-11','3.986332e-18','0.00038834306','7.498986e-12','3.693615e-13','3.0038682e-06','1.0676042e-05','7.247638e-10','2.9118738e-10','3.0038682e-06','1.0676042e-05','3.4851936e-18','1.4222335e-19','0.035424557','0.0067373845','3.6516292e-09','4.3157541e-10','0.0067373845','0.0303743','0.012184431','0.011671151'],
['Slovakia','202004','1.426949','1.0624128e-06','0.010573046','4.8821685e-06','-3.3376575e-06','0.0025018337','6.3538813e-10','1.6830047e-10','1.3694383e-15','-0.04686443','2.8786279e-05','1.056463e-05','0.014331415','0.022680018','0.0001607175','-0.00014060846','0.013662708','0.021213483','4.353098e-09','1.3910303e-09','2.0667148','5.0465164e-06','0.026576184','2.2795515e-05','-1.751458e-05','0.026052512','2.4442832e-09','4.548721e-06','3.1612448e-09','6.149012e-06','2.3098319e-09','0.040566344','4.631906e-06','1.2441932e-06','0.0027762346','0.0050550657','5.4892625e-06','-1.0674488e-05','0.0027762346','0.0050550657','2.6765012e-09','6.512832e-10','1.781464','0.06841835','2.6053789e-05','-1.36152485e-05','0.06841835','1.2438086','0.1333027','0.123241656','0.0010224552','1.5691314e-13','1.9256411e-05','5.2231025e-11','1.9810875e-11','1.69076e-06','1.0819109e-19','8.471758e-20','5.1808984e-29','2.1354967e-05','1.4182899e-10','1.3609403e-11','2.3633344e-05','6.883424e-05','7.569307e-08','3.0879136e-08','2.1438524e-05','5.9660644e-05','1.2336459e-17','7.773018e-19','0.03354248','6.25963e-12','0.000231872','1.0288593e-09','6.0943867e-10','0.00022715925','3.818822e-18','6.8367126e-12','3.012028e-18','1.1024104e-11','2.0412119e-18','0.0006106009','1.7521132e-11','7.9790607e-13','5.77767e-06','2.4569063e-05','6.3226235e-10','1.0281846e-09','5.77767e-06','2.4569063e-05','6.5654723e-18','2.7616862e-19','0.030434','0.0043382985','2.048008e-09','6.941134e-10','0.0043382985','0.017905898','0.0069124876','0.006485589'],
['Slovenia','202004','1.4383942','1.1223237e-06','0.011481081','1.4213072e-06','-1.7142953e-06','0.0028527216','6.474798e-10','1.6685109e-10','2.6760354e-14','-0.050843902','4.071791e-05','1.353125e-05','0.01811737','0.029844169','0.00017445638','-0.0001209316','0.017264139','0.027840462','5.3763745e-09','1.6395766e-09','2.0241919','5.618171e-06','0.033826705','8.603566e-06','-8.047585e-06','0.03324213','2.7847518e-09','6.372159e-06','5.0531437e-09','7.73521e-06','3.410677e-09','0.030092524','5.23457e-06','1.2552789e-06','0.003196722','0.006100127','-2.5562522e-06','-4.180888e-06','0.003196722','0.006100127','3.3599428e-09','7.229999e-10','1.7507913','0.09512564','6.846059e-06','-8.922901e-06','0.09512564','1.2126892','0.17637719','0.16516006','0.0017276242','1.7577883e-13','2.3051118e-05','5.9315435e-11','1.4051051e-11','2.392795e-06','1.0851781e-19','1.0232411e-19','9.350041e-27','3.5251793e-05','6.510639e-10','5.1849018e-11','8.756834e-05','0.00027692952','3.1608297e-07','9.356667e-08','7.934981e-05','0.0002387697','1.0552484e-17','6.764993e-19','0.04070346','7.47759e-12','0.000610758','2.4553515e-09','4.0132506e-10','0.00060220424','3.3058497e-18','1.8031044e-11','6.333774e-18','1.8972077e-11','5.2651237e-18','0.0005098305','1.1679177e-11','4.4556747e-13','7.311007e-06','2.8720253e-05','1.1822587e-09','3.8338827e-10','7.311007e-06','2.8720253e-05','5.1630244e-18','1.5580695e-19','0.04044979','0.008383054','2.5194233e-09','7.138216e-10','0.008383054','0.04391554','0.014109414','0.013456928'],
['Spain','202004','1.4818126','7.4051627e-07','0.00824336','3.7213567e-06','2.51012e-06','0.002348624','6.440326e-10','7.603585e-10','6.3198795e-14','-0.0532311','5.1254727e-05','1.6082005e-05','0.02136042','0.03608482','0.00015603543','0.00020192376','0.020349093','0.03360833','1.3898062e-08','3.7017671e-09','1.833285','3.2503747e-06','0.031197626','2.3379753e-05','1.4526719e-05','0.030855156','1.3241602e-09','4.3561163e-06','6.728985e-09','4.290461e-06','2.3321527e-09','0.027937843','2.5068898e-06','7.1182626e-07','0.0029784031','0.0052275173','7.654254e-06','1.0558743e-05','0.0029784031','0.0052275173','1.8002889e-09','4.399046e-10','1.5165551','0.068102','1.4107792e-05','1.2708416e-05','0.068102','1.0444278','0.14885566','0.1401406','0.002617169','1.41968024e-14','2.3509467e-06','1.5723423e-11','1.860743e-11','5.1128865e-07','1.506766e-19','3.6258693e-18','1.1510214e-25','4.397727e-05','1.4045904e-09','1.0391363e-10','0.00017525873','0.0005716843','8.462247e-08','1.438942e-07','0.0001588028','0.0004918469','1.7455196e-16','1.1467589e-17','0.038400657','6.1100154e-13','0.00030055197','5.368695e-10','6.815075e-10','0.00029916203','5.362159e-19','5.177063e-12','1.5133463e-17','1.7782436e-12','1.2512631e-18','0.00047736618','3.767241e-12','1.7354138e-13','4.01337e-06','1.5727626e-05','3.1534056e-10','4.9024657e-10','4.01337e-06','1.5727626e-05','3.7371267e-18','1.4007617e-19','0.021547642','0.0009476636','3.1607633e-10','4.0424772e-10','0.0009476636','0.029891606','0.0030465212','0.0028403602'],
['Sweden','202004','1.4514699','5.6221035e-07','0.005921454','6.5960994e-06','-5.146821e-06','0.0015564021','1.6711393e-10','1.9031164e-08','1.558024e-11','-0.044877503','2.5467893e-05','9.872498e-06','0.013480836','0.020892335','0.0003109185','-0.00026263695','0.012854623','0.019569052','2.0631297e-09','6.742064e-10','1.9830202','2.8760037e-06','0.020050433','3.4187884e-05','-3.2287087e-05','0.019750169','5.392332e-10','1.1318493e-06','9.65517e-10','3.129439e-06','9.002209e-10','0.017255632','1.49701145e-05','2.9853952e-06','0.009072419','0.018935794','0.00011413575','-8.4199295e-05','0.009072419','0.018935794','2.1918298e-08','3.6180121e-09','1.6856502','0.036196742','3.436896e-05','-2.159669e-05','0.036196742','0.89039063','0.10199628','0.09600752','0.002916048','4.1380595e-14','6.007303e-06','3.0786627e-11','3.0128677e-11','8.22761e-07','2.6341195e-20','8.215713e-16','2.1103058e-22','1.8714041e-05','4.684004e-11','5.912445e-12','1.0670136e-05','2.7338254e-05','6.90495e-08','6.917669e-08','9.6922495e-06','2.387916e-05','8.81767e-19','1.05822004e-19','0.046408497','8.2761977e-13','0.00020837584','7.9668266e-10','1.0362892e-09','0.00020733307','2.5674668e-19','2.6102187e-12','1.857373e-18','2.5926544e-12','1.2014793e-18','0.00013493278','2.005581e-10','4.9292927e-12','6.779794e-05','0.00038771523','4.5349214e-08','3.8903277e-08','6.779794e-05','0.00038771523','2.9438265e-16','5.736989e-18','0.027332054','0.0011199319','1.1189124e-09','7.262712e-10','0.0011199319','0.040770207','0.002494201','0.0023811425'],
['Ireland','202004','1.4726455','9.929879e-07','0.01095436','-2.0010768e-07','2.7602916e-06','0.003049678','6.255628e-10','1.8465139e-08','2.094984e-11','-0.04769666','3.5762245e-05','1.2686886e-05','0.017090565','0.027491609','9.754647e-05','0.00020238139','0.016289273','0.025688674','3.5545793e-09','1.1476186e-09','1.8691851','3.6803435e-06','0.030620571','5.371902e-06','1.1784657e-05','0.030234426','1.3348881e-09','4.0665836e-06','4.041188e-09','5.162269e-06','2.4050846e-09','0.021536576','1.3890275e-05','3.180468e-06','0.012055132','0.023810767','1.3595162e-05','2.95182e-05','0.012055132','0.023810767','1.574044e-08','3.2952165e-09','1.5880208','0.074889146','-7.808355e-07','1.4966312e-05','0.074889146','0.9612483','0.16776621','0.15767202','0.0022702052','4.8861674e-13','6.691937e-05','5.2627458e-11','2.9110776e-11','6.7542574e-06','3.5222428e-19','2.0348666e-15','2.4081037e-21','2.57677e-05','7.493756e-10','6.3511044e-11','0.00010789987','0.0003339532','7.159938e-08','1.5026126e-07','9.77997e-05','0.00028834303','3.4959275e-18','3.0756356e-19','0.036813915','2.865447e-12','0.00043752693','8.6972773e-10','6.768486e-10','0.00043239555','1.3320083e-18','1.6927761e-11','1.0481386e-17','1.2046567e-11','4.541384e-18','0.0004326899','2.236281e-10','5.656671e-12','0.00012766109','0.00067086075','4.2356906e-08','2.11146e-08','0.00012766109','0.00067086075','3.483667e-16','8.638757e-18','0.022678578','0.00511619','1.2593979e-09','8.4089063e-10','0.00511619','0.06344224','0.010138081','0.009065857'],
['Italy','202004','1.4383186','1.1281827e-06','0.011276402','2.0056286e-06','-2.18217e-07','0.0026949805','7.1645034e-10','3.1613758e-09','4.8104173e-12','-0.05584613','0.0001230999','3.543585e-05','0.046438135','0.08156065','-0.00011974879','0.00031310634','0.04421844','0.075777784','2.2696643e-08','5.779816e-09','2.0257463','7.1099344e-06','0.03847674','1.3956762e-05','-6.235064e-07','0.0377386','2.6445754e-09','9.22804e-06','7.2253914e-09','7.5547387e-06','3.5959244e-09','0.025251048','1.040558e-05','2.452155e-06','0.005849848','0.01112566','1.9075583e-06','5.4031494e-07','0.005849848','0.01112566','7.302288e-09','1.6382584e-09','1.7535385','0.081136525','8.856411e-07','3.597139e-06','0.081136525','1.0133343','0.22357689','0.20847526','0.0017407481','3.1754544e-13','2.725764e-05','2.3304164e-11','1.6212014e-11','1.4305817e-06','1.6088815e-19','1.076326e-16','2.4685788e-22','5.126071e-05','3.529385e-08','2.6039308e-09','0.0043563144','0.014295153','2.8411837e-06','2.1779003e-06','0.003946186','0.012297101','1.3779158e-15','9.6602996e-17','0.039621368','4.3359046e-11','0.00088638323','8.9578933e-10','9.623975e-10','0.0008519156','6.8386247e-18','4.1835646e-11','1.5524675e-17','1.4072837e-11','3.7041346e-18','0.00021107782','7.445074e-11','2.960018e-12','1.6353893e-05','6.742018e-05','1.2653566e-09','1.2766145e-09','1.6353893e-05','6.742018e-05','3.081928e-17','1.3644917e-18','0.029402358','0.0041635344','1.4392612e-09','1.4252689e-09','0.0041635344','0.09744603','0.03268982','0.029699715'],
['Denmark','202004','1.4541504','7.254178e-07','0.007764166','6.6325388e-06','-1.8238371e-06','0.002086503','4.3381695e-10','3.6781326e-08','6.20202e-11','-0.04561036','2.6547938e-05','1.0195974e-05','0.013877196','0.021627722','0.00030354792','-0.0001309266','0.013231181','0.020251608','2.5399896e-09','8.5694785e-10','1.964703','3.0468793e-06','0.022408556','3.263089e-05','-1.6719026e-05','0.022090128','8.8267404e-10','2.1292979e-06','2.5053764e-09','3.921533e-06','1.5614923e-09','0.02233892','1.5987518e-05','3.140649e-06','0.010405405','0.021906916','0.00016267748','-6.932102e-05','0.010405405','0.021906916','1.860194e-08','3.4114407e-09','1.689061','0.0501639','3.38109e-05','-5.0670415e-06','0.0501639','0.9407829','0.12386988','0.11649776','0.0029488106','1.0949196e-13','1.8967648e-05','5.536822e-11','4.3011844e-11','2.582271e-06','1.0042892e-19','3.6095297e-15','6.927119e-21','1.1716445e-05','8.297576e-11','9.310984e-12','1.6470727e-05','4.5014414e-05','1.0469039e-07','7.433088e-08','1.49502275e-05','3.917737e-05','1.0835265e-18','1.1331877e-19','0.052904163','5.558564e-13','0.00026799177','1.1258838e-09','9.881611e-10','0.00026658524','2.9809245e-19','3.4636183e-12','2.8529257e-18','6.974258e-12','2.0773094e-18','0.0004266102','5.919434e-10','1.1170048e-11','0.00013903','0.00085826544','2.2528374e-07','5.059317e-08','0.00013903','0.00085826544','5.976309e-16','1.02086876e-17','0.040073004','0.0037331092','2.0362712e-09','1.3193631e-09','0.0037331092','0.055689704','0.007029583','0.006663359'],
['Finland','202004','1.4764045','5.6842384e-07','0.006343564','5.063631e-06','-5.4531947e-06','0.0018166091','1.4899049e-10','1.45219365e-08','4.4743033e-12','-0.045677382','2.7358778e-05','1.0219488e-05','0.013901327','0.021850731','0.0002389606','-0.0003115985','0.013253952','0.020447304','2.3558402e-09','7.4021056e-10','1.8944384','3.1949528e-06','0.027639395','2.7546947e-05','-3.5068122e-05','0.027303575','5.6096494e-10','9.3605064e-07','6.8297024e-10','3.421602e-06','1.2739726e-09','0.023312455','9.632653e-06','2.3514312e-06','0.009254963','0.017747939','6.453311e-05','-2.4024022e-05','0.009254963','0.017747939','7.850174e-09','1.7417411e-09','1.5597574','0.048586447','2.7621687e-05','-2.4709425e-05','0.048586447','0.95556396','0.12216821','0.11590195','0.0038083817','4.1368046e-14','6.8507206e-06','1.8936294e-11','3.4223763e-11','1.0527052e-06','1.5200403e-20','2.425917e-16','2.946299e-23','3.773835e-05','1.2468658e-10','1.11834144e-11','1.9154044e-05','5.7782625e-05','5.464896e-08','1.348464e-07','1.7367125e-05','4.997645e-05','2.2138865e-18','2.0000078e-19','0.044092175','3.2169772e-12','0.0005967058','5.067621e-10','1.3689498e-09','0.00059016206','1.8517516e-19','1.4847495e-12','6.6776425e-19','1.6612449e-12','1.3503999e-18','0.0005203696','8.613625e-11','3.1948192e-12','7.7876e-05','0.00037532754','1.2817086e-08','1.4586912e-08','7.7876e-05','0.00037532754','7.354075e-17','2.074608e-18','0.028128222','0.001000368','7.315556e-10','9.475689e-10','0.001000368','0.034914684','0.0031813036','0.003025289'],
['Belgium','202004','1.429283','1.5008161e-06','0.014836582','1.1481411e-06','6.8938095e-07','0.0034561558','1.7527287e-09','2.6448321e-09','3.555274e-12','-0.04897151','3.916397e-05','1.3594847e-05','0.018261248','0.029658403','0.00015088054','-2.7046815e-05','0.017403461','0.027697006','4.311947e-09','1.3949435e-09','2.0394213','4.345513e-06','0.023952976','7.4246e-06','5.453908e-07','0.023501322','2.4884927e-09','6.9718826e-06','7.6219004e-09','6.7853675e-06','2.9190494e-09','0.041052774','7.2028292e-06','1.7346152e-06','0.00503818','0.009817803','5.6497397e-06','-1.1011714e-06','0.00503818','0.009817803','5.984505e-09','1.2851676e-09','1.7456764','0.07215693','8.157936e-06','4.0540353e-06','0.07215693','1.131666','0.15042357','0.13662985','0.0009009074','4.924929e-13','4.693501e-05','5.085197e-11','4.428153e-11','2.608251e-06','1.1632094e-18','7.573661e-17','9.68838e-23','2.4216324e-05','8.9258095e-10','7.8601285e-11','0.00013331248','0.00041101835','9.9177264e-08','9.282435e-08','0.00012082954','0.0003552003','3.6129156e-18','3.069301e-19','0.030640537','1.8863834e-12','0.00010448308','5.8290367e-10','6.216542e-10','0.00010290803','5.069734e-18','1.38070015e-11','2.4234915e-17','1.7499458e-11','2.9206929e-18','0.0009439052','7.170974e-11','2.2677207e-12','3.453907e-05','0.00016675059','6.2389502e-09','6.2957137e-09','3.453907e-05','0.00016675059','6.511031e-17','1.852966e-18','0.03443222','0.0026517482','1.1628496e-09','1.0426925e-09','0.0026517482','0.06495074','0.0041941884','0.0036497624'],
['Croatia','202004','1.4347166','1.1175947e-06','0.011183776','-3.5828403e-08','-1.6703237e-06','0.0026804588','6.275106e-10','2.3829846e-10','7.531836e-14','-0.05142056','4.8633367e-05','1.5891177e-05','0.021208504','0.03523047','0.00012374566','-0.000103196624','0.020207806','0.032849144','6.7367503e-09','1.9769448e-09','2.0458562','7.101658e-06','0.037715673','-1.3837221e-06','-8.164655e-06','0.036978874','4.283763e-09','5.4535244e-06','3.688705e-09','7.3359583e-06','3.3560805e-09','0.02535539','6.6885027e-06','1.4630971e-06','0.0035907743','0.0071309125','-1.402399e-05','-6.1806645e-06','0.0035907743','0.0071309125','9.0377466e-09','1.4796415e-09','1.7816707','0.07938803','-5.502713e-06','-9.6688345e-06','0.07938803','1.1878736','0.17064917','0.15902795','0.0018227815','2.177332e-13','2.1242422e-05','8.320499e-11','1.8388902e-11','1.5026189e-06','1.5766931e-19','3.7632502e-19','1.02316005e-25','3.833733e-05','1.2302819e-09','9.921179e-11','0.00016735547','0.0005288028','3.1827062e-07','9.893728e-08','0.00015165913','0.0004560503','2.7558323e-17','1.630751e-18','0.040012132','3.3985877e-11','0.0008527616','4.818166e-09','7.3400824e-10','0.00082592026','2.8706863e-17','2.598593e-11','5.52645e-18','1.612475e-11','4.8528896e-18','0.00024349499','1.4606587e-11','4.9590676e-13','7.460693e-06','3.2135016e-05','2.9280294e-09','8.9430396e-10','7.460693e-06','3.2135016e-05','2.7617064e-17','5.458313e-19','0.034907114','0.0046884655','3.908302e-09','9.611586e-10','0.0046884655','0.056328584','0.009608935','0.009122102']]

In [114]:
# Create the DataFrame that will be written in S3
columns = ["Country","Date","BCANGSTR","BCCMASS","BCEXTTAU","BCFLUXU","BCFLUXV","BCSCATAU","BCSMASS","DMSCMASS","DMSSMASS","DUANGSTR","DUCMASS","DUCMASS25","DUEXTT25","DUEXTTAU","DUFLUXU","DUFLUXV","DUSCAT25","DUSCATAU","DUSMASS","DUSMASS25","OCANGSTR","OCCMASS","OCEXTTAU","OCFLUXU","OCFLUXV","OCSCATAU","OCSMASS","SO2CMASS","SO2SMASS","SO4CMASS","SO4SMASS","SSANGSTR","SSCMASS","SSCMASS25","SSEXTT25","SSEXTTAU","SSFLUXU","SSFLUXV","SSSCAT25","SSSCATAU","SSSMASS","SSSMASS25","SUANGSTR","SUEXTTAU","SUFLUXU","SUFLUXV","SUSCATAU","TOTANGSTR","TOTEXTTAU","TOTSCATAU","Var_BCANGSTR","Var_BCCMASS","Var_BCEXTTAU","Var_BCFLUXU","Var_BCFLUXV","Var_BCSCATAU","Var_BCSMASS","Var_DMSCMASS","Var_DMSSMASS","Var_DUANGSTR","Var_DUCMASS","Var_DUCMASS25","Var_DUEXTT25","Var_DUEXTTAU","Var_DUFLUXU","Var_DUFLUXV","Var_DUSCAT25","Var_DUSCATAU","Var_DUSMASS","Var_DUSMASS25","Var_OCANGSTR","Var_OCCMASS","Var_OCEXTTAU","Var_OCFLUXU","Var_OCFLUXV","Var_OCSCATAU","Var_OCSMASS","Var_SO2CMASS","Var_SO2SMASS","Var_SO4CMASS","Var_SO4SMASS","Var_SSANGSTR","Var_SSCMASS","Var_SSCMASS25","Var_SSEXTT25","Var_SSEXTTAU","Var_SSFLUXU","Var_SSFLUXV","Var_SSSCAT25","Var_SSSCATAU","Var_SSSMASS","Var_SSSMASS25","Var_SUANGSTR","Var_SUEXTTAU","Var_SUFLUXU","Var_SUFLUXV","Var_SUSCATAU","Var_TOTANGSTR","Var_TOTEXTTAU","Var_TOTSCATAU"]

df_pollution_SO2 = spark.createDataFrame(info_so2, columns)
 
# Writting the results to s3

df_pollution_SO2.repartition(1).write.option("delimiter", ",").mode('overwrite').option("header", "false").csv("D://notebooks/NASA/Output/SO2.csv")

